In [1]:
from bs4 import BeautifulSoup     #网页解析，获取数据\n",
import re       #正则表达式，进行文字匹配\n",
import urllib.request,urllib.error      #制定URL，获取网页数据\n",
# import xlwt     #进行excel操作\n",
import sqlite3  #进行SQLite数据库操作\n",
import pandas as pd #存储数据及处理数据\n",
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys
import datetime
import os
import csv
from math import *
import pymysql
# from geopy.distance import geodesic
# from pyproj import Proj
pd.set_option("display.max_columns", 100)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# 20240911产业大脑新税务数据读取和处理

In [55]:
# 产业链产品对应数据
df_chain_nodes=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\十条产业链的核心产品.xlsx',dtype='str')
df_chain_nodes1=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\T_CODE_PRODUCT_20240111.xlsx',dtype='str',sheet_name='T_CODE_PRODUCT_20240111')
df_chain_nodes2=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\T_CODE_PRODUCT_20240913.xlsx',dtype='str')

# 地名和税务产品名对应数据
df_fp_total=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\t_data_sh_new_cpdm.xlsx',dtype='str')
df_fp_name=pd.read_excel(r'E:\project_data\04yulin\fp\t_data_sh_new_cpdm_202309261607.xlsx',dtype='str')
df_fp_name_=df_fp_name.loc[:,['spbm','spmc']]

df_total_admin=pd.read_csv(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\admin_division.csv',dtype='str')
df_province=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_province',dtype='str')
df_city=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_city',dtype='str')
df_dis=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_dis',dtype='str')
df_city=df_city.drop_duplicates(['type_4'],keep='first').reset_index(drop=True)
df_dis_=df_dis.loc[:,['type_4','admin_name']]
df_spbm=df_chain_nodes1.loc[df_chain_nodes1['spbm'].notnull(),['CODE','NAME','spbm','spmc']]
df_spbm1=df_spbm.drop_duplicates(subset=['spbm']).reset_index(drop=True)

# 每条产业链税务商品对应关系
df_chain_spbm=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\t_chain_core_swcpbm.xlsx',dtype='str')
# df_chain_nodes2['spbm'].fillna('aaaa',inplace=True)
# df_chain_nodes2['spmc'].fillna('aaaa',inplace=True)

In [200]:
# 连接数据库

db = pymysql.connect(
                    host='172.30.11.47',
                    port=3306,
                    user='root',    #在这里输入用户名
                    password='123456',     #在这里输入密码
                    )
cursor = db.cursor()

# home_chart_5

In [4]:
# 月度购销情况——变化
sql = "SELECT * FROM datapool.sh_1"
cursor.execute(sql)
total_fp = cursor.fetchall()


ls_col=[i[0] for i in cursor.description]
df_fp_1 = pd.DataFrame(total_fp,columns=ls_col)

# ls_spbm=df_fp_1['spbm'].unique()
latest_time=str(sorted([int(x) for x in df_fp_1['kpyf'].unique().tolist()])[-1])
df_fp_1_read=df_fp_1.loc[df_fp_1['kpyf']==latest_time].reset_index(drop=True)
for i in range(len(df_fp_1_read)):
    spbm_i=df_fp_1_read.loc[i,'spbm']
    xf_city_i=df_fp_1_read.loc[i,'xf_city']
    gf_city_i=df_fp_1_read.loc[i,'gf_city']
#     ls_xf_city_name_i=df_city.loc[df_city['type_4']==xf_city_i,'admin_name'].tolist()
#     ls_gf_city_name_i=df_city.loc[df_city['type_4']==gf_city_i,'admin_name'].tolist()
    
    ls_xf_city_name_i=df_total_admin.loc[df_total_admin['type_4']==xf_city_i,'admin_name'].tolist()
    ls_gf_city_name_i=df_total_admin.loc[df_total_admin['type_4']==gf_city_i,'admin_name'].tolist()
    
    if len(ls_xf_city_name_i)==1:
        xf_city_name_i=ls_xf_city_name_i[0]
        df_fp_1_read.loc[i,'xf_city_name']=xf_city_name_i
    else:
        print('xf_city_i',xf_city_i,ls_xf_city_name_i)
    
    if len(ls_gf_city_name_i)==1:
        gf_city_name_i=ls_gf_city_name_i[0]
        df_fp_1_read.loc[i,'gf_city_name']=gf_city_name_i
    else:
        print('gf_city_i',gf_city_i,ls_gf_city_name_i)
        
    df_sp_name=df_chain_nodes2.loc[df_chain_nodes2['spbm'].str.contains(spbm_i)]
    if len(df_sp_name)>0:
        df_sp_name1=df_sp_name.sample(1)
        sp_code=df_sp_name1['CODE'].tolist()[0]
        sp_name=df_sp_name1['NAME'].tolist()[0]
        df_fp_1_read.loc[i,'sp_code']=sp_code
        df_fp_1_read.loc[i,'sp_name']=sp_name
    else:
        print('spbm',spbm_i)

df_fp_1_read1=df_fp_1_read.astype(dtype={'sale_diff':'float'})
df_fp_1_read2=df_fp_1_read1.sort_values(by='sale_diff',ascending=False)
df_fp_1_read3=df_fp_1_read2.loc[(df_fp_1_read2['xf_city_name'].notnull())&(df_fp_1_read2['gf_city_name'].notnull())&((df_fp_1_read2['sale_diff']<=-0.1)|(df_fp_1_read2['sale_diff']>=0.1))].reset_index(drop=True)

print("name:'变化',")

ls_xf=df_fp_1_read3['xf_city_name'].unique().tolist()
ls_gf=df_fp_1_read3['gf_city_name'].unique().tolist()
ls_pro=df_fp_1_read3['sp_name'].unique().tolist()
for a in ls_xf:
    print("{name: '"+a+"'},")
for b in ls_gf:
    print("{name: '"+b+" '},")
for c in ls_pro:
    print("{name: '"+c+"'},")

for i in range(len(df_fp_1_read3)):
    name_pro=df_fp_1_read3.loc[i,'sp_name']
    name_xf=df_fp_1_read3.loc[i,'xf_city_name']
    name_gf=df_fp_1_read3.loc[i,'gf_city_name']
    name_gf=name_gf+' '
    weight_1=abs(df_fp_1_read3.loc[i,'sale_diff'])
    weight_2=df_fp_1_read3.loc[i,'sale_diff']
    if weight_2>0:
        print("{source: '"+name_xf+"',target: '"+name_pro+"',value: "+str(round(weight_1,2))+"},")
        print("{source: '"+name_pro+"',target: '"+name_gf+"',value: "+str(round(weight_1,2))+"},")
    else:
        print("{source: '"+name_xf+"',target: '"+name_pro+"',value: "+str(round(weight_1,2))+",lineStyle:{opacity:1}},")
        print("{source: '"+name_pro+"',target: '"+name_gf+"',value: "+str(round(weight_1,2))+",lineStyle:{opacity:1}},")

In [76]:
# 月度购销情况——本月
sql = "SELECT * FROM datapool.sh_2"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_2 = pd.DataFrame(total_fp,columns=ls_col)

latest_time=str(sorted([int(x) for x in df_fp_2['kpyf'].unique().tolist()])[-1])
df_fp_2_read=df_fp_2.loc[df_fp_2['kpyf']==latest_time].reset_index(drop=True)
for i in range(len(df_fp_2_read)):
    spbm_i=df_fp_2_read.loc[i,'spbm']
    xf_city_i=df_fp_2_read.loc[i,'xf_city']
    gf_city_i=df_fp_2_read.loc[i,'gf_city']
    
    ls_xf_city_name_i=df_total_admin.loc[df_total_admin['type_4']==xf_city_i,'admin_name'].tolist()
    ls_gf_city_name_i=df_total_admin.loc[df_total_admin['type_4']==gf_city_i,'admin_name'].tolist()
    
    if len(ls_xf_city_name_i)==1:
        xf_city_name_i=ls_xf_city_name_i[0]
        df_fp_2_read.loc[i,'xf_city_name']=xf_city_name_i
    else:
        print('xf_city_i',xf_city_i,ls_xf_city_name_i)
    
    if len(ls_gf_city_name_i)==1:
        gf_city_name_i=ls_gf_city_name_i[0]
        df_fp_2_read.loc[i,'gf_city_name']=gf_city_name_i
    else:
        print('gf_city_i',gf_city_i,ls_gf_city_name_i)
        
    df_sp_name=df_chain_nodes2.loc[df_chain_nodes2['spbm'].str.contains(spbm_i)]
    if len(df_sp_name)>0:
        df_sp_name1=df_sp_name.sample(1)
        sp_code=df_sp_name1['CODE'].tolist()[0]
        sp_name=df_sp_name1['NAME'].tolist()[0]
        df_fp_2_read.loc[i,'sp_code']=sp_code
        df_fp_2_read.loc[i,'sp_name']=sp_name
    else:
        print('spbm',spbm_i)
        
df_fp_1_read1=df_fp_2_read.astype(dtype={'total_sale':'float'})
df_fp_1_read2=df_fp_1_read1.sort_values(by='total_sale',ascending=False)
df_fp_1_read3=df_fp_1_read2.loc[(df_fp_1_read2['xf_city_name'].notnull())&(df_fp_1_read2['gf_city_name'].notnull())&((df_fp_1_read2['total_sale']<=-0.1)|(df_fp_1_read2['total_sale']>=10000))].reset_index(drop=True)


print("name:'上季度',")
ls_xf=df_fp_1_read3['xf_city_name'].unique().tolist()
ls_gf=df_fp_1_read3['gf_city_name'].unique().tolist()
ls_pro=df_fp_1_read3['sp_name'].unique().tolist()
for a in ls_xf:
    print("{name: '"+a+"'},")
for b in ls_gf:
    print("{name: '"+b+" '},")
for c in ls_pro:
    print("{name: '"+c+"'},")
    
for i in range(len(df_fp_1_read3)):
    name_pro=df_fp_1_read3.loc[i,'sp_name']
    name_xf=df_fp_1_read3.loc[i,'xf_city_name']
    name_gf=df_fp_1_read3.loc[i,'gf_city_name']
    name_gf=name_gf+' '
    weight_1=df_fp_1_read3.loc[i,'total_sale']/100000000
    print("{source: '"+name_xf+"',target: '"+name_pro+"',value: "+str(round(weight_1,4))+"},")
    print("{source: '"+name_pro+"',target: '"+name_gf+"',value: "+str(round(weight_1,4))+"},")

xf_city_i 3102 []
gf_city_i 6321 []
xf_city_i 6271 []
gf_city_i 6412 []
gf_city_i 6111 []
name:'上季度',
{name: '榆林市'},
{name: '上海市'},
{name: '青岛市'},
{name: '陕西省'},
{name: '泰安市'},
{name: '镇江市'},
{name: '舟山市'},
{name: '常州市'},
{name: '唐山市'},
{name: '海口市'},
{name: '甘肃省'},
{name: '昆明市'},
{name: '宁波市'},
{name: '红河哈尼族彝族自治州'},
{name: '江西省'},
{name: '丽水市'},
{name: '河南省'},
{name: '南通市'},
{name: '西安市'},
{name: '湖北省'},
{name: '天津市'},
{name: '平顶山市'},
{name: '绵阳市'},
{name: '安阳市'},
{name: '三门峡市'},
{name: '苏州市'},
{name: '厦门市'},
{name: '深圳市'},
{name: '临沂市'},
{name: '衢州市'},
{name: '河池市'},
{name: '济宁市'},
{name: '内蒙古自治区'},
{name: '福建省'},
{name: '鄂尔多斯市'},
{name: '广州市'},
{name: '张家口市'},
{name: '北京市'},
{name: '海南省'},
{name: '滨州市'},
{name: '包头市'},
{name: '青海省'},
{name: '中卫市'},
{name: '南昌市'},
{name: '宝鸡市'},
{name: '郑州市'},
{name: '上海市 '},
{name: '陕西省 '},
{name: '榆林市 '},
{name: '滨州市 '},
{name: '淄博市 '},
{name: '珠海市 '},
{name: '天津市 '},
{name: '鄂尔多斯市 '},
{name: '深圳市 '},
{name: '厦门市 '},
{name: '包头市 '},
{name: '运城市 '},

# home_chart_6

In [ ]:
# 4.年度销售数据汇总
sql = "SELECT * FROM datapool.sh_4"
cursor.execute(sql)
total_fp = cursor.fetchall()

# ls_col=['id','kpyf','spbm','xf_province','xf_city','gf_province','gf_city','total_sale']
ls_col=[i[0] for i in cursor.description]
df_fp_4 = pd.DataFrame(total_fp,columns=ls_col)

latest_time=str(sorted([int(x) for x in df_fp_4['update_time'].unique().tolist()])[-1])
df_fp_4_read=df_fp_4.loc[df_fp_4['update_time']==latest_time]

df_fp_4_read=df_fp_4_read.astype(dtype={'je':'float','se':'float'})
df_fp_4_read=df_fp_4_read.sort_values(by='je',ascending=False).reset_index(drop=True)
total_year=df_fp_4_read['je'].sum()
for i in range(len(df_fp_4_read)):
    je_i=df_fp_4_read.loc[i,'je']
    name_i=df_fp_4_read.loc[i,'chain_name']
    print("        {changeVal:"+str(round(je_i/total_year*100,2))+", value: "+str(round(je_i/100000000,2))+", name:'"+name_i+"' },")

# home_chart_7

In [111]:
# 19.	产业链年度市场份额省市分布
sql = "SELECT * FROM datapool.sh_19"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_19 = pd.DataFrame(total_fp,columns=ls_col)

latest_time=str(sorted([int(x) for x in df_fp_4['update_time'].unique().tolist()])[-1])
df_fp_19_read=df_fp_19.loc[df_fp_19['update_time']==latest_time]
df_fp_19_read=df_fp_19_read.astype(dtype={'total_je':'float','total_se':'float'})

ls_chain=df_fp_19_read['chain'].unique().tolist()

for i in ls_chain:
    # print('chain_home_chart_7')
    print("    {")
    print("        name:' "+i+"',")
    sum_i=df_fp_19_read.loc[df_fp_19_read['chain']==i]['total_je'].sum()
    print("        children:[")
    print("            {")
    print("                name: '全国',")
    print("                value: "+str(round(sum_i/10000,2))+",")
    print("                children:[")
    
    df_fp_19_read=df_fp_19_read.astype(dtype={'total_je':'float','total_se':'float'})
    df_pro=df_fp_19_read.loc[df_fp_19_read['chain']==i].groupby(['kp_year','gf_province'])['total_je'].sum().reset_index().sort_values(by='total_je',ascending=False).reset_index(drop=True)
    for a in range(len(df_pro)):
        code_province=df_pro.loc[a,'gf_province']
        ls_gf_city_name_i=df_total_admin.loc[df_total_admin['type_4']==code_province,'admin_name'].tolist()
        if len(ls_gf_city_name_i)==1:
            xf_city_name_i=ls_gf_city_name_i[0]
            print("                {")
            print("                name: '"+xf_city_name_i+"',")
            sum_a=df_pro.loc[a,'total_je']
            print("                value: "+str(round(sum_a/10000,2))+",")
            print("                children: [")
            df_city=df_fp_19_read.loc[(df_fp_19_read['chain']==i)&(df_fp_19_read['gf_province']==code_province)].reset_index()
            for b in range(len(df_city)):
                code_city=df_city.loc[b,'gf_city']
                ls_gf_city_name_b=df_total_admin.loc[df_total_admin['type_4']==code_city,'admin_name'].tolist()
                if len(ls_gf_city_name_b)==1:
                    xf_city_name_b=ls_gf_city_name_b[0]
                    je_b=df_city.loc[b,'total_je']
                    print("                    {name: '"+xf_city_name_b+"',value: "+str(round(je_b/10000,2))+"},")
#                 else:
#                     print('bbbbbb_error',code_city)
            print("                ]")
            print("                },")
#         else:
#             print('aaaaaa_error',code_province)
    print("                ]")
    print("            }")
    print("        ]")
    print("    },")
        

chain_home_chart_7
    {
        name:' 铝镁及兰炭深加工产业链',
        children:[
            {
                name: '全国',
                value: 30497076.28,
                children:[
                {
                name: '陕西省',
                value: 17818227.76,
                children: [
                    {name: '渭南市',value: 408875.0},
                    {name: '宝鸡市',value: 1428.53},
                    {name: '榆林市',value: 14060742.67},
                    {name: '延安市',value: 216978.79},
                    {name: '商洛市',value: 7260.16},
                    {name: '陕西省',value: 651423.15},
                    {name: '铜川市',value: 16187.49},
                    {name: '汉中市',value: 965.28},
                    {name: '咸阳市',value: 38806.65},
                    {name: '西安市',value: 2332437.87},
                    {name: '安康市',value: 2223.15},
                ]
                },
                {
                name: '山西省',
                value: 1763163.54,
                children: [
 

                    {name: '牡丹江市',value: 0.02},
                    {name: '七台河市',value: 310.47},
                    {name: '齐齐哈尔市',value: 415.28},
                    {name: '黑河市',value: 0.18},
                    {name: '鸡西市',value: 1636.5},
                    {name: '佳木斯市',value: 26.54},
                    {name: '哈尔滨市',value: 364.78},
                    {name: '绥化市',value: 703.52},
                    {name: '大庆市',value: 1278.84},
                    {name: '鹤岗市',value: 2174.97},
                    {name: '黑龙江省',value: 0.08},
                    {name: '双鸭山市',value: 394.04},
                ]
                },
                {
                name: '吉林省',
                value: 3648.62,
                children: [
                    {name: '吉林省',value: 0.34},
                    {name: '通化市',value: 1317.62},
                    {name: '吉林市',value: 39.02},
                    {name: '辽源市',value: 130.15},
                    {name: '白山市',value: 1104.58},
                    {

                    {name: '甘南藏族自治州',value: 164.32},
                    {name: '酒泉市',value: 1462.86},
                    {name: '金昌市',value: 1019.08},
                    {name: '天水市',value: 1418.33},
                    {name: '张掖市',value: 1245.34},
                    {name: '白银市',value: 6120.61},
                ]
                },
                {
                name: '云南省',
                value: 54525.98,
                children: [
                    {name: '西双版纳傣族自治州',value: 0.0},
                    {name: '昭通市',value: 16.8},
                    {name: '迪庆藏族自治州',value: 0.0},
                    {name: '普洱市',value: 5981.43},
                    {name: '保山市',value: 127.72},
                    {name: '怒江傈僳族自治州',value: 0.05},
                    {name: '红河哈尼族彝族自治州',value: 10.52},
                    {name: '德宏傣族景颇族自治州',value: 152.69},
                    {name: '丽江市',value: 1010.37},
                    {name: '曲靖市',value: 11515.27},
                    {name: '昆明市',value: 

                    {name: '雅安市',value: 2837.72},
                    {name: '宜宾市',value: 9404.91},
                    {name: '乐山市',value: 9202.29},
                    {name: '四川省',value: 214740.58},
                    {name: '眉山市',value: 3298.37},
                    {name: '巴中市',value: 8.5},
                    {name: '阿坝藏族羌族自治州',value: 3761.74},
                    {name: '内江市',value: 5029.62},
                    {name: '凉山彝族自治州',value: 13669.68},
                    {name: '广元市',value: 1014.04},
                    {name: '甘孜藏族自治州',value: 0.04},
                    {name: '成都市',value: 119386.74},
                    {name: '绵阳市',value: 1287.19},
                    {name: '泸州市',value: 18465.43},
                    {name: '攀枝花市',value: 3567.01},
                    {name: '德阳市',value: 3985.7},
                    {name: '南充市',value: 92.22},
                    {name: '达州市',value: 17822.03},
                    {name: '遂宁市',value: 7368.8},
                    {name: '自贡市',value:

                    {name: '浙江省',value: 0.03},
                    {name: '宁波市',value: 478.67},
                    {name: '杭州市',value: 19.84},
                ]
                },
                {
                name: '山东省',
                value: 401.09,
                children: [
                    {name: '济宁市',value: 27.89},
                    {name: '泰安市',value: 46.71},
                    {name: '威海市',value: 0.02},
                    {name: '潍坊市',value: 0.02},
                    {name: '滨州市',value: 6.22},
                    {name: '烟台市',value: 3.24},
                    {name: '菏泽市',value: 9.41},
                    {name: '青岛市',value: 65.62},
                    {name: '济南市',value: 27.36},
                    {name: '淄博市',value: 54.49},
                    {name: '枣庄市',value: 10.16},
                    {name: '德州市',value: 57.97},
                    {name: '东营市',value: 74.2},
                ]
                },
                {
                name: '黑龙江省',
          

                    {name: '重庆市',value: 107149.07},
                    {name: '重庆市',value: 21204.35},
                ]
                },
                {
                name: '湖南省',
                value: 130184.48,
                children: [
                    {name: '永州市',value: 2832.69},
                    {name: '长沙市',value: 4170.56},
                    {name: '邵阳市',value: 5473.93},
                    {name: '株洲市',value: 1817.42},
                    {name: '湘西土家族苗族自治州',value: 533.12},
                    {name: '郴州市',value: 6971.62},
                    {name: '衡阳市',value: 2919.66},
                    {name: '常德市',value: 1221.46},
                    {name: '岳阳市',value: 6331.6},
                    {name: '湖南省',value: 63754.98},
                    {name: '益阳市',value: 2649.64},
                    {name: '娄底市',value: 14668.08},
                    {name: '湘潭市',value: 16259.97},
                    {name: '张家界市',value: 215.23},
                    {name: '怀化市',value: 364

                    {name: '广元市',value: 10.0},
                    {name: '泸州市',value: 45.35},
                    {name: '成都市',value: 343.59},
                    {name: '德阳市',value: 0.09},
                    {name: '内江市',value: 21.03},
                ]
                },
                {
                name: '辽宁省',
                value: 431.89,
                children: [
                    {name: '大连市',value: 2.25},
                    {name: '沈阳市',value: 91.92},
                    {name: '营口市',value: 137.91},
                    {name: '辽阳市',value: 77.32},
                    {name: '鞍山市',value: 40.2},
                    {name: '朝阳市',value: 82.3},
                ]
                },
                {
                name: '贵州省',
                value: 419.28,
                children: [
                    {name: '铜仁市',value: 419.28},
                ]
                },
                {
                name: '湖北省',
                value: 307.66,
                children:

# home_chart_8

In [204]:
ls_chain=['氢能产业链', '铝镁及兰炭深加工产业链', '能源化工装备制造产业链', '现代煤化工产业链','文化旅游产业链', '轻纺产业链','智能无人系统和航空产业链', '新能源装备制造产业链','“碳中和”产业链', '现代特色农业产业链']

# 18.	产业链五上企业数量
sql = "SELECT * FROM datapool.sh_18"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_18 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_18['update_time'].unique().tolist()])[-1])
df_fp_18_read=df_fp_18.loc[df_fp_18['update_time']==latest_time]

df_fp_18_read_1=df_fp_18_read.loc[df_fp_18_read['city']=='6108',['kp_year','chain','five_num']]

# 17.	产业链五上企业关联企业数量
sql = "SELECT * FROM datapool.sh_17"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_17 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_17['update_time'].unique().tolist()])[-1])
df_fp_17_read=df_fp_17.loc[df_fp_17['update_time']==latest_time]

df_fp_17_read_1=df_fp_17_read.loc[df_fp_17_read['city']=='6108',['kp_year','chain','five_re_num']]

df_home_chart_8=pd.merge(left=df_fp_18_read_1,right=df_fp_17_read_1,on=['kp_year','chain'],how='left')
ls_value_1=[]
ls_value_2=[]
for i in ls_chain:
    re_five_i=df_home_chart_8.loc[df_home_chart_8['chain']==i,'five_re_num'].tolist()[0]
    five_i=df_home_chart_8.loc[df_home_chart_8['chain']==i,'five_num'].tolist()[0]
    ls_value_1.append(float(re_five_i))
    ls_value_2.append(float(five_i))
    
print('    value1:',ls_value_1,',')
print('    value2:',ls_value_2)

    value1: [3.0, 10531.0, 487.0, 37538.0, 199.0, 131.0, 12.0, 27.0, 4178.0, 38552.0]
    value2: [7.0, 635.0, 106.0, 880.0, 51.0, 45.0, 5.0, 99.0, 510.0, 888.0]


# home_chart_9

In [206]:
# 5.	产业链五上企业区县分布
ls_district1=['府谷县','靖边县','定边县','绥德县','米脂县','佳县','吴堡县','清涧县','子洲县','榆阳区','横山区','神木市']
ls_district2=['府谷','靖边','定边','绥德','米脂','佳','吴堡','清涧','子洲','榆阳','横山','神木']

sql = "SELECT * FROM datapool.sh_5"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_5 = pd.DataFrame(total_fp,columns=ls_col)

latest_time=str(sorted([int(x) for x in df_fp_5['update_time'].unique().tolist()])[-1])
df_fp_5_read=df_fp_5.loc[df_fp_5['update_time']==latest_time]
df_fp_5_read=df_fp_5_read.astype(dtype={'numb':'float'})
df_fp_5_read1=pd.merge(left=df_fp_5_read,right=df_total_admin,left_on=['xf_district'],right_on=['type_4'],how='left')
df_fp_5_read1.loc[df_fp_5_read1['admin_name'].isin(['陕西省', '榆林市']),'admin_name']='榆阳区'
df_fp_5_read1.loc[df_fp_5_read1['admin_name'].isnull(),'admin_name']='横山区'
df_fp_5_read1.loc[(~df_fp_5_read1['admin_name'].isin(ls_district1)),'admin_name']='神木市'

ls_chain=df_fp_5_read['chain_name'].unique().tolist()

for i in ls_chain:
    print("    {")
    print("        name:'"+i+"',")
    print("        data:[")
    df_i=df_fp_5_read1.loc[df_fp_5_read1['chain_name']==i].reset_index(drop=True)
    ls_year_num_i=[]
    ls_year_num_i_nor=[]
    
    for a in ls_district1:
        df_a=df_i.loc[df_i['admin_name']==a]
        if len(df_a)==0:
            ls_year_num_i.append(0)
        else:
            value_a=df_i.loc[df_i['admin_name']==a,'numb'].tolist()[0]
            ls_year_num_i.append(value_a)
    for b in range(len(ls_year_num_i)):
        ls_year_num_i_nor=round((ls_year_num_i[b]-min(ls_year_num_i))*100/(max(ls_year_num_i)-min(ls_year_num_i)))
        print('            { "name":"'+ls_district1[b]+'","value":'+str(ls_year_num_i_nor)+',"text": { "v":'+str(ls_year_num_i[b])+'}},')
    print('        ]')
    print('    },')

    {
        name:'“碳中和”产业链',
        data:[
            { "name":"府谷县","value":79,"text": { "v":15.0}},
            { "name":"靖边县","value":16,"text": { "v":3.0}},
            { "name":"定边县","value":11,"text": { "v":2.0}},
            { "name":"绥德县","value":0,"text": { "v":0}},
            { "name":"米脂县","value":5,"text": { "v":1.0}},
            { "name":"佳县","value":0,"text": { "v":0}},
            { "name":"吴堡县","value":5,"text": { "v":1.0}},
            { "name":"清涧县","value":0,"text": { "v":0}},
            { "name":"子洲县","value":5,"text": { "v":1.0}},
            { "name":"榆阳区","value":21,"text": { "v":4.0}},
            { "name":"横山区","value":21,"text": { "v":4.0}},
            { "name":"神木市","value":100,"text": { "v":19.0}},
        ]
    },
    {
        name:'现代特色农业产业链',
        data:[
            { "name":"府谷县","value":9,"text": { "v":3.0}},
            { "name":"靖边县","value":45,"text": { "v":7.0}},
            { "name":"定边县","value":64,"text": { "v":9.0}},
            { "n

In [211]:
ls_chain=['氢能产业链', '铝镁及兰炭深加工产业链', '能源化工装备制造产业链', '现代煤化工产业链','文化旅游产业链', '轻纺产业链','智能无人系统和航空产业链', '新能源装备制造产业链','“碳中和”产业链', '现代特色农业产业链']
df_fp_5=df_fp_5.astype(dtype={'numb':'float'})
df_5_to_4=df_fp_5.groupby(['chain_name'])['numb'].sum().reset_index()
ls_value_3=[]
for i in ls_chain:
    ls_value_3.append(df_5_to_4.loc[df_5_to_4['chain_name']==i,'numb'].tolist()[0])
# print('    value1:',ls_value_1)
print('    value2:',ls_value_3)

    value2: [1.0, 106.0, 27.0, 545.0, 22.0, 19.0, 3.0, 93.0, 67.0, 99.0]


# detail_chart

In [17]:
ls_district1=['府谷县','靖边县','定边县','绥德县','米脂县','佳县','吴堡县','清涧县','子洲县','榆阳区','横山区','神木市']
ls_district2=['府谷','靖边','定边','绥德','米脂','佳','吴堡','清涧','子洲','榆阳','横山','神木']
ls_detail=['“碳中和”产业链','现代特色农业产业链','氢能产业链','能源化工装备制造产业链','轻纺产业链','文化旅游产业链','智能无人系统和航空产业链','新能源装备制造产业链','铝镁及兰炭深加工产业链','现代煤化工产业链']


# detail_chart_1

In [31]:
# 4.	年度销售数据汇总
sql = "SELECT * FROM datapool.sh_4"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_4 = pd.DataFrame(total_fp,columns=ls_col)

latest_time=str(sorted([int(x) for x in df_fp_4['update_time'].unique().tolist()])[-1])
df_fp_4_read=df_fp_4.loc[df_fp_4['update_time']==latest_time]
df_fp_4_read=df_fp_4_read.astype(dtype={'je':'float','se':'float'})
df_fp_4_read_add=df_fp_4_read.loc[:,['chain_name','kp_year','je']]
df_fp_4_read_add=df_fp_4_read_add.rename(columns={'kp_year':'year','chain_name':'chain'})
df_detail1=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\detail_01_产业链销售总额及增速_年度.csv')
df_detail1=pd.concat([df_detail1,df_fp_4_read_add],axis=0)
df_detail1=df_detail1.astype(dtype={'year':'str','je':'float','rate':'float'})

latest_year=str(sorted([int(x) for x in df_detail1['year'].unique().tolist()])[-1])
last_year=str(sorted([int(x) for x in df_detail1['year'].unique().tolist()])[-2])
for i in ls_detail:
    latest_je=df_detail1.loc[(df_detail1['chain']==i)&(df_detail1['year']==latest_year),'je'].tolist()[0]
    last_je=df_detail1.loc[(df_detail1['chain']==i)&(df_detail1['year']==last_year),'je'].tolist()[0]
    print(latest_je,last_je)
    df_detail1.loc[(df_detail1['chain']==i)&(df_detail1['year']==latest_year),'rate']=round((latest_je-last_je)/last_je*100,2)
    
df_detail1.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\detail_01_产业链销售总额及增速_年度.csv',index=False,mode='w')

14573964213.21 22406126484.370003
612251914.49 793421740.89
12537538.1 60053026.07
2055123419.44 4417608999.9
52271002.77 44103883.900000006
11305250.29 46321413.47
6447552.26 33158808.89
2707221712.02 5057361033.91
39255489469.49 43882850628.78
366281704790.15 433848040881.15


# detail_chart_2

In [42]:
df_detail_2=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_年度.csv',dtype='str')
df_detail_2=df_detail_2.astype(dtype={'rate':'float'})

detail_2_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

# 6.	产业链税收季度变化
sql = "SELECT * FROM datapool.sh_6"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_6 = pd.DataFrame(total_fp,columns=ls_col)
df_fp_6=df_fp_6.rename(columns={'chain_name':'chain'})
df_fp_6=df_fp_6.astype(dtype={'se':'float'})
df_fp_6=df_fp_6.loc[:,['kpyf','se','chain']]
df_fp_6.loc[:,'year']=df_fp_6['kpyf'].str[:4]
df_fp_6.loc[:,'je']=0
df_fp_6.loc[:,['kpyf','je','se','chain','year']].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',header=None,index=False,mode='a+')

# 3.	季度产品销售数据汇总
sql = "SELECT * FROM datapool.sh_3"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_3 = pd.DataFrame(total_fp,columns=ls_col)

df_spbm_now=pd.merge(left=df_fp_3,right=df_chain_spbm.loc[:,['spbm','spmc','chain']],left_on=['spbm','chain_name'],right_on=['spbm','chain'],how='left')
df_spbm_now1=df_spbm_now.loc[df_spbm_now['spmc'].notnull()]
df_spbm_now2=df_spbm_now1.loc[:,['spbm','kpyf','je','chain']]
df_spbm_now2.loc[:,'year']=df_spbm_now2['kpyf'].str[:4]
df_spbm_now3=df_spbm_now2.loc[~df_spbm_now2['spbm'].isin(ls_fp_code_deselect_pro)]

df_spbm_quarter=pd.concat([detail_2_quarter,df_spbm_now3],axis=0)
df_spbm_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',index=False)

# df_fp_6=df_fp_6.rename(columns={'chain_name':'chain'})
# df_fp_6=df_fp_6.astype(dtype={'se':'float'})
# df_fp_6=df_fp_6.loc[:,['kpyf','se','chain']]
# df_fp_6.loc[:,'year']=df_fp_6['kpyf'].str[:4]
detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
# detail_1_quarter=pd.concat([detail_1_quarter,df_fp_6],axis=0)
# detail_1_quarter
detail_1_quarter=detail_1_quarter.astype(dtype={'se':'float'})

ls_date=sorted(df_spbm_quarter['kpyf'].unique().tolist())[-6:]
ls_result_detail_2=[]
df_result_detail_2=pd.DataFrame()
for i in ls_detail:
    ls_chain_i=[]
    ls_chain_spbm=[x[:-1] for x in df_chain_spbm.loc[df_chain_spbm['chain']==i,'spbm_t'].unique().tolist()]
    
    detail_2_year_=df_spbm_quarter.loc[(df_spbm_quarter['kpyf'].isin(ls_date))&(df_spbm_quarter['chain']==i)&(~df_spbm_quarter['spbm'].isin(ls_fp_code_deselect_pro))]
    detail_2_year_=detail_2_year_.astype(dtype={'je':'float'})
    detail_2_year_1=detail_2_year_.groupby(['chain','spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    detail_2_year_2=detail_2_year_1.loc[detail_2_year_1['spbm'].isin(ls_chain_spbm)].head(3)
    ls_spbm=detail_2_year_2['spbm'].unique().tolist()
    for j in range(1,len(ls_date)):
        se_i_now=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        ls_i=[i,ls_date[j],'chain_se',se_i_now/10000]
        ls_result_detail_2.append(ls_i)
        ls_chain_i.append(ls_i)
        
        for a in ls_spbm:
            df_pro_now=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            df_pro_last=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j-1])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            if len(df_pro_now)==0:
                pro_a_now=0
            else:
                pro_a_now=df_pro_now['je'].tolist()[0]
                
            if len(df_pro_last)==0:
                pro_a_last=0
            else:
                pro_a_last=df_pro_last['je'].tolist()[0]
                
            if pro_a_last==0 and pro_a_now!=0:
                pro_a_rate=100
            elif pro_a_last==0 and pro_a_now==0:
                pro_a_rate=0
            else:
                pro_a_rate=(pro_a_now-pro_a_last)*100/pro_a_last
            ls_a=[i,ls_date[j],a,pro_a_rate]
            ls_result_detail_2.append(ls_a)
            ls_chain_i.append(ls_a)
    df_chain_i=pd.DataFrame(ls_chain_i,columns=['chain','kpyf','spbm','rate'])
    for c in range(len(df_chain_i)):
        spbm_c=df_chain_i.loc[c,'spbm']
        df_code_name=df_chain_nodes2.loc[df_chain_nodes2['spbm'].str.contains(spbm_c)]
        if len(df_code_name)>0:
            df_code_name1=df_code_name.sample(1)
            code=df_code_name1['CODE'].tolist()[0]
            name=df_code_name1['NAME'].tolist()[0]
            df_chain_i.loc[c,'CODE']=code
            df_chain_i.loc[c,'NAME']=name
    df_result_detail_2=pd.concat([df_result_detail_2,df_chain_i],axis=0)
    df_result_detail_2=df_result_detail_2.drop_duplicates(subset=['chain','kpyf','spbm','rate'],keep='first')
    
df_result_detail_2.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',index=False,mode='w')

# detail_chart_3

In [201]:
# 8.	当年销售金额前十位非经销五上企业
sql = "SELECT * FROM datapool.sh_8"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_8 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_8['update_time'].unique().tolist()])[-1])
df_fp_8_read=df_fp_8.loc[df_fp_8['update_time']==latest_time]

df_fp_8_read=df_fp_8_read.rename(columns={'chain_name':'chain','kp_year':'year'})
df_fp_8_read.loc[:,'xf_dis_code']=df_fp_8_read['xf_shxydm'].str[2:8]

df1mer=pd.merge(left=df_fp_8_read,right=df_total_admin,left_on=['xf_dis_code'],right_on=['type_4'],how='left')
df1mer.loc[df1mer['admin_name'].isnull(),'admin_name']='神木市'
df1mer.loc[(df1mer['admin_name']=='榆林市')|(df1mer['admin_name']=='陕西省'),'admin_name']='榆阳区'
df1mer=df1mer.rename(columns={'admin_name':'xf_district','total_je':'je'})
df1mer=df1mer.astype(dtype={'je':'float'})
df1mer.loc[:,'je_w']=df1mer['je']/10000

df1mer1=df1mer.loc[:,['xf_shxydm','xf_nsrmc','xf_hydm','chain','year','xf_district','je','je_w']]
df1mer1.loc[:,'update_time']=latest_time
df1mer1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_03_季度销售生产企业_年度.csv',header=None,index=False,mode='a+')

# detail_chart_4

In [202]:
# 7.	当年销售金额前十位经销五上企业
sql = "SELECT * FROM datapool.sh_7"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_7 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_7['update_time'].unique().tolist()])[-1])
df_fp_7_read=df_fp_7.loc[df_fp_7['update_time']==latest_time]

df_fp_7_read=df_fp_7_read.rename(columns={'chain_name':'chain','kp_year':'year'})
df_fp_7_read.loc[:,'xf_dis_code']=df_fp_7_read['xf_shxydm'].str[2:8]

df1mer=pd.merge(left=df_fp_7_read,right=df_total_admin,left_on=['xf_dis_code'],right_on=['type_4'],how='left')
df1mer.loc[df1mer['admin_name'].isnull(),'admin_name']='神木市'
df1mer.loc[(df1mer['admin_name']=='榆林市')|(df1mer['admin_name']=='陕西省'),'admin_name']='榆阳区'
df1mer=df1mer.rename(columns={'admin_name':'xf_district','total_je':'je'})
df1mer=df1mer.astype(dtype={'je':'float'})
df1mer.loc[:,'je_w']=df1mer['je']/10000

df1mer1=df1mer.loc[:,['xf_shxydm','xf_nsrmc','xf_hydm','chain','year','xf_district','je','je_w']]
df1mer1.loc[:,'update_time']=latest_time
df1mer1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业_年度.csv',header=None,index=False,mode='a+')

# detail_chart_5

In [91]:
# 9.	产业链非经销五上企业区县分布
sql = "SELECT * FROM datapool.sh_9"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_9 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_9['update_time'].unique().tolist()])[-1])
df_fp_9_read=df_fp_9.loc[df_fp_9['update_time']==latest_time]

df_fp_9_read1=pd.merge(left=df_fp_9_read,right=df_total_admin,left_on=['xf_district'],right_on=['type_4'],how='left')
df_fp_9_read1.loc[(df_fp_9_read1['admin_name'].isnull())|(df_fp_9_read1['admin_name']=='陕西省')|
                  (df_fp_9_read1['admin_name']=='榆林市')|(df_fp_9_read1['admin_name']=='莲湖区'),'admin_name']='榆阳区'
df_fp_9_read1=df_fp_9_read1.rename(columns={'kp_year':'year','chain_name':'chain','admin_name':'district_nor','numb':'xf_shxydm'})
df_fp_9_read2=df_fp_9_read1.loc[:,['year','chain','district_nor','xf_shxydm']]
df_fp_9_read2.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_05_本地产业链非经销企业区县分布_年度.csv',header=None,index=False,mode='a+')

# detail_chart_6和detail_chart_7已有

# detail_chart_8、9、10的销售和购买的基础数据生成

In [106]:
# 14.	产业链产品交易季度金额变化-购买
sql = "SELECT * FROM datapool.sh_14"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_14 = pd.DataFrame(total_fp,columns=ls_col)
df_fp_14=df_fp_14.rename(columns={'kp_year':'kpyf','chain_name':'chain','total_je':'je'})
df_fp_14.loc[:,'xf_city']='无'
df_fp_14.loc[:,'year']=df_fp_14['kpyf'].str[:4]
df_fp_14_=df_fp_14.loc[:,['xf_city','spbm','kpyf','je','chain','year']]
df_fp_14_.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_8910_产业链产品交易情况_季度_购买.csv',header=None,index=False,mode='a+')

# 3.	季度产品销售数据汇总
sql = "SELECT * FROM datapool.sh_3"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_3 = pd.DataFrame(total_fp,columns=ls_col)
df_fp_3=df_fp_3.rename(columns={'chain_name':'chain'})
df_fp_3.loc[:,'gf_city']='无'
df_fp_3.loc[:,'year']=df_fp_3['kpyf'].str[:4]
df_fp_3_=df_fp_3.loc[:,['gf_city','spbm','kpyf','je','chain','year']]
df_fp_3_.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_8910_产业链产品交易情况_季度_销售.csv',header=None,index=False,mode='a+')


# detail_chart_8_销售

In [253]:
# 10.	当年产业链产品交易情况-销售
sql = "SELECT * FROM datapool.sh_10"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_10 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_10['update_time'].unique().tolist()])[-1])
df_fp_10_read=df_fp_10.loc[df_fp_10['update_time']==latest_time]

detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]
date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]

df_spbm_now=pd.merge(left=df_fp_10_read,right=df_chain_spbm.loc[:,['spbm','spmc','chain']],left_on=['spbm','chain_name'],right_on=['spbm','chain'],how='left')
df_spbm_now1=df_spbm_now.loc[df_spbm_now['spmc'].notnull(),['kp_year','chain_name','spbm','total_je','update_time','spmc']].reset_index(drop=True)
df_spbm_now1=df_spbm_now1.rename(columns={'chain_name':'chain','kp_year':'year','total_je':'je','update_time':'kpyf'})
df_spbm_now1=df_spbm_now1.astype(dtype={'je':'float'})

df_result_detail_8=pd.DataFrame()
for i in ls_detail:
    df_i=df_spbm_now1.loc[df_spbm_now1['chain']==i]
    if len(df_i)>0:
        df_i_=df_i.sort_values(by=['je'],ascending=False).head(15)
        df_i_1=df_i_.loc[:,['spbm','chain','kpyf','je','spmc']].reset_index(drop=True)
        for a in range(len(df_i_1)):
            spbm_a=df_i_1.loc[a,'spbm']
            je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_a),'je'].tolist())
            je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_a),'je'].tolist())
            je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_a),'je'].tolist())
            if je_tongbi==0:
                df_i_1.loc[a,'rate_tongbi']=1
            else:
                df_i_1.loc[a,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi

            if je_huanbi==0:
                df_i_1.loc[a,'rate_huanbi']=1
            else:
                df_i_1.loc[a,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
            df_i_1.loc[a,'je_quarter']=je_now

            df_sp_name=df_chain_nodes2.loc[df_chain_nodes2['spbm'].str.contains(spbm_a)]
            if len(df_sp_name)>0:
                df_sp_name1=df_sp_name.sample(1)
                sp_code=df_sp_name1['CODE'].tolist()[0]
                sp_name=df_sp_name1['NAME'].tolist()[0]
                df_i_1.loc[a,'CODE']=sp_code
                df_i_1.loc[a,'NAME']=sp_name
            df_i_1.loc[a,'je_w']=df_i_1.loc[a,'je']/10000
        df_i_2=df_i_1.loc[:,['spbm','chain','kpyf','je','rate_tongbi','rate_huanbi','je_quarter','CODE','NAME','spmc','je_w']]
        df_result_detail_8=pd.concat([df_result_detail_8,df_i_2],axis=0)
df_result_detail_8.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_销售.csv',header=None,index=False,mode='a+')

# detail_chart_8_购买

In [254]:
# 11.	当年产业链产品交易情况-购买
sql = "SELECT * FROM datapool.sh_11"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_11 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_11['update_time'].unique().tolist()])[-1])
df_fp_11_read=df_fp_11.loc[df_fp_11['update_time']==latest_time]

detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]
date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]

df_spbm_now=pd.merge(left=df_fp_11_read,right=df_chain_spbm.loc[:,['spbm','spmc','chain']],left_on=['spbm','chain_name'],right_on=['spbm','chain'],how='left')
df_spbm_now1=df_spbm_now.loc[df_spbm_now['spmc'].notnull(),['kp_year','chain_name','spbm','total_je','update_time','spmc']].reset_index(drop=True)
df_spbm_now1=df_spbm_now1.rename(columns={'chain_name':'chain','kp_year':'year','total_je':'je','update_time':'kpyf'})
df_spbm_now1=df_spbm_now1.astype(dtype={'je':'float'})

df_result_detail_8=pd.DataFrame()
for i in ls_detail:
    df_i=df_spbm_now1.loc[df_spbm_now1['chain']==i]
    if len(df_i)>0:
        df_i_=df_i.sort_values(by=['je'],ascending=False).head(15)
        df_i_1=df_i_.loc[:,['spbm','chain','kpyf','je','spmc']].reset_index(drop=True)
        for a in range(len(df_i_1)):
            spbm_a=df_i_1.loc[a,'spbm']
            je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_a),'je'].tolist())
            je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_a),'je'].tolist())
            je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_a),'je'].tolist())
            if je_tongbi==0:
                df_i_1.loc[a,'rate_tongbi']=1
            else:
                df_i_1.loc[a,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi

            if je_huanbi==0:
                df_i_1.loc[a,'rate_huanbi']=1
            else:
                df_i_1.loc[a,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
            df_i_1.loc[a,'je_quarter']=je_now

            df_sp_name=df_chain_nodes2.loc[df_chain_nodes2['spbm'].str.contains(spbm_a)]
            if len(df_sp_name)>0:
                df_sp_name1=df_sp_name.sample(1)
                sp_code=df_sp_name1['CODE'].tolist()[0]
                sp_name=df_sp_name1['NAME'].tolist()[0]
                df_i_1.loc[a,'CODE']=sp_code
                df_i_1.loc[a,'NAME']=sp_name
            df_i_1.loc[a,'je_w']=df_i_1.loc[a,'je']/10000
        df_i_2=df_i_1.loc[:,['spbm','chain','kpyf','je','rate_tongbi','rate_huanbi','je_quarter','CODE','NAME','spmc','je_w']]
        df_result_detail_8=pd.concat([df_result_detail_8,df_i_2],axis=0)
df_result_detail_8.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_购买.csv',header=None,index=False,mode='a+')

# detail_chart_9_销售

In [272]:
# 12.	当年产业链产品交易目的地城市分布-销售
sql = "SELECT * FROM datapool.sh_12"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_12 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_12['update_time'].unique().tolist()])[-1])
df_fp_12_read=df_fp_12.loc[df_fp_12['update_time']==latest_time]
df_fp_12_read=df_fp_12_read.rename(columns={'kp_year':'year','chain_name':'chain','total_je':'je','update_time':'kpyf'})
df_fp_12_read=df_fp_12_read.astype(dtype={'je':'float'})

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_detail_8_sell=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_销售.csv',dtype='str')
df_detail_9=pd.DataFrame()

for i in ls_detail:
    df_i=df_detail_8_sell.loc[(df_detail_8_sell['chain']==i)&(~df_detail_8_sell['spbm'].isin(ls_fp_code_deselect_pro))&(df_detail_8_sell['kpyf']==latest_time)].head(3)
    ls_spbm_i=df_i['spbm'].tolist()
    df_9_i=df_fp_13_read.loc[(df_fp_13_read['spbm'].isin(ls_spbm_i))&(df_fp_13_read['chain']==i)]
    df_9_i__=pd.merge(left=df_9_i,right=df_i.loc[:,['spbm','CODE','NAME']],on=['spbm'],how='left')
    if len(df_9_i)>0:
        df_9_i_=pd.merge(left=df_9_i__,right=df_total_admin,left_on=['xf_city'],right_on=['type_4'],how='left').reset_index(drop=True)
        df_9_i_1=df_9_i_.loc[df_9_i_['admin_name'].notnull(),['chain','spbm','admin_name','kpyf','year','je','CODE','NAME']]
        df_9_i_1=df_9_i_1.rename(columns={'admin_name':'xf_city'})
        df_9_i_2=df_9_i_1.groupby(['spbm'])['je'].apply(lambda x: x.sort_values(ascending=False).head(5)).reset_index()
        df_9_i_3=df_9_i_2.loc[:,['spbm','je']]
        df_9_i_4=pd.merge(left=df_9_i_3,right=df_9_i_1,on=['spbm','je'],how='left')
        df_9_i_4=df_9_i_4.loc[:,['chain','spbm','xf_city','kpyf','year','je','CODE','NAME']]
        df_9_i_4.loc[:,'spmc']='无'
        df_9_i_4.loc[:,'je_w']=df_9_i_4['je']/10000
    df_detail_9=pd.concat([df_detail_9,df_9_i_4],axis=0)
df_detail_9.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_销售.csv',header=None,index=False,mode='a+')

# detail_chart_9_购买

In [271]:
# 13.	当年产业链产品交易目的地城市分布-购买
sql = "SELECT * FROM datapool.sh_13"
cursor.execute(sql)
total_fp = cursor.fetchall()

ls_col=[i[0] for i in cursor.description]
df_fp_13 = pd.DataFrame(total_fp,columns=ls_col)
latest_time=str(sorted([int(x) for x in df_fp_13['update_time'].unique().tolist()])[-1])
df_fp_13_read=df_fp_13.loc[df_fp_13['update_time']==latest_time]
df_fp_13_read=df_fp_13_read.rename(columns={'kp_year':'year','chain_name':'chain','total_je':'je','update_time':'kpyf'})
df_fp_13_read=df_fp_13_read.astype(dtype={'je':'float'})

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_detail_8_sell=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_购买.csv',dtype='str')
df_detail_9_sold=pd.DataFrame()

for i in ls_detail:
    df_i=df_detail_8_sell.loc[(df_detail_8_sell['chain']==i)&(~df_detail_8_sell['spbm'].isin(ls_fp_code_deselect_pro))&(df_detail_8_sell['kpyf']==latest_time)].head(3)
    ls_spbm_i=df_i['spbm'].tolist()
    df_9_i=df_fp_13_read.loc[(df_fp_13_read['spbm'].isin(ls_spbm_i))&(df_fp_13_read['chain']==i)]
    df_9_i__=pd.merge(left=df_9_i,right=df_i.loc[:,['spbm','CODE','NAME']],on=['spbm'],how='left')
    if len(df_9_i)>0:
        df_9_i_=pd.merge(left=df_9_i__,right=df_total_admin,left_on=['xf_city'],right_on=['type_4'],how='left').reset_index(drop=True)
        df_9_i_1=df_9_i_.loc[df_9_i_['admin_name'].notnull(),['chain','spbm','admin_name','kpyf','year','je','CODE','NAME']]
        df_9_i_1=df_9_i_1.rename(columns={'admin_name':'xf_city'})
        df_9_i_2=df_9_i_1.groupby(['spbm'])['je'].apply(lambda x: x.sort_values(ascending=False).head(5)).reset_index()
        df_9_i_3=df_9_i_2.loc[:,['spbm','je']]
        df_9_i_4=pd.merge(left=df_9_i_3,right=df_9_i_1,on=['spbm','je'],how='left')
        df_9_i_4=df_9_i_4.loc[:,['chain','spbm','xf_city','kpyf','year','je','CODE','NAME']]
        df_9_i_4.loc[:,'spmc']='无'
        df_9_i_4.loc[:,'je_w']=df_9_i_4['je']/10000
    df_detail_9_sold=pd.concat([df_detail_9_sold,df_9_i_4],axis=0)
df_detail_9_sold.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_购买.csv',header=None,index=False,mode='a+')

# detail_chart_10_销售

In [284]:
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]
date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]

df_detail_8_sell=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_销售.csv',dtype='str')
detail_10_info=pd.DataFrame()

for i in ls_detail:
    df_i=df_detail_8_sell.loc[(df_detail_8_sell['chain']==i)&(~df_detail_8_sell['spbm'].isin(ls_fp_code_deselect_pro))&(df_detail_8_sell['kpyf']==latest_time)].head(3)
    ls_spbm_i=df_i['spbm'].tolist()
    for a in ls_spbm_i:
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_13))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(1,len(ls_date_13)):
            je_now=df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je']
            date_now_b=ls_date_13[b]
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=100
            else:
                rate_tongbi=(je_now-je_last_year)*100/je_last_year

            date_last_quarter=ls_date_13[b-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=100
            else:
                rate_huanbi=(je_now-je_last_quarter)*100/je_last_quarter

            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je_last_year']=je_last_year
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_tongbi']=rate_tongbi
        df_a_10_2=pd.merge(left=df_a_10_1,right=df_i.loc[:,['spbm','CODE','NAME']],on=['spbm'],how='left')
        df_a_10_2.loc[:,'spmc']='无'
        df_a_10_2.loc[:,'je_w']=df_a_10_1['je']/10000
        df_a_10_2.loc[:,'je_last_year_w']=df_a_10_1['je_last_year']/10000
        detail_10_info=pd.concat([detail_10_info,df_a_10_2],axis=0)
        detail_10_info=detail_10_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
detail_10_info.loc[detail_10_info['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_销售.csv',index=False,mode='w')

# detail_chart_10_购买

In [285]:
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]
date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]

df_detail_8_sell=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_购买.csv',dtype='str')
detail_10_info=pd.DataFrame()

for i in ls_detail:
    df_i=df_detail_8_sell.loc[(df_detail_8_sell['chain']==i)&(~df_detail_8_sell['spbm'].isin(ls_fp_code_deselect_pro))&(df_detail_8_sell['kpyf']==latest_time)].head(3)
    ls_spbm_i=df_i['spbm'].tolist()
    for a in ls_spbm_i:
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_13))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(1,len(ls_date_13)):
            je_now=df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je']
            date_now_b=ls_date_13[b]
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=100
            else:
                rate_tongbi=(je_now-je_last_year)*100/je_last_year

            date_last_quarter=ls_date_13[b-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=100
            else:
                rate_huanbi=(je_now-je_last_quarter)*100/je_last_quarter

            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je_last_year']=je_last_year
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_tongbi']=rate_tongbi
        df_a_10_2=pd.merge(left=df_a_10_1,right=df_i.loc[:,['spbm','CODE','NAME']],on=['spbm'],how='left')
        df_a_10_2.loc[:,'spmc']='无'
        df_a_10_2.loc[:,'je_w']=df_a_10_1['je']/10000
        df_a_10_2.loc[:,'je_last_year_w']=df_a_10_1['je_last_year']/10000
        detail_10_info=pd.concat([detail_10_info,df_a_10_2],axis=0)
        detail_10_info=detail_10_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
detail_10_info.loc[detail_10_info['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_购买.csv',index=False,mode='w')

# 生成sql语句

In [17]:
insert into ADS_YL_CYDN_CYLFP
--现代特色农业产业链
SELECT kp_year,
       xf_shxydm,
       gf_shxydm,
       spbm,
       je,
       se,
       uuid,
       '1' step,
       '现代特色农业产业链' chain_
  from ADS_YL_CYDN_FPJCSJ
 WHERE gf_shxydm LIKE '9%'
   AND xf_shxydm LIKE '9%'
   AND xf_shxydm IN (SELECT ent_shxydm
                       from ads_yl_cydn_cylnsrxx
                      WHERE chain_name = '现代特色农业产业链');
--碳中和”产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1' step,'“碳中和”产业链'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='“碳中和”产业链');



NameError: name 'a' is not defined

In [7]:
df_ent=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\core_ent_sql.csv',dtype='str')
ls_chain=df_ent['chain_name'].unique().tolist()
for i in ls_chain:
    # print(i)
    a=i
    str_df=f"""--{a}
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'{a}'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='{a}');"""
    print(str_df)

--现代特色农业产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'现代特色农业产业链'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='现代特色农业产业链');
--“碳中和”产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'“碳中和”产业链'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='“碳中和”产业链');
--现代煤化工产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'现代煤化工产业链'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='现代煤化工产业链');
--智能无人系统和航空产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'智能无人系统和航空产业链'  cha

In [2]:
str_df="""--现代煤化工产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'现代煤化工产业链'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='现代煤化工产业链');"""

In [4]:
print(str_df)

--现代煤化工产业链
insert into ADS_YL_CYDN_CYLFP
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,uuid,'1'  step,'现代煤化工产业链'  chain_
from ADS_YL_CYDN_FPJCSJ
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm IN (SELECT ent_shxydm from ads_yl_cydn_cylnsrxx WHERE chain_name='现代煤化工产业链');


In [14]:
现代特色农业产业链
3
13 9193
4
“碳中和”产业链
15
10 26501
16
现代煤化工产业链
7
1 334655
8
智能无人系统和航空产业链
4
1 174
5
文化旅游产业链
7
37 25131
8
轻纺产业链
1
33 313
2
20 333
3
能源化工装备制造产业链
12
1 21745
13
新能源装备制造产业链
1
42 298
2
1 299
3
铝镁及兰炭深加工产业链
9
1 63817
10
氢能产业链
1
3 4
2
7 11
3

Hello, world!


In [8]:
ls_chain

['现代特色农业产业链',
 '“碳中和”产业链',
 '现代煤化工产业链',
 '智能无人系统和航空产业链',
 '文化旅游产业链',
 '轻纺产业链',
 '能源化工装备制造产业链',
 '新能源装备制造产业链',
 '铝镁及兰炭深加工产业链',
 '氢能产业链']

In [12]:
ls_chain_step=[4,16,8,5,8,3,13,3,10,3]
for i in range(len(ls_chain)):
    chain_name=ls_chain[i]
    chain_step=ls_chain_step[i]
    str_1=f'--{chain_name}'
    print(str_1)
    for h in range(chain_step):
        a=str(h+2)
        b=str(h+1)
        
        
        str_df1=f"""--{chain_name}
insert into ADS_YL_CYDN_CYLFP
SELECT home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,
       home_7.se,home_7.uuid,'{a}' step,'{chain_name}'  chain_
from(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.uuid 
FROM ADS_YL_CYDN_FPJCSJ t1
LEFT JOIN (select * from ADS_YL_CYDN_CYLFP where chain_='{chain_name}') t2 ON t1.uuid = t2.uuid
WHERE t2.uuid IS NULL
)  home_7 
inner join (
select kp_year ,gf_shxydm ,spbm, sum(je),sum(se)  
from ADS_YL_CYDN_CYLFP
where 
step='{b}'
and chain_='{chain_name}'
group by kp_year,gf_shxydm ,spbm 
) t21 ON home_7.xf_shxydm=t21.gf_shxydm
and home_7.spbm=t21.spbm
;"""
        print(str_df1)
        

--现代特色农业产业链
--现代特色农业产业链
insert into ADS_YL_CYDN_CYLFP
SELECT home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,
       home_7.se,home_7.uuid,'2' step,'现代特色农业产业链'  chain_
from(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.uuid 
FROM ADS_YL_CYDN_FPJCSJ t1
LEFT JOIN (select * from ADS_YL_CYDN_CYLFP where chain_='现代特色农业产业链') t2 ON t1.uuid = t2.uuid
WHERE t2.uuid IS NULL
)  home_7 
inner join (
select kp_year ,gf_shxydm ,spbm, sum(je),sum(se)  
from ADS_YL_CYDN_CYLFP
where 
step='1'
and chain_='现代特色农业产业链'
group by kp_year,gf_shxydm ,spbm 
) t21 ON home_7.xf_shxydm=t21.gf_shxydm
and home_7.spbm=t21.spbm
;
--现代特色农业产业链
insert into ADS_YL_CYDN_CYLFP
SELECT home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,
       home_7.se,home_7.uuid,'3' step,'现代特色农业产业链'  chain_
from(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.uuid 
FROM ADS_YL_CYDN_FPJCSJ t1
LEFT JOIN (select * from ADS_YL_CYDN_CYLFP where chain_='现代特色农业产业

In [37]:
ls_chain_step=[4,16,8,5,8,3,13,3,10,3]
for i in range(2):
    a=str(i+2)
    b=str(i+1)
    sql=f"""insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_)
SELECT 
home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,home_7.se,home_7.id,{a} as step,'氢能产业链' as chain_
from
(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.id as id
FROM basedata_total.t_chain_2023_fp_base t1
LEFT JOIN (select * from basedata_total.t_chain_2023_fp_2 where chain_='氢能产业链') t2 ON t1.id = t2.id
WHERE t2.id IS NULL
) as home_7 
inner join 
(
select 
kp_year ,gf_shxydm ,spbm, sum(je),sum(se)  
from
basedata_total.t_chain_2023_fp_2
where 
step={b}
group by
gf_shxydm ,spbm 
)
as t21
ON 
home_7.xf_shxydm=t21.gf_shxydm
and home_7.spbm=t21.spbm
;"""
    print('# 循环取数')
    print(sql)
    print('')

# 循环取数
insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_)
SELECT 
home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,home_7.se,home_7.id,2 as step,'氢能产业链' as chain_
from
(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.id as id
FROM basedata_total.t_chain_2023_fp_base t1
LEFT JOIN basedata_total.t_chain_2023_fp_2 t2 ON t1.id = t2.id
WHERE t2.id IS NULL
) as home_7 
inner join 
(
select 
kp_year ,gf_shxydm ,spbm, sum(je),sum(se)  
from
basedata_total.t_chain_2023_fp_2
where 
step=1
group by
gf_shxydm ,spbm 
)
as t21
ON 
home_7.xf_shxydm=t21.gf_shxydm
and home_7.spbm=t21.spbm
;

# 循环取数
insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_)
SELECT 
home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,home_7.se,home_7.id,3 as step,'氢能产业链' as chain_
from
(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.id as id
FROM based

In [29]:
df_ent=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\core_ent_sql.csv',dtype='str')
ls_chain=df_ent['chain_name'].unique().tolist()
for a in ls_chain:
    
    sql1=f"""insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_) 
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,id,1 as step,'{a}' as chain_
from
basedata_total.t_chain_2023_fp_base
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm 
IN (SELECT ent_shxydm from basedata_total.t_chain_core_ent_sql WHERE chain_name='{a}');"""
    print(sql1)
    print('')

insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_) 
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,id,1 as step,'现代特色农业产业链' as chain_
from
basedata_total.t_chain_2023_fp_base
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm 
IN (SELECT ent_shxydm from basedata_total.t_chain_core_ent_sql WHERE chain_name='现代特色农业产业链');

insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_) 
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,id,1 as step,'“碳中和”产业链' as chain_
from
basedata_total.t_chain_2023_fp_base
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm 
IN (SELECT ent_shxydm from basedata_total.t_chain_core_ent_sql WHERE chain_name='“碳中和”产业链');

insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_) 
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,id,1 as step,'现代煤化工产业链' as chain_
from
basedata_total.t_chain_2023_fp_base
WHERE 

In [26]:
print(sql1)

insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_) 
SELECT 
kp_year, xf_shxydm,gf_shxydm,spbm,je,se,id,1 as step,'现代煤化工产业链' as chain_
from
basedata_total.t_chain_2023_fp_base
WHERE gf_shxydm LIKE '9%' 
AND xf_shxydm LIKE '9%'
AND xf_shxydm 
IN (SELECT ent_shxydm from basedata_total.t_chain_core_ent_sql WHERE chain_name='现代煤化工产业链');


In [8]:
print(sql)

insert into basedata_total.t_chain_2023_fp_2(kp_year,xf_shxydm,gf_shxydm,spbm,je,se,id,step,chain_)
SELECT 
home_7.kp_year,home_7.xf_shxydm,home_7.gf_shxydm,home_7.spbm,home_7.je,home_7.se,home_7.id,5 as step,'“碳中和”产业链' as chain_
from
(
SELECT t1.kp_year,t1.xf_shxydm,t1.gf_shxydm,t1.spbm,t1.je,t1.se,t1.id as id
FROM basedata_total.t_chain_2023_fp_base t1
LEFT JOIN basedata_total.t_chain_2023_fp_2 t2 ON t1.id = t2.id
WHERE t2.id IS NULL
) as home_7 
inner join 
(
select 
kp_year ,gf_shxydm ,spbm, sum(je),sum(se)  
from
basedata_total.t_chain_2023_fp_2
where 
step=4
group by
gf_shxydm ,spbm 
)
as t21
ON 
home_7.xf_shxydm=t21.gf_shxydm
and home_7.spbm=t21.spbm
;


# 税务数据测试

In [2]:
df_ent=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\core_ent_sql.csv',dtype='str')

df_02=pd.read_csv(r'E:\project_项目\04榆林产业大脑\税务数据对接\t_data_sh_new_qyjfkpsj_2023_5_202408151019.csv',dtype='str')
df_03=pd.read_csv(r'E:\project_项目\04榆林产业大脑\税务数据对接\t_data_sh_new_qyjfkpsj_2023_5_202408151017.csv',dtype='str')
df=pd.concat([df_02,df_03],axis=0)
df=df.reset_index()
df=df.astype(dtype={'je':'float','se':'float'})

In [4]:
len(set(df['xf_shxydm'].unique().tolist()+df['gf_shxydm'].unique().tolist()))

398605

In [5]:
df

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se
0,0,916108935622301822,榆林人民大厦有限责任公司,H6190,91100000100016448C,中国进出口银行,NaN,3070402000000000000,202302,3535.85,212.15
1,1,91610826MA70D0YT4K,绥德德迩优汽修服务有限公司,F5263,9160824677904634M,靖边县宝顺运输有限公司,NaN,1090310019900000000,202302,41.59,5.41
2,2,91610822MABNDGW14R,府谷县紫瑞尚品国际餐饮服务有限公司,H6190,9160000783675775T,陕西国龙奥星环境修复有限公司,NaN,3070402000000000000,202302,792.08,7.92
3,3,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,9610822MA708MR72L,府谷县中远辰治金有限公司,NaN,1070101010100000000,202302,331.86,43.14
4,4,91610826MA70D0YT4K,绥德德迩优汽修服务有限公司,F5263,9160824677904634M,靖边县宝顺运输有限公司,NaN,1090310019900000000,202302,284.07,36.93
...,...,...,...,...,...,...,...,...,...,...,...
3792748,1893648,92652801MA780PUB6E,库尔勒阳光华泽酒店,NaN,91610802MA7G47BU0R,陕西德泰民丰农业有限公司,A0514,3070402000000000000,202303,156.44,1.56
3792749,1893649,92650109MA7A55TR5L,乌鲁木齐市米东区博世启洋电动工具专卖,NaN,9161082167791359XQ,神木市金环工程技术有限公司,M7484,1090132030000000000,202303,445.54,4.46
3792750,1893650,916108216911259528,神木市志瑞加油站,F5265,91810821MA70950B2G,神木昊森煤业有限公司,NaN,1070101010100000000,202303,663.72,86.28
3792751,1893651,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,91810132552334746A,中宏汇成本建设工程有限公司,NaN,1070101010100000000,202303,265.48,34.52


In [39]:
# df_1=pd.merge(left=df,right=df_ent,left_on=['xf_shxydm'],right_on=['ent_shxydm'],how='inner')
# df_2=df_1.loc[df_1['je']>0]

df_11=pd.merge(left=df,right=df_ent,left_on=['xf_shxydm'],right_on=['ent_shxydm'],how='left')
df_12=df_11.loc[df_11['chain_name']!='现代煤化工产业链']
df_13=df_11.loc[df_11['chain_name']=='现代煤化工产业链']

In [38]:
df_add1=pd.merge(left=df_12,right=df_13.loc[:,['gf_shxydm','spbm']],left_on=['xf_shxydm','spbm'],right_on=['gf_shxydm','spbm'],how='inner')
df_add1_=df_add1.loc[:,['index','xf_shxydm','xf_nsrmc','xf_hydm','gf_shxydm_x','gf_nsrmc','gf_hydm','spbm','kpyf','je','se','chain_name','ent_shxydm','ent_name']]
df_add1=df_add1_.rename(columns={'gf_shxydm_x':'gf_shxydm'})
df_add1

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,195,91610802730401623T,榆林市榆阳区牛圈壕利明加油站,F5265,9160133MA6W02Q75W,陕西新汉驰汽车贸易有限公司,NaN,1070101010100000000,202302,336.28,43.72,NaN,NaN,NaN
1,195,91610802730401623T,榆林市榆阳区牛圈壕利明加油站,F5265,9160133MA6W02Q75W,陕西新汉驰汽车贸易有限公司,NaN,1070101010100000000,202302,336.28,43.72,NaN,NaN,NaN
2,195,91610802730401623T,榆林市榆阳区牛圈壕利明加油站,F5265,9160133MA6W02Q75W,陕西新汉驰汽车贸易有限公司,NaN,1070101010100000000,202302,336.28,43.72,NaN,NaN,NaN
3,195,91610802730401623T,榆林市榆阳区牛圈壕利明加油站,F5265,9160133MA6W02Q75W,陕西新汉驰汽车贸易有限公司,NaN,1070101010100000000,202302,336.28,43.72,NaN,NaN,NaN
4,195,91610802730401623T,榆林市榆阳区牛圈壕利明加油站,F5265,9160133MA6W02Q75W,陕西新汉驰汽车贸易有限公司,NaN,1070101010100000000,202302,336.28,43.72,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741386,1876264,91640205MA774W2G9G,石嘴山市恒韦气体销售有限公司,NaN,91610893MA70A71Y8U,陕西丰硕泓昌能源化工有限公司,F5281,1070203060000000000,202303,29955.22,3894.18,NaN,NaN,NaN
741387,1876264,91640205MA774W2G9G,石嘴山市恒韦气体销售有限公司,NaN,91610893MA70A71Y8U,陕西丰硕泓昌能源化工有限公司,F5281,1070203060000000000,202303,29955.22,3894.18,NaN,NaN,NaN
741388,1876264,91640205MA774W2G9G,石嘴山市恒韦气体销售有限公司,NaN,91610893MA70A71Y8U,陕西丰硕泓昌能源化工有限公司,F5281,1070203060000000000,202303,29955.22,3894.18,NaN,NaN,NaN
741389,1876264,91640205MA774W2G9G,石嘴山市恒韦气体销售有限公司,NaN,91610893MA70A71Y8U,陕西丰硕泓昌能源化工有限公司,F5281,1070203060000000000,202303,29955.22,3894.18,NaN,NaN,NaN


In [29]:
df_14=pd.concat([df_13,df_add1],axis=0)
df_14

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
3,3,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,9610822MA708MR72L,府谷县中远辰治金有限公司,NaN,1070101010100000000,202302,331.86,43.14,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
6,6,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,9160821MA70BD6A54,神木市中泓泰商贸有限公司,NaN,1070101010100000000,202302,3316.85,431.19,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
8,8,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,91016131MA6TX48R7P,陕西昊涵予建筑工程有限公司,NaN,1070101010100000000,202302,471.68,61.32,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
10,10,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,914001010000623689,山西东方林龙装饰材料有限公司,NaN,1070101010100000000,202302,220.35,28.65,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
11,11,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,991610802MA70CJ7R2B,陕西大唐风范教育科技有限公司输林分公司,NaN,1070101010100000000,202302,88.50,11.50,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294090,1885579,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3242888.20,421575.47,NaN,NaN,NaN
1294091,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294092,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294093,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN


In [37]:
df_12=df_11.loc[~df_11['index'].isin(df_14['index'].tolist())]
df_13=df_14

In [52]:
df_base=df.loc[df['je']>0].groupby(['xf_shxydm','gf_shxydm','spbm'])['je'].sum().reset_index()
df_base=df_base.reset_index()

In [53]:
df_base

,index,xf_shxydm,gf_shxydm,spbm,je
0,0,91100000400000712N,91610821305529183C,3040101000000000000,141509.43
1,1,91110000051421390A,91610802050400635F,3040602990000000000,188679.24
2,2,91110000051421390A,91610823305726101A,3040602990000000000,100000.00
3,3,91110000051421390A,91610824590270069T,3040602990000000000,100000.00
4,4,91110000051421390A,916108250817158759,3040602990000000000,100000.00
...,...,...,...,...,...
1746564,1746564,93640522MA75X2AW5B,91610831MA709N6L13,1010118000000000000,15000.00
1746565,1746565,93652302MA776K302Y,9161082222403383X0,3070401000000000000,1297.03
1746566,1746566,93652824MA78FWFU7D,91610802MA7FQG5U6K,3070401000000000000,700.00
1746567,1746567,93653201MA77KC120A,91610825667999078A,1010115030300000000,13484.40


In [54]:
df_11=pd.merge(left=df_base,right=df_ent,left_on=['xf_shxydm'],right_on=['ent_shxydm'],how='left')
df_12=df_11.loc[df_11['chain_name']!='现代煤化工产业链']
df_13=df_11.loc[df_11['chain_name']=='现代煤化工产业链']

In [58]:
df_13.loc[:,['step']]=1
df_13

,index,xf_shxydm,gf_shxydm,spbm,je,chain_name,ent_shxydm,ent_name,step
283705,283705,916100000504386259,91130501MABM6LG685,1020102000000000000,5.592268e+06,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283706,283706,916100000504386259,91610821MA703CB846,1020102000000000000,1.213386e+06,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283707,283707,916100000504386259,916108220786281646,1020102000000000000,2.335615e+06,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283708,283708,916100000504386259,91610822555655537N,1020102000000000000,5.661996e+05,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283709,283709,916100000504386259,916108225593843242,1020102000000000000,5.304706e+06,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
...,...,...,...,...,...,...,...,...,...
1289902,1289902,91610893MA70AULF26,91620000719024628N,1070213019900000000,1.809829e+09,现代煤化工产业链,91610893MA70AULF26,中石油兰州石化榆林化工有限公司,1
1289903,1289903,91610893MA70AULF26,91620102551256790P,1100101020100000000,1.075200e+02,现代煤化工产业链,91610893MA70AULF26,中石油兰州石化榆林化工有限公司,1
1289904,1289904,91610893MA70AULF26,91620104332139009E,1070101050100000000,8.274308e+05,现代煤化工产业链,91610893MA70AULF26,中石油兰州石化榆林化工有限公司,1
1289905,1289905,91610893MA70AULF26,91620104MA74ETHC5G,1070101050100000000,1.089834e+06,现代煤化工产业链,91610893MA70AULF26,中石油兰州石化榆林化工有限公司,1


In [63]:
df_add1=pd.merge(left=df_12,right=df_13.loc[:,['gf_shxydm','spbm']],left_on=['xf_shxydm','spbm'],right_on=['gf_shxydm','spbm'],how='inner')
df_add1_=df_add1.loc[:,['index','xf_shxydm','gf_shxydm_x','spbm','je','chain_name','ent_shxydm','ent_name']]
df_add1=df_add1_.rename(columns={'gf_shxydm_x':'gf_shxydm'})
df_add1.loc[:,['step']]=3
df_add1

,index,xf_shxydm,gf_shxydm,spbm,je,chain_name,ent_shxydm,ent_name,step
0,1265,91110000710925526U,91610000305753978J,1070101030100000000,1202611.89,NaN,NaN,NaN,3
1,1265,91110000710925526U,91610000305753978J,1070101030100000000,1202611.89,NaN,NaN,NaN,3
2,1266,91110000710925526U,916100003057540069,1070101030100000000,1470458.16,NaN,NaN,NaN,3
3,1266,91110000710925526U,916100003057540069,1070101030100000000,1470458.16,NaN,NaN,NaN,3
4,1267,91110000710925526U,91610800671529239R,1070101030100000000,505500.17,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...
131895,1655477,92610824MA7091001D,91610825MA70CD2899,1070101030100000000,261130.98,NaN,NaN,NaN,3
131896,1655477,92610824MA7091001D,91610825MA70CD2899,1070101030100000000,261130.98,NaN,NaN,NaN,3
131897,1655480,92610824MA7091001D,93610824MA703AJR59,1070101030100000000,4961.06,NaN,NaN,NaN,3
131898,1655480,92610824MA7091001D,93610824MA703AJR59,1070101030100000000,4961.06,NaN,NaN,NaN,3


In [61]:
df_14=pd.concat([df_13,df_add1],axis=0)
df_14

,index,xf_shxydm,gf_shxydm,spbm,je,chain_name,ent_shxydm,ent_name,step
283705,283705,916100000504386259,91130501MABM6LG685,1020102000000000000,5592267.61,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283706,283706,916100000504386259,91610821MA703CB846,1020102000000000000,1213386.37,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283707,283707,916100000504386259,916108220786281646,1020102000000000000,2335615.09,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283708,283708,916100000504386259,91610822555655537N,1020102000000000000,566199.64,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
283709,283709,916100000504386259,916108225593843242,1020102000000000000,5304706.21,现代煤化工产业链,916100000504386259,府谷县金宏湾矿业有限公司,1
...,...,...,...,...,...,...,...,...,...
266824,1654539,92610824MA708ARX2M,91610824L04761483X,1070101030100000000,217051.51,NaN,NaN,NaN,2
266825,1654540,92610824MA708ARX2M,91610824MA70CAR12T,1070101030100000000,35398.23,NaN,NaN,NaN,2
266826,1654542,92610824MA708ARX2M,91610825MA70ATFC7F,1070101030100000000,1643.36,NaN,NaN,NaN,2
266827,1654538,92610824MA708ARX2M,91610824691114663P,1070101010100000000,86601.78,NaN,NaN,NaN,2


In [62]:
df_12=df_11.loc[~(df_11['index'].isin(df_14['index'].tolist()))]
df_13=df_14

In [68]:
i='现代煤化工产业链'
df_base=df.loc[df['je']>0].groupby(['xf_shxydm','gf_shxydm','spbm'])['je'].sum().reset_index()
df_base=df_base.reset_index()
print(i)

step=1
flag=1

df_11=pd.merge(left=df_base,right=df_ent,left_on=['xf_shxydm'],right_on=['ent_shxydm'],how='left')
df_12=df_11.loc[df_11['chain_name']!=i]
df_13=df_11.loc[df_11['chain_name']==i]

df_13.loc[:,['step']]=step

while flag==1:
    print(step)
    df_13_=df_13.groupby(['gf_shxydm','spbm','step'])['je'].sum().reset_index()
    df_add1=pd.merge(left=df_12,right=df_13_.loc[df_13_['step']==step,['gf_shxydm','spbm']],left_on=['xf_shxydm','spbm'],right_on=['gf_shxydm','spbm'],how='inner')
    if len(df_add1)==0:
        flag=0
    else:
        df_add1_=df_add1.loc[:,['index','xf_shxydm','gf_shxydm_x','spbm','je','chain_name','ent_shxydm','ent_name']]
        df_add1=df_add1_.rename(columns={'gf_shxydm_x':'gf_shxydm'})

        step+=1
        df_add1.loc[:,['step']]=step
        df_14=pd.concat([df_13,df_add1],axis=0)
        print(len(df_add1),len(df_14))

        df_12=df_11.loc[~(df_11['index'].isin(df_14['index'].tolist()))]
        df_13=df_14

现代煤化工产业链
1
164727 282513
2
45250 327763
3
5364 333127
4
1395 334522
5
119 334641
6
13 334654
7
1 334655
8


In [69]:
df_13.to_csv(r'E:\project_项目\04榆林产业大脑\税务数据对接\现代煤化工1.csv',index=False)

In [70]:
# 循环整合
ls_chain=df_ent['chain_name'].unique().tolist()
df_total_chain_fp=pd.DataFrame()
for i in ls_chain:
    df_base=df.loc[df['je']>0].groupby(['xf_shxydm','gf_shxydm','spbm'])['je'].sum().reset_index()
    df_base=df_base.reset_index()
    print(i)
    
    step=1
    flag=1
    
    df_11=pd.merge(left=df_base,right=df_ent,left_on=['xf_shxydm'],right_on=['ent_shxydm'],how='left')
    df_12=df_11.loc[df_11['chain_name']!=i]
    df_13=df_11.loc[df_11['chain_name']==i]
    
    df_13.loc[:,['step']]=step
    
    while flag==1:
        print(step)
        df_13_=df_13.groupby(['gf_shxydm','spbm','step'])['je'].sum().reset_index()
        df_add1=pd.merge(left=df_12,right=df_13_.loc[df_13_['step']==step,['gf_shxydm','spbm']],left_on=['xf_shxydm','spbm'],right_on=['gf_shxydm','spbm'],how='inner')
        if len(df_add1)==0:
            flag=0
        else:
            df_add1_=df_add1.loc[:,['index','xf_shxydm','gf_shxydm_x','spbm','je','chain_name','ent_shxydm','ent_name']]
            df_add1=df_add1_.rename(columns={'gf_shxydm_x':'gf_shxydm'})

            step+=1
            df_add1.loc[:,['step']]=step
            df_14=pd.concat([df_13,df_add1],axis=0)
            print(len(df_add1),len(df_14))

            df_12=df_11.loc[~(df_11['index'].isin(df_14['index'].tolist()))]
            df_13=df_14
    df_13=df_13.reset_index()
    df_13.loc[:,['chain_name']]=i
    df_total_chain_fp=pd.concat([df_13,df_total_chain_fp],axis=0)

现代特色农业产业链
1
1450 8980
2
200 9180
3
13 9193
4
“碳中和”产业链
1
2408 4787
2
3772 8559
3
6098 14657
4
6100 20757
5
2778 23535
6
1284 24819
7
717 25536
8
271 25807
9
356 26163
10
116 26279
11
76 26355
12
58 26413
13
32 26445
14
46 26491
15
10 26501
16
现代煤化工产业链
1
164727 282513
2
45250 327763
3
5364 333127
4
1395 334522
5
119 334641
6
13 334654
7
1 334655
8
智能无人系统和航空产业链
1
15 160
2
12 172
3
1 173
4
1 174
5
文化旅游产业链
1
3270 4014
2
7694 11708
3
2691 14399
4
5477 19876
5
2734 22610
6
2484 25094
7
37 25131
8
轻纺产业链
1
33 313
2
20 333
3
能源化工装备制造产业链
1
513 2118
2
484 2602
3
2100 4702
4
4306 9008
5
6767 15775
6
3996 19771
7
1230 21001
8
505 21506
9
143 21649
10
87 21736
11
8 21744
12
1 21745
13
新能源装备制造产业链
1
42 298
2
1 299
3
铝镁及兰炭深加工产业链
1
5119 11683
2
8062 19745
3
15575 35320
4
13907 49227
5
6590 55817
6
6471 62288
7
1512 63800
8
16 63816
9
1 63817
10
氢能产业链
1
3 4
2
7 11
3


In [41]:
df_13.groupby(['xf_shxydm','gf_shxydm','spbm'])['je'].sum().reset_index()

,xf_shxydm,gf_shxydm,spbm,je
0,916100000504386259,91130501MABM6LG685,1020102000000000000,5.592268e+06
1,916100000504386259,91610821MA703CB846,1020102000000000000,1.213386e+06
2,916100000504386259,916108220786281646,1020102000000000000,2.335615e+06
3,916100000504386259,91610822555655537N,1020102000000000000,5.661996e+05
4,916100000504386259,916108225593843242,1020102000000000000,5.304706e+06
...,...,...,...,...
118741,91610893MA70AULF26,91620000719024628N,1070213019900000000,1.809829e+09
118742,91610893MA70AULF26,91620102551256790P,1100101020100000000,1.075200e+02
118743,91610893MA70AULF26,91620104332139009E,1070101050100000000,8.274308e+05
118744,91610893MA70AULF26,91620104MA74ETHC5G,1070101050100000000,1.089834e+06


In [4]:
df_add11=pd.read_csv(r'E:\project_项目\04榆林产业大脑\税务数据对接\_SELECT_home_7_xf_shxydm_home_7_xf_nsrmc_home_7_xf_hydm_home_7_g_202408191332.csv',dtype='str')

In [5]:
df_add11.loc[:,'sql']='yes'
df_add111=df_add11.loc[:,['xf_shxydm','gf_shxydm','spbm','je','sql']]
df_add1=df_add1.astype(dtype={'je':'str','se':'str'})
df_merge_1=pd.merge(left=df_add1,right=df_add111,on=['xf_shxydm','gf_shxydm','spbm','je'],how='left')

NameError: name 'df_add1' is not defined

In [ ]:
df_merge_1.loc[df_merge_1['sql']!='yes']

In [69]:
df_add1

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,17,91610821794104572T,神木市开元加油站,F5265,9160821MA70BQ7PX1,陕西卓纵点贸易有限公司,NaN,1070101010100000000,202302,328.32,42.68,NaN,NaN,NaN
1,102995,91610821794104572T,神木市开元加油站,F5265,9113040010552648X6,中国煤炭地质总局一二九勘探队,NaN,1070101010100000000,202302,884.96,115.04,NaN,NaN,NaN
2,333107,91610821794104572T,神木市开元加油站,F5265,91310230564784445R,上海基强建设工程有限公司,NaN,1070101010100000000,202302,300.00,39.00,NaN,NaN,NaN
3,427278,91610821794104572T,神木市开元加油站,F5265,91340202MA8N4QC956,芜湖云中驿物流有限公司,NaN,1070101010100000000,202302,1641.59,213.41,NaN,NaN,NaN
4,582663,91610821794104572T,神木市开元加油站,F5265,91430100395980276X,三一能源装备有限公司,NaN,1070101010100000000,202302,200.00,26.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294090,1885579,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3242888.20,421575.47,NaN,NaN,NaN
1294091,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294092,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294093,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN


In [6]:
df_12

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,0,916108935622301822,榆林人民大厦有限责任公司,H6190,91100000100016448C,中国进出口银行,NaN,3070402000000000000,202302,3535.85,212.15,NaN,NaN,NaN
1,1,91610826MA70D0YT4K,绥德德迩优汽修服务有限公司,F5263,9160824677904634M,靖边县宝顺运输有限公司,NaN,1090310019900000000,202302,41.59,5.41,NaN,NaN,NaN
2,2,91610822MABNDGW14R,府谷县紫瑞尚品国际餐饮服务有限公司,H6190,9160000783675775T,陕西国龙奥星环境修复有限公司,NaN,3070402000000000000,202302,792.08,7.92,NaN,NaN,NaN
4,4,91610826MA70D0YT4K,绥德德迩优汽修服务有限公司,F5263,9160824677904634M,靖边县宝顺运输有限公司,NaN,1090310019900000000,202302,284.07,36.93,NaN,NaN,NaN
5,5,91610802MA70FXDT12,陕西北升宏泰建设工程有限公司,E4891,9116227971899890,准格尔旗美日煤炭有限责任公司,NaN,1080105010100000000,202302,0.00,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792747,1893647,91610821MA708P1Q0P,神木市义丰厚机电有限公司,F5281,91650000228860955P,国家能源集团新疆能源有限责任公司,NaN,1090201990000000000,202303,8495.58,1104.42,NaN,NaN,NaN
3792748,1893648,92652801MA780PUB6E,库尔勒阳光华泽酒店,NaN,91610802MA7G47BU0R,陕西德泰民丰农业有限公司,A0514,3070402000000000000,202303,156.44,1.56,NaN,NaN,NaN
3792749,1893649,92650109MA7A55TR5L,乌鲁木齐市米东区博世启洋电动工具专卖,NaN,9161082167791359XQ,神木市金环工程技术有限公司,M7484,1090132030000000000,202303,445.54,4.46,NaN,NaN,NaN
3792750,1893650,916108216911259528,神木市志瑞加油站,F5265,91810821MA70950B2G,神木昊森煤业有限公司,NaN,1070101010100000000,202303,663.72,86.28,NaN,NaN,NaN


In [7]:
df_13

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
3,3,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,9610822MA708MR72L,府谷县中远辰治金有限公司,NaN,1070101010100000000,202302,331.86,43.14,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
6,6,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,9160821MA70BD6A54,神木市中泓泰商贸有限公司,NaN,1070101010100000000,202302,3316.85,431.19,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
8,8,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,91016131MA6TX48R7P,陕西昊涵予建筑工程有限公司,NaN,1070101010100000000,202302,471.68,61.32,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
10,10,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,914001010000623689,山西东方林龙装饰材料有限公司,NaN,1070101010100000000,202302,220.35,28.65,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
11,11,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,991610802MA70CJ7R2B,陕西大唐风范教育科技有限公司输林分公司,NaN,1070101010100000000,202302,88.50,11.50,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792588,1893488,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91650000625555217H,特变电工新疆新能源股份有限公司,NaN,1070101010100000000,202303,813.27,105.73,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
3792648,1893548,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,916523236934288930,呼图壁县晋新煤化工有限公司,NaN,1070101010100000000,202303,1015.04,131.96,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
3792689,1893589,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,91652900MA7831ML8X,新疆吉尔特电器有限公司,NaN,1070101010100000000,202303,1246.02,161.98,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
3792732,1893632,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,91650106MA796Y9U6T,疆隆超盛涛商贸有限公司,NaN,1070101010100000000,202303,35.41,4.60,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司


In [14]:
df_group=pd.read_csv(r'E:\project_项目\04榆林产业大脑\税务数据对接\_select_gf_shxydm_spbm_sum_je_from_basedata_total_t_chain_2023_f_202408191430.csv',dtype='str')
df_group

,gf_shxydm,spbm,sum(je)
0,9610822MA708MR72L,1070101010100000000,1021.24
1,9160821MA70BD6A54,1070101010100000000,3699.69
2,91016131MA6TX48R7P,1070101010100000000,471.68
3,914001010000623689,1070101010100000000,220.35
4,991610802MA70CJ7R2B,1070101010100000000,1428.32
...,...,...,...
95576,91640100694303443J,1070101010100000000,176.99
95577,91640100MA75WRP528,1070101010100000000,256.64
95578,91640221750809917D,1070101010100000000,219.47
95579,91640100MA76MRH82M,1070101010100000000,225.66


In [23]:
df_group.loc[df_group['gf_shxydm']=='91610822MA7Mu77u27']

,gf_shxydm,spbm,sum(je)


In [17]:
df_merge_group=pd.merge(left=df_gf_sp,right=df_group,on=['gf_shxydm','spbm'],how='outer')
df_merge_group.loc[df_merge_group['sum(je)'].isnull()]

,gf_shxydm,spbm,je,sum(je)
77838,91610822MA7Mu77u27,1020102000000000000,-766576.99,NaN


In [22]:
df_gf_sp
df_gf_sp.loc[df_gf_sp['gf_shxydm']=='91610822MA7Mu77u27']

,gf_shxydm,spbm,je
77838,91610822MA7Mu77u27,1020102000000000000,-766576.99


In [18]:
df_13.loc[df_13['je']<0]

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
15,15,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,97730185MA0D8JU138,石家庄立富机械设备租赁有限公司,NaN,1070101010100000000,202302,-8.85,-1.15,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
19,19,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91670822583528754G,府谷县昊田煤电冶化有限公司,NaN,1070101010100000000,202302,-13.27,-1.73,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
91,91,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,9260802MA7068055B,榆林市榆阳区海兵装饰材料销售门市,NaN,1070101010100000000,202302,-1.77,-0.23,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司
118,118,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91670821MA70AFDX8F,神木市富鼎能源有限公司,NaN,1070101010100000000,202302,-62.55,-8.13,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
262,262,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91670822583528754G,府谷县昊田煤电冶化有限公司,NaN,1070101010100000000,202302,-26.55,-3.45,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787948,1888848,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91650106MA795GJK3F,新疆吴创商贸有限公司,NaN,1070101010100000000,202303,-29.09,-3.78,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
3789567,1890467,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91650104MA7N1PX398,新疆丰琳育能源服务有限公司,NaN,1070101010100000000,202303,-8.85,-1.15,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
3790639,1891539,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91650105766849630Q,新疆成汇工程管理有限公司,NaN,1070101010100000000,202303,-138.61,-18.02,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
3791706,1892606,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91650000697817263C,昌吉市开源煤矿,NaN,1070101010100000000,202303,-8.85,-1.15,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司


In [13]:
df_gf_sp

,gf_shxydm,spbm,je
0,90050623MA0Q7DBJX8,1070101010100000000,1491.22
1,9010132MA6UQ9K25N,1070101030100000000,269.91
2,901120116732827038J,1070101030100000000,694.69
3,901309846741628581,1070101010100000000,407.08
4,90361062MA6YK2WL95,1070101010100000000,247.79
...,...,...,...
95577,991610802MA70CJ7R2B,1070101010100000000,1428.32
95578,991610822MA70DH365R,1070101010100000000,333.63
95579,991610824MA7G9RKUXM,1070101030100000000,442.48
95580,991610825305725424C,1070101010100000000,353.98


In [30]:
# pd.set_option('mode.use_inf_as_null', True)
df_gf_sp=df_13.groupby(['gf_shxydm','spbm'])['je'].sum().reset_index()
df_gf_sp_1=df_gf_sp.loc[df_gf_sp['gf_shxydm']!='91610822MA7Mu77u27']

In [31]:
# 276410 362951
df_gf_sp=df_13.groupby(['gf_shxydm','spbm'])['je'].sum().reset_index()
df_py_merge=pd.merge(left=df_12,right=df_gf_sp_1.loc[:,['gf_shxydm','spbm']],on=['gf_shxydm','spbm'],how='inner')

In [32]:
df_py_merge

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,33,91610802730419962E,榆林市榆阳区杏焉村加油站,F5265,9160104MABP57G837,西安映企品牌管理有限公司,NaN,1070101010100000000,202302,369.03,47.97,NaN,NaN,NaN
1,52,91610893305600461G,榆林万达石油燃气有限公司,F5162,9160104MABP57G837,西安映企品牌管理有限公司,NaN,1070101010100000000,202302,588.50,76.50,NaN,NaN,NaN
2,426,91610893305600461G,榆林万达石油燃气有限公司,F5162,9160104MABP57G837,西安映企品牌管理有限公司,NaN,1070101010100000000,202302,730.09,94.91,NaN,NaN,NaN
3,649,91610893305600461G,榆林万达石油燃气有限公司,F5162,9160104MABP57G837,西安映企品牌管理有限公司,NaN,1070101010100000000,202303,309.73,40.27,NaN,NaN,NaN
4,34,91610822MA70DYKT8G,陕西中油华韬能源有限公司府谷杨家畔加油站,F5265,9610822MA708MR72L,府谷县中远辰治金有限公司,NaN,1070101010100000000,202302,331.86,43.14,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362946,1891211,91610821570664722Q,神木市榆神高速锦界服务区南区中东加油站,F5265,91650109MA776HRW98,乌鲁木齐荣泰盛祥电子科技有限公司,NaN,1070101010100000000,202303,345.13,44.87,NaN,NaN,NaN
362947,1891462,9161082158352469X1,神木市保泰加油站,F5265,91650000697817263C,昌吉市开源煤矿,NaN,1070101010100000000,202303,442.48,57.52,NaN,NaN,NaN
362948,1892749,91652325MA79HY6872,新疆银丰恒耀商贸有限公司,NaN,91610823MA70G5X00N,榆林浩海融昌煤炭运销有限公司,F5161,1020101000000000000,202303,169061.95,21978.05,NaN,NaN,NaN
362949,1893242,916108007869853146,榆林泰裕油气有限公司开发区航空加油站,F5265,91652201MA775W6A00,哈密市盛镁镁业有限公司,NaN,1070101010100000000,202303,267.26,34.74,NaN,NaN,NaN


In [33]:
df_sql_merge=pd.read_csv(r'E:\project_项目\04榆林产业大脑\税务数据对接\_SELECT_home_7_xf_shxydm_home_7_xf_nsrmc_home_7_xf_hydm_home_7_g_202408191644.csv',dtype='str')
df_sql_merge

,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,id
0,91140931MA0MR1UM80,保德县晋洲煤炭有限公司,NaN,91610822MA7BK9K43D,府谷县忠正煤炭运销有限公司,F5161,1020101000000000000,202302,850115.11,110514.96,148625
1,91140931MA0MR1UM80,保德县晋洲煤炭有限公司,NaN,916108223056392271,榆林承泽能源化工有限公司,F5161,1020101000000000000,202303,644949.24,83843.4,2156484
2,91150291552838725H,国能供应链内蒙古有限公司,NaN,91610821770043971N,中国神华能源股份有限公司神东煤炭分公司,B0610,1090201080000000000,202302,8482.41,1102.71,41348
3,91150291552838725H,国能供应链内蒙古有限公司,NaN,91610821770043971N,中国神华能源股份有限公司神东煤炭分公司,B0610,1090201080000000000,202302,4638.33,602.99,64322
4,91150291552838725H,国能供应链内蒙古有限公司,NaN,91610821770043971N,中国神华能源股份有限公司神东煤炭分公司,B0610,1090201080000000000,202302,137486.72,17873.28,77164
...,...,...,...,...,...,...,...,...,...,...,...
276405,916210007190521432,中国石油天然气股份有限公司甘肃庆阳销售分公司,NaN,916108005671245495,榆林市塞上运输有限公司,G5421,1070101010100000000,202303,547.79,71.21,3777523
276406,91620900719043845H,中国石油天然气股份有限公司甘肃酒泉销售分公司,NaN,91610821MA70ADQJ0B,神木市锐源精煤科技有限公司,F5181,1070101010100000000,202303,1971.67,256.33,3780045
276407,916210007190521432,中国石油天然气股份有限公司甘肃庆阳销售分公司,NaN,91610825MA7035QG6M,陕西佰瑞达新能源科技发展有限公司,C2519,1070101010100000000,202303,353.98,46.02,3780244
276408,916210007190521432,中国石油天然气股份有限公司甘肃庆阳销售分公司,NaN,91610824MA70E5AY1U,陕西拓源油气开采有限公司,D4511,1070101010100000000,202303,340.71,44.29,3780830


In [35]:
df_sql_merge_1=df_sql_merge.loc[:,['xf_shxydm','gf_shxydm','spbm','je']]

df_py_merge_1=df_py_merge.loc[:,['xf_shxydm','gf_shxydm','spbm','je']]
df_py_merge_1=df_py_merge_1.astype(dtype={'je':'str'})
df_py_merge_1.loc[:,'merge']='py'
df_py_sql_merge_1=pd.merge(left=df_sql_merge_1,right=df_py_merge_1,on=['xf_shxydm','gf_shxydm','spbm','je'],how='outer')

In [38]:
df_py_sql_merge_1.loc[df_py_sql_merge_1['merge'].notnull()]

,xf_shxydm,gf_shxydm,spbm,je,merge
0,91140931MA0MR1UM80,91610822MA7BK9K43D,1020101000000000000,850115.11,py
1,91140931MA0MR1UM80,916108223056392271,1020101000000000000,644949.24,py
2,91140931MA0MR1UM80,916108223056392271,1020101000000000000,644949.24,py
3,91140931MA0MR1UM80,916108223056392271,1020101000000000000,644949.24,py
4,91140931MA0MR1UM80,916108223056392271,1020101000000000000,644949.24,py
...,...,...,...,...,...
473648,916108257521442858,91652801792286560G,1070101010100000000,308.85,py
473649,916108257521442858,91652801792286560G,1070101010100000000,308.85,py
473650,91610825MA70DQ7146,916502006792893487,1070101010100000000,176.99,py
473651,91610825MA70DQ7146,916502006792893487,1070101010100000000,176.99,py


In [37]:
df_py_sql_merge_1.loc[df_py_sql_merge_1['merge'].isnull()]

,xf_shxydm,gf_shxydm,spbm,je,merge
8,91110106MA005RPX7Y,91610800694936013W,1090201080000000000,54.07,NaN
21,91110106MA005RPX7Y,91610800694936013W,1090201080000000000,216.64,NaN
22,91110106MA005RPX7Y,91610800694936013W,1090201080000000000,30347.89,NaN
31,91120118MA06DW1E1K,91610822MA70F2DW10,1070204020000000000,229018.35,NaN
32,91120118MA06DW1E1K,91610822MA70F2DW10,1070204020000000000,229018.35,NaN
...,...,...,...,...,...
312473,91640323750838208R,91610826MA6TH1GD86,1070101010100000000,807091.15,NaN
312475,916108215706647063,91620981MAC5P4115C,1070101010100000000,204.42,NaN
312477,916108215706647063,91620105MA71P9NLX1,1070101010100000000,389.38,NaN
312482,916210007190521432,91610825MA70BPFT0A,1070101010100000000,32709.69,NaN


In [64]:
df_add1=pd.merge(left=df_12,right=df_13.loc[:,['gf_shxydm','spbm']],left_on=['xf_shxydm','spbm'],right_on=['gf_shxydm','spbm'],how='inner')
df_add1_=df_add1.loc[:,['index','xf_shxydm','xf_nsrmc','xf_hydm','gf_shxydm_x','gf_nsrmc','gf_hydm','spbm','kpyf','je','se','chain_name','ent_shxydm','ent_name']]
df_add1=df_add1_.rename(columns={'gf_shxydm_x':'gf_shxydm'})
df_add1

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,17,91610821794104572T,神木市开元加油站,F5265,9160821MA70BQ7PX1,陕西卓纵点贸易有限公司,NaN,1070101010100000000,202302,328.32,42.68,NaN,NaN,NaN
1,102995,91610821794104572T,神木市开元加油站,F5265,9113040010552648X6,中国煤炭地质总局一二九勘探队,NaN,1070101010100000000,202302,884.96,115.04,NaN,NaN,NaN
2,333107,91610821794104572T,神木市开元加油站,F5265,91310230564784445R,上海基强建设工程有限公司,NaN,1070101010100000000,202302,300.00,39.00,NaN,NaN,NaN
3,427278,91610821794104572T,神木市开元加油站,F5265,91340202MA8N4QC956,芜湖云中驿物流有限公司,NaN,1070101010100000000,202302,1641.59,213.41,NaN,NaN,NaN
4,582663,91610821794104572T,神木市开元加油站,F5265,91430100395980276X,三一能源装备有限公司,NaN,1070101010100000000,202302,200.00,26.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294090,1885579,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3242888.20,421575.47,NaN,NaN,NaN
1294091,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294092,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294093,1890806,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN


In [66]:
df_13.loc[(df_13['gf_shxydm']=='91610821794104572T')&(df_13['spbm']=='1070101010100000000')]

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
707699,707699,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,91610821794104572T,神木市开元加油站,F5265,1070101010100000000,202302,176.99,23.01,现代煤化工产业链,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司


In [67]:
df_12.loc[(df_12['xf_shxydm']=='91610821794104572T')&(df_12['spbm']=='1070101010100000000')]

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
17,17,91610821794104572T,神木市开元加油站,F5265,9160821MA70BQ7PX1,陕西卓纵点贸易有限公司,NaN,1070101010100000000,202302,328.32,42.68,NaN,NaN,NaN
102995,102995,91610821794104572T,神木市开元加油站,F5265,9113040010552648X6,中国煤炭地质总局一二九勘探队,NaN,1070101010100000000,202302,884.96,115.04,NaN,NaN,NaN
333107,333107,91610821794104572T,神木市开元加油站,F5265,91310230564784445R,上海基强建设工程有限公司,NaN,1070101010100000000,202302,300.00,39.00,NaN,NaN,NaN
427278,427278,91610821794104572T,神木市开元加油站,F5265,91340202MA8N4QC956,芜湖云中驿物流有限公司,NaN,1070101010100000000,202302,1641.59,213.41,NaN,NaN,NaN
582663,582663,91610821794104572T,神木市开元加油站,F5265,91430100395980276X,三一能源装备有限公司,NaN,1070101010100000000,202302,200.00,26.00,NaN,NaN,NaN
1047383,1047383,91610821794104572T,神木市开元加油站,F5265,91610821661184337L,陕西星光市政工程有限公司,E4813,1070101010100000000,202302,234.51,30.49,NaN,NaN,NaN
1211532,1211532,91610821794104572T,神木市开元加油站,F5265,91610132MAB0XRC23C,陕西秦都泓万项目管理有限公司,M7482,1070101010100000000,202302,404.42,52.58,NaN,NaN,NaN
1216337,1216337,91610821794104572T,神木市开元加油站,F5265,91610802MA70CH8F5Q,陕西方源恒泰建筑工程有限公司,E4790,1070101010100000000,202302,1309.74,170.26,NaN,NaN,NaN
1218967,1218967,91610821794104572T,神木市开元加油站,F5265,916108210569281665,陕西麟腾建设工程有限公司,E5011,1070101010100000000,202302,370.80,48.20,NaN,NaN,NaN
1364153,1364153,91610821794104572T,神木市开元加油站,F5265,91610821552197956U,神木市海源建设工程有限公司,E4910,1070101010100000000,202302,915.04,118.96,NaN,NaN,NaN


In [35]:
df_11.loc[~df_11['index'].isin(df_11['index'].tolist())]

,index,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name


In [34]:
df_12['chain_name'].unique().tolist()

[]

In [16]:
df_12.columns.tolist()

['xf_shxydm',
 'xf_nsrmc',
 'xf_hydm',
 'gf_shxydm',
 'gf_nsrmc',
 'gf_hydm',
 'spbm',
 'kpyf',
 'je',
 'se',
 'chain_name',
 'ent_shxydm',
 'ent_name']

In [17]:
df_add1=pd.merge(left=df_12,right=df_chain_1.loc[:,['gf_shxydm','spbm']],left_on=['xf_shxydm','spbm'],right_on=['gf_shxydm','spbm'],how='inner')
df_add1_=df_add1.loc[:,['xf_shxydm','xf_nsrmc','xf_hydm','gf_shxydm_x','gf_nsrmc','gf_hydm','spbm','kpyf','je','se','chain_name','ent_shxydm','ent_name']]
df_add1=df_add1_.rename(columns={'gf_shxydm_x':'gf_shxydm'})
df_add1

,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,91610821794104572T,神木市开元加油站,F5265,9160821MA70BQ7PX1,陕西卓纵点贸易有限公司,NaN,1070101010100000000,202302,328.32,42.68,NaN,NaN,NaN
1,91610821794104572T,神木市开元加油站,F5265,9113040010552648X6,中国煤炭地质总局一二九勘探队,NaN,1070101010100000000,202302,884.96,115.04,NaN,NaN,NaN
2,91610821794104572T,神木市开元加油站,F5265,91310230564784445R,上海基强建设工程有限公司,NaN,1070101010100000000,202302,300.00,39.00,NaN,NaN,NaN
3,91610821794104572T,神木市开元加油站,F5265,91340202MA8N4QC956,芜湖云中驿物流有限公司,NaN,1070101010100000000,202302,1641.59,213.41,NaN,NaN,NaN
4,91610821794104572T,神木市开元加油站,F5265,91430100395980276X,三一能源装备有限公司,NaN,1070101010100000000,202302,200.00,26.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294090,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3242888.20,421575.47,NaN,NaN,NaN
1294091,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294092,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN
1294093,91650100397997581W,新疆新投能源开发有限责任公司,NaN,91610800MA7052096Q,陕西鸿立隆实业有限公司,B0610,1020101000000000000,202303,3915446.91,509008.09,NaN,NaN,NaN


In [13]:
df_chain_12=pd.concat([])

,xf_shxydm,xf_nsrmc,xf_hydm,gf_shxydm,gf_nsrmc,gf_hydm,spbm,kpyf,je,se,chain_name,ent_shxydm,ent_name
0,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,9610822MA708MR72L,府谷县中远辰治金有限公司,NaN,1070101010100000000,202302,331.86,43.14,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
1,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,97730185MA0D8JU138,石家庄立富机械设备租赁有限公司,NaN,1070101010100000000,202302,-8.85,-1.15,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
2,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91670822583528754G,府谷县昊田煤电冶化有限公司,NaN,1070101010100000000,202302,-13.27,-1.73,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
3,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,91911105MA718KER3W,陕西大钧勤业建筑工程有限公司,NaN,1070101010100000000,202302,176.99,23.01,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
4,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,916909285778268946,旬阳县金辉矿业有限公司,NaN,1070101010100000000,202302,412.39,53.61,现代煤化工产业链,91610800552165727D,中国石化销售有限公司陕西榆林石油分公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
434805,91610822MA70B4TQ8T,陕西能源冯家塔矿业运营有限责任公司,L7219,91610000583547998F,陕西投资集团有限公司,J6999,3040101000000000000,202303,0.00,0.00,现代煤化工产业链,91610822MA70B4TQ8T,陕西能源冯家塔矿业运营有限责任公司
434806,916108000880800750,榆林市腾宇矿建工程有限公司,E5012,91610800694936013W,国家能源集团陕西神延煤炭有限责任公司,B0610,3050100000000000000,202303,166138.61,1661.39,现代煤化工产业链,916108000880800750,榆林市腾宇矿建工程有限公司
434807,916108000880800750,榆林市腾宇矿建工程有限公司,E5012,91610800797941851R,陕西文信建筑工程有限公司,E5090,3050100000000000000,202303,0.00,0.00,现代煤化工产业链,916108000880800750,榆林市腾宇矿建工程有限公司
434811,91610000661170904Y,陕西嘉泰矿山工程有限公司,E4840,91610000691106065G,府谷县弘建煤矿有限公司,B0610,3050100000000000000,202303,1251009.17,112590.83,现代煤化工产业链,91610000661170904Y,陕西嘉泰矿山工程有限公司


In [16]:
df_2.groupby(['chain_name','spbm'])['je'].sum().reset_index()

,chain_name,spbm,je
0,“碳中和”产业链,1020101000000000000,5.165095e+09
1,“碳中和”产业链,1020102000000000000,1.191880e+08
2,“碳中和”产业链,1020199000000000000,1.586672e+08
3,“碳中和”产业链,1040111020000000000,1.075221e+04
4,“碳中和”产业链,1040199990000000000,6.717000e+01
...,...,...,...
956,铝镁及兰炭深加工产业链,3040101000000000000,7.169811e+05
957,铝镁及兰炭深加工产业链,3040502019900000000,6.003982e+05
958,铝镁及兰炭深加工产业链,3049900000000000000,3.821662e+06
959,铝镁及兰炭深加工产业链,3050100000000000000,1.400000e+04


In [15]:
16673388422.039913-1.667339e+10

-1577.9600868225098

In [13]:
df_2.groupby(['chain_name'])['je'].sum().reset_index()

,chain_name,je
0,“碳中和”产业链,1.667339e+10
1,文化旅游产业链,2.687930e+07
2,新能源装备制造产业链,3.331354e+09
3,智能无人系统和航空产业链,8.576344e+06
4,氢能产业链,4.084653e+07
5,现代煤化工产业链,3.633059e+11
6,现代特色农业产业链,6.330744e+08
7,能源化工装备制造产业链,2.842794e+09
8,轻纺产业链,3.475162e+07
9,铝镁及兰炭深加工产业链,3.848797e+10


# 信用平台数据对接

In [3]:
df=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\core_ent_sql.xlsx',dtype='str')
df.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\core_ent_sql.csv',index=False)

In [4]:
df=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\core_ent_sql.csv',dtype='str')
df

,chain_name,ent_shxydm,ent_name
0,现代特色农业产业链,9161089357352467X6,榆林市四海食品配送有限责任公司
1,“碳中和”产业链,91610800694925584L,榆林市西北化工有限公司
2,现代煤化工产业链,91610802727360605W,榆林市牛家梁煤炭集运有限责任公司
3,智能无人系统和航空产业链,91610800H13100629H,西部机场集团榆林机场有限公司
4,现代特色农业产业链,916108226679507926,榆林九州通医药有限公司
...,...,...,...
1214,现代煤化工产业链,916100006984015773,神木县香水河矿业有限公司
1215,现代煤化工产业链,916100007769889684,陕西能源凉水井矿业有限责任公司
1216,现代煤化工产业链,91610821MA708FCP5T,神木市大洗选煤企业管理有限公司
1217,现代煤化工产业链,91610821305529183C,陕西能源煤炭铁路运销有限公司


In [8]:
df_1=pd.read_csv(r'E:\project_项目\04榆林产业大脑\信用平台数据对接\SELECT_FROM_DATAPOOL_T_CHAIN_COMPANY_LIMIT_20_ORDER_BY_RAND__202407221111.csv',dtype='str')
df_2=pd.read_csv(r'E:\project_项目\04榆林产业大脑\信用平台数据对接\SELECT_FROM_CHAINAPP_T_CHAIN_GRAPH_WHERE_DESCRIPTION_重点产业链__202407221053.csv',dtype='str')

In [26]:
df_1.head(1)
ls=[]
for i in range(len(df_1)):
    ls_i=df_1.loc[i,'PRODUCT_CODE']
    # print(type(ls_i))
    if type(ls_i)!=float:
        ls_i_=ls_i.split(';')
        ls.extend(ls_i_)
ls1=list(set(ls))
print(len(ls1))
df_chain_nodes1.loc[df_chain_nodes1['ID'].isin(ls1)].to_csv(r'E:\project_项目\04榆林产业大脑\信用平台数据对接\产品数据字典.csv',index=False)

69


In [64]:
# 产业链对应的税务产品列表
df_chain_nodes1=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\T_CODE_PRODUCT_20231026.xlsx',dtype='str',sheet_name='T_CODE_PRODUCT_20231026')
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

df_chain_core_ent=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype='str',sheet_name='Sheet1')
df_chain_core_ent_c=df_chain_core_ent.loc[df_chain_core_ent['chain'].notnull()]
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()

df_swcpdm=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\t_data_sh_new_cpdm.xlsx',dtype='str')
df_swcpdm_1=df_swcpdm.loc[:,['spbm','spmc','spbmjc','hzx']]

df_2024=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\税务数据2024待融合.xlsx',dtype='str')

df_result=pd.DataFrame()
df_result1=pd.DataFrame()
for i in ls_chain:
    print(i)
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    ls_spbm=df_chain_spbm['spbm'].tolist()
    ls_spbm_1=[x for ls1 in ls_spbm for x in ls1.split(';')]
    ls_spbm_2=list(set(ls_spbm_1))
    df_swcpdm_i=df_swcpdm_1.loc[df_swcpdm_1['spbm'].isin(ls_spbm_2)]
    df_swcpdm_i_1=df_swcpdm_i.drop_duplicates(subset=['spbm'])
    df_swcpdm_i_1.loc[:,'chain']=i
    df_result=pd.concat([df_result,df_swcpdm_i_1],axis=0)
    print(len(ls_spbm_2))
    print(len(df_swcpdm_i_1))
    
    df_2024_i=df_2024.loc[df_2024['chain'].str.contains(i)]
    df_2024_i.loc[:,'chain']=i
    df_result1=pd.concat([df_result1,df_2024_i],axis=0)
    # print(ls_spbm)
    
df_re_1=pd.concat([df_result,df_result1],axis=0)
df_re_2=df_re_1.drop_duplicates(subset=['spbm','chain'])
df_re_2.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_swcpbm.csv',index=False)

现代特色农业产业链
58
58
“碳中和”产业链
109
109
现代煤化工产业链
77
74
智能无人系统和航空产业链
43
42
文化旅游产业链
22
22
轻纺产业链
31
31
能源化工装备制造产业链
38
37
新能源装备制造产业链
108
107
铝镁及兰炭深加工产业链
80
79
氢能产业链
41
41


C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_8368\1119136908.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swcpdm_i_1.loc[:,'chain']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_8368\1119136908.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swcpdm_i_1.loc[:,'chain']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_8368\1119136908.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

# 税务数据读取

In [69]:
df_202202=pd.DataFrame()
ls_02=os.listdir('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent_fp')
for i in ls_02:
    if i.startswith('202202'):
        print(i)
        df_i=pd.read_csv(r'E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent_fp\\'+i)
        df_202202=pd.concat([df_202202,df_i],axis=0)
df_202202=df_202202.drop_duplicates(subset=['id']).reset_index(drop=True)
df_202202.to_csv(r'E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\202202.csv',index=False,mode='w')

202202.csv
2022021.csv
2022022.csv


In [7]:
# 发票数据都是按季度的增量更新
# pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\202002.csv',encoding='gbk',dtype='str')
ls_file=os.listdir('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent')
for i in ls_file:
    if i.endswith('.csv'):
        print(i)
        df_fp=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent\\'+i,dtype='str')
        print(df_fp.head(5))
# df_fp=df_fp.astype(dtype={'je':'float','se':'float'})

202301tanzhonghegf.csv
            gf_shxydm     gf_nsrmc gf_hydm                 spbm     gf_sjswjg   
0  1110000005477532XY     榆林市邮政管理局   S9226  1080124020000000000  国家税务总局陕西省税务局  \
1  1110000005477532XY     榆林市邮政管理局   S9226  1090409040000000000  国家税务总局陕西省税务局   
2  1110000005477532XY     榆林市邮政管理局   S9226  1090413050000000000  国家税务总局陕西省税务局   
3  1110000005477532XY     榆林市邮政管理局   S9226  3040301990000000000  国家税务总局陕西省税务局   
4  11100000MB0175329K  中华人民共和国榆林海关   S9226  1070203060000000000  国家税务总局陕西省税务局   

      gf_dsswjg                 gf_qxswjg    kpyf  sum(je)  
0  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理二股  202303        0  
1  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理二股  202303   148.52  
2  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理二股  202303    59.41  
3  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理二股  202303  18029.7  
4  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理一股  202302  1061.95  
202301tanzhonghexf.csv
            xf_shxydm                    xf_nsrmc xf_hydm   
0  12100000436676829L       黄河水利委员会晋陕蒙接壤

            gf_shxydm                    gf_nsrmc gf_hydm   
0  12100000436676829L       黄河水利委员会晋陕蒙接壤地区水土保持监督局   N7620  \
1  12100000436676829L       黄河水利委员会晋陕蒙接壤地区水土保持监督局   N7620   
2  12100000436676829L       黄河水利委员会晋陕蒙接壤地区水土保持监督局   N7620   
3  12100000755206947M  黄河水土保持绥德治理监督局（绥德水土保持科学试验站）   M7461   
4  12100000755206947M  黄河水土保持绥德治理监督局（绥德水土保持科学试验站）   M7461   

                  spbm     gf_sjswjg     gf_dsswjg               gf_qxswjg   
0  3040301990000000000  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市榆阳区税务局青山路税务分局  \
1  3070402000000000000  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市榆阳区税务局青山路税务分局   
2  3070402000000000000  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市榆阳区税务局青山路税务分局   
3  3040105000000000000  国家税务总局陕西省税务局  国家税务总局榆林市税务局      国家税务总局绥德县税务局第二税务分局   
4  3040105000000000000  国家税务总局陕西省税务局  国家税务总局榆林市税务局      国家税务总局绥德县税务局第二税务分局   

     kpyf             sum(je)  
0  202302              638.61  
1  202302   47072.01000000001  
2  202303   52860.28999999999  
3  202302           522830.19  
4  2

            xf_shxydm               xf_nsrmc xf_hydm                 spbm   
0  126100004366899378                   榆林学院   P8341  3040101000000000000  \
1  126108004366923514      黄河水利委员会榆林水文水资源勘测局   N7690  3040101000000000000   
2  126108004366923514      黄河水利委员会榆林水文水资源勘测局   N7690  3040101000000000000   
3  126108007379906869            榆林市公路局清涧公路段   G5443  3050100000000000000   
4  12610800MB29513799  榆林传媒中心（榆林日报社、榆林广播电视台）   R8750  3040203000000000000   

      xf_sjswjg     xf_dsswjg                 xf_qxswjg    kpyf    sum(je)  
0  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理一股  202302  145631.07  
1  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理一股  202302  441584.16  
2  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理一股  202303   396039.6  
3  国家税务总局陕西省税务局  国家税务总局榆林市税务局        国家税务总局清涧县税务局玉家河税务所  202302  650475.25  
4  国家税务总局陕西省税务局  国家税务总局榆林市税务局  国家税务总局榆林市税务局第二税务分局征收管理一股  202302  110377.36  


#

# sql语句汇总

In [ ]:
# 根据产业链的企业筛选得到
# home_7产业链市场份额省市分布
# home_8产业链企业数量 quarter_ent_shxydm 字段空间要留足够大，20000家企业
# home_9企业区县分布



# 连接数据库

In [2]:
# 连接数据库 t data sh new qyifkpsi 2023 1
db = pymysql.connect(
                    host='172.30.11.47',
                    port=3306,
                    user='root',    #在这里输入用户名
                    password='123456',     #在这里输入密码
                    )
cursor = db.cursor()
sql = "SELECT * FROM datapool.t_data_sh_new_qyjfkpsj_2023_5 WHERE gf_shxydm LIKE '9%' AND xf_shxydm LIKE '9%'"
cursor.execute(sql)
total_fp = cursor.fetchall()
cursor.close()
db.close()

In [3]:
df_col = pd.read_csv(r'E:\project_basedata\chain_supply\chain_result_0413\t_data_sh_new_qyjfkpsj_202304202046.csv',dtype = 'str')
ls_col = df_col.columns.tolist()
df_fp = pd.DataFrame(total_fp,columns=ls_col)
df_fp

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time
0,4000056,00,91610821735376041U,神木市大砭窑气化煤有限责任公司加油站,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局店塔税务分局,91670821MA70FU2813,神木市语昊机电设备维修有限公司,None,None,None,None,*柴油*柴油0#,1070101030100000000,351.33,45.67,53.72,升,202303,2023-12-08 10:40:41
1,4000085,00,91610802X23585072J,榆林市榆阳区刘千河加油站,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局上郡路税务分局,91600893MA708NUD3B,榆林市笨笨商贸有新公司,None,None,None,None,*汽油*92#汽油,1070101010100000000,154.87,20.13,25.04,升,202303,2023-12-08 10:40:41
2,4000090,00,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理一股,9140822MA9L9DHX5P,河南京徽供应链管理有限公司,None,None,None,None,*汽油*95号车用汽油(ⅥB),1070101010100000000,1558.41,202.59,213.28,升,202303,2023-12-08 10:40:41
3,4000161,00,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局上郡路税务分局,991610802MA70CJ7R2B,陕西大唐风范教育科技有限公司输林分公司,None,None,None,None,*汽油*92号 车用汽油(ⅥB),1070101010100000000,329.2,42.8,46.5,升,202303,2023-12-08 10:40:41
4,4000291,00,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局上郡路税务分局,9630100MA758GMQ4N,青海厚福实业有限公司,None,None,None,None,*汽油*92号 车用汽油(ⅥB),1070101010100000000,516.82,67.18,73,升,202303,2023-12-08 10:40:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3870149,8667093,81,916108216911259528,神木市志瑞加油站,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局孙家岔税务分局,91810821MA70950B2G,神木昊森煤业有限公司,None,None,None,None,*汽油*汽油,1070101010100000000,663.72,86.28,104.96,升,202303,2023-12-16 11:03:11
3870150,8668514,81,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局上郡路税务分局,91810132552334746A,中宏汇成本建设工程有限公司,None,None,None,None,*汽油*92号 车用汽油(ⅥB),1070101010100000000,265.48,34.52,38.62,升,202303,2023-12-16 11:05:47
3870151,8668537,81,916108210521172433,神木市铧山加油有限公司,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,92810821MA7EDGQH1B,神木市麟州喜宴酒店,None,None,None,None,*柴油*柴油,1070101030100000000,469.03,60.97,74.02,升,202302,2023-12-16 11:05:47
3870152,8668689,81,916108216911259528,神木市志瑞加油站,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局孙家岔税务分局,91810821MA70950B2G,神木昊森煤业有限公司,None,None,None,None,*汽油*汽油,1070101010100000000,1977.88,257.12,304.19,升,202302,2023-12-16 11:05:47


# 开始

In [4]:
# 产业链产品对应数据
df_chain_nodes=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\十条产业链的核心产品.xlsx',dtype='str')
df_chain_nodes1=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\T_CODE_PRODUCT_20240111.xlsx',dtype='str',sheet_name='T_CODE_PRODUCT_20240111')
# 地名和税务产品名对应数据
df_fp_name=pd.read_excel(r'E:\project_data\04yulin\fp\t_data_sh_new_cpdm_202309261607.xlsx',dtype='str')
df_fp_name_=df_fp_name.loc[:,['spbm','spmc']]
df_province=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_province',dtype='str')
df_city=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_city',dtype='str')
df_dis=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_dis',dtype='str')
df_city=df_city.drop_duplicates(['type_4'],keep='first').reset_index(drop=True)
df_dis_=df_dis.loc[:,['type_4','admin_name']]
df_spbm=df_chain_nodes1.loc[df_chain_nodes1['spbm'].notnull(),['CODE','NAME','spbm','spmc']]
df_spbm1=df_spbm.drop_duplicates(subset=['spbm']).reset_index(drop=True)

In [6]:
df_chain_nodes1=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_swcpbm.csv',dtype='str')

In [7]:
# 产业链核心五上企业名单
df_chain_core_ent=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype='str',sheet_name='Sheet1')
df_chain_core_ent_c=df_chain_core_ent.loc[df_chain_core_ent['chain'].notnull()]
ls_chain_core_ent=df_chain_core_ent.loc[df_chain_core_ent['chain'].notnull()]['统一社会信用代码'].unique().tolist()

In [5]:
df_fp_b=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\202302.csv',dtype='str')
df_fp_b=df_fp_b.astype(dtype={'je':'float','se':'float'})
df_fp_b1=df_fp_b.loc[(df_fp_b['xf_shxydm'].isin(ls_chain_core_ent))|(df_fp_b['gf_shxydm'].isin(ls_chain_core_ent))]
df_fp=df_fp_b.loc[(df_fp_b['xf_hydm']!='-')&(df_fp_b['gf_hydm']!='-')&(df_fp_b['xf_hydm'].notnull())&(df_fp_b['gf_hydm'].notnull())&(~df_fp_b['xf_shxydm'].str.startswith('11'))&(~df_fp_b['gf_shxydm'].str.startswith('11'))&(~df_fp_b['xf_shxydm'].str.startswith('00'))&(~df_fp_b['gf_shxydm'].str.startswith('00'))]

In [8]:
# 发票购销双方的位置信息补全
df_fp_=df_fp.copy()
df_fp_.loc[df_fp_['xf_sjswjg'].notnull(),'xf_province']=df_fp_.loc[df_fp_['xf_sjswjg'].notnull()]['xf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['xf_dsswjg'].notnull(),'xf_city']=df_fp_.loc[df_fp_['xf_dsswjg'].notnull()]['xf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['xf_qxswjg'].notnull(),'xf_district']=df_fp_.loc[df_fp_['xf_qxswjg'].notnull()]['xf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()
df_fp_.loc[df_fp_['gf_sjswjg'].notnull(),'gf_province']=df_fp_.loc[df_fp_['gf_sjswjg'].notnull()]['gf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['gf_dsswjg'].notnull(),'gf_city']=df_fp_.loc[df_fp_['gf_dsswjg'].notnull()]['gf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['gf_qxswjg'].notnull(),'gf_district']=df_fp_.loc[df_fp_['gf_qxswjg'].notnull()]['gf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()

df_fp_.loc[(df_fp_['xf_province'].isnull()),'xf_province_code']=df_fp_['xf_shxydm'].str[2:4]
df_fp_.loc[(df_fp_['gf_province'].isnull()),'gf_province_code']=df_fp_['gf_shxydm'].str[2:4]

df_fp_.loc[(df_fp_['xf_province'].isnull()),'xf_city_code']=df_fp_['xf_shxydm'].str[2:6]
df_fp_.loc[(df_fp_['gf_province'].isnull()),'gf_city_code']=df_fp_['gf_shxydm'].str[2:6]

df_fp_.loc[(df_fp_['xf_province'].isnull()),'xf_district_code']=df_fp_['xf_shxydm'].str[2:8]
df_fp_.loc[(df_fp_['gf_province'].isnull()),'gf_district_code']=df_fp_['gf_shxydm'].str[2:8]

df_fp_1=pd.merge(left=df_fp_,right=df_province,left_on=['xf_province_code'],right_on=['type_4'],how='left')
df_fp_1.rename(columns={'admin_name':'xf_admin_pro'},inplace=True)
df_fp_1.drop(columns=['type_4'],inplace=True)
df_fp_2=pd.merge(left=df_fp_1,right=df_province,left_on=['gf_province_code'],right_on=['type_4'],how='left')
df_fp_2.rename(columns={'admin_name':'gf_admin_pro'},inplace=True)
df_fp_2.drop(columns=['type_4'],inplace=True)

df_fp_3=pd.merge(left=df_fp_2,right=df_city,left_on=['xf_city_code'],right_on=['type_4'],how='left')
df_fp_3.rename(columns={'admin_name':'xf_admin_city'},inplace=True)
df_fp_3.drop(columns=['type_4'],inplace=True)
df_fp_4=pd.merge(left=df_fp_3,right=df_city,left_on=['gf_city_code'],right_on=['type_4'],how='left')
df_fp_4.rename(columns={'admin_name':'gf_admin_city'},inplace=True)
df_fp_4.drop(columns=['type_4'],inplace=True)

df_fp_5=pd.merge(left=df_fp_4,right=df_dis_,left_on=['xf_district_code'],right_on=['type_4'],how='left')
df_fp_5.rename(columns={'admin_name':'xf_admin_dis'},inplace=True)
df_fp_5.drop(columns=['type_4'],inplace=True)
df_fp_6=pd.merge(left=df_fp_5,right=df_dis_,left_on=['gf_district_code'],right_on=['type_4'],how='left')
df_fp_6.rename(columns={'admin_name':'gf_admin_dis'},inplace=True)
df_fp_6.drop(columns=['type_4'],inplace=True)

df_fp_6_=df_fp_6.copy()
df_fp_6_.loc[df_fp_6_['xf_province'].isnull(),'xf_province']=df_fp_6_['xf_admin_pro']
df_fp_6_.loc[df_fp_6_['gf_province'].isnull(),'gf_province']=df_fp_6_['gf_admin_pro']
df_fp_6_.loc[df_fp_6_['xf_city'].isnull(),'xf_city']=df_fp_6_['xf_admin_city']
df_fp_6_.loc[df_fp_6_['gf_city'].isnull(),'gf_city']=df_fp_6_['gf_admin_city']
df_fp_6_.loc[df_fp_6_['xf_district'].isnull(),'xf_district']=df_fp_6_['xf_admin_dis']
df_fp_6_.loc[df_fp_6_['gf_district'].isnull(),'gf_district']=df_fp_6_['gf_admin_dis']

# 刨除经销环节
# 购销双方均为FG类企业的数据不要
# df_fp_6_1=df_fp_6_.loc[~(((df_fp_6_['xf_hydm'].str[0]=='F')|(df_fp_6_['xf_hydm'].str[0]=='G'))&((df_fp_6_['gf_hydm'].str[0]=='F')|(df_fp_6_['gf_hydm'].str[0]=='G')))]
# df_fp_6_2_=df_fp_6_1.loc[(df_fp_6_1['xf_city'].notnull())&(df_fp_6_1['gf_city'].notnull())]
df_fp_6_2_=df_fp_6_.loc[(df_fp_6_['xf_city'].notnull())&(df_fp_6_['gf_city'].notnull())]
# df_fp_6_2_1=df_fp_6_2_.loc[df_fp_6_2_['je']>10000]
# df_fp_6_.loc[(df_fp_6_['xf_hydm'].str[0]!='F')&(df_fp_6_['xf_hydm'].str[0]!='G')&(df_fp_6_['gf_hydm'].str[0]!='F')&(df_fp_6_['gf_hydm'].str[0]!='G')]

In [9]:
df_fp_6_2_=df_fp_6_2_.astype(dtype={'je':'float','se':'float'})
print(df_fp_6_2_.iloc[301,16],df_fp_6_2_.iloc[301,17],df_fp_6_2_['kpyf'].unique().tolist())
print(df_fp_6_2_.iloc[301])

1352.83 81.17 ['202302', '202303', '202301']
id                              4180175
xzqh                                 11
xf_shxydm            91110000780201646K
xf_nsrmc                  北京歌华开元大酒店有限公司
xf_hydm                            None
xf_sjswjg                          None
xf_dsswjg                          None
xf_qxswjg                          None
gf_shxydm            916100006847787282
gf_nsrmc                    府谷县能东煤矿有限公司
gf_hydm                           B0610
gf_sjswjg                  国家税务总局陕西省税务局
gf_dsswjg                  国家税务总局榆林市税务局
gf_qxswjg           国家税务总局府谷县税务局老高川税务分局
hwmc                           *餐饮服务*餐费
spbm                3070401000000000000
je                              1352.83
se                                81.17
sl                                    2
jldw                               None
kpyf                             202303
update_time         2023-12-08 18:27:18
xf_province                          北京
xf_city                            

In [93]:
# home_5季度购销情况变化_本季度
# 不看煤油气
# 购销金额前五产品的前五购销地，且只看榆林与外地之间的交流
deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油',
              '发电及供电量','洗煤','售电量','天然气','风力发电','商品混凝土','风力发电','国内道路货物运输服务','水泥','其他加工劳务',
              '其他有形动产经营租赁服务','液化天然气','燃料油',
              '居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务','工程服务','火力发电']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()
df_fp_4_3=df_fp_6_2_.groupby(['kpyf','spbm','xf_province','xf_city','gf_province','gf_city'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
df_result_3=df_fp_4_3.loc[df_fp_4_3['xf_city']!=df_fp_4_3['gf_city']]
# 排序总金额前五的产品
# df_result_3_=df_result_3.loc[df_result_3['NAME'].notnull()]
df_result_4=df_result_3.loc[~(df_result_3['spbm'].isin(ls_fp_code_deselect_pro))]
df_result_5=df_result_4.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False)

# 排序金额前五的地方
ls_pro=df_result_5['spbm'].unique().tolist()
df_result_1=pd.DataFrame()
count=1
flag=1
while flag<=5:
    df_i=df_result_4.loc[df_result_4['spbm']==ls_pro[count-1]].head(5)
    if len(df_i)<3:
        count+=1
        continue
    else:
        df_i_=pd.merge(left=df_i,right=df_spbm1,on=['spbm'],how='left')
        if len(df_i_.loc[df_i_['CODE'].isnull()])==len(df_i_):
            count+=1
            continue
        else:
            df_result_1=pd.concat([df_result_1,df_i_])
            count+=1
            flag+=1
df_result_1.loc[:,'je_yiyuan']=df_result_1['je']/100000000
df_result_4.loc[:,'je_yiyuan']=df_result_4['je']/100000000
df_result_1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'home_05_季度购销情况变化_本季度.csv',header=None,index=False,mode='a+')
df_result_4.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'home_05_季度购销情况_全.csv',header=None,index=False,mode='a+')
# ,header=None

In [100]:
# home_5季度购销情况变化_变化
# 前一季度
df_last=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'home_05_季度购销情况_全.csv',dtype='str')
df_last=df_last.astype(dtype={'je_yiyuan':'float'})
df_last=df_last.loc[df_last['je_yiyuan']>0]
date_now=sorted(df_last['kpyf'].unique().tolist())[-1]
print(date_now)
# year=date_now[:4]
# quarter=date_now[-2:]
# if quarter=='01':
#     last_year=str(int(year)-1)
# else:
#     last_year=year
date_last=sorted(df_last['kpyf'].unique().tolist())[-2]
print(date_last)
# last_year+quarter_list[quarter_list.index(quarter)-1]

df_last_=df_last.loc[(df_last['kpyf']==date_last),['spbm','xf_province','xf_city','gf_province','gf_city','je_yiyuan']]
df_last_.rename(columns={'je_yiyuan':'je_last'},inplace=True)
df_now_=df_last.loc[(df_last['kpyf']==date_now),['spbm','xf_province','xf_city','gf_province','gf_city','je_yiyuan']]
df_now_.rename(columns={'je_yiyuan':'je_now'},inplace=True)
df_result_4_=pd.merge(left=df_last_,right=df_now_,on=['spbm','xf_province','xf_city','gf_province','gf_city'],how='outer')
df_result_4_.loc[:,'kpyf']=date_now
# 计算变化率
df_result_4_.loc[(df_result_4_['je_now'].notnull())&(df_result_4_['je_last'].notnull())&(df_result_4_['je_last']!=0),'rate']=(df_result_4_['je_now']-df_result_4_['je_last'])/df_result_4_['je_last']
df_result_4_.loc[(df_result_4_['je_now'].isnull())&(df_result_4_['je_last'].notnull())&(df_result_4_['je_last']!=0),'rate']=-1
df_result_4_.loc[(df_result_4_['je_now'].notnull())&(df_result_4_['je_last'].isnull())&(df_result_4_['je_last']==0),'rate']=1
df_result_4_1=df_result_4_.loc[df_result_4_['rate'].notnull()].copy()
df_result_4_1.loc[:,'abs_rate']=round(abs(df_result_4_1['rate']),2)

# 排序
df_result_4_2=df_result_4_1.loc[(df_result_4_1['je_now']>=10000)|(df_result_4_1['je_last']>=10000)]
df_result_4_3=df_result_4_2.sort_values(by='abs_rate',ascending=False)
ls_spbm_change=df_result_4_3['spbm'].unique().tolist()

df_result_5_change=pd.DataFrame()
count=1
flag=1
while flag<=5:
    df_i=df_result_4_3.loc[df_result_4_3['spbm']==ls_spbm_change[count-1]].head(5)
    if len(df_i)<4:
        # print(count,'len(df_i)',len(df_i))
        count+=1
        continue
    else:
        df_i_=pd.merge(left=df_i,right=df_spbm1,on=['spbm'],how='left')
        if len(df_i_.loc[df_i_['CODE'].isnull()])==len(df_i_):
            # print(count,ls_spbm_change[count-1])
            count+=1
            continue
        else:
            df_result_5_change=pd.concat([df_result_5_change,df_i_])
            count+=1
            flag+=1

df_result_5_change.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'home_05_季度购销情况变化_变化.csv',index=False,mode='w')
# ,header=None

202304
202303


In [94]:
# 根据产业链的企业筛选得到
# home_6产业链销售市场分季度数据

date_now=sorted(df_fp_6_2_['kpyf'].unique().tolist())[-1]
df_chain_je_se=pd.DataFrame()
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_.loc[(df_fp_6_2_['xf_shxydm'].isin(ls_shxydm))]
    
    if len(df_chain_fp_now)==0:
        ls_i_result=[date_now,0,0,i,date_now[:4]]
        df_area_market_2=pd.DataFrame([ls_i_result],columns=['kpyf','je','se','chain','year'])
    else:
        df_area_market_=df_chain_fp_now.groupby(['kpyf'])['je'].sum().reset_index()
        df_area_market_1=df_chain_fp_now.groupby(['kpyf'])['se'].sum().reset_index()
        df_area_market_2=pd.merge(left=df_area_market_,right=df_area_market_1,on=['kpyf'],how='inner')
        df_area_market_2.loc[:,'chain']=i
        df_area_market_2.loc[:,'year']=df_area_market_2['kpyf'].str[:4]
    df_chain_je_se=pd.concat([df_chain_je_se,df_area_market_2])
    
# df_chain_je_se.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_季度.csv',header=None,index=False,mode='a+')
df_chain_je_se.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_季度.csv',header=None,index=False,mode='a+',encoding='utf8')

# ,header=None
# |(df_fp_6_2_['gf_shxydm'].isin(ls_shxydm))

In [101]:
# 根据产业链的企业筛选得到
# home_6产业链销售市场年数据
df_total_chain_je=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_季度.csv',dtype={'year':'str'})
df_total_chain_je1=df_total_chain_je.groupby(['year','chain'])['je'].sum().reset_index()
df_total_chain_je2=df_total_chain_je.groupby(['year','chain'])['se'].sum().reset_index()
df_total_chain_je_=pd.merge(left=df_total_chain_je1,right=df_total_chain_je2,on=['year','chain'],how='inner')

df_total_chain_je_.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_全年.csv',index=False,mode='w')
# header=None,

In [10]:
# 根据产业链的企业筛选得到
# home_7产业链市场份额省市分布
# home_8产业链企业数量 quarter_ent_shxydm 字段空间要留足够大，20000家企业
# home_9企业区县分布

date_now=sorted(df_fp_6_2_['kpyf'].unique().tolist())[-1]
year_now=date_now[:4]
df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()

# home_7产业链市场份额省市分布
df_reault_chain_market=pd.DataFrame()
# home_8产业链企业数量
ls_chain_num=[]
# home_9企业区县分布
df_chain_district_num=pd.DataFrame()

for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    df_chain_fp_now=df_fp_6_2_1.loc[(df_fp_6_2_1['xf_shxydm'].isin(ls_shxydm))|(df_fp_6_2_1['gf_shxydm'].isin(ls_shxydm))]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        df_chain_fp_now_1=df_chain_fp_now.copy()
        # print(i,'before',len(df_chain_fp_now_1))
        ls_gf_shxydm=df_chain_fp_now['gf_shxydm'].unique().tolist()
        ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
        df_fp_6_2_2=df_fp_6_2_1.loc[~(df_fp_6_2_1.index.isin(df_chain_fp_now_1.index.tolist()))]
        
        flag=1
        while flag==1:
            pro_a=df_chain_fp_now_1.loc[(df_chain_fp_now_1['gf_shxydm'].isin(ls_gf_shxydm_))&
                                        (df_chain_fp_now_1['spbm'].isin(ls_chain_spbm))]['spbm'].unique().tolist()
            df_add=df_fp_6_2_2.loc[(df_fp_6_2_2['xf_shxydm'].isin(ls_gf_shxydm_))&(df_fp_6_2_2['spbm'].isin(pro_a))]
            print(len(df_add))
            if len(df_add)==0:
                print(len(df_add))
                flag=0
            else:
                ls_gf_shxydm=df_add.loc[df_add['gf_city']=='榆林市']['gf_shxydm'].unique().tolist()
                ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
                df_fp_6_2_2=df_fp_6_2_2.loc[~(df_fp_6_2_2.index.isin(df_add.index.tolist()))]
                df_chain_fp_now_1=pd.concat([df_chain_fp_now_1,df_add],axis=0)
                # print(i,'after',len(df_chain_fp_now_1))
        
        df_chain_fp_now_1=df_chain_fp_now_1.drop_duplicates()
        df_chain_fp_now_1=df_chain_fp_now_1.loc[df_chain_fp_now_1['je']>0]
        print(i,'after_all',len(df_chain_fp_now_1))
        
        # home_7产业链市场份额省市分布
        df_province_i=df_chain_fp_now_1.groupby(['gf_province'])['je'].sum().reset_index()
        df_province_i.rename(columns={'je':'province_je'},inplace=True)
        df_city_i=df_chain_fp_now_1.groupby(['gf_province','gf_city'])['je'].sum().reset_index()
        df_city_i.rename(columns={'je':'city_je'},inplace=True)
        df_area_market=pd.merge(left=df_city_i,right=df_province_i,on=['gf_province'],how='left')
        df_area_market.loc[:,'chain']=i
        df_area_market.loc[:,'kpyf']=date_now
        df_area_market.loc[:,'year']=year_now
        df_area_market_=df_area_market.sort_values(by='province_je',ascending=False)
        df_reault_chain_market=pd.concat([df_reault_chain_market,df_area_market_])
        
        # home_8产业链企业数量
        chain_xf=df_chain_fp_now_1.loc[(df_chain_fp_now_1['xf_city']=='榆林市')]
        chain_gf=df_chain_fp_now_1.loc[(df_chain_fp_now_1['gf_city']=='榆林市')]
        quarter_sell_ent_xf=chain_xf['xf_shxydm'].unique().tolist()
        quarter_sell_ent_gf=chain_gf['gf_shxydm'].unique().tolist()
        quarter_sell_ent=list(set(quarter_sell_ent_xf+quarter_sell_ent_gf))
        quarter_sell_ent_=[x for x in quarter_sell_ent if x in ls_shxydm]
        str_quarter_sell_ent=';'.join(quarter_sell_ent)
        str_quarter_sell_ent_=';'.join(quarter_sell_ent_)
        ls_chain_num.append([i,len(quarter_sell_ent),len(quarter_sell_ent_),str_quarter_sell_ent,str_quarter_sell_ent_,date_now,year_now])
        
        # home_9企业区县分布
        df_district_xf=chain_xf.loc[chain_xf['xf_shxydm'].isin(ls_shxydm)]
        xf_district_1=df_district_xf.groupby(['xf_district','kpyf'])['xf_shxydm'].apply(lambda x:';'.join(x)).reset_index()
        xf_district_1.rename(columns={'xf_shxydm':'xf_shxydm_str'},inplace=True)
        xf_district_2=df_district_xf.groupby(['xf_district','kpyf'])['xf_shxydm'].count().reset_index()
        xf_district=pd.merge(left=xf_district_1,right=xf_district_2,on=['xf_district','kpyf'],how='inner')
        xf_district.rename(columns={'xf_district':'district','xf_shxydm':'num','xf_shxydm_str':'shxydm_str'},inplace=True)
        
        df_district_gf=chain_gf.loc[chain_gf['gf_shxydm'].isin(ls_shxydm)]
        gf_district_1=df_district_gf.groupby(['gf_district','kpyf'])['gf_shxydm'].apply(lambda x:';'.join(x)).reset_index()
        gf_district_1.rename(columns={'gf_shxydm':'gf_shxydm_str'},inplace=True)
        gf_district_2=df_district_gf.groupby(['gf_district','kpyf'])['gf_shxydm'].count().reset_index()
        gf_district=pd.merge(left=gf_district_1,right=gf_district_2,on=['gf_district','kpyf'],how='inner')
        gf_district.rename(columns={'gf_district':'district','gf_shxydm':'num','gf_shxydm_str':'shxydm_str'},inplace=True)
        
        district_ent=pd.concat([xf_district,gf_district],axis=0)
        district_ent_1=district_ent.groupby(['district','kpyf'])['shxydm_str'].apply(lambda x:';'.join(x)).reset_index()
        for a in range(len(district_ent_1)):
            str_a=district_ent_1.loc[a,'shxydm_str']
            list_a=str_a.split(';')
            list_a_=list(set(list_a))
            district_ent_1.loc[a,'num']=len(list_a_)
            district_ent_1.loc[a,'shxydm_str_1']=';'.join(list_a_)
        district_ent_1.loc[:,'chain']=i
        district_ent_1.loc[:,'year']=district_ent_1['kpyf'].str[:4]
        district_ent_1.drop(columns=['shxydm_str'],axis=1,inplace=True)
        df_chain_district_num=pd.concat([df_chain_district_num,district_ent_1])
        
df_reault_chain_market.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_本季度.csv',header=None,index=False,mode='a+')
pd.DataFrame(ls_chain_num,columns=['chain','quarter_num','quarter_5_num','quarter_ent_shxydm','quarter_ent_5_shxydm','kpyf','year']).to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_本季度.csv',header=None,index=False,mode='a+')
df_chain_district_num.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_本季度.csv',header=None,index=False,mode='a+')
# ,header=None

KeyboardInterrupt: 

In [ ]:
现代特色农业产业链 after_all 76095
“碳中和”产业链 after_all 28107
现代煤化工产业链 after_all 291923
智能无人系统和航空产业链 after_all 1038
文化旅游产业链 after_all 10478
轻纺产业链 after_all 1543
能源化工装备制造产业链 after_all 19196
新能源装备制造产业链 after_all 7946
铝镁及兰炭深加工产业链 after_all 52000
氢能产业链 after_all 21

In [102]:
# 根据产业链的企业筛选得到
# home_7产业链市场份额省市分布
# home_8产业链企业数量
# home_9企业区县分布

# home_7产业链市场份额省市分布_全年
df_07_=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_本季度.csv',dtype={'kpyf':'str','year':'str'})
df_province_7=df_07_.groupby(['year','chain','gf_province'])['city_je'].sum().reset_index()
df_province_7.rename(columns={'city_je':'province_je'},inplace=True)
df_city_7=df_07_.groupby(['year','chain','gf_province','gf_city'])['city_je'].sum().reset_index()
df_area_market_total_year=pd.merge(left=df_city_7,right=df_province_7,on=['year','chain','gf_province'],how='left')
df_area_market_total_year.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_年度.csv',index=False,mode='w')

# home_8产业链企业数量
df_08_=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_本季度.csv',dtype={'kpyf':'str','year':'str'})
date_now=sorted(df_08_['kpyf'].unique().tolist())[-1]
district_ent_1=df_08_.groupby(['year','chain'])['quarter_ent_shxydm'].apply(lambda x:';'.join(x)).reset_index()
district_ent_2=df_08_.groupby(['year','chain'])['quarter_ent_5_shxydm'].apply(lambda x:';'.join(x)).reset_index()
district_ent_3=pd.merge(left=district_ent_1,right=district_ent_2,on=['year','chain'],how='inner')
for a in range(len(district_ent_3)):
    # year_a=district_ent_1.loc[a,'year']
    chain_a=district_ent_3.loc[a,'chain']
    str_a=district_ent_3.loc[a,'quarter_ent_shxydm']
    str_b=district_ent_3.loc[a,'quarter_ent_5_shxydm']
    list_a=str_a.split(';')
    list_a_=list(set(list_a))
    list_b=str_b.split(';')
    list_b_=list(set(list_b))
    
    district_ent_3.loc[a,'year_num']=len(list_a_)
    district_ent_3.loc[a,'quarter_num']=df_08_.loc[(df_08_['kpyf']==date_now)&(df_08_['chain']==chain_a)]['quarter_num'].tolist()[-1]
    district_ent_3.loc[a,'year_5_num']=len(list_b_)
    district_ent_3.loc[a,'quarter_5_num']=df_08_.loc[(df_08_['kpyf']==date_now)&(df_08_['chain']==chain_a)]['quarter_5_num'].tolist()[-1]
district_ent_3.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_年度.csv',index=False,mode='w')

# home_9企业区县分布
ls_district1=['府谷县','靖边县','定边县','绥德县','米脂县','佳县','吴堡县','清涧县','子洲县','榆阳区','横山区','神木市']
ls_district2=['府谷','靖边','定边','绥德','米脂','佳','吴堡','清涧','子洲','榆阳','横山','神木']

df_09_=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_本季度.csv',dtype={'kpyf':'str','year':'str'})
for a in range(len(df_09_)):
    district_a=df_09_.loc[a,'district']
    for b in range(len(ls_district2)):
        if ls_district2[b] in district_a:
            df_09_.loc[a,'district_nor']=ls_district1[b]
        if district_a=='榆林高新技术产业开发区' or district_a=='榆林经济技术开发区' or district_a=='榆林市':
            df_09_.loc[a,'district_nor']='榆阳区'

df_09_=df_09_.loc[df_09_['district_nor'].notnull()]
district_ent_1=df_09_.groupby(['year','chain','district_nor'])['shxydm_str_1'].apply(lambda x:';'.join(x)).reset_index()
for a in range(len(district_ent_1)):
    str_a=district_ent_1.loc[a,'shxydm_str_1']
    list_a=str_a.split(';')
    list_a_=list(set(list_a))
    district_ent_1.loc[a,'year_num']=len(list_a_)
    
district_ent_1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_年度.csv',index=False,mode='w')

In [96]:
# 根据产业链的企业筛选得到
# detail_1历年产业链销售总额及增速
# detail_2产业链税收及前三非煤油气主要产品销售额变化
deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_fp_6_2_1=df_fp_6_2_.copy()
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
date_now=sorted(df_fp_6_2_1['kpyf'].unique().tolist())[-1]
year_now=date_now[:4]
detail_1_quarter=pd.DataFrame()
detail_2_quarter=pd.DataFrame()

for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_1.loc[(df_fp_6_2_1['xf_shxydm'].isin(ls_shxydm))]
    # detail_1历年产业链销售总额及增速
    if len(df_chain_fp_now)==0:
        print(i)
        df_area_market_2=pd.DataFrame([[date_now,0,0,i,year_now]],columns=['kpyf','je','se','chain','year'])
        detail_2_i_=pd.DataFrame()
    else:
        df_area_market_=df_chain_fp_now.groupby(['kpyf'])['je'].sum().reset_index()
        df_area_market_1=df_chain_fp_now.groupby(['kpyf'])['se'].sum().reset_index()
        df_area_market_2=pd.merge(left=df_area_market_,right=df_area_market_1,on=['kpyf'],how='inner')
        df_area_market_2.loc[:,'chain']=i
        df_area_market_2.loc[:,'year']=df_area_market_2['kpyf'].str[:4]
        
    # detail_2产业链税收及前三非煤油气主要产品销售额变化
        detail_2_je_i=df_chain_fp_now.groupby(['spbm','kpyf'])['je'].sum().reset_index()
        detail_2_se_i=df_chain_fp_now.groupby(['spbm','kpyf'])['se'].sum().reset_index()
        detail_2_i=pd.merge(left=detail_2_je_i,right=detail_2_se_i,on=['spbm','kpyf'],how='inner')
        detail_2_i_=detail_2_i.loc[~(detail_2_i['spbm'].isin(ls_fp_code_deselect_pro))].copy()
        detail_2_i_.loc[:,'chain']=i
        detail_2_i_.loc[:,'year']=detail_2_i_['kpyf'].str[:4]
        
    detail_1_quarter=pd.concat([detail_1_quarter,df_area_market_2])
    detail_2_quarter=pd.concat([detail_2_quarter,detail_2_i_])

detail_1_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',header=None,index=False,mode='a+')
detail_2_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',header=None,index=False,mode='a+')
# ,header=None

In [13]:
# 根据产业链的企业筛选得到
# detail_1全年
# detail_2全年
# best

# detail_01历年产业链销售总额及增速
df_detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
df_detail_1_year=df_detail_1_quarter.groupby(['chain','year'])['je'].sum().reset_index()
ls_year=sorted(df_detail_1_year['year'].unique().tolist())
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()

for i in ls_chain:
    for j in range(1,len(ls_year)):
        je_now=df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'je'].tolist()[0]
        je_last=df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j-1]),'je'].tolist()[0]
        if je_last==0:
            df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'rate']=100
        else:
            df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'rate']=(je_now-je_last)*100/je_last
df_detail_1_year.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_年度.csv',index=False,mode='w')

# detail_02产业链税收及前三非煤油气主要产品销售额变化
detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_2_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})

df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

ls_date=sorted(detail_2_quarter['kpyf'].unique().tolist())[-6:]
ls_result_detail_2=[]
df_result_detail_2=pd.DataFrame()

for i in ls_chain:
    ls_chain_i=[]
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    df_chain_spbm_=df_chain_spbm.loc[:,['CODE','NAME','spbm','spmc']]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    detail_2_year_=detail_2_quarter.loc[(detail_2_quarter['kpyf'].isin(ls_date))&(detail_2_quarter['chain']==i)]
    detail_2_year_1=detail_2_year_.groupby(['chain','spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    detail_2_year_2=detail_2_year_1.loc[detail_2_year_1['spbm'].isin(ls_chain_spbm)].head(3)
    ls_spbm=detail_2_year_2['spbm'].unique().tolist()
    for j in range(1,len(ls_date)):
        
        # se_i_last=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j-1])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        se_i_now=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        # if se_i_last==0 and se_i_now!=0:
        #     se_rate=100
        # elif se_i_last==0 and se_i_now==0:
        #     se_rate=0
        # else:
        #     se_rate=(se_i_now-se_i_last)*100/se_i_last
        ls_i=[i,ls_date[j],'chain_se',se_i_now/10000]
        ls_result_detail_2.append(ls_i)
        ls_chain_i.append(ls_i)
        
        for a in ls_spbm:
            df_pro_now=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            df_pro_last=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j-1])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            if len(df_pro_now)==0:
                pro_a_now=0
            else:
                pro_a_now=df_pro_now['je'].tolist()[0]
                
            if len(df_pro_last)==0:
                pro_a_last=0
            else:
                pro_a_last=df_pro_last['je'].tolist()[0]
                
            if pro_a_last==0 and pro_a_now!=0:
                pro_a_rate=100
            elif pro_a_last==0 and pro_a_now==0:
                pro_a_rate=0
            else:
                pro_a_rate=(pro_a_now-pro_a_last)*100/pro_a_last
            ls_a=[i,ls_date[j],a,pro_a_rate]
            ls_result_detail_2.append(ls_a)
            ls_chain_i.append(ls_a)
    df_chain_i=pd.DataFrame(ls_chain_i,columns=['chain','kpyf','spbm','rate'])
    df_chain_i_re=pd.merge(left=df_chain_i,right=df_chain_spbm_,on=['spbm'],how='left')
    df_result_detail_2=pd.concat([df_result_detail_2,df_chain_i_re],axis=0)
    df_result_detail_2=df_result_detail_2.drop_duplicates(subset=['chain','kpyf','spbm','rate','spmc'],keep='first')

df_detail_2=pd.DataFrame(ls_result_detail_2,columns=['chain','kpyf','spbm','rate'])
df_detail_2=df_detail_2.astype(dtype={'spbm':'str'})
df_detail_3=pd.merge(left=df_detail_2,right=df_spbm1,on=['spbm'],how='left')
#df_detail_3.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',index=False,mode='w')
df_result_detail_2.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',index=False,mode='w')
# ,header=None

In [97]:
# 根据产业链的企业筛选得到
# detail_03季度销售前十生产企业（万元）
# detail_04季度销售前十经销企业（万元）

df_fp_6_2_1=df_fp_6_2_.copy()
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_3_quarter=pd.DataFrame()
detail_4_quarter=pd.DataFrame()
detail_4_quarter_no5=pd.DataFrame()
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_1.loc[(df_fp_6_2_1['xf_shxydm'].isin(ls_shxydm))]
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    if len(df_chain_fp_now)==0:
        continue
    else:
        detail_34_quarter=df_chain_fp_now.groupby(['xf_shxydm','xf_nsrmc','xf_hydm','kpyf','xf_district'])['je'].sum().reset_index()
        detail_34_quarter.loc[:,'je_w']=detail_34_quarter['je']/10000
        detail_34_quarter.loc[:,'hydm_type']=detail_34_quarter['xf_hydm'].str[0]
        detail_34_quarter.loc[:,'year']=detail_34_quarter['kpyf'].str[:4]
        detail_34_quarter.loc[:,'chain']=i

        # detail_03季度销售前十生产企业（万元）,hydm=非FG
        detail_3_quarter_i=detail_34_quarter.loc[detail_34_quarter['hydm_type'].isin(['A','B','C','E','I','M','R'])]
        detail_3_quarter=pd.concat([detail_3_quarter,detail_3_quarter_i])
        # detail_04季度销售前十经销企业（万元）
        detail_4_quarter_i=detail_34_quarter.loc[detail_34_quarter['hydm_type'].isin(['F','G'])]
        detail_4_quarter=pd.concat([detail_4_quarter,detail_4_quarter_i])
        # detail_04季度销售前十经销企业（万元，非五上）
        df_chain_fp_now_1=df_chain_fp_now.copy()
        # print(i,'before',len(df_chain_fp_now_1))
        ls_gf_shxydm=df_chain_fp_now['gf_shxydm'].unique().tolist()
        ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
        df_fp_6_2_2=df_fp_6_2_1.loc[~(df_fp_6_2_1.index.isin(df_chain_fp_now_1.index.tolist()))]
        
        flag=1
        while flag==1:
            pro_a=df_chain_fp_now_1.loc[(df_chain_fp_now_1['gf_shxydm'].isin(ls_gf_shxydm_))&(df_chain_fp_now_1['spbm'].isin(ls_chain_spbm))]['spbm'].unique().tolist()
            df_add=df_fp_6_2_2.loc[(df_fp_6_2_2['xf_shxydm'].isin(ls_gf_shxydm_))&(df_fp_6_2_2['spbm'].isin(pro_a))]

            if len(df_add)==0:
                flag=0
            else:
                ls_gf_shxydm=df_add.loc[df_add['gf_city']=='榆林市']['gf_shxydm'].unique().tolist()
                ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
                df_fp_6_2_2=df_fp_6_2_2.loc[~(df_fp_6_2_2.index.isin(df_add.index.tolist()))]
                df_chain_fp_now_1=pd.concat([df_chain_fp_now_1,df_add],axis=0)
                # print(i,'after',len(df_chain_fp_now_1))
        
        df_chain_fp_now_1=df_chain_fp_now_1.drop_duplicates()
        df_chain_fp_now_1=df_chain_fp_now_1.loc[df_chain_fp_now_1['je']>0]
        print(i,'after_all',len(df_chain_fp_now_1))
        
        # df_chain_fp_now_1=df_chain_fp_now.copy()
        # ls_gf_shxydm=df_chain_fp_now['gf_shxydm'].unique().tolist()
        # ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
        # df_fp_6_2_2=df_fp_6_2_1.loc[~(df_fp_6_2_1.index.isin(df_chain_fp_now_1.index.tolist()))]
        # 
        # flag=1
        # while flag==1:
        #     df_add=pd.DataFrame()
        #     for a in ls_gf_shxydm:
        #         pro_a=df_chain_fp_now_1.loc[df_chain_fp_now_1['gf_shxydm']==a]['spbm'].unique().tolist()
        #         df_sell_a=df_fp_6_2_2.loc[(df_fp_6_2_2['xf_shxydm']==a)&(df_fp_6_2_2['spbm'].isin(pro_a))]
        #         df_add=pd.concat([df_add,df_sell_a],axis=0)
        #     if len(df_add)==0:
        #         flag=0
        #     else:
        #         ls_gf_shxydm=df_add.loc[df_add['gf_city']=='榆林市']['gf_shxydm'].unique().tolist()
        #         ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
        #         df_fp_6_2_2=df_fp_6_2_2.loc[~(df_fp_6_2_2.index.isin(df_add.index.tolist()))]
        #         df_chain_fp_now_1=pd.concat([df_chain_fp_now_1,df_add],axis=0)
        # 
        # df_chain_fp_now_1=df_chain_fp_now_1.drop_duplicates()
        # df_chain_fp_now_1=df_chain_fp_now_1.loc[df_chain_fp_now_1['je']>0]
        
        detail_34_quarter=df_chain_fp_now_1.groupby(['xf_shxydm','xf_nsrmc','xf_hydm','kpyf','xf_district'])['je'].sum().reset_index()
        detail_34_quarter.loc[:,'je_w']=detail_34_quarter['je']/10000
        detail_34_quarter.loc[:,'hydm_type']=detail_34_quarter['xf_hydm'].str[0]
        detail_34_quarter.loc[:,'year']=detail_34_quarter['kpyf'].str[:4]
        detail_34_quarter.loc[:,'chain']=i
        
        detail_4_quarter_i=detail_34_quarter.loc[detail_34_quarter['hydm_type'].isin(['F','G'])]
        detail_4_quarter_no5=pd.concat([detail_4_quarter_no5,detail_4_quarter_i])
    
detail_3_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_03_季度销售生产企业_季度.csv',header=None,index=False,mode='a+')
detail_4_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业_季度.csv',header=None,index=False,mode='a+')
detail_4_quarter_no5.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业非五上_季度.csv',header=None,index=False,mode='a+')
# ,header=None

现代特色农业产业链 after_all 61664
“碳中和”产业链 after_all 6057
现代煤化工产业链 after_all 120366
智能无人系统和航空产业链 after_all 98
文化旅游产业链 after_all 8880
轻纺产业链 after_all 400
能源化工装备制造产业链 after_all 7444
新能源装备制造产业链 after_all 360
铝镁及兰炭深加工产业链 after_all 20801
氢能产业链 after_all 2


In [7]:
# 根据产业链的企业筛选得到
# detail_03季度销售前十生产企业（万元）
# detail_04季度销售前十经销企业（万元）
# detail_05本地产业链生产企业区县分布

ls_chain=df_chain_core_ent_c['chain'].unique().tolist()

ls_district1=['府谷县','靖边县','定边县','绥德县','米脂县','佳县','吴堡县','清涧县','子洲县','榆阳区','横山区','神木市']
ls_district2=['府谷','靖边','定边','绥德','米脂','佳','吴堡','清涧','子洲','榆阳','横山','神木']
detail_3_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_03_季度销售生产企业_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_4_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_4_quarter_no5=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业非五上_季度.csv',dtype={'kpyf':'str','year':'str'})

detail_3_year=detail_3_quarter.groupby(['xf_shxydm','xf_nsrmc','xf_hydm','chain','year','xf_district'])['je'].sum().reset_index()
detail_4_year=detail_4_quarter.groupby(['xf_shxydm','xf_nsrmc','xf_hydm','chain','year','xf_district'])['je'].sum().reset_index()
detail_4_no5_year=detail_4_quarter_no5.groupby(['xf_shxydm','xf_nsrmc','xf_hydm','chain','year','xf_district'])['je'].sum().reset_index()

latest_year=sorted(detail_3_quarter['year'].unique().tolist())[-1]
detail_3_quarter=pd.DataFrame()
detail_4_quarter=pd.DataFrame()
detail_5_quarter=pd.DataFrame()
detail_4_quarter_no5_=pd.DataFrame()

for i in ls_chain:
     # detail_03季度销售前十生产企业（万元）
    detail_3_year_1=detail_3_year.loc[(detail_3_year['year']==latest_year)&(detail_3_year['chain']==i)].sort_values(by='je',ascending=False)
    detail_3_year_1=detail_3_year_1.loc[detail_3_year_1['je']>0].reset_index(drop=True)
    detail_3_year_2=detail_3_year_1.head(10)
    detail_3_quarter=pd.concat([detail_3_quarter,detail_3_year_2],axis=0)
    # 
    # detail_04季度销售前十经销企业（万元）
    detail_4_year_1=detail_4_year.loc[(detail_4_year['year']==latest_year)&(detail_4_year['chain']==i)].sort_values(by='je',ascending=False)
    detail_4_year_1=detail_4_year_1.loc[detail_4_year_1['je']>0].reset_index(drop=True)
    detail_4_year_2=detail_4_year_1.head(10)
    detail_4_quarter=pd.concat([detail_4_quarter,detail_4_year_2],axis=0)
    # 
    # detail_04季度非五上销售前十经销企业（万元）
    detail_4_year_1=detail_4_no5_year.loc[(detail_4_no5_year['year']==latest_year)&(detail_4_no5_year['chain']==i)].sort_values(by='je',ascending=False)
    detail_4_year_1=detail_4_year_1.loc[detail_4_year_1['je']>0].reset_index(drop=True)
    detail_4_year_2=detail_4_year_1.head(10)
    detail_4_quarter_no5_=pd.concat([detail_4_quarter_no5_,detail_4_year_2],axis=0)
    
    # detail_05本地产业链生产企业区县分布
    for a in range(len(detail_3_year_1)):
        district_a=detail_3_year_1.loc[a,'xf_district']
        for b in range(len(ls_district2)):
            if ls_district2[b] in district_a:
                detail_3_year_1.loc[a,'district_nor']=ls_district1[b]
        if district_a=='榆林高新技术产业开发区' or district_a=='榆林经济技术开发区' or district_a=='榆林市':
                detail_3_year_1.loc[a,'district_nor']='榆阳区'
    try:
        detail_3_year_1_=detail_3_year_1.loc[detail_3_year_1['district_nor'].notnull()]
        detail_5_year_1=detail_3_year_1_.groupby(['year','chain','district_nor'])['xf_shxydm'].count().reset_index()
    except:
        detail_5_year_1=pd.DataFrame()
        
    detail_5_quarter=pd.concat([detail_5_quarter,detail_5_year_1],axis=0)
    
detail_5_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_05_本地产业链非经销企业区县分布_年度.csv',index=False,mode='w')
# detail_4_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_04_季度销售经销企业_年度.csv',index=False,mode='w')
# detail_4_quarter_no5_.loc[:,'je_w']=detail_4_quarter_no5_['je']/10000
# detail_4_quarter_no5_.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_04_季度销售经销企业非五上_年度.csv',index=False,mode='w')
# detail_3_quarter.loc[:,'je_w']=detail_3_quarter['je']/10000
# detail_3_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_03_季度销售生产企业_年度.csv',index=False,mode='w')

In [85]:
detail_4_no5_year

,xf_shxydm,xf_nsrmc,xf_hydm,chain,year,xf_district,je
0,91150921MA0QJM75XM,陕西山能盛世供应链管理有限公司,F5161,现代煤化工产业链,2022,榆林高新技术产业开发区,81861.46
1,91150921MA0QJM75XM,陕西山能盛世供应链管理有限公司,F5161,铝镁及兰炭深加工产业链,2023,榆林高新技术产业开发区,5468860.29
2,91610000052129199L,陕西广通现代物流有限公司,G5431,现代煤化工产业链,2023,西安市莲湖区,797155.08
3,91610000052129199L,陕西广通现代物流有限公司,G5431,铝镁及兰炭深加工产业链,2022,西安市莲湖区,1044497.34
4,91610000054751360A,陕西省煤炭运销集团建宏投资发展有限公司,F5161,现代煤化工产业链,2022,西安高新技术产业开发区,46942205.19
...,...,...,...,...,...,...,...
6603,93610802MA70CK1M6J,榆林市榆阳区中农科创种植农民专业合作社,F5211,现代特色农业产业链,2022,榆林市榆阳区,1382555.28
6604,93610828052138271M,佳县红润枣业专业合作社,F5115,现代特色农业产业链,2022,佳县,41000.00
6605,93610828052138271M,佳县红润枣业专业合作社,F5115,现代特色农业产业链,2023,佳县,190000.00
6606,93610831562243709Y,子洲县永望农产品销售农民专业合作社,F5223,现代特色农业产业链,2022,子洲县,188196.67


In [98]:
# 根据产业链的企业筛选得到
# detail_08季度产业链产品交易情况_销售
# detail_09季度产品交易情况_销售
# detail_10产品交易金额变化_销售
df_fp_6_2_1=df_fp_6_2_.copy()
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_8910=pd.DataFrame()

for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_1.loc[(df_fp_6_2_1['xf_shxydm'].isin(ls_shxydm))]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        df_i=df_chain_fp_now.groupby(['gf_city','spbm','kpyf'])['je'].sum().reset_index()
        df_i.loc[:,'chain']=i
        df_i.loc[:,'year']=df_i['kpyf'].str[:4]
        detail_8910=pd.concat([detail_8910,df_i],axis=0)
detail_8910.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',header=None,index=False,mode='a+')
# ,header=None

In [33]:
# 根据产业链的企业筛选得到
# detail_08年度产业链产品交易情况_销售
# detail_09季度产品交易情况_销售
# detail_10产品交易金额变化_销售

ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_detail_3=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',dtype='str')
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]
detail_08_info=pd.DataFrame()
detail_09_area=pd.DataFrame()
detail_10_info=pd.DataFrame()

for i in ls_chain:
    # detail_08年度产业链产品交易情况_销售
    df_i=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)]
    df_i_chain=df_i.groupby(['spbm','chain','kpyf'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    df_chain_spbm_=df_chain_spbm.loc[:,['CODE','NAME','spbm','spmc']]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    df_i_chain_1=df_i_chain.loc[df_i_chain['spbm'].isin(ls_chain_spbm)]
    df_i_chain_top=df_i_chain_1.head(15).reset_index(drop=True)
    for j in range(len(df_i_chain_top)):
        spbm_j=df_i_chain_top.loc[j,'spbm']
        je_nian=df_i_chain_top.loc[j,'je']
        je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        if je_tongbi==0:
            df_i_chain_top.loc[j,'rate_tongbi']=1
        else:
            df_i_chain_top.loc[j,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi
            
        if je_huanbi==0:
            df_i_chain_top.loc[j,'rate_huanbi']=1
        else:
            df_i_chain_top.loc[j,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
        
        df_i_chain_top.loc[j,'je_quarter']=je_now
    df_i_chain_top_=pd.merge(left=df_i_chain_top,right=df_chain_spbm_,on=['spbm'],how='left')
    df_i_chain_top_.loc[:,'je_w']=df_i_chain_top_['je']/10000
    df_i_chain_top_=df_i_chain_top_.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top_])
    # detail_08_info=detail_08_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
    
    # detail_09季度产品交易情况_销售
    df_i_chain_9=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)]
    df_i_chain_9_1=df_i_chain_top_.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(3)
    ls_spbm_top_3_=df_i_chain_9_1['spbm'].unique().tolist()
    # ls_spbm_top_3_=df_detail_3.loc[(df_detail_3['chain']==i)&(df_detail_3['kpyf']==date_now)]['spbm'].tolist()
    # ls_spbm_top_3_=[x for x in ls_spbm_top_3_ if x!='chain_se']
    
    for a in ls_spbm_top_3_:
        df_a=df_i_chain_9.loc[df_i_chain_9['spbm']==a]
        df_a_top5=df_a.groupby(['chain','spbm','gf_city','kpyf','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(5)
        df_a_top5_=pd.merge(left=df_a_top5,right=df_chain_spbm_,on=['spbm'],how='left')
        df_a_top5_.loc[:,'je_w']=df_a_top5_['je']/10000
        detail_09_area=pd.concat([detail_09_area,df_a_top5_])
        detail_09_area=detail_09_area.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
        
        # detail_10产品交易金额变化_销售
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_13))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(1,len(ls_date_13)):
            je_now=df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je']
            date_now_b=ls_date_13[b]
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=100
            else:
                rate_tongbi=(je_now-je_last_year)*100/je_last_year
                
            date_last_quarter=ls_date_13[b-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=100
            else:
                rate_huanbi=(je_now-je_last_quarter)*100/je_last_quarter
                
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je_last_year']=je_last_year
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_tongbi']=rate_tongbi

        df_a_10_1_=pd.merge(left=df_a_10_1,right=df_chain_spbm_,on=['spbm'],how='left')
        df_a_10_1_.loc[:,'je_w']=df_a_10_1_['je']/10000
        df_a_10_1_.loc[:,'je_last_year_w']=df_a_10_1_['je_last_year']/10000
        detail_10_info=pd.concat([detail_10_info,df_a_10_1_])
        detail_10_info=detail_10_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')

detail_08_info.loc[detail_08_info['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_销售.csv',index=False,mode='w')
detail_09_area.loc[detail_09_area['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_销售.csv',index=False,mode='w')
detail_10_info.loc[detail_10_info['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_销售.csv',index=False,mode='w')

In [99]:
# 根据产业链的企业筛选得到
# detail_08年度产业链产品交易情况_购买
# detail_09季度产品交易情况_购买
# detail_10产品交易金额变化_购买
df_fp_6_2_1=df_fp_6_2_.copy()
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_8910=pd.DataFrame()

for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_1.loc[(df_fp_6_2_1['gf_shxydm'].isin(ls_shxydm))]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        df_i=df_chain_fp_now.groupby(['xf_city','spbm','kpyf'])['je'].sum().reset_index()
        df_i.loc[:,'chain']=i
        df_i.loc[:,'year']=df_i['kpyf'].str[:4]
        detail_8910=pd.concat([detail_8910,df_i],axis=0)
    
detail_8910.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',header=None,index=False,mode='a+')
# ,header=None

In [32]:
# 根据产业链的企业筛选得到
# detail_08年度产业链产品交易情况_购买
# detail_09季度产品交易情况_购买
# detail_10产品交易金额变化_购买
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

detail_08_info=pd.DataFrame()
detail_09_area=pd.DataFrame()
detail_10_info=pd.DataFrame()

for i in ls_chain:
    # detail_08年度产业链产品交易情况_购买
    df_i=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)]
    df_i_chain=df_i.groupby(['spbm','chain','kpyf'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    df_chain_spbm_=df_chain_spbm.loc[:,['CODE','NAME','spbm','spmc']]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    df_i_chain_1=df_i_chain.loc[df_i_chain['spbm'].isin(ls_chain_spbm)]
    df_i_chain_top=df_i_chain_1.head(15).reset_index(drop=True)
    for j in range(len(df_i_chain_top)):
        spbm_j=df_i_chain_top.loc[j,'spbm']
        je_nian=df_i_chain_top.loc[j,'je']
        je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        if je_tongbi==0:
            df_i_chain_top.loc[j,'rate_tongbi']=1
        else:
            df_i_chain_top.loc[j,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi
            
        if je_huanbi==0:
            df_i_chain_top.loc[j,'rate_huanbi']=1
        else:
            df_i_chain_top.loc[j,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
        
        df_i_chain_top.loc[j,'je_quarter']=je_now
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top])
    df_i_chain_top_=pd.merge(left=df_i_chain_top,right=df_chain_spbm_,on=['spbm'],how='left')
    df_i_chain_top_.loc[:,'je_w']=df_i_chain_top_['je']/10000
    df_i_chain_top_=df_i_chain_top_.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top_])
    # detail_08_info=detail_08_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
    
    # detail_09季度产品交易情况_购买
    df_i_chain_9=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)]
    df_i_chain_9_1=df_i_chain_top_.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(3)
    ls_spbm_top_3=df_i_chain_9_1['spbm'].unique().tolist()
    
    for a in ls_spbm_top_3:
        df_a=df_i_chain_9.loc[df_i_chain_9['spbm']==a]
        df_a_top5=df_a.groupby(['chain','spbm','xf_city','kpyf','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(5)
        df_a_top5_=pd.merge(left=df_a_top5,right=df_chain_spbm_,on=['spbm'],how='left')
        df_a_top5_.loc[:,'je_w']=df_a_top5_['je']/10000
        detail_09_area=pd.concat([detail_09_area,df_a_top5_])
        detail_09_area=detail_09_area.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
        
        # detail_10产品交易金额变化_购买
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_13))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(1,len(ls_date_13)):
            je_now=df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je']
            date_now_b=ls_date_13[b]
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=100
            else:
                rate_tongbi=(je_now-je_last_year)*100/je_last_year
                
            date_last_quarter=ls_date_13[b-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=100
            else:
                rate_huanbi=(je_now-je_last_quarter)*100/je_last_quarter
                
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je_last_year']=je_last_year
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_tongbi']=rate_tongbi

        df_a_10_1_=pd.merge(left=df_a_10_1,right=df_chain_spbm_,on=['spbm'],how='left')
        df_a_10_1_.loc[:,'je_w']=df_a_10_1_['je']/10000
        df_a_10_1_.loc[:,'je_last_year_w']=df_a_10_1_['je_last_year']/10000
        detail_10_info=pd.concat([detail_10_info,df_a_10_1_])
        detail_10_info=detail_10_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
        
detail_08_info.loc[detail_08_info['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_购买.csv',index=False,mode='w')
detail_09_area.loc[detail_09_area['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_购买.csv',index=False,mode='w')
detail_10_info.loc[detail_10_info['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_购买.csv',index=False,mode='w')

#

# 新版备用

In [ ]:
# 根据产业链的企业筛选得到
# detail_08年度产业链产品交易情况_购买
# detail_09季度产品交易情况_购买
# detail_10产品交易金额变化_购买
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]
detail_08_info=pd.DataFrame()
detail_09_area=pd.DataFrame()
detail_10_info=pd.DataFrame()

for i in ls_chain:
    # detail_08年度产业链产品交易情况_购买
    df_i=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)]
    df_i_chain=df_i.groupby(['spbm','chain','kpyf'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    df_chain_spbm_=df_chain_spbm.loc[:,['CODE','NAME','spbm','spmc']]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    df_i_chain_1=df_i_chain.loc[df_i_chain['spbm'].isin(ls_chain_spbm)]
    df_i_chain_top=df_i_chain_1.head(15).reset_index(drop=True)
    for j in range(len(df_i_chain_top)):
        spbm_j=df_i_chain_top.loc[j,'spbm']
        je_nian=df_i_chain_top.loc[j,'je']
        je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        if je_tongbi==0:
            df_i_chain_top.loc[j,'rate_tongbi']=1
        else:
            df_i_chain_top.loc[j,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi
            
        if je_huanbi==0:
            df_i_chain_top.loc[j,'rate_huanbi']=1
        else:
            df_i_chain_top.loc[j,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
        
        df_i_chain_top.loc[j,'je_quarter']=je_now
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top])
    df_i_chain_top_=pd.merge(left=df_i_chain_top,right=df_chain_spbm_,on=['spbm'],how='left')
    df_i_chain_top_.loc[:,'je_w']=df_i_chain_top_['je']/10000
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top_])
    detail_08_info=detail_08_info.drop_duplicates(subset=['chain','kpyf','spbm','je','spmc'],keep='first')
    
    # detail_09季度产品交易情况_购买
    df_i_chain_9=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm'].isin(ls_chain_spbm))&(~detail_8910['spbm'].isin(ls_fp_code_deselect_pro))]
    df_i_chain_9_1=df_i_chain_9.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(3)
    ls_spbm_top_3=df_i_chain_9_1['spbm'].unique().tolist()
    
    for a in ls_spbm_top_3:
        df_a=df_i_chain_9.loc[df_i_chain_9['spbm']==a]
        df_a_top5=df_a.groupby(['chain','spbm','xf_city','kpyf','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(5)
        detail_09_area=pd.concat([detail_09_area,df_a_top5])
        
        # detail_10产品交易金额变化_购买
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_13))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(1,len(ls_date_13)):
            je_now=df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je']
            date_now_b=ls_date_13[b]
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=100
            else:
                rate_tongbi=(je_now-je_last_year)*100/je_last_year
                
            date_last_quarter=ls_date_13[b-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=100
            else:
                rate_huanbi=(je_now-je_last_quarter)*100/je_last_quarter
                
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je_last_year']=je_last_year
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_tongbi']=rate_tongbi
        detail_10_info=pd.concat([detail_10_info,df_a_10_1])
        
detail_08_info_1=pd.merge(left=detail_08_info,right=df_spbm1,on=['spbm'],how='left')
detail_08_info_1.loc[:,'je_w']=detail_08_info_1['je']/10000
detail_08_info_1.loc[detail_08_info_1['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_购买.csv',index=False,mode='w')

detail_09_area_1=pd.merge(left=detail_09_area,right=df_spbm1,on=['spbm'],how='left')
detail_09_area_1.loc[:,'je_w']=detail_09_area_1['je']/10000
detail_09_area_1.loc[detail_09_area_1['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_购买.csv',index=False,mode='w')

detail_10_info_1=pd.merge(left=detail_10_info,right=df_spbm1,on=['spbm'],how='left')
detail_10_info_1.loc[:,'je_w']=detail_10_info_1['je']/10000
detail_10_info_1.loc[:,'je_last_year_w']=detail_10_info_1['je_last_year']/10000
detail_10_info_1.loc[detail_10_info_1['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_购买.csv',index=False,mode='w')

In [ ]:
# 根据产业链的企业筛选得到
# detail_08年度产业链产品交易情况_销售
# detail_09季度产品交易情况_销售
# detail_10产品交易金额变化_销售

ls_chain=df_chain_core_ent_c['chain'].unique().tolist()
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})
ls_date_13=sorted(detail_8910['kpyf'].unique().tolist())[-13:]

deselect_pro=['原煤','液化石油气','收费的电力供应服务','金属制品、设备修理','柴油','维修、清洁和其他服务','其他建筑安装服务','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','煤炭采选产品','其他未列明煤炭采选产品','液化天然气（LNG）供应量','其他石油制品','其他建筑服务']
ls_fp_code_deselect_pro=df_chain_nodes1.loc[df_chain_nodes1['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_detail_3=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',dtype='str')
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

date_now=ls_date_13[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_13[-2]
detail_08_info=pd.DataFrame()
detail_09_area=pd.DataFrame()
detail_10_info=pd.DataFrame()

for i in ls_chain:
    # detail_08年度产业链产品交易情况_销售
    df_i=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)]
    df_i_chain=df_i.groupby(['spbm','chain','kpyf'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    df_chain_spbm_=df_chain_spbm.loc[:,['CODE','NAME','spbm','spmc']]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    df_i_chain_1=df_i_chain.loc[df_i_chain['spbm'].isin(ls_chain_spbm)]
    df_i_chain_top=df_i_chain_1.head(15).reset_index(drop=True)
    for j in range(len(df_i_chain_top)):
        spbm_j=df_i_chain_top.loc[j,'spbm']
        je_nian=df_i_chain_top.loc[j,'je']
        je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        if je_tongbi==0:
            df_i_chain_top.loc[j,'rate_tongbi']=1
        else:
            df_i_chain_top.loc[j,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi
            
        if je_huanbi==0:
            df_i_chain_top.loc[j,'rate_huanbi']=1
        else:
            df_i_chain_top.loc[j,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
        
        df_i_chain_top.loc[j,'je_quarter']=je_now
    df_i_chain_top_=pd.merge(left=df_i_chain_top,right=df_chain_spbm_,on=['spbm'],how='left')
    df_i_chain_top_.loc[:,'je_w']=df_i_chain_top_['je']/10000
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top_])
    
    # detail_09季度产品交易情况_销售
    df_i_chain_9=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(~detail_8910['spbm'].isin(ls_fp_code_deselect_pro))]
    df_i_chain_9_1=df_i_chain_top_.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(3)
    # ls_spbm_top_3=df_i_chain_9_1['spbm'].unique().tolist()
    # ls_spbm_top_3_=ls_spbm_top_3
    ls_spbm_top_3_=df_detail_3.loc[(df_detail_3['chain']==i)&(df_detail_3['kpyf']==date_now)]['spbm'].tolist()
    ls_spbm_top_3_=[x for x in ls_spbm_top_3_ if x!='chain_se']
    
    for a in ls_spbm_top_3_:
        df_a=df_i_chain_9.loc[df_i_chain_9['spbm']==a]
        df_a_top5=df_a.groupby(['chain','spbm','gf_city','kpyf','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(5)
        df_a_top5_=pd.merge(left=df_a_top5,right=df_chain_spbm_,on=['spbm'],how='left')
        df_a_top5_.loc[:,'je_w']=df_a_top5_['je']/10000
        detail_09_area=pd.concat([detail_09_area,df_a_top5_])
        
        # detail_10产品交易金额变化_销售
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_13))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(1,len(ls_date_13)):
            je_now=df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je']
            date_now_b=ls_date_13[b]
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=100
            else:
                rate_tongbi=(je_now-je_last_year)*100/je_last_year
                
            date_last_quarter=ls_date_13[b-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=100
            else:
                rate_huanbi=(je_now-je_last_quarter)*100/je_last_quarter
                
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'je_last_year']=je_last_year
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[df_a_10_1['kpyf']==ls_date_13[b],'rate_tongbi']=rate_tongbi

        df_a_10_1_=pd.merge(left=df_a_10_1,right=df_chain_spbm_,on=['spbm'],how='left')
        df_a_10_1_.loc[:,'je_w']=df_a_10_1_['je']/10000
        df_a_10_1_.loc[:,'je_last_year_w']=df_a_10_1_['je_last_year']/10000
        detail_10_info=pd.concat([detail_10_info,df_a_10_1_])

# detail_08_info_1=pd.merge(left=detail_08_info,right=df_spbm1,on=['spbm'],how='left')
# detail_08_info_1.loc[:,'je_w']=detail_08_info_1['je']/10000
# detail_08_info_1.loc[detail_08_info_1['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_销售.csv',index=False,mode='w')
# 
# detail_09_area_1=pd.merge(left=detail_09_area,right=df_spbm1,on=['spbm'],how='left')
# detail_09_area_1.loc[:,'je_w']=detail_09_area_1['je']/10000
# detail_09_area_1.loc[detail_09_area_1['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_销售.csv',index=False,mode='w')
# 
# detail_10_info_1=pd.merge(left=detail_10_info,right=df_spbm1,on=['spbm'],how='left')
# detail_10_info_1.loc[:,'je_w']=detail_10_info_1['je']/10000
# detail_10_info_1.loc[:,'je_last_year_w']=detail_10_info_1['je_last_year']/10000
# detail_10_info_1.loc[detail_10_info_1['NAME'].notnull()].to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_销售.csv',index=False,mode='w')

In [7]:
# 根据产业链的企业筛选得到
# detail_1全年
# detail_2全年

# detail_01历年产业链销售总额及增速
df_detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
df_detail_1_year=df_detail_1_quarter.groupby(['chain','year'])['je'].sum().reset_index()
ls_year=sorted(df_detail_1_year['year'].unique().tolist())
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()

for i in ls_chain:
    for j in range(1,len(ls_year)):
        je_now=df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'je'].tolist()[0]
        je_last=df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j-1]),'je'].tolist()[0]
        if je_last==0:
            df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'rate']=100
        else:
            df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'rate']=(je_now-je_last)*100/je_last
df_detail_1_year.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_年度.csv',index=False,mode='w')

# detail_02产业链税收及前三非煤油气主要产品销售额变化
detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_2_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',dtype={'kpyf':'str','year':'str','spbm':'str'})

df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]

ls_date=sorted(detail_2_quarter['kpyf'].unique().tolist())[-6:]
ls_result_detail_2=[]

for i in ls_chain:
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    df_chain_spbm_=df_chain_spbm.loc[:,['CODE','NAME','spbm','spmc']]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    detail_2_year_=detail_2_quarter.loc[(detail_2_quarter['kpyf'].isin(ls_date))&(detail_2_quarter['chain']==i)]
    detail_2_year_1=detail_2_year_.groupby(['chain','spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    detail_2_year_2=detail_2_year_1.loc[detail_2_year_1['spbm'].isin(ls_chain_spbm)].head(3)
    ls_spbm=detail_2_year_2['spbm'].unique().tolist()
    for j in range(1,len(ls_date)):
        
        # se_i_last=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j-1])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        se_i_now=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        # if se_i_last==0 and se_i_now!=0:
        #     se_rate=100
        # elif se_i_last==0 and se_i_now==0:
        #     se_rate=0
        # else:
        #     se_rate=(se_i_now-se_i_last)*100/se_i_last
        ls_i=[i,ls_date[j],'chain_se',se_i_now/10000]
        ls_result_detail_2.append(ls_i)
        
        for a in ls_spbm:
            df_pro_now=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            df_pro_last=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j-1])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            if len(df_pro_now)==0:
                pro_a_now=0
            else:
                pro_a_now=df_pro_now['je'].tolist()[0]
                
            if len(df_pro_last)==0:
                pro_a_last=0
            else:
                pro_a_last=df_pro_last['je'].tolist()[0]
                
            if pro_a_last==0 and pro_a_now!=0:
                pro_a_rate=100
            elif pro_a_last==0 and pro_a_now==0:
                pro_a_rate=0
            else:
                pro_a_rate=(pro_a_now-pro_a_last)*100/pro_a_last
            ls_a=[i,ls_date[j],a,pro_a_rate]
            ls_result_detail_2.append(ls_a)

df_detail_2=pd.DataFrame(ls_result_detail_2,columns=['chain','kpyf','spbm','rate'])
df_detail_2=df_detail_2.astype(dtype={'spbm':'str'})
df_detail_3=pd.merge(left=df_detail_2,right=df_spbm1,on=['spbm'],how='left')
df_detail_3.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',index=False,mode='w')
# ,header=None

In [36]:
# 根据产业链的企业筛选得到
# home_7产业链市场份额省市分布
# home_8产业链企业数量 quarter_ent_shxydm 字段空间要留足够大，20000家企业
# home_9企业区县分布

date_now=sorted(df_fp_6_2_['kpyf'].unique().tolist())[-1]
year_now=date_now[:4]
df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['chain'].notnull()]
ls_chain=df_chain_core_ent_c['chain'].unique().tolist()

# home_7产业链市场份额省市分布
df_reault_chain_market=pd.DataFrame()
# home_8产业链企业数量
ls_chain_num=[]
# home_9企业区县分布
df_chain_district_num=pd.DataFrame()

for i in ls_chain:
    ls_shxydm=df_chain_core_ent_c.loc[df_chain_core_ent_c['chain']==i]['统一社会信用代码'].unique().tolist()
    df_chain_spbm=df_chain_nodes2.loc[df_chain_nodes2['chain'].str.contains(i)]
    ls_chain_spbm=df_chain_spbm['spbm'].unique().tolist()
    
    df_chain_fp_now=df_fp_6_2_1.loc[(df_fp_6_2_1['xf_shxydm'].isin(ls_shxydm))|(df_fp_6_2_1['gf_shxydm'].isin(ls_shxydm))]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        df_chain_fp_now_1=df_chain_fp_now.copy()
        print(i,'before',len(df_chain_fp_now_1))
        ls_gf_shxydm=df_chain_fp_now['gf_shxydm'].unique().tolist()
        ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
        df_fp_6_2_2=df_fp_6_2_1.loc[~(df_fp_6_2_1.index.isin(df_chain_fp_now_1.index.tolist()))]
        
        flag=1
        while flag==1:
            df_add=pd.DataFrame()
            for a in ls_gf_shxydm:
                pro_a=df_chain_fp_now_1.loc[df_chain_fp_now_1['gf_shxydm']==a]['spbm'].unique().tolist()
                df_sell_a=df_fp_6_2_2.loc[(df_fp_6_2_2['xf_shxydm']==a)&(df_fp_6_2_2['spbm'].isin(pro_a))]
                df_add=pd.concat([df_add,df_sell_a],axis=0)
                print(ls_gf_shxydm.index(a),'/',len(ls_gf_shxydm))
            if len(df_add)==0:
                flag=0
            else:
                ls_gf_shxydm=df_add.loc[df_add['gf_city']=='榆林市']['gf_shxydm'].unique().tolist()
                ls_gf_shxydm_=[x for x in ls_gf_shxydm if x not in ls_shxydm]
                df_fp_6_2_2=df_fp_6_2_2.loc[~(df_fp_6_2_2.index.isin(df_add.index.tolist()))]
                df_chain_fp_now_1=pd.concat([df_chain_fp_now_1,df_add],axis=0)
                print(i,'after',len(df_chain_fp_now_1))
        
        df_chain_fp_now_1=df_chain_fp_now_1.drop_duplicates()
        df_chain_fp_now_1=df_chain_fp_now_1.loc[df_chain_fp_now_1['je']>0]
        print(i,'after_all',len(df_chain_fp_now_1))
        
        # home_7产业链市场份额省市分布
        df_province_i=df_chain_fp_now_1.groupby(['gf_province'])['je'].sum().reset_index()
        df_province_i.rename(columns={'je':'province_je'},inplace=True)
        df_city_i=df_chain_fp_now_1.groupby(['gf_province','gf_city'])['je'].sum().reset_index()
        df_city_i.rename(columns={'je':'city_je'},inplace=True)
        df_area_market=pd.merge(left=df_city_i,right=df_province_i,on=['gf_province'],how='left')
        df_area_market.loc[:,'chain']=i
        df_area_market.loc[:,'kpyf']=date_now
        df_area_market.loc[:,'year']=year_now
        df_area_market_=df_area_market.sort_values(by='province_je',ascending=False)
        df_reault_chain_market=pd.concat([df_reault_chain_market,df_area_market_])
        
        # home_8产业链企业数量
        chain_xf=df_chain_fp_now_1.loc[(df_chain_fp_now_1['xf_city']=='榆林市')&(df_chain_fp_now_1['spbm'].isin(ls_chain_spbm))]
        chain_gf=df_chain_fp_now_1.loc[(df_chain_fp_now_1['gf_city']=='榆林市')&(df_chain_fp_now_1['spbm'].isin(ls_chain_spbm))]
        quarter_sell_ent_xf=chain_xf['xf_shxydm'].unique().tolist()
        quarter_sell_ent_gf=chain_gf['gf_shxydm'].unique().tolist()
        quarter_sell_ent=list(set(quarter_sell_ent_xf+quarter_sell_ent_gf))
        quarter_sell_ent_=[x for x in quarter_sell_ent if x in ls_shxydm]
        str_quarter_sell_ent=';'.join(quarter_sell_ent)
        str_quarter_sell_ent_=';'.join(quarter_sell_ent_)
        ls_chain_num.append([i,len(quarter_sell_ent),len(quarter_sell_ent_),str_quarter_sell_ent,str_quarter_sell_ent_,date_now,year_now])
        
        # home_9企业区县分布
        df_district_xf=chain_xf.loc[chain_xf['xf_shxydm'].isin(ls_shxydm)]
        xf_district_1=df_district_xf.groupby(['xf_district','kpyf'])['xf_shxydm'].apply(lambda x:';'.join(x)).reset_index()
        xf_district_1.rename(columns={'xf_shxydm':'xf_shxydm_str'},inplace=True)
        xf_district_2=df_district_xf.groupby(['xf_district','kpyf'])['xf_shxydm'].count().reset_index()
        xf_district=pd.merge(left=xf_district_1,right=xf_district_2,on=['xf_district','kpyf'],how='inner')
        xf_district.rename(columns={'xf_district':'district','xf_shxydm':'num','xf_shxydm_str':'shxydm_str'},inplace=True)
        
        df_district_gf=chain_gf.loc[chain_gf['gf_shxydm'].isin(ls_shxydm)]
        gf_district_1=df_district_gf.groupby(['gf_district','kpyf'])['gf_shxydm'].apply(lambda x:';'.join(x)).reset_index()
        gf_district_1.rename(columns={'gf_shxydm':'gf_shxydm_str'},inplace=True)
        gf_district_2=df_district_gf.groupby(['gf_district','kpyf'])['gf_shxydm'].count().reset_index()
        gf_district=pd.merge(left=gf_district_1,right=gf_district_2,on=['gf_district','kpyf'],how='inner')
        gf_district.rename(columns={'gf_district':'district','gf_shxydm':'num','gf_shxydm_str':'shxydm_str'},inplace=True)
        
        district_ent=pd.concat([xf_district,gf_district],axis=0)
        district_ent_1=district_ent.groupby(['district','kpyf'])['shxydm_str'].apply(lambda x:';'.join(x)).reset_index()
        for a in range(len(district_ent_1)):
            str_a=district_ent_1.loc[a,'shxydm_str']
            list_a=str_a.split(';')
            list_a_=list(set(list_a))
            district_ent_1.loc[a,'num']=len(list_a_)
            district_ent_1.loc[a,'shxydm_str_1']=';'.join(list_a_)
        district_ent_1.loc[:,'chain']=i
        district_ent_1.loc[:,'year']=district_ent_1['kpyf'].str[:4]
        district_ent_1.drop(columns=['shxydm_str'],axis=1,inplace=True)
        df_chain_district_num=pd.concat([df_chain_district_num,district_ent_1])
        
df_reault_chain_market.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_本季度.csv',index=False,mode='a+')
pd.DataFrame(ls_chain_num,columns=['chain','quarter_num','quarter_5_num','quarter_ent_shxydm','quarter_ent_5_shxydm','kpyf','year']).to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_本季度.csv',index=False,mode='a+')
df_chain_district_num.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_本季度.csv',index=False,mode='a+')
# ,header=None

现代特色农业产业链 before 98359
0 / 1691
1 / 1691
2 / 1691
3 / 1691
4 / 1691
5 / 1691
6 / 1691
7 / 1691
8 / 1691
9 / 1691
10 / 1691
11 / 1691
12 / 1691
13 / 1691
14 / 1691
15 / 1691
16 / 1691
17 / 1691
18 / 1691
19 / 1691
20 / 1691
21 / 1691
22 / 1691
23 / 1691
24 / 1691
25 / 1691
26 / 1691
27 / 1691
28 / 1691
29 / 1691
30 / 1691
31 / 1691
32 / 1691
33 / 1691
34 / 1691
35 / 1691
36 / 1691
37 / 1691
38 / 1691
39 / 1691
40 / 1691
41 / 1691
42 / 1691
43 / 1691
44 / 1691
45 / 1691
46 / 1691
47 / 1691
48 / 1691
49 / 1691
50 / 1691
51 / 1691
52 / 1691
53 / 1691
54 / 1691
55 / 1691
56 / 1691
57 / 1691
58 / 1691
59 / 1691
60 / 1691
61 / 1691
62 / 1691
63 / 1691
64 / 1691
65 / 1691
66 / 1691
67 / 1691
68 / 1691
69 / 1691
70 / 1691
71 / 1691
72 / 1691
73 / 1691
74 / 1691
75 / 1691
76 / 1691
77 / 1691
78 / 1691
79 / 1691
80 / 1691
81 / 1691
82 / 1691
83 / 1691
84 / 1691
85 / 1691
86 / 1691
87 / 1691
88 / 1691
89 / 1691
90 / 1691
91 / 1691
92 / 1691
93 / 1691
94 / 1691
95 / 1691
96 / 1691
97 / 1691
98 / 16

754 / 1691
755 / 1691
756 / 1691
757 / 1691
758 / 1691
759 / 1691
760 / 1691
761 / 1691
762 / 1691
763 / 1691
764 / 1691
765 / 1691
766 / 1691
767 / 1691
768 / 1691
769 / 1691
770 / 1691
771 / 1691
772 / 1691
773 / 1691
774 / 1691
775 / 1691
776 / 1691
777 / 1691
778 / 1691
779 / 1691
780 / 1691
781 / 1691
782 / 1691
783 / 1691
784 / 1691
785 / 1691
786 / 1691
787 / 1691
788 / 1691
789 / 1691
790 / 1691
791 / 1691
792 / 1691
793 / 1691
794 / 1691
795 / 1691
796 / 1691
797 / 1691
798 / 1691
799 / 1691
800 / 1691
801 / 1691
802 / 1691
803 / 1691
804 / 1691
805 / 1691
806 / 1691
807 / 1691
808 / 1691
809 / 1691
810 / 1691
811 / 1691
812 / 1691
813 / 1691
814 / 1691
815 / 1691
816 / 1691
817 / 1691
818 / 1691
819 / 1691
820 / 1691
821 / 1691
822 / 1691
823 / 1691
824 / 1691
825 / 1691
826 / 1691
827 / 1691
828 / 1691
829 / 1691
830 / 1691
831 / 1691
832 / 1691
833 / 1691
834 / 1691
835 / 1691
836 / 1691
837 / 1691
838 / 1691
839 / 1691
840 / 1691
841 / 1691
842 / 1691
843 / 1691
844 / 1691

1459 / 1691
1460 / 1691
1461 / 1691
1462 / 1691
1463 / 1691
1464 / 1691
1465 / 1691
1466 / 1691
1467 / 1691
1468 / 1691
1469 / 1691
1470 / 1691
1471 / 1691
1472 / 1691
1473 / 1691
1474 / 1691
1475 / 1691
1476 / 1691
1477 / 1691
1478 / 1691
1479 / 1691
1480 / 1691
1481 / 1691
1482 / 1691
1483 / 1691
1484 / 1691
1485 / 1691
1486 / 1691
1487 / 1691
1488 / 1691
1489 / 1691
1490 / 1691
1491 / 1691
1492 / 1691
1493 / 1691
1494 / 1691
1495 / 1691
1496 / 1691
1497 / 1691
1498 / 1691
1499 / 1691
1500 / 1691
1501 / 1691
1502 / 1691
1503 / 1691
1504 / 1691
1505 / 1691
1506 / 1691
1507 / 1691
1508 / 1691
1509 / 1691
1510 / 1691
1511 / 1691
1512 / 1691
1513 / 1691
1514 / 1691
1515 / 1691
1516 / 1691
1517 / 1691
1518 / 1691
1519 / 1691
1520 / 1691
1521 / 1691
1522 / 1691
1523 / 1691
1524 / 1691
1525 / 1691
1526 / 1691
1527 / 1691
1528 / 1691
1529 / 1691
1530 / 1691
1531 / 1691
1532 / 1691
1533 / 1691
1534 / 1691
1535 / 1691
1536 / 1691
1537 / 1691
1538 / 1691
1539 / 1691
1540 / 1691
1541 / 1691
1542

138 / 930
139 / 930
140 / 930
141 / 930
142 / 930
143 / 930
144 / 930
145 / 930
146 / 930
147 / 930
148 / 930
149 / 930
150 / 930
151 / 930
152 / 930
153 / 930
154 / 930
155 / 930
156 / 930
157 / 930
158 / 930
159 / 930
160 / 930
161 / 930
162 / 930
163 / 930
164 / 930
165 / 930
166 / 930
167 / 930
168 / 930
169 / 930
170 / 930
171 / 930
172 / 930
173 / 930
174 / 930
175 / 930
176 / 930
177 / 930
178 / 930
179 / 930
180 / 930
181 / 930
182 / 930
183 / 930
184 / 930
185 / 930
186 / 930
187 / 930
188 / 930
189 / 930
190 / 930
191 / 930
192 / 930
193 / 930
194 / 930
195 / 930
196 / 930
197 / 930
198 / 930
199 / 930
200 / 930
201 / 930
202 / 930
203 / 930
204 / 930
205 / 930
206 / 930
207 / 930
208 / 930
209 / 930
210 / 930
211 / 930
212 / 930
213 / 930
214 / 930
215 / 930
216 / 930
217 / 930
218 / 930
219 / 930
220 / 930
221 / 930
222 / 930
223 / 930
224 / 930
225 / 930
226 / 930
227 / 930
228 / 930
229 / 930
230 / 930
231 / 930
232 / 930
233 / 930
234 / 930
235 / 930
236 / 930
237 / 930


KeyboardInterrupt: 

# 老版备用

In [3]:
# 所有发票产品的对应关系
df_chain_nodes2=df_chain_nodes1.loc[df_chain_nodes1['spbm'].notnull(),['CODE','NAME','spbm']]
ls_chain_nodes2=df_chain_nodes2.values.tolist()
ls_result=[]
for i in ls_chain_nodes2:
    ls_i=i[-1].split(';')
    if len(ls_i)>1:
        for j in ls_i:
            ls_j=[i[0],i[1],j]
            ls_result.append(ls_j)
    else:
        ls_result.append(i)
        
df_chain_nodes2=pd.DataFrame(ls_result,columns=['CODE','NAME','spbm'])
df_chain_nodes2_=df_chain_nodes2.drop_duplicates(['spbm'],keep='first').reset_index(drop=True)

In [31]:
# 老版
# home_6产业链销售市场分季度数据
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
df_chain_je_se=pd.DataFrame()

date_now=sorted(df_fp_6_2_['kpyf'].unique().tolist())[-1]

for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_.loc[df_fp_6_2_['spbm'].isin(ls_spbm)]
    
    if len(df_chain_fp_now)==0:
        ls_i_result=[date_now,0,0,i,date_now[:4]]
        df_area_market_2=pd.DataFrame([ls_i_result],columns=['kpyf','je','se','chain','year'])
    else:
        df_area_market_=df_chain_fp_now.groupby(['kpyf'])['je'].sum().reset_index()
        df_area_market_1=df_chain_fp_now.groupby(['kpyf'])['se'].sum().reset_index()
        df_area_market_2=pd.merge(left=df_area_market_,right=df_area_market_1,on=['kpyf'],how='inner')
        df_area_market_2.loc[:,'chain']=i
        df_area_market_2.loc[:,'year']=df_area_market_2['kpyf'].str[:4]
    
    df_chain_je_se=pd.concat([df_chain_je_se,df_area_market_2])
    
df_chain_je_se.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_季度.csv',header=None,index=False,mode='a+')

# home_6产业链销售市场年数据
df_total_chain_je=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_季度.csv',dtype={'year':'str'})
df_total_chain_je1=df_total_chain_je.groupby(['year','chain'])['je'].sum().reset_index()
df_total_chain_je2=df_total_chain_je.groupby(['year','chain'])['se'].sum().reset_index()
df_total_chain_je_=pd.merge(left=df_total_chain_je1,right=df_total_chain_je2,on=['year','chain'],how='inner')

df_total_chain_je_.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_06_产业链销售金额及税额_全年.csv',index=False,mode='w')
# header=None,

In [50]:
# 老版
# home_7产业链市场份额省市分布
# home_8产业链企业数量 quarter_ent_shxydm 字段空间要留足够大，60000-100000
# home_9企业区县分布

date_now=sorted(df_fp_6_2_['kpyf'].unique().tolist())[-1]
year_now=date_now[:4]
df_fp_6_2_1=df_fp_6_2_.copy()

df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
# home_7产业链市场份额省市分布
df_reault_chain_market=pd.DataFrame()
# home_8产业链企业数量
ls_chain_num=[]
# home_9企业区县分布
df_chain_district_num=pd.DataFrame()

for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp_now=df_fp_6_2_1.loc[df_fp_6_2_1['spbm'].isin(ls_spbm)]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        # home_7产业链市场份额省市分布
        df_province_i=df_chain_fp_now.groupby(['gf_province'])['je'].sum().reset_index()
        df_province_i.rename(columns={'je':'province_je'},inplace=True)
        df_city_i=df_chain_fp_now.groupby(['gf_province','gf_city'])['je'].sum().reset_index()
        df_city_i.rename(columns={'je':'city_je'},inplace=True)
        df_area_market=pd.merge(left=df_city_i,right=df_province_i,on=['gf_province'],how='left')
        df_area_market.loc[:,'chain']=i
        df_area_market.loc[:,'kpyf']=date_now
        df_area_market.loc[:,'year']=year_now
        df_area_market_=df_area_market.sort_values(by='province_je',ascending=False)
        df_reault_chain_market=pd.concat([df_reault_chain_market,df_area_market_])
        # home_8产业链企业数量
        chain_xf=df_chain_fp_now.loc[df_chain_fp_now['xf_city']=='榆林市']
        chain_gf=df_chain_fp_now.loc[df_chain_fp_now['gf_city']=='榆林市']
        quarter_sell_ent_xf=chain_xf['xf_shxydm'].unique().tolist()
        quarter_sell_ent_gf=chain_gf['gf_shxydm'].unique().tolist()
        quarter_sell_ent=list(set(quarter_sell_ent_xf+quarter_sell_ent_gf))
        str_quarter_sell_ent=';'.join(quarter_sell_ent)
        ls_chain_num.append([i,len(quarter_sell_ent),str_quarter_sell_ent,date_now,year_now])
        # home_9企业区县分布
        xf_district_1=chain_xf.groupby(['xf_district','kpyf'])['xf_shxydm'].apply(lambda x:';'.join(x)).reset_index()
        xf_district_1.rename(columns={'xf_shxydm':'xf_shxydm_str'},inplace=True)
        xf_district_2=chain_xf.groupby(['xf_district','kpyf'])['xf_shxydm'].count().reset_index()
        xf_district=pd.merge(left=xf_district_1,right=xf_district_2,on=['xf_district','kpyf'],how='inner')
        xf_district.rename(columns={'xf_district':'district','xf_shxydm':'num','xf_shxydm_str':'shxydm_str'},inplace=True)

        gf_district_1=chain_gf.groupby(['gf_district','kpyf'])['gf_shxydm'].apply(lambda x:';'.join(x)).reset_index()
        gf_district_1.rename(columns={'gf_shxydm':'gf_shxydm_str'},inplace=True)
        gf_district_2=chain_gf.groupby(['gf_district','kpyf'])['gf_shxydm'].count().reset_index()
        gf_district=pd.merge(left=gf_district_1,right=gf_district_2,on=['gf_district','kpyf'],how='inner')
        gf_district.rename(columns={'gf_district':'district','gf_shxydm':'num','gf_shxydm_str':'shxydm_str'},inplace=True)

        district_ent=pd.concat([xf_district,gf_district],axis=0)
        district_ent_1=district_ent.groupby(['district','kpyf'])['shxydm_str'].apply(lambda x:';'.join(x)).reset_index()
        for a in range(len(district_ent_1)):
            str_a=district_ent_1.loc[a,'shxydm_str']
            list_a=str_a.split(';')
            list_a_=list(set(list_a))
            district_ent_1.loc[a,'num']=len(list_a_)
            district_ent_1.loc[a,'shxydm_str_1']=';'.join(list_a_)
        district_ent_1.loc[:,'chain']=i
        district_ent_1.loc[:,'year']=district_ent_1['kpyf'].str[:4]
        district_ent_1.drop(columns=['shxydm_str'],axis=1,inplace=True)
        df_chain_district_num=pd.concat([df_chain_district_num,district_ent_1])
df_reault_chain_market.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_本季度.csv',header=None,index=False,mode='a+')
pd.DataFrame(ls_chain_num,columns=['chain','quarter_num','quarter_ent_shxydm','kpyf','year']).to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_本季度.csv',header=None,index=False,mode='a+')
df_chain_district_num.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_本季度.csv',header=None,index=False,mode='a+')
# ,header=None

In [52]:
# 老版
# home_7产业链市场份额省市分布
# home_8产业链企业数量
# home_9企业区县分布
# home_7产业链市场份额省市分布_全年
df_07_=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_本季度.csv',dtype={'kpyf':'str','year':'str'})
df_province_7=df_07_.groupby(['year','chain','gf_province'])['province_je'].sum().reset_index()
df_city_7=df_07_.groupby(['year','chain','gf_province','gf_city'])['city_je'].sum().reset_index()
df_area_market_total_year=pd.merge(left=df_city_7,right=df_province_7,on=['year','chain','gf_province'],how='left')
df_area_market_total_year.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_07_市场份额_年度.csv',index=False,mode='w')

# home_8产业链企业数量
df_08_=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_本季度.csv',dtype={'kpyf':'str','year':'str'})
date_now=sorted(df_08_['kpyf'].unique().tolist())[-1]
district_ent_1=df_08_.groupby(['year','chain'])['quarter_ent_shxydm'].apply(lambda x:';'.join(x)).reset_index()
for a in range(len(district_ent_1)):
    # year_a=district_ent_1.loc[a,'year']
    chain_a=district_ent_1.loc[a,'chain']
    str_a=district_ent_1.loc[a,'quarter_ent_shxydm']
    list_a=str_a.split(';')
    list_a_=list(set(list_a))
    district_ent_1.loc[a,'year_num']=len(list_a_)
    district_ent_1.loc[a,'quarter_num']=df_08_.loc[(df_08_['kpyf']==date_now)&(df_08_['chain']==chain_a)]['quarter_num'].tolist()[-1]
district_ent_1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_产业链企业数量_年度.csv',index=False,mode='w')

# home_9企业区县分布
df_09_=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_本季度.csv',dtype={'kpyf':'str','year':'str'})
district_ent_1=df_09_.groupby(['year','chain','district'])['shxydm_str_1'].apply(lambda x:';'.join(x)).reset_index()
for a in range(len(district_ent_1)):
    str_a=district_ent_1.loc[a,'shxydm_str_1']
    list_a=str_a.split(';')
    list_a_=list(set(list_a))
    district_ent_1.loc[a,'year_num']=len(list_a_)
district_ent_1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_09_产业链企业区县分布_年度.csv',index=False,mode='w')

In [200]:
# 老版
# detail_1历年产业链销售总额及增速
# detail_2产业链税收及前三非煤油气主要产品销售额变化
deselect_pro=['原煤','液化石油气','柴油','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','热力']
ls_fp_code_deselect_pro=df_chain_nodes2.loc[df_chain_nodes2['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
date_now=sorted(df_fp_6_2_1['kpyf'].unique().tolist())[-1]
year_now=date_now[:4]
detail_1_quarter=pd.DataFrame()
detail_2_quarter=pd.DataFrame()

for i in ls_chain:
    # detail_1历年产业链销售总额及增速
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp_now=df_fp_4_2_sell.loc[df_fp_4_2_sell['spbm'].isin(ls_spbm)]
    if len(df_chain_fp_now)==0:
        print(i)
        df_area_market_2=pd.DataFrame([[date_now,0,0,i,year_now]],columns=['kpyf','je','se','chain','year'])
        detail_2_i_=pd.DataFrame()
    else:
        df_area_market_=df_chain_fp_now.groupby(['kpyf'])['je'].sum().reset_index()
        df_area_market_1=df_chain_fp_now.groupby(['kpyf'])['se'].sum().reset_index()
        df_area_market_2=pd.merge(left=df_area_market_,right=df_area_market_1,on=['kpyf'],how='inner')
        df_area_market_2.loc[:,'chain']=i
        df_area_market_2.loc[:,'year']=df_area_market_2['kpyf'].str[:4]
        
    # detail_2产业链税收及前三非煤油气主要产品销售额变化
        detail_2_je_i=df_chain_fp_now.groupby(['spbm','kpyf'])['je'].sum().reset_index()
        detail_2_se_i=df_chain_fp_now.groupby(['spbm','kpyf'])['se'].sum().reset_index()
        detail_2_i=pd.merge(left=detail_2_je_i,right=detail_2_se_i,on=['spbm','kpyf'],how='inner')
        detail_2_i_=detail_2_i.loc[~(detail_2_i['spbm'].isin(ls_fp_code_deselect_pro))].copy()
        detail_2_i_.loc[:,'chain']=i
        detail_2_i_.loc[:,'year']=detail_2_i_['kpyf'].str[:4]
    
    detail_1_quarter=pd.concat([detail_1_quarter,df_area_market_2])
    detail_2_quarter=pd.concat([detail_2_quarter,detail_2_i_])

detail_1_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',header=None,index=False,mode='a+')
detail_2_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',header=None,index=False,mode='a+')
# ,header=None

In [19]:
# 老版
# detail_01全年
# detail_02全年

# detail_01历年产业链销售总额及增速
df_detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
df_detail_1_year=df_detail_1_quarter.groupby(['chain','year'])['je'].sum().reset_index()
ls_year=df_detail_1_year['year'].unique().tolist()
ls_chain=df_detail_1_quarter['chain'].unique().tolist()

for i in ls_chain:
    for j in range(1,len(ls_year)):
        je_now=df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'je'].tolist()[0]
        je_last=df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j-1]),'je'].tolist()[0]
        if je_last==0:
            df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'rate']=100
        else:
            df_detail_1_year.loc[(df_detail_1_year['chain']==i)&(df_detail_1_year['year']==ls_year[j]),'rate']=(je_now-je_last)*100/je_last
df_detail_1_year.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_年度.csv',index=False,mode='w')

# detail_02产业链税收及前三非煤油气主要产品销售额变化
detail_1_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链销售总额及增速_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_2_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_02_产业链税收及前三销售产品销售额变化_季度.csv',dtype={'kpyf':'str','year':'str'})

ls_date=sorted(detail_2_quarter['kpyf'].unique().tolist())[-6:]
ls_result_detail_2=[]

for i in ls_chain:
    ls_result_i=[]
    detail_2_year_=detail_2_quarter.loc[(detail_2_quarter['kpyf'].isin(ls_date))&(detail_2_quarter['chain']==i)]
    detail_2_year_1=detail_2_year_.groupby(['chain','spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    detail_2_year_2=detail_2_year_1.head(3)
    ls_spbm=detail_2_year_2['spbm'].unique().tolist()
    for j in range(1,len(ls_date)):
        
        se_i_last=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j-1])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        se_i_now=detail_1_quarter.loc[(detail_1_quarter['kpyf']==ls_date[j])&(detail_1_quarter['chain']==i),'se'].tolist()[0]
        if se_i_last==0 and se_i_now!=0:
            se_rate=100
        elif se_i_last==0 and se_i_now==0:
            se_rate=0
        else:
            se_rate=(se_i_now-se_i_last)*100/se_i_last
        ls_i=[i,ls_date[j],'chain_se_rate',se_rate]
        ls_result_detail_2.append(ls_i)
        
        for a in ls_spbm:
            df_pro_now=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            df_pro_last=detail_2_year_.loc[(detail_2_year_['kpyf']==ls_date[j-1])&(detail_2_year_['chain']==i)&(detail_2_year_['spbm']==a)]
            if len(df_pro_now)==0:
                pro_a_now=0
            else:
                pro_a_now=df_pro_now['je'].tolist()[0]
                
            if len(df_pro_last)==0:
                pro_a_last=0
            else:
                pro_a_last=df_pro_last['je'].tolist()[0]
                
            if pro_a_last==0 and pro_a_now!=0:
                pro_a_rate=100
            elif pro_a_last==0 and pro_a_now==0:
                pro_a_rate=0
            else:
                pro_a_rate=(pro_a_now-pro_a_last)*100/pro_a_last
            ls_a=[i,ls_date[j],a,pro_a_rate]
            ls_result_detail_2.append(ls_a)

df_detail_2=pd.DataFrame(ls_result_detail_2,columns=['chain','kpyf','spbm','se_rate'])
df_detail_2=df_detail_2.astype(dtype={'spbm':'str'})
df_detail_3=pd.merge(left=df_detail_2,right=df_chain_nodes2_,on=['spbm'],how='left')
df_detail_3.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',index=False,mode='w')
# ,header=None

In [36]:
# 老版
# detail_03季度销售前十生产企业（万元）
# detail_04季度销售前十经销企业（万元）
# detail_05本地产业链生产企业区县分布
deselect_pro=['原煤','液化石油气','柴油','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','热力']
ls_fp_code_deselect_pro=df_chain_nodes2.loc[df_chain_nodes2['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()

df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
detail_04_quarter=pd.DataFrame()
detail_05_quarter=pd.DataFrame()

for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp_now=df_fp_4_2_sell.loc[df_fp_4_2_sell['spbm'].isin(ls_spbm)]
    
    detail_45_quarter=df_chain_fp_now.groupby(['xf_shxydm','xf_nsrmc','xf_hydm','kpyf','xf_district'])['je'].sum().reset_index()
    detail_45_quarter.loc[:,'hydm_type']=detail_45_quarter['xf_hydm'].str[0]
    detail_45_quarter.loc[:,'year']=detail_45_quarter['kpyf'].str[:4]
    detail_45_quarter.loc[:,'chain']=i
    
    # detail_03季度销售前十生产企业（万元）,hydm=非FG
    detail_04_quarter_i=detail_45_quarter.loc[detail_45_quarter['hydm_type'].isin(['A','B','C','E','I','M','R'])]
    detail_04_quarter=pd.concat([detail_04_quarter,detail_04_quarter_i])
    # detail_04季度销售前十经销企业（万元）
    detail_05_quarter_i=detail_45_quarter.loc[detail_45_quarter['hydm_type'].isin(['F','G'])]
    detail_05_quarter=pd.concat([detail_05_quarter,detail_05_quarter_i])
    
detail_04_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_03_季度销售生产企业_季度.csv',header=None,index=False,mode='a+')
detail_05_quarter.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业_季度.csv',header=None,index=False,mode='a+')
# ,header=None

In [39]:
# 老版
# detail_03季度销售前十生产企业（万元）
# detail_04季度销售前十经销企业（万元）
# detail_05本地产业链生产企业区县分布
detail_04_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_03_季度销售生产企业_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_05_quarter=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_04_季度销售经销企业_季度.csv',dtype={'kpyf':'str','year':'str'})
detail_04_year=detail_04_quarter.groupby(['xf_shxydm','xf_nsrmc','chain','year','xf_district'])['je'].sum().reset_index()
detail_05_year=detail_05_quarter.groupby(['xf_shxydm','xf_nsrmc','chain','year','xf_district'])['je'].sum().reset_index()
latest_year=sorted(detail_04_quarter['year'].unique().tolist())[-1]
latest_quarter=sorted(detail_04_quarter['kpyf'].unique().tolist())[-1]
df_04_result=pd.DataFrame()
df_05_result=pd.DataFrame()
df_06_result=pd.DataFrame()
for i in ls_chain:
    
    # detail_03季度销售前十生产企业（万元）
    detail_04_year_1=detail_04_year.loc[(detail_04_year['year']==latest_year)&(detail_04_year['chain']==i)].sort_values(by='je',ascending=False)
    detail_04_year_2=detail_04_year_1.head(10)
    df_04_result=pd.concat([df_04_result,detail_04_year_2],axis=0)
    # 
    # detail_04季度销售前十经销企业（万元）
    detail_05_year_1=detail_05_year.loc[(detail_05_year['year']==latest_year)&(detail_05_year['chain']==i)].sort_values(by='je',ascending=False)
    detail_05_year_2=detail_05_year_1.head(10)
    df_05_result=pd.concat([df_05_result,detail_05_year_2],axis=0)
    # 
    # detail_05本地产业链生产企业区县分布
    detail_06_year_1=detail_04_year_1.groupby(['year','chain','xf_district'])['xf_shxydm'].count().reset_index()
    df_06_result=pd.concat([df_06_result,detail_06_year_1],axis=0)
    
df_06_result.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_05_本地产业链生产企业区县分布_年度.csv',index=False,mode='w')
df_05_result.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_04_季度销售经销企业_年度.csv',index=False,mode='w')
df_04_result.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_03_季度销售生产企业_年度.csv',index=False,mode='w')

In [ ]:
# 老版
# detail_08季度产业链产品交易情况_销售
# detail_09季度产品交易情况_销售
# detail_10产品交易金额变化_销售
df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()

detail_8910=pd.DataFrame()

for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp_now=df_fp_4_2_sell.loc[df_fp_4_2_sell['spbm'].isin(ls_spbm)]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        df_i=df_chain_fp_now.groupby(['xf_city','gf_city','spbm','kpyf'])['je'].sum().reset_index()
        df_i.loc[:,'chain']=i
        df_i.loc[:,'year']=df_i['kpyf'].str[:4]
        detail_8910=pd.concat([detail_8910,df_i],axis=0)
    
detail_8910.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',header=None,index=False,mode='a+')
# ,header=None

In [ ]:
# 老版
# detail_08年度产业链产品交易情况_销售
# detail_09季度产品交易情况_销售
# detail_10产品交易金额变化_销售
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_销售.csv',dtype={'kpyf':'str','year':'str'})
ls_date_12=sorted(detail_8910['kpyf'].unique().tolist())
ls_date_12_=ls_date_12[-12:]
deselect_pro=['原煤','液化石油气','柴油','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','热力']
ls_fp_code_deselect_pro=df_chain_nodes2.loc[df_chain_nodes2['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()
df_detail_3=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',dtype='str')

df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_sell=df_fp_6_2_1.loc[df_fp_6_2_1['xf_city']=='榆林市']
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()

date_now=ls_date_12[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_12[-2]
detail_08_info=pd.DataFrame()
detail_09_area=pd.DataFrame()
detail_10_info=pd.DataFrame()

for i in ls_chain:
    # detail_08年度产业链产品交易情况_销售
    df_i=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['year']==year_now)]
    df_i_chain=df_i.groupby(['spbm','chain','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    df_i_chain_top=df_i_chain.head(15).reset_index(drop=True)
    for j in range(len(df_i_chain_top)):
        spbm_j=df_i_chain_top.loc[j,'spbm']
        je_nian=df_i_chain_top.loc[j,'je']
        je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        if je_tongbi==0:
            df_i_chain_top.loc[j,'rate_tongbi']=1
        else:
            df_i_chain_top.loc[j,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi
            
        if je_huanbi==0:
            df_i_chain_top.loc[j,'rate_huanbi']=1
        else:
            df_i_chain_top.loc[j,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi
        
        df_i_chain_top.loc[j,'je_quarter']=je_now
        df_i_chain_top.loc[j,'je_tu']=je_nian-je_now
        
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top])
    
    # detail_09季度产品交易情况_销售
    df_i_chain_9=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(~detail_8910['spbm'].isin(ls_fp_code_deselect_pro))]
    df_i_chain_9_1=df_i_chain_9.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(3)
    ls_spbm_top_3=df_i_chain_9_1['spbm'].unique().tolist()
    ls_spbm_top_3_=df_detail_3.loc[(df_detail_3['chain']==i)&(detail_8910['kpyf']==date_now)]['spbm'].tolist()
    ls_spbm_top_3_=[x for x in ls_spbm_top_3_ if x!='chain_se_rate']
    
    for a in ls_spbm_top_3:
        df_a=df_i_chain_9.loc[df_i_chain_9['spbm']==a]
        df_a_top5=df_a.groupby(['chain','spbm','gf_city','kpyf','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(5)
        detail_09_area=pd.concat([detail_09_area,df_a_top5])
        
        # detail_10产品交易金额变化_销售
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_12_))].reset_index(drop=True)
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index()
        for b in range(len(df_a_10_1)):
            je_now=df_a_10_1.loc[b,'je']
            date_now_b=df_a_10_1.loc[b,'kpyf']
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            rate_tongbi=(je_now-je_last_year)/je_last_year
            date_last_quarter=ls_date_12[ls_date_12.index(date_now_b)-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            rate_huanbi=(je_now-je_last_quarter)/je_last_quarter
            df_a_10_1.loc[b,'je_last_year']=je_last_year
            df_a_10_1.loc[b,'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[b,'rate_tongbi']=rate_tongbi
        detail_10_info=pd.concat([detail_10_info,df_a_10_1])

detail_08_info.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_销售.csv',index=False,mode='w')
detail_09_area.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_销售.csv',index=False,mode='w')
detail_10_info.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_销售.csv',index=False,mode='w')

In [96]:
# 老版
# detail_08年度产业链产品交易情况_购买
# detail_09季度产品交易情况_购买
# detail_10产品交易金额变化_购买
df_fp_6_2_1=df_fp_6_2_.copy()
df_fp_4_2_buy=df_fp_6_2_1.loc[df_fp_6_2_1['gf_city']=='榆林市']
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()

detail_8910=pd.DataFrame()

for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp_now=df_fp_4_2_buy.loc[df_fp_4_2_buy['spbm'].isin(ls_spbm)]
    if len(df_chain_fp_now)==0:
        print(i)
        continue
    else:
        df_i=df_chain_fp_now.groupby(['xf_city','gf_city','spbm','kpyf'])['je'].sum().reset_index()
        df_i.loc[:,'chain']=i
        df_i.loc[:,'year']=df_i['kpyf'].str[:4]
        detail_8910=pd.concat([detail_8910,df_i],axis=0)
    
detail_8910.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',header=None,index=False,mode='a+')
# ,header=None

现代煤化工产业链
文化旅游产业链


In [130]:
# 老版
# detail_08年度产业链产品交易情况_购买
# detail_09季度产品交易情况_购买
# detail_10产品交易金额变化_购买
detail_8910=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_8910_产业链产品交易情况_季度_购买.csv',dtype={'kpyf':'str','year':'str'})
ls_date_12=sorted(detail_8910['kpyf'].unique().tolist())
ls_date_12_=ls_date_12[-12:]
deselect_pro=['原煤','液化石油气','柴油','燃料油','汽油','发电及供电量','洗煤','售电量','天然气','居民服务','热力']
ls_fp_code_deselect_pro=df_chain_nodes2.loc[df_chain_nodes2['NAME'].isin(deselect_pro)]['spbm'].unique().tolist()
df_detail_3=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\'+'detail_02_'+'产业链税收及前三销售产品销售额变化_年度.csv',dtype='str')

df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()

date_now=ls_date_12[-1]
year_now=date_now[:4]
date_tongbi=str(int(year_now)-1)+date_now[-2:]
date_huanbi=ls_date_12[-2]
detail_08_info=pd.DataFrame()
detail_09_area=pd.DataFrame()
detail_10_info=pd.DataFrame()

for i in ls_chain:
    # detail_08年度产业链产品交易情况_销售
    df_i=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['year']==year_now)]
    df_i_chain=df_i.groupby(['spbm','chain','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    df_i_chain_top=df_i_chain.head(15).reset_index(drop=True)
    for j in range(len(df_i_chain_top)):
        spbm_j=df_i_chain_top.loc[j,'spbm']
        je_nian=df_i_chain_top.loc[j,'je']
        je_now=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_tongbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_tongbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        je_huanbi=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_huanbi)&(detail_8910['spbm']==spbm_j),'je'].tolist())
        if je_tongbi==0:
            df_i_chain_top.loc[j,'rate_tongbi']=1
        else:
            df_i_chain_top.loc[j,'rate_tongbi']=(je_now-je_tongbi)/je_tongbi
            
        if je_huanbi==0:
            df_i_chain_top.loc[j,'rate_huanbi']=1
        else:
            df_i_chain_top.loc[j,'rate_huanbi']=(je_now-je_huanbi)/je_huanbi

        df_i_chain_top.loc[j,'je_quarter']=je_now
        df_i_chain_top.loc[j,'je_tu']=je_nian-je_now
        
    detail_08_info=pd.concat([detail_08_info,df_i_chain_top])
    
    # detail_09季度产品交易情况_销售
    df_i_chain_9=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['kpyf']==date_now)&(~detail_8910['spbm'].isin(ls_fp_code_deselect_pro))]
    df_i_chain_9_1=df_i_chain_9.groupby(['spbm'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(3)
    ls_spbm_top_3=df_i_chain_9_1['spbm'].unique().tolist()
    ls_spbm_top_3_=df_detail_3.loc[(df_detail_3['chain']==i)&(detail_8910['kpyf']==date_now)]['spbm'].tolist()
    ls_spbm_top_3_=[x for x in ls_spbm_top_3_ if x!='chain_se_rate']
    
    for a in ls_spbm_top_3:
        df_a=df_i_chain_9.loc[df_i_chain_9['spbm']==a]
        df_a_top5=df_a.groupby(['chain','spbm','gf_city','kpyf','year'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(5)
        detail_09_area=pd.concat([detail_09_area,df_a_top5])
        
        # detail_10产品交易金额变化_销售
        df_a_10=detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf'].isin(ls_date_12_))]
        df_a_10_1=df_a_10.groupby(['chain','spbm','kpyf'])['je'].sum().reset_index().sort_values(by='kpyf',ascending=True)
        for b in range(len(df_a_10_1)):
            je_now=df_a_10_1.loc[b,'je']
            date_now_b=df_a_10_1.loc[b,'kpyf']
            date_last_year=str(int(date_now_b[:4])-1)+date_now_b[-2:]
            je_last_year=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_year)]['je'].tolist())
            if je_last_year==0:
                rate_tongbi=1
            else:
                rate_tongbi=(je_now-je_last_year)/je_last_year
                
            date_last_quarter=ls_date_12[ls_date_12.index(date_now_b)-1]
            je_last_quarter=sum(detail_8910.loc[(detail_8910['chain']==i)&(detail_8910['spbm']==a)&(detail_8910['kpyf']==date_last_quarter)]['je'].tolist())
            if je_last_quarter==0:
                rate_huanbi=1
            else:
                rate_huanbi=(je_now-je_last_quarter)/je_last_quarter
            df_a_10_1.loc[b,'je_last_year']=je_last_year
            df_a_10_1.loc[b,'rate_huanbi']=rate_huanbi
            df_a_10_1.loc[b,'rate_tongbi']=rate_tongbi
        detail_10_info=pd.concat([detail_10_info,df_a_10_1])

detail_08_info.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_08_产业链产品交易情况_年度_购买.csv',index=False,mode='w')
detail_09_area.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_09_产品交易情况_购买.csv',index=False,mode='w')
detail_10_info.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_10_产品交易金额变化_购买.csv',index=False,mode='w')

# 税务数据中的企业购销产业链产品金额汇总

In [11]:
"','".join(ls_chain_core_ent)

"9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70

In [68]:
df_chain_pro=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\十条产业链的核心产品.xlsx',header=None,dtype='str',sheet_name='chain_pro')
ls_1_ch=df_chain_pro[0].unique().tolist()
for i in ls_1_ch:
    ls_i=df_chain_pro.loc[df_chain_pro[0]==i][1].unique().tolist()
    print(i,len(ls_i),ls_i)

“碳中和”产业链 80 ['1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000'

In [71]:
df_xf=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype='str',sheet_name='2023xfsp')
df_gf=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype='str',sheet_name='2023gfsp')
df_xf_chain1=pd.merge(left=df_xf,right=df_chain_core_ent_c,left_on=['xf_shxydm'],right_on=['统一社会信用代码'],how='left')
df_gf_chain1=pd.merge(left=df_gf,right=df_chain_core_ent_c,left_on=['gf_shxydm'],right_on=['统一社会信用代码'],how='left')

In [73]:
df_xf_chain1=pd.merge(left=df_xf,right=df_chain_core_ent_c,left_on=['xf_shxydm'],right_on=['统一社会信用代码'],how='left')
df_gf_chain1=pd.merge(left=df_gf,right=df_chain_core_ent_c,left_on=['gf_shxydm'],right_on=['统一社会信用代码'],how='left')

In [74]:
df_gf_chain1

,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,kpyf,spbm,sum(je),组织机构代码,数据处理地,单位详细名称,报表类别,主要业务活动(或主要产品)1,统一社会信用代码,chain
0,916100000504386259,府谷县金宏湾矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,202301,1070101010100000000,676.99,050438625,610822105204,府谷县金宏湾矿业有限公司,B,采矿业烟煤和无烟煤开采洗选,916100000504386259,现代煤化工产业链
1,916100000504386259,府谷县金宏湾矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,202301,1070208010000000000,118.81,050438625,610822105204,府谷县金宏湾矿业有限公司,B,采矿业烟煤和无烟煤开采洗选,916100000504386259,现代煤化工产业链
2,916100000504386259,府谷县金宏湾矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,202301,1090407020000000000,297.03,050438625,610822105204,府谷县金宏湾矿业有限公司,B,采矿业烟煤和无烟煤开采洗选,916100000504386259,现代煤化工产业链
3,916100000504386259,府谷县金宏湾矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,202301,3070402000000000000,187.13,050438625,610822105204,府谷县金宏湾矿业有限公司,B,采矿业烟煤和无烟煤开采洗选,916100000504386259,现代煤化工产业链
4,916100000504386259,府谷县金宏湾矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,202302,1040201250000000000,19306.93,050438625,610822105204,府谷县金宏湾矿业有限公司,B,采矿业烟煤和无烟煤开采洗选,916100000504386259,现代煤化工产业链
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86023,93610828052138271M,佳县红润枣业专业合作社,F5115,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局金明寺税务分局,202303,3050100000000000000,-396039.6,052138271,610828001240,佳县红润枣业专业合作社,E,红枣零售,93610828052138271M,现代特色农业产业链
86024,93610828052138271M,佳县红润枣业专业合作社,F5115,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局金明寺税务分局,202303,3070401000000000000,177.23,052138271,610828001240,佳县红润枣业专业合作社,E,红枣零售,93610828052138271M,现代特色农业产业链
86025,93610831305343143G,子洲县秦北中药材农民专业合作社,A0171,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局子洲县税务局淮宁湾税务分局,202303,1070205010000000000,69200,305343143,610831200200,子洲县秦北中药材农民专业合作社,E,中药材批发,93610831305343143G,现代特色农业产业链
86026,93610831562243709Y,子洲县永望农产品销售农民专业合作社,F5223,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局子洲县税务局双湖峪税务分局,202303,1070204070000000000,0,562243709,610831104243,子洲县永望农产品销售农民专业合作社,E,中药材批发,93610831562243709Y,现代特色农业产业链


In [107]:
ls=['91610824MA708T379E','91610800223930221M','91610800MA70319P01','91610824MA7045KE7K','91610800552165727D','9161089357352467X6','91610893713586830T','91610821681569381A','91610828MA7035J19Q','91610821552174447C','916108303522676814','91610800MA70DD196U','93610802MA703FR92F','91610802061927221J','91610827MA7040FP1T','91610822224032386A','9161082405479693X8','91610800766331292B','91610893735373473F','91610800223931531Y','91610800698417755J','91610893MA70C3XL5R','91610893567149850M','91610821071285285M','916108243055203063','91610802081747367D','91610800794100846T','91610831593341926R','91610800661152976L','916108226679507926','91610893MA703AHQ4K','91610827MA7032FT4X','91610800776976043J','916108210648495992','91610800677943334K','91610824MA703BB57G','91610802MA7031R94N','91610821224000157T','91610822559385992C','91610000755214488A','916108226679923350','91610824MA703AAQXB','916108315706692914','91610800567137999E','916100006984015773','91610000694922009G','91610827061945163F','91610806MA70ADRH09','9161000055219304XB','91610824MA70A3CU40','91610802MA7055P507','91610821583543612D','91610802MA703BWC8F','91610821MA70BKG73Y','9161082130538851XE','916108217353454137','91610802MA704Y7M9M','91610000752131054Q','916108280504343257','91610800675112705T','91610821MA70978GX5','9161000056714796XP','9161000056377267XH','91610821677921872D','9161000056710680XM','916108005521962839','91610000924003893Q','91610821748622598U','91610822MA7097U17U','91610000681575597D','916100005556789748','91610821MA703J6063','91610800MA70988L3J','91610831MA709N6L13','91610825305721597M','91610821675105396U','916108215637902455','91610823555664310J','91610821305384658A','9161000071978482XF','916108226715375708','91610821338638431X','916108007588160388','91610000059683299M','91610000677916061H','91610000755225347C','91610822687983735A','91610822667970734M','91610821786998975C','91610000755202794R','91610827MA705CXF2K','91610821054788032W','91610000773808634G','91610800223930635R','91610823MA70CG8N8F','916108216847887231','916108277836898514','9161082167510000XL','91610821078617967F','91610822694938211C','91610800555695483W','91610821MA7098KX87','91610821786996363L','91610822786997948R','91610802667987720K','916108230880722358','91610000681575589J','91610800694936013W','91610800MA7036T07K','91610802773841215D','916100007100518818','91610823MA703ELY6H','91610822MA703BPQ8E','91610821770043971N','916108217552325636','916100005556826316','91610800MA70F0X441','916100006751366436','9161082108806100X8','91610822677931237W','91610000562225156P','91610000567118552C','91610000MA6TG40X28','91610824675115447J','91610000758832169D','9161082176632501XX','9161000071357096XA','916100000596527830','91610802573522607J','91610893MA70BUNR99','916108313056661962','91610822748616366T','91610821305337210M','91610800677914736G','9161082271355816XC','916100002240051930','91610000710052833G','91610824667959658W','91610806586970396Q','916108226911259879','91610800752129712H','91610821675144336D','91610000559374302W','91610822MA7059MH04','91610800776976764F','9161000071350425X8','916108215637906599','91610822735347005J','916108003054001360','91610000719708587F','91610821MA703BX57B','91610821675145005D','916108215735218745','916108253055693618','916100007769886696','91610821305496043G','91610822797925923U','91610000691125485H','91610821552195029G','9161082257351819X6','91610821MA70EU1M4C','91610000305340444Y','91610821067914579R','91610821684770443G','9161080266116183XF','91610821694911481E','91610000691106065G','91610821MA709DR57H','91610821758816740M','91610000X23586219E','91610828MA709TR85H','9161082158075123X8','91610821MA703AYG0F','91610800059688380U','916100006847787282','91610831MABN470E3F','91610825338670220D','91610821MA70CNJ41E','91610800661157478J','91610822667973185H','916100000712732418','91610829MA709G5C21','91610822MA7036PA3C','91610821664110536C','91610822MA70DY9C9C','91610822078636033K','9161080276255849XM','9161000077382866XG','916100007099182651','91610821691135173P','91610821562228197B','916100000880671537','91610822MA708N4B1M','91610802687968340T','916108003057434031','91610821064821756N','91610822078635727F','916108225593913130','9161082271355912XF','916108007869525471','91610821MA703YBK2D','91610821MA703BEC31','91610821074501043F','91610824583543516F','91610822677911949C','9161000075884416XH','91610802223962506N','91610821MA70950B2G','91610821684784298E','916100006847510073','91610824MA7031PR97','916108006911278008','91610821MA7033LY65','916108216879865066','91610822694929032M','9161082155216971XT','91610821MA7037WA08','91610800MA708M1D63','91610821563791942T','91610821555689542X','916100002239672519','91610821MA70DCFF18','91610800598779185W','91610821MA703GB77B','916100009240134699','91610821MA6TGWKM6C','91610802684771219B','91610000567138887F','91610104MA6U6WPP9U','916108257869852184','91610800MA703RJE5P','916100006847780075','91610000567129825J','91610823MA7032KGXX','91610806735348921D','91610821675117004H','91610822555655537N','91610822MA70DF1676','916100005521646440','91610821MA70DKG314','91610000773830604T','91610802MA704D189B','9161000071005330XQ','91610000224061857H','9161000005968331X1','91610821563789930M','91610822087963650K','916100005593700612','91610000577835002Q','91610824338721649N','91610821305529570Q','91610000224057049W','91610821MA70CG834J','91610893MA7056GH49','91610000664134933F','91610893MA7030276T','91610821596655947E','91610821684790305C','91610822671520461A','91610000698436454D','91610822MA708KQDXA','9161082156710197X5','916100005622201527','916108215807826858','91610806MA703A6H0H','9161082266796860XC','916100007135040226','91610893MA70C8BN5R','91610822737993836U','91610821050402075G','916108220745134073','91610000580775995H','91610800305315580J','91610821MA70EC2YXC','9161000092400161X7','91610821677948768B','916108217869620404','91610800593317221R','91610000593327534K','91610000710055196X','9161082269490584XB','91610821562219223W','916108216751414422','916100007552218217','91610821691126138J','91610000779910293J','91610000577813946E','91610800MA703EW32C','91610821MA7034TW33','9161082155566904XM','916108216911367669','91610821583535073B','9161000075521877X8','91610821555695272T','91610802MA705D6C9K','91610830555673671H','91610821MA70D3PF0H','916108226847965391','916108230882445003','916108217552057845','91610000924044062B','91610822MA704CN193','91610800H13100629H','91610821MA70362218','916108212240045457','91610000766346072G','91610822552168012N','91610000567136590B','91610000710052665E','91610000567138334L','91610822586971540Q','916108215869581355','91610802305755922F','91610821770017642B','916108065869736343','91610800MA703DR38N','91610822766320980Q','91610821MA703LQGXF','91610821677948098N','91610821559365318N','91610821338596382X','91610821305412030B','91610821677923560K','91610806MA70391C58','91610802MA703BFH02','91610823MA7030E21X','916108303385722175','916100006879777738','91610821790779716D','916108215521580763','9161082630547280X6','916100005556550942','91610821719766197G','91610825088196684W','916108255521915890','916100006984339153','91610000758819036E','91610826MA70333X3J','91610000X23586251R','916100005556688484','91610821570668862F','91610821MA7094W97M','91610823MA708U9U8T','91610821MA703TKP7H','91610800691127712R','916108212240035186','91610000577813989U','9161080072737250XT','916108212240019344','916108226949327095','91610822694914308A','91610000773800042F','91610821580772153T','916108216715160756','91610824305379402Y','916108006751155433','91610821MA70C2Q06B','916100007135061593','91610800064810491X','91610821755222453P','916108225835113037','91610824555662817L','91610822567149623M','916108007450080310','916108006751043185','91610821766336843W','91610800677948717M','91610822MA704K1D9T','916108216779105329','91610822MA703A0A5W','916108000679086973','916108215556868517','91610822050401881R','916108210712746156','91610802MA7038KK91','91610821684773951D','91610821755200326A','91610821MA70AY2U52','91610821305529183C','91610000748646493C','91610802MA70A6FN4M','91610822710053561Q','916108217625768311','91610822563762148E','91610800088193408Q','9161082255566867X2','91610821677922285W','916100007799482405','91610826MA70CG498N','91610000732641997R','91610000223967235K','91610822MA7030C7X0','91610821MA709HTD69','91610821MA704NDFX6','91610821MA708P1D4Q','916100006984084291','9161082859666136XJ','91610800MA7036C41M','916108232240609348','91610800061931191K','91610800694930906W','91610800691145662A','916108215966606585','91610828MA7030X571','91610821071269955P','91610821794117621M','91610000790751174B','916108227663422157','916108253057555513','91610000563779951C','916100005556584045','91610800748646530U','9161082173534658XU','916100007100533691','91610000710051814A','916100000521118534','916108250817158759','91610825687982230J','91610823MA7030FM17','91610800770011101Y','916108216679832524','916108003385989430','91610821677901740Y','91610000797940752N','91610821671537060C','91610821MA709MTL5A','91610000MA6TG1L91C','916108226779018201','916108006679710248','916108215593687385','916108215807683576','91610821MA703BE64X','91610800078619321P','916108215835267406','91610823755214998B','9161082569842068XN','91610824305645491Q','91610821586998516U','91610000745041552L','91610000567129809W','916108006641132516','9161082155568221XT','916108225671304988','91610822MA709EAN9R','916100007769889684','91610000224003921H','9161082179790070XW','91610800575649337Q','916100007273509845','916108222240334186','916100007799307022','91610822762574940L','91610822567124282E','91610000681575570M','916108250859923274','91610822671525115L','91610893MA70AKUT7Y','916108225778382625','916108225637837070','91610822735351493Q','9161082258078631XM','91610000059666958C','916108216847868401','91610821MA70DTWN1E','91610822684783682Q','91610822305318853B','91610821MA703FB4X5','91610000681597913D','91610824352260279W','916108216751367583','916108066879758908','916100005556762074','91610000687954512E','91610824577821241J','916100002240321328','91610821567141533Q','91610800684795675B','91610000X23601087L','91610821MA704UUN2D','91610000687954301Y','916108213055523324','91610000223965838G','91610800598796380J','91610824580788446J','91610822583526636G','91610000732652506M','91610822MA709N423A','91610822MA709Y7E08','916108270596677901','91610822071284629R','91610821567114877Q','91610821MA70AFDX8F','91610000598765891B','91610000671527778F','91610823MA70D01L9W','916108215593819240','9161082167793717XC','91610824093913721Q','916108225556745861','9161082155566058X5','91610800552158025D','916108937869808637','916108225869772994','9161000071005277XF','91610823MA70D1G21X','91610827338623053A','91610000745037668H','91610824305602280L','916108020569016092','91610000059676961F','91610823056903460W','91610821770025546B','91610821MA705B6C33','91610821MA70DT2L42','91610806078612699W','916108020547928919','91610000924001302H','91610800223930176L','91610821305323134R','916108226911466229','916100006815955200','91610822580776779W','91610822586987462E','916100006984246809','91610000577805794Q','916108310619449025','91610800MA70AE5B4E','91610000748616526W','91610000766333263G','91610823MA70GABW64','91610821MA7050AM0A','91610822773826787T','91610822555680564K','9161082158697432XC','91610822MA70A6HJ4U','91610821583541000M','91610806305365529M','916100006879524029','91610800730410562R','91610822766330636W','91610000710924961H','91610824MA70DAAF42','91610821671501594M','91610821783666617B','91610800596685070J','91610802577836419K','91610800MA70371J7F','91610800223959534H','91610821338575709B','91610822555666885M','91610806586963508Y','91610802691100835M','916108006815878706','916108000679234595','91610000713558602R','91610822MA708YJX74','916108006984228963','91610821MA7035W466','91610800088079154R','91610821691147676K','91610828MA703PB208','91610821MA708UWL0R','91610800MA703QW35X','91610826MA7033UK93','91610821MA7098LA63','916108213057510246','9161080066117645XR','91610821664127514E','91610821MA703PT760','91610822MA708Y049A','9161080230559847XB','91610806691107682J','91610000786958834R','91610802552178923N','91610825770016498A','91610821MA703QFH1K','916108295933288522','91610826064845432U','9161082830551626X8','91610824MA70404L9N','91610821681590201J','9161082256224308XJ','91610822MA709AGT5N','91610822797911572J','91610822MA70384L5F','91610821776988992B','91610800056930530N','91610822575627197H','916108006611675026','916100000817239128','91610822552179029C','91610825305646531X','916108216911304797','91610000577827678B','91610822667965572L','91610822MA7093A6XP','91610821675145697Y','916108220569493036','91610823MA703DDX4B','91610800735340321T','91610822552193103H','91610893MA70AULF26','91610821MA708FCP5T','91610822563764290Y','916108225869795931','91610821MA70A5UY7L','91610828MA703JJY7E','91610821563795767X','91610823671529749H','91610824570657119L','91610802078614125H','91610000684797582J','91610827552179360X','91610821MA70ERP17F','91610824052125550G','91610823305604112X','91610822681555895X','91610800790767520W','91610821305769822Y','916108217100521205','91610000580766765J','9161082130559046X1','916108217799382026','916108023386836119','916108253057807499','91610800677905610M','916108237907734373','916108217700423972','91610806067948461P','91610000797915039F','91610806MA7096L418','91610000797938927G','916108250881238057','916108222240302172','916108253055136327','91610821MA70D3P75Q','91610821681582199M','9161082257069098XN','91610821305712199T','9161082167792431X4','91610822056917184G','91610822MA70CYYF7Q','916108935521883122','91610000710052067J','91610822MA7032CD47','91610800MA7036E79W','91610000586982709M','91610806755247976C','916108233054100434','916108216911169173','91610821338655688X','91610824338748948N','91610000X23606048P','91610821MA703QH21F','91610821573521946X','916108225987548772','916108257941124337','91610821675114583D','91610000681599492L','91610822573535061G','91610823MA7035CD25','916100005671063088','916108216911087145','916100009240011505','916108216847628063','91610821586985766Y','91610821MA70C2PMXJ','91610822093914935U','916108220648211664','91610821MA703316XP','916108225593843242','93610830MA703AM46D','91610821MA709UHQ1A','916100006815997672','91610821681588961J','91610822MA70F1FP48','91610824586989142N','916108227486349192','91610821555656230Q','91610822755225080Y','916108243054516389','91610821MA70BFPF2Y','91610822752112793E','91610822224030882U','91610821MA70A9ND11','91610000710054484G','91610821061948145J','91610824MA70D7CY47','916108215556937286','916108216879635742','91610821064803785M','916108216815903409','916108227135586612','91610800590253023H','916108227450323733','91610822577800053T','91610821MA708QC6XL','91610822687992703H','91610821MA70B7PF0Y','91610830305391161C','916108007273474788','91610822MA704G881Y','91610800MA709M13XP','916108006984411145','91610828352252148H','91610802050400635F','916108006641075481','91610822224032482K','91610821577821532A','91610822681564126J','9161080271004864XG','916108007979285936','91610830710064009G','91610830MA7095T23Y','91610821555699740T','91610821713506992M','91610824093912366E','91610821694922041J','916100002240314638','916108210817431714','91610821305453131A','91610825305557133R','91610829MA704APE27','9161000059027254XF','91610800MA70AXHK6D','916108237941275554','91610821305664764W','91610821698418344C','91610824MA7043WQ6F','91610000735388763B','916108225521605984','91610802MA703WM89H','916108246949093223','91610821074539041A','91610000687952218H','91610000741268506A','91610800755204183B','91610000773844942E','91610823MA709HYNXD','91610822596695017M','91610821L047817240','91610827MA7094AT02','916108225735291204','91610000730404314B','93610831562243709Y','916108003054061837','91610824559383436L','916108225521576557','916108213055993173','916108255902738171','91610824713561318K','91610800061909963N','91610800691107981R','916100006911035934','91610822224033581D','91610821MA70CG4B44','91610821MA70BJ9W0C','916108225807676532','91610821MA703A1W1A','91610824074522522Q','91610829786975896A','91610827MA70FK575C','91610827L04797814E','91610822694908266R','91610800694921348L','916108213053142541','91610825MA7033R222','91610800MA70B34385','91610821677912765F','91610822567139548R','91610827MA703CCPXF','916108255807949859','91610806MA703PKD8X','91610802MA70BMX34M','91610800675123180G','916108020880671880','916108933057186623','91610824590274190L','91610821338609446E','91610825MA703C1J05','91610821681551456T','91610824MA7045E8X5','91610821570659229E','91610822081718611M','916108257100585227','916108250939154365','91610822593312017P','91610800719785144W','91610802MA70CCRJ58','91610822MA708MR72L','91610800MA708L7213','91610800671529239R','91610822MA708UM64E','9161082806191181XW','916108000880800750','91610821MA708GH3X4','9161082530538018XY','91610821664106588B','91610821MA70A5L79B','91610800MA7096937L','91610821081708229J','91610822MA70317P8A','91610800MA7035TJX3','91610802MA70AWAY3D','91610821698418854R','91610822MA7040Q74E','91610825MA703D773C','91610000224030946L','91610831MA70323L26','91610802MA703J1L6T','91610802563788284P','916108213057842808','91610821MA7094M77U','91610822MA704D4D9G','91610822562224372N','91610821352278970Y','91610821MA70DKQ35X','91610821MA704G7F25','91610821338609534W','91610825MA70A9HP1Q','91610821555684127G','9161082230534244X0','916108215556718796','91610800067912397E','916108280679278399','91610802MA70ATEH12','91610831305767755N','91610800590260549L','9161082230539678XQ','91610828MA703GAP8E','91610824059660193A','916108220786281646','916108226715094065','91610825MA703BP13J','91610826MA703MQK5P','91610822MA703HB02A','91610800752146344X','91610824MA704D9D0X','91610802MA7034XX71','91610000770001587F','91610821694924442T','91610822094023050T','91610821305767771W','91610824MA709B7K7N','916108213385789858','91610825MA7031MW0H','9161082269494526XA','91610823MA7030WJ5A','91610800061935707U','916108060547617531','916108230521323633','916108216879709774','91610824MA70EFL6XT','916108253054304235','916100006911194994','91610821586969627M','91610800MA708G9G5C','916108213056545585','9161082256712766XC','91610822766305764C','91610825305710716N','9161082175884213XN','91610802338661033H','91610821766308324Q','91610000924013143K','916108222240319575','9161080006481082X4','91610826MA70FQTC5E','91610824067940013A','91610802MA7038GD6G','91610821338639979E','91610802MA703HWE04','91610800436676327U','91610822MA70BE3W9P','91610831MA708R007W','91610828MA7032YP28','91610822MA708J3C27','91610800710051400F','91610821694922463H','91610000064835373N','91610823MA70341H2K','916108223056392271','916108223057890497','91610823MA70G73N3H','91610822MA7BK9K43D','916108250648189605','916108307303912262','91610825MA70318W1T','91610800691131228D','91610821762578722U','91610831710065220P','91610822MA703WCN8Q','91610824MA708NC530','91610823305726101A','91610821MA70FXBP6K','91610824590270069T','91610823MA704R7J5F','916108225902933593','91610800694918499D','91610822552154948T','916108226879560162','91610822552163879U','916108006984024304','91610821MA70381K88','91610824059693913C','91610000567118544H','91610824684776511K','91610821MA70CRQ8X1','91610821MA703FP576','916108060786458026','91610822563772274J','91610800681582586L','91610825MA7097R42F','91610822MA70FGBR66','91610802MA703F8M84','91610800748618329J','916108930882378452','91610826583507806A','91610821671537108L','91610000305329931J','916108230881953581']

In [ ]:
'91610824MA708T379E', '91610800223930221M', '91610800MA70319P01', '91610824MA7045KE7K', '91610800552165727D', '9161089357352467X6', '91610893713586830T', '91610821681569381A', '91610828MA7035J19Q', '91610821552174447C', '916108303522676814', '91610800MA70DD196U', '93610802MA703FR92F', '91610802061927221J', '91610827MA7040FP1T', '91610822224032386A', '9161082405479693X8', '91610800766331292B', '91610893735373473F', '91610800223931531Y', '91610800698417755J', '91610893MA70C3XL5R', '91610893567149850M', '91610821071285285M', '916108243055203063', '91610802081747367D', '91610800794100846T', '91610831593341926R', '91610800661152976L', '916108226679507926', '91610893MA703AHQ4K', '91610827MA7032FT4X', '91610800776976043J', '916108210648495992', '91610800677943334K', '91610824MA703BB57G', '91610802MA7031R94N', '91610821224000157T', '91610822559385992C', '91610000755214488A', '916108226679923350', '91610824MA703AAQXB', '916108315706692914', '91610800567137999E', '916100006984015773', '91610000694922009G', '91610827061945163F', '91610806MA70ADRH09', '9161000055219304XB', '91610824MA70A3CU40', '91610802MA7055P507', '91610821583543612D', '91610802MA703BWC8F', '91610821MA70BKG73Y', '9161082130538851XE', '916108217353454137', '91610802MA704Y7M9M', '91610000752131054Q', '916108280504343257', '91610800675112705T', '91610821MA70978GX5', '9161000056714796XP', '9161000056377267XH', '91610821677921872D', '9161000056710680XM', '916108005521962839', '91610000924003893Q', '91610821748622598U', '91610822MA7097U17U', '91610000681575597D', '916100005556789748', '91610821MA703J6063', '91610800MA70988L3J', '91610831MA709N6L13', '91610825305721597M', '91610821675105396U', '916108215637902455', '91610823555664310J', '91610821305384658A', '9161000071978482XF', '916108226715375708', '91610821338638431X', '916108007588160388', '91610000059683299M', '91610000677916061H', '91610000755225347C', '91610822687983735A', '91610822667970734M', '91610821786998975C', '91610000755202794R', '91610827MA705CXF2K', '91610821054788032W', '91610000773808634G', '91610800223930635R', '91610823MA70CG8N8F', '916108216847887231', '916108277836898514', '9161082167510000XL', '91610821078617967F', '91610822694938211C', '91610800555695483W', '91610821MA7098KX87', '91610821786996363L', '91610822786997948R', '91610802667987720K', '916108230880722358', '91610000681575589J', '91610800694936013W', '91610800MA7036T07K', '91610802773841215D', '916100007100518818', '91610823MA703ELY6H', '91610822MA703BPQ8E', '91610821770043971N', '916108217552325636', '916100005556826316', '91610800MA70F0X441', '916100006751366436', '9161082108806100X8', '91610822677931237W', '91610000562225156P', '91610000567118552C', '91610000MA6TG40X28', '91610824675115447J', '91610000758832169D', '9161082176632501XX', '9161000071357096XA', '916100000596527830', '91610802573522607J', '91610893MA70BUNR99', '916108313056661962', '91610822748616366T', '91610821305337210M', '91610800677914736G', '9161082271355816XC', '916100002240051930', '91610000710052833G', '91610824667959658W', '91610806586970396Q', '916108226911259879', '91610800752129712H', '91610821675144336D', '91610000559374302W', '91610822MA7059MH04', '91610800776976764F', '9161000071350425X8', '916108215637906599', '91610822735347005J', '916108003054001360', '91610000719708587F', '91610821MA703BX57B', '91610821675145005D', '916108215735218745', '916108253055693618', '916100007769886696', '91610821305496043G', '91610822797925923U', '91610000691125485H', '91610821552195029G', '9161082257351819X6', '91610821MA70EU1M4C', '91610000305340444Y', '91610821067914579R', '91610821684770443G', '9161080266116183XF', '91610821694911481E', '91610000691106065G', '91610821MA709DR57H', '91610821758816740M', '91610000X23586219E', '91610828MA709TR85H', '9161082158075123X8', '91610821MA703AYG0F', '91610800059688380U', '916100006847787282', '91610831MABN470E3F', '91610825338670220D', '91610821MA70CNJ41E', '91610800661157478J', '91610822667973185H', '916100000712732418', '91610829MA709G5C21', '91610822MA7036PA3C', '91610821664110536C', '91610822MA70DY9C9C', '91610822078636033K', '9161080276255849XM', '9161000077382866XG', '916100007099182651', '91610821691135173P', '91610821562228197B', '916100000880671537', '91610822MA708N4B1M', '91610802687968340T', '916108003057434031', '91610821064821756N', '91610822078635727F', '916108225593913130', '9161082271355912XF', '916108007869525471', '91610821MA703YBK2D', '91610821MA703BEC31', '91610821074501043F', '91610824583543516F', '91610822677911949C', '9161000075884416XH', '91610802223962506N', '91610821MA70950B2G', '91610821684784298E', '916100006847510073', '91610824MA7031PR97', '916108006911278008', '91610821MA7033LY65', '916108216879865066', '91610822694929032M', '9161082155216971XT', '91610821MA7037WA08', '91610800MA708M1D63', '91610821563791942T', '91610821555689542X', '916100002239672519', '91610821MA70DCFF18', '91610800598779185W', '91610821MA703GB77B', '916100009240134699', '91610821MA6TGWKM6C', '91610802684771219B', '91610000567138887F', '91610104MA6U6WPP9U', '916108257869852184', '91610800MA703RJE5P', '916100006847780075', '91610000567129825J', '91610823MA7032KGXX', '91610806735348921D', '91610821675117004H', '91610822555655537N', '91610822MA70DF1676', '916100005521646440', '91610821MA70DKG314', '91610000773830604T', '91610802MA704D189B', '9161000071005330XQ', '91610000224061857H', '9161000005968331X1', '91610821563789930M', '91610822087963650K', '916100005593700612', '91610000577835002Q', '91610824338721649N', '91610821305529570Q', '91610000224057049W', '91610821MA70CG834J', '91610893MA7056GH49', '91610000664134933F', '91610893MA7030276T', '91610821596655947E', '91610821684790305C', '91610822671520461A', '91610000698436454D', '91610822MA708KQDXA', '9161082156710197X5', '916100005622201527', '916108215807826858', '91610806MA703A6H0H', '9161082266796860XC', '916100007135040226', '91610893MA70C8BN5R', '91610822737993836U', '91610821050402075G', '916108220745134073', '91610000580775995H', '91610800305315580J', '91610821MA70EC2YXC', '9161000092400161X7', '91610821677948768B', '916108217869620404', '91610800593317221R', '91610000593327534K', '91610000710055196X', '9161082269490584XB', '91610821562219223W', '916108216751414422', '916100007552218217', '91610821691126138J', '91610000779910293J', '91610000577813946E', '91610800MA703EW32C', '91610821MA7034TW33', '9161082155566904XM', '916108216911367669', '91610821583535073B', '9161000075521877X8', '91610821555695272T', '91610802MA705D6C9K', '91610830555673671H', '91610821MA70D3PF0H', '916108226847965391', '916108230882445003', '916108217552057845'

In [38]:
df_chain_core_ent_xf=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype='str',sheet_name='2023xf')
df_chain_core_ent_gf=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype='str',sheet_name='2023gf')
# str1="','".join(ls_chain_core_ent)

In [45]:
df_xf_chain=pd.merge(left=df_chain_core_ent_xf,right=df_chain_core_ent_c,left_on=['xf_shxydm'],right_on=['统一社会信用代码'],how='left')
df_xf_chain.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\xf_chain.csv',index=False)

In [46]:
df_gf_chain=pd.merge(left=df_chain_core_ent_gf,right=df_chain_core_ent_c,left_on=['gf_shxydm'],right_on=['统一社会信用代码'],how='left')
df_gf_chain.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\gf_chain.csv',index=False)

In [57]:
df_chain_core_ent_c.loc[df_chain_core_ent_c['统一社会信用代码'].isin([x for x in ls_chain_core_ent if x not in ls_xf])].to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\xf_queshi.csv',index=False)
df_chain_core_ent_c.loc[df_chain_core_ent_c['统一社会信用代码'].isin([x for x in ls_chain_core_ent if x not in ls_gf])].to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\gf_queshi.csv',index=False)


In [95]:
df_xf=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype={'spbm':'str','kpyf':'str'},sheet_name='2023xfsp')
df_gf=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_core_ent.xlsx',dtype={'spbm':'str','kpyf':'str'},sheet_name='2023gfsp')
df_xf_chain1=pd.merge(left=df_xf,right=df_chain_core_ent_c,left_on=['xf_shxydm'],right_on=['统一社会信用代码'],how='left')
df_gf_chain1=pd.merge(left=df_gf,right=df_chain_core_ent_c,left_on=['gf_shxydm'],right_on=['统一社会信用代码'],how='left')

df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
ls_result=[]
df_result_fp_chain2=pd.DataFrame()
for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    print(i,len(ls_spbm))
    # (df_gf_chain1['spbm'].isin(ls_spbm))&
    df_fp_xf_i=df_xf_chain1.loc[(df_xf_chain1['chain']==i)]
    df_fp_xf_i_=df_fp_xf_i.groupby(['chain','kpyf','spbm'])['sum(je)'].sum().reset_index()
    df_fp_xf_i_=df_fp_xf_i_.rename(columns={'sum(je)':'xf_je'})
    
    df_fp_gf_i=df_gf_chain1.loc[(df_gf_chain1['chain']==i)]
    df_fp_gf_i_=df_fp_gf_i.groupby(['chain','kpyf','spbm'])['sum(je)'].sum().reset_index()
    df_fp_gf_i_=df_fp_gf_i_.rename(columns={'sum(je)':'gf_je'})
    
    df_result_fp_i=pd.merge(left=df_fp_xf_i_,right=df_fp_gf_i_,on=['chain','kpyf','spbm'],how='outer')
    df_result_fp_chain2=pd.concat([df_result_fp_chain2,df_result_fp_i],axis=0)

智能无人系统和航空产业链 38
轻纺产业链 26
“碳中和”产业链 82
氢能产业链 29
铝镁及兰炭深加工产业链 62
现代特色农业产业链 47
现代煤化工产业链 56
文化旅游产业链 15
能源化工装备制造产业链 35
新能源装备制造产业链 82


In [97]:
df_result_fp_chain2.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\base_data\chain_spbm_je.csv',index=False)

In [85]:
df_xf_chain1.loc[(df_xf_chain1['chain']=='智能无人系统和航空产业链')].groupby(['spbm'])['sum(je)'].sum().reset_index()

,spbm,sum(je)
0,3040101000000000000,-136740.20
1,3040104010000000000,487772.45
2,3040105000000000000,2793588.92
3,3040201010000000000,118800.00
4,3040201990000000000,169811.33
5,3040401010100000000,1288009.55
6,3040401019900000000,15827.83
7,3040801990000000000,4245.28
8,3049900000000000000,943.40
9,3070401000000000000,43807.55


In [ ]:
# 选择发票当中购方或销方为产业链五上企业的30000条数据
select * from datapool.t_data_sh_new_qyjfkpsj_2023_5 where gf_shxydm in ('9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70GG2Y1H','91610800MA703QPKXC','91610802061916178Q','916108007273435491','91610000567118552C','91610893713586830T','916108007869525471','916108007979389782','9161080271004864XG','91610800661152976L','91610802MA704Y7M9M','91610802MA70G53D6M','91610000797940752N','91610802MA7031R94N','91610802MA703WM89H','916108005778342619','91610800752124911M','916108007588338747','91610802223962506N','916108006751043185','91610800677914736G','916108007450080310','91610800773819114A','91610800794100846T','916108006641075481','916108003385989430','91610800552165727D','91610802577836419K','9161080266116183XF','91610800684799641Q','916108006984024304','916108006641132516','916108007625527097','91610802MA70BMX34M','91610800MA70BE0G9E','91610800MA703EW32C','91610800MA70AXHK6D','91610800MA7036C41M','91610800MA704GNH0Q','91610802MA703BWC8F','91610800059688380U','91610800305302050M','91610800MA709M13XP','91610893MA703DEY9X','91610800694918499D','91610800MA7037RA9M','91610822MA7097U17U','91610824MA704D9D0X','91610800067912098G','91610802MA7034XX71','916108933057186623','91610800730410562R','91610800593317221R','91610800MA70988L3J','916108006984411145','91610800677943334K','91610893MA703AHQ4K','91610800675112705T','91610800MA703DR38N','91610893MA703K4C4E','916108006611675026','91610893MA70C3XL5R','91610893MA70AKUT7Y','91610893MA7030276T','91610800067912397E','91610802081747367D','91610800078619321P','91610800223959534H','916108000679086973','91610104MA6U6WPP9U','91610822MA708UM64E','91610802MA70FP667W','91610800698417755J','91610802MA70F2QM2B','91610802MA70CCRJ58','916108003054061837','91610893MA70BUNR99','91610893567149850M','91610800694921348L','91610893MA7092E2XX','91610800MA70319P01','91610800776976764F','91610800555695483W','91610893MA7056GH49','91610800748618329J','91610800064810491X','91610802338656146X','9161080230559847XB','91610800691100720J','91610000661160001T','91610800MA7035TJX3','91610800305315580J','91610800074521407G','916108000679234595','91610802573522607J','91610800661157478J','91610800088193408Q','91610800MA704B3A2P','91610800088079154R','916108020569016092','91610800MA7036T07K','91610000X2358669XB','91610800061931191K','91610802338661033H','91610800694930906W','91610800575649337Q','916108023055573957','916108000939802706','916100002239672519','916108026879737809','916100007100518818','91610802MA70A6FN4M','91610000X23586219E','91610800671536834E','91610800691127712R','916100006815997672','91610802667987720K','91610802MA7038FT0M','91610802552178923N','91610000223967235K','916108006679710248','91610000745037668H','91610000223965838G','91610802570668811N','916108006751155433','91610000710051814A','91610000766346072G','91610800598779185W','91610802MA708F2G8K','91610802MA709Y9P37','916108007588160388','916108003054001360','91610800691131228D','91610000786958834R','91610800MA703RJE5P','91610802MA7055P507','916108020547928919','916100005593700612','91610802681595256T','91610800MA704B3W2A','916100006847510073','916108020880671880','91610000735388763B','91610000773844942E','91610800752129712H','916108007979285936','91610000X23586251R','91610000752131054Q','91610802MA7038KK91','91610802773841215D','916108005778349158','9161080009394451XQ','91610800061909963N','91610802MA7038GD6G','93610802MA703FR92F','91610000338700928R','91610800552158025D','91610802691100835M','91610802563788284P','91610802MA703J1L6T','91610800MA7036E79W','91610802061927221J','91610800590253023H','9161080276255849XM','91610802MA704D189B','91610821MA703YBK2D','91610800MA703QW35X','91610802305755922F','91610802078614125H','91610802MA70D7448F','91610802MA70ATEH12','91610000059676961F','91610802MA7038MAX1','91610000593327534K','91610802MA703HWE04','916108023386836119','9161080006481082X4','91610800590260549L','91610800MA70371J7F','91610000797938927G','91610800596685070J','9161000056714796XP','91610800684794322L','91610823555664310J','91610823755214998B','91610000755225347C','91610000758832169D','916108006815878706','916108006911278008','916100006984246809','916100006984339153','91610000710055196X','916100007273509845','91610000755214488A','91610000779910293J','91610000758810349Q','916108232240609348','91610000224061857H','916108935521883122','916100005556550942','916100006815955200','91610000687954301Y','91610000790751174B','91610823MA7035CD25','916108230880722358','91610823305604112X','916108230881953581','916108237907734373','91610823MA703ELY6H','91610823MA70CG8N8F','91610823671529749H','91610823MA7032KGXX','91610823MA70D1G21X','91610823MA70G73N3H','91610823MA703DDX4B','91610823MA7097DG1P','916108237941275554','91610823MA7034CJ9Q','91610823MA70GABW64','91610823MA7032U61A','91610823MA7030E21X','91610823MA709HYNXD','916108233054100434','91610823MA70EP4G0U','916100006984084291','916108230521323633','916108230882445003','91610823MA704R7J5F','91610000773800042F','91610000586982709M','91610823056903460W','91610823MA708N92XX','91610823305726101A','91610823MA708U9U8T','91610823MA70CM4M03','91610823MA70341H2K','91610893MA70F7MT80','91610823MA7030FM17','91610893MA70AULF26','91610800684795675B','91610893088247840A','916108930882378452','91610800567137999E','91610823MA70D01L9W','91610823MA7030WJ5A','91610822552193103H','91610822583542433E','916100006751366436','91610000687952218H','91610822694938211C','916100007769886696','9161082222403383X0','916108225593913130','91610822567139548R','91610800671529239R','916108222240319575','91610822MA70CYYF7Q','91610800223930635R','916108225593639100','91610822MA703A0A5W','91610822583526038L','91610822MA709EAN9R','91610822710053561Q','916108226715094065','91610822MA708J3C27','916108223387099570','9161082266411342X1','9161082267152645XB','916108226984006033','91610822MA703WCN8Q','91610822MA70DF1676','916108222240308741','916108222240302172','91610822224030882U','91610000577805794Q','91610822224032482K','916108227100544680','91610822766320980Q','91610822586971540Q','91610822071284629R','91610822MA70BE3W9P','91610822MA70F1FP48','91610822MA7BK9K43D','91610822MA709Y0M1T','91610822MA709Y7E08','91610822MA704D4D9G','91610822924032280H','91610822MA709N423A','91610822MA709HGW9P','91610822MA7040Q74E','91610822MA70G7EX5Y','91610000661170904Y','91610000224030946L','91610822797925923U','916108225933488027','91610822224032386A','91610000MA6TG1L91C','916108223056392271','9161082230539678XQ','91610822555655537N','91610822762574940L','9161082206868156X4','916100002240321328','91610822684783682Q','91610822735351493Q','916108227135586612','91610822752112793E','91610822224041784B','91610822573535061G','91610822698443996B','91610822552179029C','91610000580766765J','91610822081718611M','91610822677931237W','91610822MA703BPQ8E','91610000559374302W','91610822694908266R','91610822694929032M','91610000797915039F','91610822093914935U','91610822555666885M','916108223386303333','91610822583546661K','91610822MA70317P8A','91610822671525115L','91610822577800053T','91610822087963650K','91610822797937238T','91610822667973185H','916100005556762074','91610822687981123J','91610822567149623M','916108225869772994','9161082269494526XA','916108227663422157','916108225593843242','916108226911466229','91610822575627197H','91610822677911949C','91610822056917184G','91610822MA7059MH04','91610000770001587F','91610800719785144W','91610000567108776E','91610000758819036E','91610822786997948R','91610822563772274J','91610000224057049W','916108225521605984','91610822MA709DFT7W','916108225835113037','91610822MA703HB02A','916100000504386259','916108225869795931','91610822MA708MR72L','91610822735347005J','91610800770011101Y','91610800675123180G','9161000077382866XG','916108220569493036','91610822MA70DY9C9C','916100000817239128','91610000713558602R','91610822570658605X','91610000758843706Y','91610822MA7036PA3C','91610000061914033T','91610822224033581D','916108220619348353','91610822694914308A','9161082273039809X5','916100000521118534','91610822552168012N','91610000580775995H','916108227486349192','91610822766312091G','9161082276633061X2','91610822MA708Y049A','91610822710054302H','91610822687992703H','91610822766330636W','91610822MA70FGBR66','91610822078635727F','91610822078636033K','91610822MA704G881Y','91610822667965572L','916108227799344049','91610822MA70B4TQ8T','916108226911259879','916108222240334186','91610822MA70A6HJ4U','91610822MA709R8R4H','9161082258078527XY','91610822687983735A','91610822710054791E','91610822MA708R672M','91610822577843598F','916108225869837874','9161082257069098XN','91610822694927918T','91610822797911572J','916108225637837070','916108226949183843','916108226949327095','91610822MA704CN193','9161082271355912XF','91610822567124282E','916108225671304988','91610000710054484G','916108225735414537','916100002240314638','916108225778382625','91610822773826787T','91610000567136590B','91610822078640366D','91610822094023050T','9161000056710680XM','91610822681555895X','91610000924044062B','91610822552163879U','91610822305318853B','91610000567138334L','916108226879560162','91610822567139193Q','91610000681575589J','916108226679923350','91610821MA70343P02','916108930881217117','91610822305621270Y','916100006879777738','91610822050401881R','91610822MA704K1D9T','91610822748606993Y','91610822MA70B3171E','916108225807966659','91610822MA70AB7K22','91610822MA7030P565','91610822755225080Y','91610000X23606048P','916100005556688484','91610000598765891B','916100009240407903','91610822766305764C','91610000924030840U','9161000055935482XF','91610000567118544H','91610000719708587F','91610822562241391F','916108225521576557','91610822MA7030C7X0','9161082255566867X2','916108225735291204','916108226847965391','9161082271355816XC','91610000567129809W','91610000687954512E','91610822667970734M','91610822748616366T','91610000681575597D','91610000691106065G','916108227869769281','9161082269490584XB','916108223057890497','91610000562225156P','91610822737993836U','91610822MA7093A6XP','91610822563764290Y','916108225556745861','9161082266796860XC','91610822MA708N4B1M','91610000681575570M','9161082256712766XC','91610822727369589U','91610822580776779W','916108225902933593','916108220648211664','9161082207127640X1','91610822559385992C','9161082256224308XJ','9161082258078631XM','91610822590295784U','91610822593312017P','91610822596695017M','916108225987548772','916108226779018201','91610822681564126J','91610822MA709AGT5N','91610822MA70A9UE46','91610822MA708KQDXA','91610822552191175G','91610822555680564K','91610822552154948T','91610822671520461A','916100006847787282','91610822MA708YJX74','91610000567103318K','91610822563762148E','91610822562224372N','91610822555688793Q','9161082257351819X6','91610822583526636G','916100007099182651','916100005671063088','916108225807676532','91610822580784090L','9161000059027254XF','91610822MA7032CD47','916108220745134073','91610822766331508H','9161000056377267XH','916100005556789748','91610000567138887F','916108226715375708','916108227450323733','9161082230534244X0','9161082208819745XP','91610822573530391G','91610000681599492L','91610822MA7039625F','916108220786281646','91610000078649651R','91610000305788468G','91610822586987462E','91610822MA70384L5F','91610822MA703BUW9C','91610824MA708G4C05','916108245756426370','91610824352260279W','91610824338607352A','91610824MA709B7K7N','91610824MA7045E8X5','916108243055203063','91610824MA7043WQ6F','91610824MA7045KE7K','91610824684776511K','91610824MA7052RM4M','91610824305602280L','91610824MA70404L9N','91610824MA708T379E','91610824713561318K','91610824590274190L','9161082405479693X8','916108246949093223','91610800677905610M','91610000698419427R','91610824338748948N','91610824MA70A3CU40','91610824067940013A','91610824MA7031PR97','91610824093912366E','91610824052125550G','91610824555662817L','91610824586989142N','91610824MA70DAAF42','91610824667959658W','91610824338721649N','91610824305379402Y','916108245593799981','91610824583543516F','91610824MA703AAQXB','91610824675115447J','91610824MA70EFL6XT','91610824MABQ6NL698','91610824580788446J','91610824590270069T','91610824MA70D7CY47','91610824577821241J','916108003057434031','91610824059660193A','91610824559383436L','916108243054516389','91610824MA703BB57G','91610824MA708NC530','91610824305645491Q','91610824074522522Q','91610824093913721Q','91610824059693913C','91610824570657119L','91610825710058346A','916108253055799146','916108253057807499','91610825MA703BP13J','916108253055693618','91610825064823620K','91610825MA70GL610E','91610825MA703GRQ4W','91610825094008096H','91610825088196684W','916108250939461010','91610825305710716N','91610825MA7K37P61M','91610825MA70GHF09N','91610825MA70GGYFX2','916108253055136327','91610825MA708W73XU','9161082530550930XA','91610825056929273K','91610825MA7097R42F','91610825MA703D773C','91610825059666253M','916108250648189605','916108257100585227','916108255807949859','91610825770016498A','916108252241200281','916108257941124337','91610825305646531X','91610825305721597M','916108257869947471','916108255902738171','91610825MA704RJJ1E','91610825MA7031MW0H','916108250817158759','9161082530538018XY','91610825305557133R','916108253057060719','91610800MA70B34385','916108253054304235','916108253057555513','91610825MA70GGKC69','91610825338670220D','91610825MA7033R222','916108250939154365','916108257869852184','916108250881238057','91610825687982230J','91610825MA703C1J05','9161082569842068XN','916108250859923274','91610825MA70A9HP1Q','916108255521915890','9161082509390645XX','91610825MA70318W1T','91610826MA7033UK93','91610826MA70CG498N','91610800223930176L','91610826583507806A','916108266641120996','916108265593534054','91610800X236250463','9161082630547280X6','916108266949220255','91610826MA70333X3J','91610826MA703MQK5P','91610826064845432U','91610826MA70FQTC5E','91610826590266393W','91610827MA705CXF2K','91610827MA703CCPXF','91610827L04797814E','91610827MA7032FT4X','91610827MA70FK575C','916108277836898514','91610827061945163F','916100007552218217','91610827MA703BQ601','91610827552179360X','91610827MA708YWL19','9161082730562026XQ','91610827MA7094AT02','916108270596677901','91610827338623053A','91610827MA7040FP1T','91610828MA7030X571','91610828MA7MYJ0G84','9161082830551626X8','91610828MA703PB208','91610828MA709TR85H','91610828MA7035J19Q','91610828MA703GAP8E','93610828570669865Y','916108280679278399','93610828MA703MKE86','91610828562200557R','91610828MA7BC7C940','916108280504343257','93610828052138271M','91610828MA708R8J36','93610828559385247F','916108283057501879','9161082859666136XJ','91610828677919502C','91610828661169321T','91610828352252148H','936108285735062871','916108285966825230','91610828MA709KLN51','91610828088015337K','91610800735340321T','91610828MA708URR8U','91610828MA70E0UY5E','91610800MA70AG0322','916108283057533375','91610000305329931J','91610828575612673A','91610828MA70AN8Q0W','9161082806191181XW','91610828MA7032YP28','91610828MA703JJY7E','91610828773832562F','916108295933288522','91610829MA709G5C21','91610829067914085M','91610829MA704APE27','91610829MA70DMLR50','91610829790753292M','91610800790767520W','91610829786975896A','91610829586955030X','91610829MA709Q3L0H','91610830677947677D','916108307769831550','91610830MA7093KD0T','916108305756016913','91610830305391161C','91610830056947332A','916108303522676814','91610830786966180U','91610830MA7038868K','91610830MA70APFT59','91610830719750582L','916108307303912262','916108303056153482','91610830MA7033XL6Y','91610830719747359N','91610830224270253G','91610830MA7039PL1Q','91610830MA7095T23Y','91610830MA708W9N6T','91610830MA7036GJX0','916108303385722175','91610830MA70AC6U08','91610830MA704Q1P39','91610830338743565A','91610830MA7037YE6B','91610830710064009G','91610830555673671H','93610830MA703AM46D','91610830MA7036G47D','91610830MA7050672C','91610830305481837E','91610831MA70323L26','916108000880800750','91610831MA70B73M8U','91610831MA708WCR8K','91610831305767755N','91610831710065220P','91610831727348008E','91610831593341926R','916108310619449025','91610831MA703CJF2E','916108315706692914','91610831MA708R007W','91610831MA70B3MD4G','91610831MA70BD1H0Y','93610831562243709Y','916108313056661962','916108310939512426','91610831MA709N6L13','9161083157781616X6','916100000880671537','91610831MABN470E3F','93610831305343143G','9161082167793717XC','916108213055993173','91610821675105396U','91610821MA7035W466','91610821MA70EU1M4C','916108216847887231','91610800755204183B','91610821MA70978GX5','91610821MA70DTWN1E','91610821MA70DCFF18','91610821MA70FJR21E','91610821305767771W','916100007799307022','91610821677922285W','916100003053012185','91610000748616526W','91610821563791942T','91610821766308324Q','91610821786996363L','91610821MA70CNJ41E','91610000563779951C','916108217135706793','91610821MA70CRQ8X1','91610000732641997R','916100007799482405','91610000059666958C','91610821555656230Q','91610821583545538T','9161000075884416XH','91610821MA703FP576','916108215556937286','91610821MA70362218','91610821MA703BE64X','91610821664106588B','9161082156710197X5','91610821MA708QC6XL','91610821MA70C2Q06B','91610821MA70DKG314','916108217869920396','91610821MA70C2PMXJ','91610821719766197G','91610000710924961H','91610821067914579R','91610821071269955P','91610821MA70BJ9W0C','91610821MA7098JB19','916108215966606585','91610821MA70EC2YXC','91610821MA70CGPH7F','91610821583541000M','91610821559390206W','91610821667969549M','9161082175884213XN','916108217100521205','91610821MA703PT760','91610821338639979E','91610821MA703BHW6U','91610821783666617B','91610821MA70FXBP6K','91610821681596021L','916108216879709774','91610800598796380J','91610800691145662A','91610821691135173P','916108212240045457','91610821224000157T','91610821MA703GB77B','91610821681569381A','916100006847780075','916108216779159829','916108213385789858','91610821583543612D','9161082130538851XE','916108210648495992','91610000664134933F','91610821596655947E','91610821MA709DR57H','916108217552325636','91610821088123194U','91610000745041552L','91610000687992877M','91610821555689542X','91610821677901740Y','91610821677923560K','91610821MA70BFPF2Y','91610821MA709PAA2A','91610000748618126L','91610821MA70EGPX9H','916100002240051930','91610000924001302H','916108215835267406','916108215521580763','916108216751414422','916100005556584045','916108216779105329','91610821562228197B','91610821MA703XYH1W','91610821570659229E','916108217625768311','91610821305496043G','916108216815570738','91610821MA708GH3X4','9161082167510000XL','91610821MA703BEC31','91610821MA703LQGXF','916100000596527830','91610821MA703HPX1T','91610821305323134R','91610821MA703N1B5U','91610821573521946X','91610821MA703A1W1A','91610821305384658A','9161080066117645XR','91610821MA70955UX1','91610821MA70AY2U52','91610000577813989U','91610821MA70CG4B44','91610821694922041J','91610000064835373N','91610821562219223W','916100007100533691','91610000732652506M','916108217353454137','91610821MA7094W97M','91610821MA70381K88','91610000773808634G','91610821MA70BTPE2D','91610000305465386R','91610821MA70FPJ19J','91610821671501594M','91610821MA703316XP','91610000X23601087L','9161000071978482XF','916100006911194994','9161082158697432XC','91610821MA705B6C33','91610821MA6TGWKM6C','91610000710052067J','91610821770025546B','91610000924013143K','916108216911367669','91610821694911481E','91610000671527778F','91610000698436454D','91610821675114583D','91610821664110536C','916108215593819240','91610821681579985G','91610821563789930M','91610000766333263G','916108217869620404','91610821786962796K','91610821677948768B','91610821681588961J','91610000681597913D','91610821684790305C','91610821MA703BX57B','91610821MA703TKP7H','91610821MA70BKG73Y','91610821MA70FUAP10','91610821790760484L','91610821078617967F','91610821MA7098LA63','91610821MA7033LY65','91610821338609446E','91610821338609534W','91610821MA70B7PF0Y','91610821567141533Q','91610821681590201J','916108216815903409','91610000305340444Y','91610000059683299M','916100005622201527','916108215869581355','91610000924003893Q','916100005556826316','91610821684773951D','916100007135061593','91610821691126138J','9161000055219304XB','91610821061925744C','91610821L047817240','916108215807683576','916108216679832524','9161082155216971XT','91610821552195029G','91610821586985766Y','9161082155568221XT','9161082167792431X4','91610821081708229J','91610000773830604T','91610821675119317B','91610821061948145J','9161000092400161X7','91610821074501043F','916108215637906599','916108216847868401','91610821MA70AFP8XK','91610821790779716D','91610821MA709UHQ1A','91610821MA703J6063','9161082108806100X8','91610821305529570Q','91610821MA70950B2G','91610821MA703FB4X5','91610821MA7098KX87','91610821064821756N','91610821305712199T','91610821MA70GQ7Q5E','91610806691107682J','91610821580782810W','9161082155566904XM','91610821586998516U','916100007135040226','91610806741251560K','91610821MA7050AM0A','91610821766336843W','916108216879865066','91610821691147676K','916108216715437436','91610821755200326A','91610821687963603D','91610000224003921H','91610821050402075G','91610821675141573X','9161000071005330XQ','916108212240019344','91610821305412030B','9161082168796362X0','91610821586969627M','91610821MA703QH21F','91610821338655688X','91610821352278970Y','91610821562249481F','91610821MA704UUN2D','91610821MA704G7F25','91610821694924442T','91610000730404314B','91610821570668862F','9161000005968331X1','916108216879635742','91610000MA6TG40X28','91610821555699740T','916100005521646440','91610000710052833G','916100009240134699','91610000567129825J','9161000075521877X8','91610821770031292X','91610821776988992B','91610800MA708L7213','91610821MA703QFH1K','916108216715343455','91610821675144336D','91610893MA70C8BN5R','91610821MA70A5UY7L','91610821MA70ERP17F','91610000741268506A','91610821MA70DKQ35X','91610000577835002Q','916108216715106698','91610821563795767X','91610821770033124M','9161000071350425X8','91610821770043971N','91610821675117004H','916108212240035186','916108215556718796','91610821786998238R','916108216751367583','91610821684770443G','91610821677921872D','91610821338575709B','91610821698418344C','916108216847628063','916108217799382026','91610821698418854R','91610821713506992M','91610821681551456T','91610821MA709MTL5A','916108215593687385','91610821691122380D','91610000677916061H','91610821555684127G','91610000671528076Q','91610821694922463H','916108216911087145','916108216911169173','91610821675145697Y','916108216911304797','91610821583535073B','916108210817431714','91610821MA7034TW33','91610821671537108L','91610821580772153T','916108217552057845','916108215735218745','9161082173534658XU','9161082155219611XA','91610821071278958H','916108210569000161','9161000071005277XF','91610000691125485H','91610821577821532A','91610821671537060C','91610821MA704NDFX6','91610000710052665E','91610821071285285M','91610000577827678B','916108213056545585','916108213057842808','91610000694922009G','916108216715160756','91610821MA7094M77U','9161082155566058X5','9161082179790070XW','91610821338596382X','916108210712746156','91610821567114877Q','91610000577813946E','9161082169841866XW','916100000712732418','91610821586963639H','9161000071357096XA','91610821064803785M','91610821552174447C','91610821MA704JTT54','916108066879758908','91610000684797582J','91610806MA703A6H0H','91610806586970396Q','91610806MA703PKD8X','916108060547617531','91610800694936013W','91610806MA708E9E3C','916108007588273274','91610821MA708UWL0R','91610806MA70ADRH09','91610806305365529M','916108060786458026','91610806078612699W','91610821MA703DEQ3U','916100006879524029','91610800MA70F0X441','91610806586963508Y','91610806MA7096L418','91610800MA708M1D63','91610806MA70391C58','916100009240011505','916108065869736343','91610821758816740M','91610821794117621M','91610821MA70CG834J','91610821MA70D3P75Q','91610821MA70D3PF0H','91610821770017642B','91610821684784298E','91610821MA70AFDX8F','91610821664127514E','91610821MA70CXKU84','91610821MA70A9ND11','9161082176632501XX','91610821675145005D','91610821054788032W','916108217700423972','91610821770013983N','91610821MA70DT2L42','91610821681582199M','91610806MA709B0F16','91610821MA7037WA08','916108213057510246','91610821555695272T','916108215556868517','9161082158075123X8','91610821762578722U','91610821755222453P','91610821MA70A5L79B','916108213053142541','91610821074539041A','916108215637902455','91610806067948461P','91610821MA703AYG0F','91610800752146344X','9161082130559046X1','91610806755247976C','916108217588481798','91610821305664764W','91610821766312040R','91610821786998975C','91610821677912765F','91610821559365318N','91610821MA703BRT54','91610806735348921D','91610821748622598U','91610000748646493C','91610821MA709HTD69','916108215671436353','91610821338638431X','91610821305453131A','91610821305769822Y','91610821305337210M','916108213055523324','916108215807826858','91610821MA708P1D4Q','916100006984015773','916100007769889684','91610821MA708FCP5T','91610821305529183C','91610821677948098N') and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm not in ('000000000000000','000000','88888888888888888888','000011000000000') and xf_shxydm not in ('000000000000000','000000','88888888888888888888','000011000000000') and kpyf='202302' limit 30000

In [ ]:
# 2023年产业链核心企业购销数据
# 核心企业做销方的销售金额数据汇总 2197
select * from (select xf_shxydm, xf_nsrmc, xf_hydm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj_2023_5 where xf_shxydm in ('9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70GG2Y1H','91610800MA703QPKXC','91610802061916178Q','916108007273435491','91610000567118552C','91610893713586830T','916108007869525471','916108007979389782','9161080271004864XG','91610800661152976L','91610802MA704Y7M9M','91610802MA70G53D6M','91610000797940752N','91610802MA7031R94N','91610802MA703WM89H','916108005778342619','91610800752124911M','916108007588338747','91610802223962506N','916108006751043185','91610800677914736G','916108007450080310','91610800773819114A','91610800794100846T','916108006641075481','916108003385989430','91610800552165727D','91610802577836419K','9161080266116183XF','91610800684799641Q','916108006984024304','916108006641132516','916108007625527097','91610802MA70BMX34M','91610800MA70BE0G9E','91610800MA703EW32C','91610800MA70AXHK6D','91610800MA7036C41M','91610800MA704GNH0Q','91610802MA703BWC8F','91610800059688380U','91610800305302050M','91610800MA709M13XP','91610893MA703DEY9X','91610800694918499D','91610800MA7037RA9M','91610822MA7097U17U','91610824MA704D9D0X','91610800067912098G','91610802MA7034XX71','916108933057186623','91610800730410562R','91610800593317221R','91610800MA70988L3J','916108006984411145','91610800677943334K','91610893MA703AHQ4K','91610800675112705T','91610800MA703DR38N','91610893MA703K4C4E','916108006611675026','91610893MA70C3XL5R','91610893MA70AKUT7Y','91610893MA7030276T','91610800067912397E','91610802081747367D','91610800078619321P','91610800223959534H','916108000679086973','91610104MA6U6WPP9U','91610822MA708UM64E','91610802MA70FP667W','91610800698417755J','91610802MA70F2QM2B','91610802MA70CCRJ58','916108003054061837','91610893MA70BUNR99','91610893567149850M','91610800694921348L','91610893MA7092E2XX','91610800MA70319P01','91610800776976764F','91610800555695483W','91610893MA7056GH49','91610800748618329J','91610800064810491X','91610802338656146X','9161080230559847XB','91610800691100720J','91610000661160001T','91610800MA7035TJX3','91610800305315580J','91610800074521407G','916108000679234595','91610802573522607J','91610800661157478J','91610800088193408Q','91610800MA704B3A2P','91610800088079154R','916108020569016092','91610800MA7036T07K','91610000X2358669XB','91610800061931191K','91610802338661033H','91610800694930906W','91610800575649337Q','916108023055573957','916108000939802706','916100002239672519','916108026879737809','916100007100518818','91610802MA70A6FN4M','91610000X23586219E','91610800671536834E','91610800691127712R','916100006815997672','91610802667987720K','91610802MA7038FT0M','91610802552178923N','91610000223967235K','916108006679710248','91610000745037668H','91610000223965838G','91610802570668811N','916108006751155433','91610000710051814A','91610000766346072G','91610800598779185W','91610802MA708F2G8K','91610802MA709Y9P37','916108007588160388','916108003054001360','91610800691131228D','91610000786958834R','91610800MA703RJE5P','91610802MA7055P507','916108020547928919','916100005593700612','91610802681595256T','91610800MA704B3W2A','916100006847510073','916108020880671880','91610000735388763B','91610000773844942E','91610800752129712H','916108007979285936','91610000X23586251R','91610000752131054Q','91610802MA7038KK91','91610802773841215D','916108005778349158','9161080009394451XQ','91610800061909963N','91610802MA7038GD6G','93610802MA703FR92F','91610000338700928R','91610800552158025D','91610802691100835M','91610802563788284P','91610802MA703J1L6T','91610800MA7036E79W','91610802061927221J','91610800590253023H','9161080276255849XM','91610802MA704D189B','91610821MA703YBK2D','91610800MA703QW35X','91610802305755922F','91610802078614125H','91610802MA70D7448F','91610802MA70ATEH12','91610000059676961F','91610802MA7038MAX1','91610000593327534K','91610802MA703HWE04','916108023386836119','9161080006481082X4','91610800590260549L','91610800MA70371J7F','91610000797938927G','91610800596685070J','9161000056714796XP','91610800684794322L','91610823555664310J','91610823755214998B','91610000755225347C','91610000758832169D','916108006815878706','916108006911278008','916100006984246809','916100006984339153','91610000710055196X','916100007273509845','91610000755214488A','91610000779910293J','91610000758810349Q','916108232240609348','91610000224061857H','916108935521883122','916100005556550942','916100006815955200','91610000687954301Y','91610000790751174B','91610823MA7035CD25','916108230880722358','91610823305604112X','916108230881953581','916108237907734373','91610823MA703ELY6H','91610823MA70CG8N8F','91610823671529749H','91610823MA7032KGXX','91610823MA70D1G21X','91610823MA70G73N3H','91610823MA703DDX4B','91610823MA7097DG1P','916108237941275554','91610823MA7034CJ9Q','91610823MA70GABW64','91610823MA7032U61A','91610823MA7030E21X','91610823MA709HYNXD','916108233054100434','91610823MA70EP4G0U','916100006984084291','916108230521323633','916108230882445003','91610823MA704R7J5F','91610000773800042F','91610000586982709M','91610823056903460W','91610823MA708N92XX','91610823305726101A','91610823MA708U9U8T','91610823MA70CM4M03','91610823MA70341H2K','91610893MA70F7MT80','91610823MA7030FM17','91610893MA70AULF26','91610800684795675B','91610893088247840A','916108930882378452','91610800567137999E','91610823MA70D01L9W','91610823MA7030WJ5A','91610822552193103H','91610822583542433E','916100006751366436','91610000687952218H','91610822694938211C','916100007769886696','9161082222403383X0','916108225593913130','91610822567139548R','91610800671529239R','916108222240319575','91610822MA70CYYF7Q','91610800223930635R','916108225593639100','91610822MA703A0A5W','91610822583526038L','91610822MA709EAN9R','91610822710053561Q','916108226715094065','91610822MA708J3C27','916108223387099570','9161082266411342X1','9161082267152645XB','916108226984006033','91610822MA703WCN8Q','91610822MA70DF1676','916108222240308741','916108222240302172','91610822224030882U','91610000577805794Q','91610822224032482K','916108227100544680','91610822766320980Q','91610822586971540Q','91610822071284629R','91610822MA70BE3W9P','91610822MA70F1FP48','91610822MA7BK9K43D','91610822MA709Y0M1T','91610822MA709Y7E08','91610822MA704D4D9G','91610822924032280H','91610822MA709N423A','91610822MA709HGW9P','91610822MA7040Q74E','91610822MA70G7EX5Y','91610000661170904Y','91610000224030946L','91610822797925923U','916108225933488027','91610822224032386A','91610000MA6TG1L91C','916108223056392271','9161082230539678XQ','91610822555655537N','91610822762574940L','9161082206868156X4','916100002240321328','91610822684783682Q','91610822735351493Q','916108227135586612','91610822752112793E','91610822224041784B','91610822573535061G','91610822698443996B','91610822552179029C','91610000580766765J','91610822081718611M','91610822677931237W','91610822MA703BPQ8E','91610000559374302W','91610822694908266R','91610822694929032M','91610000797915039F','91610822093914935U','91610822555666885M','916108223386303333','91610822583546661K','91610822MA70317P8A','91610822671525115L','91610822577800053T','91610822087963650K','91610822797937238T','91610822667973185H','916100005556762074','91610822687981123J','91610822567149623M','916108225869772994','9161082269494526XA','916108227663422157','916108225593843242','916108226911466229','91610822575627197H','91610822677911949C','91610822056917184G','91610822MA7059MH04','91610000770001587F','91610800719785144W','91610000567108776E','91610000758819036E','91610822786997948R','91610822563772274J','91610000224057049W','916108225521605984','91610822MA709DFT7W','916108225835113037','91610822MA703HB02A','916100000504386259','916108225869795931','91610822MA708MR72L','91610822735347005J','91610800770011101Y','91610800675123180G','9161000077382866XG','916108220569493036','91610822MA70DY9C9C','916100000817239128','91610000713558602R','91610822570658605X','91610000758843706Y','91610822MA7036PA3C','91610000061914033T','91610822224033581D','916108220619348353','91610822694914308A','9161082273039809X5','916100000521118534','91610822552168012N','91610000580775995H','916108227486349192','91610822766312091G','9161082276633061X2','91610822MA708Y049A','91610822710054302H','91610822687992703H','91610822766330636W','91610822MA70FGBR66','91610822078635727F','91610822078636033K','91610822MA704G881Y','91610822667965572L','916108227799344049','91610822MA70B4TQ8T','916108226911259879','916108222240334186','91610822MA70A6HJ4U','91610822MA709R8R4H','9161082258078527XY','91610822687983735A','91610822710054791E','91610822MA708R672M','91610822577843598F','916108225869837874','9161082257069098XN','91610822694927918T','91610822797911572J','916108225637837070','916108226949183843','916108226949327095','91610822MA704CN193','9161082271355912XF','91610822567124282E','916108225671304988','91610000710054484G','916108225735414537','916100002240314638','916108225778382625','91610822773826787T','91610000567136590B','91610822078640366D','91610822094023050T','9161000056710680XM','91610822681555895X','91610000924044062B','91610822552163879U','91610822305318853B','91610000567138334L','916108226879560162','91610822567139193Q','91610000681575589J','916108226679923350','91610821MA70343P02','916108930881217117','91610822305621270Y','916100006879777738','91610822050401881R','91610822MA704K1D9T','91610822748606993Y','91610822MA70B3171E','916108225807966659','91610822MA70AB7K22','91610822MA7030P565','91610822755225080Y','91610000X23606048P','916100005556688484','91610000598765891B','916100009240407903','91610822766305764C','91610000924030840U','9161000055935482XF','91610000567118544H','91610000719708587F','91610822562241391F','916108225521576557','91610822MA7030C7X0','9161082255566867X2','916108225735291204','916108226847965391','9161082271355816XC','91610000567129809W','91610000687954512E','91610822667970734M','91610822748616366T','91610000681575597D','91610000691106065G','916108227869769281','9161082269490584XB','916108223057890497','91610000562225156P','91610822737993836U','91610822MA7093A6XP','91610822563764290Y','916108225556745861','9161082266796860XC','91610822MA708N4B1M','91610000681575570M','9161082256712766XC','91610822727369589U','91610822580776779W','916108225902933593','916108220648211664','9161082207127640X1','91610822559385992C','9161082256224308XJ','9161082258078631XM','91610822590295784U','91610822593312017P','91610822596695017M','916108225987548772','916108226779018201','91610822681564126J','91610822MA709AGT5N','91610822MA70A9UE46','91610822MA708KQDXA','91610822552191175G','91610822555680564K','91610822552154948T','91610822671520461A','916100006847787282','91610822MA708YJX74','91610000567103318K','91610822563762148E','91610822562224372N','91610822555688793Q','9161082257351819X6','91610822583526636G','916100007099182651','916100005671063088','916108225807676532','91610822580784090L','9161000059027254XF','91610822MA7032CD47','916108220745134073','91610822766331508H','9161000056377267XH','916100005556789748','91610000567138887F','916108226715375708','916108227450323733','9161082230534244X0','9161082208819745XP','91610822573530391G','91610000681599492L','91610822MA7039625F','916108220786281646','91610000078649651R','91610000305788468G','91610822586987462E','91610822MA70384L5F','91610822MA703BUW9C','91610824MA708G4C05','916108245756426370','91610824352260279W','91610824338607352A','91610824MA709B7K7N','91610824MA7045E8X5','916108243055203063','91610824MA7043WQ6F','91610824MA7045KE7K','91610824684776511K','91610824MA7052RM4M','91610824305602280L','91610824MA70404L9N','91610824MA708T379E','91610824713561318K','91610824590274190L','9161082405479693X8','916108246949093223','91610800677905610M','91610000698419427R','91610824338748948N','91610824MA70A3CU40','91610824067940013A','91610824MA7031PR97','91610824093912366E','91610824052125550G','91610824555662817L','91610824586989142N','91610824MA70DAAF42','91610824667959658W','91610824338721649N','91610824305379402Y','916108245593799981','91610824583543516F','91610824MA703AAQXB','91610824675115447J','91610824MA70EFL6XT','91610824MABQ6NL698','91610824580788446J','91610824590270069T','91610824MA70D7CY47','91610824577821241J','916108003057434031','91610824059660193A','91610824559383436L','916108243054516389','91610824MA703BB57G','91610824MA708NC530','91610824305645491Q','91610824074522522Q','91610824093913721Q','91610824059693913C','91610824570657119L','91610825710058346A','916108253055799146','916108253057807499','91610825MA703BP13J','916108253055693618','91610825064823620K','91610825MA70GL610E','91610825MA703GRQ4W','91610825094008096H','91610825088196684W','916108250939461010','91610825305710716N','91610825MA7K37P61M','91610825MA70GHF09N','91610825MA70GGYFX2','916108253055136327','91610825MA708W73XU','9161082530550930XA','91610825056929273K','91610825MA7097R42F','91610825MA703D773C','91610825059666253M','916108250648189605','916108257100585227','916108255807949859','91610825770016498A','916108252241200281','916108257941124337','91610825305646531X','91610825305721597M','916108257869947471','916108255902738171','91610825MA704RJJ1E','91610825MA7031MW0H','916108250817158759','9161082530538018XY','91610825305557133R','916108253057060719','91610800MA70B34385','916108253054304235','916108253057555513','91610825MA70GGKC69','91610825338670220D','91610825MA7033R222','916108250939154365','916108257869852184','916108250881238057','91610825687982230J','91610825MA703C1J05','9161082569842068XN','916108250859923274','91610825MA70A9HP1Q','916108255521915890','9161082509390645XX','91610825MA70318W1T','91610826MA7033UK93','91610826MA70CG498N','91610800223930176L','91610826583507806A','916108266641120996','916108265593534054','91610800X236250463','9161082630547280X6','916108266949220255','91610826MA70333X3J','91610826MA703MQK5P','91610826064845432U','91610826MA70FQTC5E','91610826590266393W','91610827MA705CXF2K','91610827MA703CCPXF','91610827L04797814E','91610827MA7032FT4X','91610827MA70FK575C','916108277836898514','91610827061945163F','916100007552218217','91610827MA703BQ601','91610827552179360X','91610827MA708YWL19','9161082730562026XQ','91610827MA7094AT02','916108270596677901','91610827338623053A','91610827MA7040FP1T','91610828MA7030X571','91610828MA7MYJ0G84','9161082830551626X8','91610828MA703PB208','91610828MA709TR85H','91610828MA7035J19Q','91610828MA703GAP8E','93610828570669865Y','916108280679278399','93610828MA703MKE86','91610828562200557R','91610828MA7BC7C940','916108280504343257','93610828052138271M','91610828MA708R8J36','93610828559385247F','916108283057501879','9161082859666136XJ','91610828677919502C','91610828661169321T','91610828352252148H','936108285735062871','916108285966825230','91610828MA709KLN51','91610828088015337K','91610800735340321T','91610828MA708URR8U','91610828MA70E0UY5E','91610800MA70AG0322','916108283057533375','91610000305329931J','91610828575612673A','91610828MA70AN8Q0W','9161082806191181XW','91610828MA7032YP28','91610828MA703JJY7E','91610828773832562F','916108295933288522','91610829MA709G5C21','91610829067914085M','91610829MA704APE27','91610829MA70DMLR50','91610829790753292M','91610800790767520W','91610829786975896A','91610829586955030X','91610829MA709Q3L0H','91610830677947677D','916108307769831550','91610830MA7093KD0T','916108305756016913','91610830305391161C','91610830056947332A','916108303522676814','91610830786966180U','91610830MA7038868K','91610830MA70APFT59','91610830719750582L','916108307303912262','916108303056153482','91610830MA7033XL6Y','91610830719747359N','91610830224270253G','91610830MA7039PL1Q','91610830MA7095T23Y','91610830MA708W9N6T','91610830MA7036GJX0','916108303385722175','91610830MA70AC6U08','91610830MA704Q1P39','91610830338743565A','91610830MA7037YE6B','91610830710064009G','91610830555673671H','93610830MA703AM46D','91610830MA7036G47D','91610830MA7050672C','91610830305481837E','91610831MA70323L26','916108000880800750','91610831MA70B73M8U','91610831MA708WCR8K','91610831305767755N','91610831710065220P','91610831727348008E','91610831593341926R','916108310619449025','91610831MA703CJF2E','916108315706692914','91610831MA708R007W','91610831MA70B3MD4G','91610831MA70BD1H0Y','93610831562243709Y','916108313056661962','916108310939512426','91610831MA709N6L13','9161083157781616X6','916100000880671537','91610831MABN470E3F','93610831305343143G','9161082167793717XC','916108213055993173','91610821675105396U','91610821MA7035W466','91610821MA70EU1M4C','916108216847887231','91610800755204183B','91610821MA70978GX5','91610821MA70DTWN1E','91610821MA70DCFF18','91610821MA70FJR21E','91610821305767771W','916100007799307022','91610821677922285W','916100003053012185','91610000748616526W','91610821563791942T','91610821766308324Q','91610821786996363L','91610821MA70CNJ41E','91610000563779951C','916108217135706793','91610821MA70CRQ8X1','91610000732641997R','916100007799482405','91610000059666958C','91610821555656230Q','91610821583545538T','9161000075884416XH','91610821MA703FP576','916108215556937286','91610821MA70362218','91610821MA703BE64X','91610821664106588B','9161082156710197X5','91610821MA708QC6XL','91610821MA70C2Q06B','91610821MA70DKG314','916108217869920396','91610821MA70C2PMXJ','91610821719766197G','91610000710924961H','91610821067914579R','91610821071269955P','91610821MA70BJ9W0C','91610821MA7098JB19','916108215966606585','91610821MA70EC2YXC','91610821MA70CGPH7F','91610821583541000M','91610821559390206W','91610821667969549M','9161082175884213XN','916108217100521205','91610821MA703PT760','91610821338639979E','91610821MA703BHW6U','91610821783666617B','91610821MA70FXBP6K','91610821681596021L','916108216879709774','91610800598796380J','91610800691145662A','91610821691135173P','916108212240045457','91610821224000157T','91610821MA703GB77B','91610821681569381A','916100006847780075','916108216779159829','916108213385789858','91610821583543612D','9161082130538851XE','916108210648495992','91610000664134933F','91610821596655947E','91610821MA709DR57H','916108217552325636','91610821088123194U','91610000745041552L','91610000687992877M','91610821555689542X','91610821677901740Y','91610821677923560K','91610821MA70BFPF2Y','91610821MA709PAA2A','91610000748618126L','91610821MA70EGPX9H','916100002240051930','91610000924001302H','916108215835267406','916108215521580763','916108216751414422','916100005556584045','916108216779105329','91610821562228197B','91610821MA703XYH1W','91610821570659229E','916108217625768311','91610821305496043G','916108216815570738','91610821MA708GH3X4','9161082167510000XL','91610821MA703BEC31','91610821MA703LQGXF','916100000596527830','91610821MA703HPX1T','91610821305323134R','91610821MA703N1B5U','91610821573521946X','91610821MA703A1W1A','91610821305384658A','9161080066117645XR','91610821MA70955UX1','91610821MA70AY2U52','91610000577813989U','91610821MA70CG4B44','91610821694922041J','91610000064835373N','91610821562219223W','916100007100533691','91610000732652506M','916108217353454137','91610821MA7094W97M','91610821MA70381K88','91610000773808634G','91610821MA70BTPE2D','91610000305465386R','91610821MA70FPJ19J','91610821671501594M','91610821MA703316XP','91610000X23601087L','9161000071978482XF','916100006911194994','9161082158697432XC','91610821MA705B6C33','91610821MA6TGWKM6C','91610000710052067J','91610821770025546B','91610000924013143K','916108216911367669','91610821694911481E','91610000671527778F','91610000698436454D','91610821675114583D','91610821664110536C','916108215593819240','91610821681579985G','91610821563789930M','91610000766333263G','916108217869620404','91610821786962796K','91610821677948768B','91610821681588961J','91610000681597913D','91610821684790305C','91610821MA703BX57B','91610821MA703TKP7H','91610821MA70BKG73Y','91610821MA70FUAP10','91610821790760484L','91610821078617967F','91610821MA7098LA63','91610821MA7033LY65','91610821338609446E','91610821338609534W','91610821MA70B7PF0Y','91610821567141533Q','91610821681590201J','916108216815903409','91610000305340444Y','91610000059683299M','916100005622201527','916108215869581355','91610000924003893Q','916100005556826316','91610821684773951D','916100007135061593','91610821691126138J','9161000055219304XB','91610821061925744C','91610821L047817240','916108215807683576','916108216679832524','9161082155216971XT','91610821552195029G','91610821586985766Y','9161082155568221XT','9161082167792431X4','91610821081708229J','91610000773830604T','91610821675119317B','91610821061948145J','9161000092400161X7','91610821074501043F','916108215637906599','916108216847868401','91610821MA70AFP8XK','91610821790779716D','91610821MA709UHQ1A','91610821MA703J6063','9161082108806100X8','91610821305529570Q','91610821MA70950B2G','91610821MA703FB4X5','91610821MA7098KX87','91610821064821756N','91610821305712199T','91610821MA70GQ7Q5E','91610806691107682J','91610821580782810W','9161082155566904XM','91610821586998516U','916100007135040226','91610806741251560K','91610821MA7050AM0A','91610821766336843W','916108216879865066','91610821691147676K','916108216715437436','91610821755200326A','91610821687963603D','91610000224003921H','91610821050402075G','91610821675141573X','9161000071005330XQ','916108212240019344','91610821305412030B','9161082168796362X0','91610821586969627M','91610821MA703QH21F','91610821338655688X','91610821352278970Y','91610821562249481F','91610821MA704UUN2D','91610821MA704G7F25','91610821694924442T','91610000730404314B','91610821570668862F','9161000005968331X1','916108216879635742','91610000MA6TG40X28','91610821555699740T','916100005521646440','91610000710052833G','916100009240134699','91610000567129825J','9161000075521877X8','91610821770031292X','91610821776988992B','91610800MA708L7213','91610821MA703QFH1K','916108216715343455','91610821675144336D','91610893MA70C8BN5R','91610821MA70A5UY7L','91610821MA70ERP17F','91610000741268506A','91610821MA70DKQ35X','91610000577835002Q','916108216715106698','91610821563795767X','91610821770033124M','9161000071350425X8','91610821770043971N','91610821675117004H','916108212240035186','916108215556718796','91610821786998238R','916108216751367583','91610821684770443G','91610821677921872D','91610821338575709B','91610821698418344C','916108216847628063','916108217799382026','91610821698418854R','91610821713506992M','91610821681551456T','91610821MA709MTL5A','916108215593687385','91610821691122380D','91610000677916061H','91610821555684127G','91610000671528076Q','91610821694922463H','916108216911087145','916108216911169173','91610821675145697Y','916108216911304797','91610821583535073B','916108210817431714','91610821MA7034TW33','91610821671537108L','91610821580772153T','916108217552057845','916108215735218745','9161082173534658XU','9161082155219611XA','91610821071278958H','916108210569000161','9161000071005277XF','91610000691125485H','91610821577821532A','91610821671537060C','91610821MA704NDFX6','91610000710052665E','91610821071285285M','91610000577827678B','916108213056545585','916108213057842808','91610000694922009G','916108216715160756','91610821MA7094M77U','9161082155566058X5','9161082179790070XW','91610821338596382X','916108210712746156','91610821567114877Q','91610000577813946E','9161082169841866XW','916100000712732418','91610821586963639H','9161000071357096XA','91610821064803785M','91610821552174447C','91610821MA704JTT54','916108066879758908','91610000684797582J','91610806MA703A6H0H','91610806586970396Q','91610806MA703PKD8X','916108060547617531','91610800694936013W','91610806MA708E9E3C','916108007588273274','91610821MA708UWL0R','91610806MA70ADRH09','91610806305365529M','916108060786458026','91610806078612699W','91610821MA703DEQ3U','916100006879524029','91610800MA70F0X441','91610806586963508Y','91610806MA7096L418','91610800MA708M1D63','91610806MA70391C58','916100009240011505','916108065869736343','91610821758816740M','91610821794117621M','91610821MA70CG834J','91610821MA70D3P75Q','91610821MA70D3PF0H','91610821770017642B','91610821684784298E','91610821MA70AFDX8F','91610821664127514E','91610821MA70CXKU84','91610821MA70A9ND11','9161082176632501XX','91610821675145005D','91610821054788032W','916108217700423972','91610821770013983N','91610821MA70DT2L42','91610821681582199M','91610806MA709B0F16','91610821MA7037WA08','916108213057510246','91610821555695272T','916108215556868517','9161082158075123X8','91610821762578722U','91610821755222453P','91610821MA70A5L79B','916108213053142541','91610821074539041A','916108215637902455','91610806067948461P','91610821MA703AYG0F','91610800752146344X','9161082130559046X1','91610806755247976C','916108217588481798','91610821305664764W','91610821766312040R','91610821786998975C','91610821677912765F','91610821559365318N','91610821MA703BRT54','91610806735348921D','91610821748622598U','91610000748646493C','91610821MA709HTD69','916108215671436353','91610821338638431X','91610821305453131A','91610821305769822Y','91610821305337210M','916108213055523324','916108215807826858','91610821MA708P1D4Q','916100006984015773','916100007769889684','91610821MA708FCP5T','91610821305529183C','91610821677948098N') and xf_hydm is not null and gf_hydm is not null group by xf_shxydm, xf_nsrmc, xf_hydm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 5895
select * from (select xf_shxydm, xf_nsrmc, xf_hydm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, spbm, sum(je) from datapool.t_data_sh_new_qyjfkpsj_2023_5 where xf_shxydm in ('9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70GG2Y1H','91610800MA703QPKXC','91610802061916178Q','916108007273435491','91610000567118552C','91610893713586830T','916108007869525471','916108007979389782','9161080271004864XG','91610800661152976L','91610802MA704Y7M9M','91610802MA70G53D6M','91610000797940752N','91610802MA7031R94N','91610802MA703WM89H','916108005778342619','91610800752124911M','916108007588338747','91610802223962506N','916108006751043185','91610800677914736G','916108007450080310','91610800773819114A','91610800794100846T','916108006641075481','916108003385989430','91610800552165727D','91610802577836419K','9161080266116183XF','91610800684799641Q','916108006984024304','916108006641132516','916108007625527097','91610802MA70BMX34M','91610800MA70BE0G9E','91610800MA703EW32C','91610800MA70AXHK6D','91610800MA7036C41M','91610800MA704GNH0Q','91610802MA703BWC8F','91610800059688380U','91610800305302050M','91610800MA709M13XP','91610893MA703DEY9X','91610800694918499D','91610800MA7037RA9M','91610822MA7097U17U','91610824MA704D9D0X','91610800067912098G','91610802MA7034XX71','916108933057186623','91610800730410562R','91610800593317221R','91610800MA70988L3J','916108006984411145','91610800677943334K','91610893MA703AHQ4K','91610800675112705T','91610800MA703DR38N','91610893MA703K4C4E','916108006611675026','91610893MA70C3XL5R','91610893MA70AKUT7Y','91610893MA7030276T','91610800067912397E','91610802081747367D','91610800078619321P','91610800223959534H','916108000679086973','91610104MA6U6WPP9U','91610822MA708UM64E','91610802MA70FP667W','91610800698417755J','91610802MA70F2QM2B','91610802MA70CCRJ58','916108003054061837','91610893MA70BUNR99','91610893567149850M','91610800694921348L','91610893MA7092E2XX','91610800MA70319P01','91610800776976764F','91610800555695483W','91610893MA7056GH49','91610800748618329J','91610800064810491X','91610802338656146X','9161080230559847XB','91610800691100720J','91610000661160001T','91610800MA7035TJX3','91610800305315580J','91610800074521407G','916108000679234595','91610802573522607J','91610800661157478J','91610800088193408Q','91610800MA704B3A2P','91610800088079154R','916108020569016092','91610800MA7036T07K','91610000X2358669XB','91610800061931191K','91610802338661033H','91610800694930906W','91610800575649337Q','916108023055573957','916108000939802706','916100002239672519','916108026879737809','916100007100518818','91610802MA70A6FN4M','91610000X23586219E','91610800671536834E','91610800691127712R','916100006815997672','91610802667987720K','91610802MA7038FT0M','91610802552178923N','91610000223967235K','916108006679710248','91610000745037668H','91610000223965838G','91610802570668811N','916108006751155433','91610000710051814A','91610000766346072G','91610800598779185W','91610802MA708F2G8K','91610802MA709Y9P37','916108007588160388','916108003054001360','91610800691131228D','91610000786958834R','91610800MA703RJE5P','91610802MA7055P507','916108020547928919','916100005593700612','91610802681595256T','91610800MA704B3W2A','916100006847510073','916108020880671880','91610000735388763B','91610000773844942E','91610800752129712H','916108007979285936','91610000X23586251R','91610000752131054Q','91610802MA7038KK91','91610802773841215D','916108005778349158','9161080009394451XQ','91610800061909963N','91610802MA7038GD6G','93610802MA703FR92F','91610000338700928R','91610800552158025D','91610802691100835M','91610802563788284P','91610802MA703J1L6T','91610800MA7036E79W','91610802061927221J','91610800590253023H','9161080276255849XM','91610802MA704D189B','91610821MA703YBK2D','91610800MA703QW35X','91610802305755922F','91610802078614125H','91610802MA70D7448F','91610802MA70ATEH12','91610000059676961F','91610802MA7038MAX1','91610000593327534K','91610802MA703HWE04','916108023386836119','9161080006481082X4','91610800590260549L','91610800MA70371J7F','91610000797938927G','91610800596685070J','9161000056714796XP','91610800684794322L','91610823555664310J','91610823755214998B','91610000755225347C','91610000758832169D','916108006815878706','916108006911278008','916100006984246809','916100006984339153','91610000710055196X','916100007273509845','91610000755214488A','91610000779910293J','91610000758810349Q','916108232240609348','91610000224061857H','916108935521883122','916100005556550942','916100006815955200','91610000687954301Y','91610000790751174B','91610823MA7035CD25','916108230880722358','91610823305604112X','916108230881953581','916108237907734373','91610823MA703ELY6H','91610823MA70CG8N8F','91610823671529749H','91610823MA7032KGXX','91610823MA70D1G21X','91610823MA70G73N3H','91610823MA703DDX4B','91610823MA7097DG1P','916108237941275554','91610823MA7034CJ9Q','91610823MA70GABW64','91610823MA7032U61A','91610823MA7030E21X','91610823MA709HYNXD','916108233054100434','91610823MA70EP4G0U','916100006984084291','916108230521323633','916108230882445003','91610823MA704R7J5F','91610000773800042F','91610000586982709M','91610823056903460W','91610823MA708N92XX','91610823305726101A','91610823MA708U9U8T','91610823MA70CM4M03','91610823MA70341H2K','91610893MA70F7MT80','91610823MA7030FM17','91610893MA70AULF26','91610800684795675B','91610893088247840A','916108930882378452','91610800567137999E','91610823MA70D01L9W','91610823MA7030WJ5A','91610822552193103H','91610822583542433E','916100006751366436','91610000687952218H','91610822694938211C','916100007769886696','9161082222403383X0','916108225593913130','91610822567139548R','91610800671529239R','916108222240319575','91610822MA70CYYF7Q','91610800223930635R','916108225593639100','91610822MA703A0A5W','91610822583526038L','91610822MA709EAN9R','91610822710053561Q','916108226715094065','91610822MA708J3C27','916108223387099570','9161082266411342X1','9161082267152645XB','916108226984006033','91610822MA703WCN8Q','91610822MA70DF1676','916108222240308741','916108222240302172','91610822224030882U','91610000577805794Q','91610822224032482K','916108227100544680','91610822766320980Q','91610822586971540Q','91610822071284629R','91610822MA70BE3W9P','91610822MA70F1FP48','91610822MA7BK9K43D','91610822MA709Y0M1T','91610822MA709Y7E08','91610822MA704D4D9G','91610822924032280H','91610822MA709N423A','91610822MA709HGW9P','91610822MA7040Q74E','91610822MA70G7EX5Y','91610000661170904Y','91610000224030946L','91610822797925923U','916108225933488027','91610822224032386A','91610000MA6TG1L91C','916108223056392271','9161082230539678XQ','91610822555655537N','91610822762574940L','9161082206868156X4','916100002240321328','91610822684783682Q','91610822735351493Q','916108227135586612','91610822752112793E','91610822224041784B','91610822573535061G','91610822698443996B','91610822552179029C','91610000580766765J','91610822081718611M','91610822677931237W','91610822MA703BPQ8E','91610000559374302W','91610822694908266R','91610822694929032M','91610000797915039F','91610822093914935U','91610822555666885M','916108223386303333','91610822583546661K','91610822MA70317P8A','91610822671525115L','91610822577800053T','91610822087963650K','91610822797937238T','91610822667973185H','916100005556762074','91610822687981123J','91610822567149623M','916108225869772994','9161082269494526XA','916108227663422157','916108225593843242','916108226911466229','91610822575627197H','91610822677911949C','91610822056917184G','91610822MA7059MH04','91610000770001587F','91610800719785144W','91610000567108776E','91610000758819036E','91610822786997948R','91610822563772274J','91610000224057049W','916108225521605984','91610822MA709DFT7W','916108225835113037','91610822MA703HB02A','916100000504386259','916108225869795931','91610822MA708MR72L','91610822735347005J','91610800770011101Y','91610800675123180G','9161000077382866XG','916108220569493036','91610822MA70DY9C9C','916100000817239128','91610000713558602R','91610822570658605X','91610000758843706Y','91610822MA7036PA3C','91610000061914033T','91610822224033581D','916108220619348353','91610822694914308A','9161082273039809X5','916100000521118534','91610822552168012N','91610000580775995H','916108227486349192','91610822766312091G','9161082276633061X2','91610822MA708Y049A','91610822710054302H','91610822687992703H','91610822766330636W','91610822MA70FGBR66','91610822078635727F','91610822078636033K','91610822MA704G881Y','91610822667965572L','916108227799344049','91610822MA70B4TQ8T','916108226911259879','916108222240334186','91610822MA70A6HJ4U','91610822MA709R8R4H','9161082258078527XY','91610822687983735A','91610822710054791E','91610822MA708R672M','91610822577843598F','916108225869837874','9161082257069098XN','91610822694927918T','91610822797911572J','916108225637837070','916108226949183843','916108226949327095','91610822MA704CN193','9161082271355912XF','91610822567124282E','916108225671304988','91610000710054484G','916108225735414537','916100002240314638','916108225778382625','91610822773826787T','91610000567136590B','91610822078640366D','91610822094023050T','9161000056710680XM','91610822681555895X','91610000924044062B','91610822552163879U','91610822305318853B','91610000567138334L','916108226879560162','91610822567139193Q','91610000681575589J','916108226679923350','91610821MA70343P02','916108930881217117','91610822305621270Y','916100006879777738','91610822050401881R','91610822MA704K1D9T','91610822748606993Y','91610822MA70B3171E','916108225807966659','91610822MA70AB7K22','91610822MA7030P565','91610822755225080Y','91610000X23606048P','916100005556688484','91610000598765891B','916100009240407903','91610822766305764C','91610000924030840U','9161000055935482XF','91610000567118544H','91610000719708587F','91610822562241391F','916108225521576557','91610822MA7030C7X0','9161082255566867X2','916108225735291204','916108226847965391','9161082271355816XC','91610000567129809W','91610000687954512E','91610822667970734M','91610822748616366T','91610000681575597D','91610000691106065G','916108227869769281','9161082269490584XB','916108223057890497','91610000562225156P','91610822737993836U','91610822MA7093A6XP','91610822563764290Y','916108225556745861','9161082266796860XC','91610822MA708N4B1M','91610000681575570M','9161082256712766XC','91610822727369589U','91610822580776779W','916108225902933593','916108220648211664','9161082207127640X1','91610822559385992C','9161082256224308XJ','9161082258078631XM','91610822590295784U','91610822593312017P','91610822596695017M','916108225987548772','916108226779018201','91610822681564126J','91610822MA709AGT5N','91610822MA70A9UE46','91610822MA708KQDXA','91610822552191175G','91610822555680564K','91610822552154948T','91610822671520461A','916100006847787282','91610822MA708YJX74','91610000567103318K','91610822563762148E','91610822562224372N','91610822555688793Q','9161082257351819X6','91610822583526636G','916100007099182651','916100005671063088','916108225807676532','91610822580784090L','9161000059027254XF','91610822MA7032CD47','916108220745134073','91610822766331508H','9161000056377267XH','916100005556789748','91610000567138887F','916108226715375708','916108227450323733','9161082230534244X0','9161082208819745XP','91610822573530391G','91610000681599492L','91610822MA7039625F','916108220786281646','91610000078649651R','91610000305788468G','91610822586987462E','91610822MA70384L5F','91610822MA703BUW9C','91610824MA708G4C05','916108245756426370','91610824352260279W','91610824338607352A','91610824MA709B7K7N','91610824MA7045E8X5','916108243055203063','91610824MA7043WQ6F','91610824MA7045KE7K','91610824684776511K','91610824MA7052RM4M','91610824305602280L','91610824MA70404L9N','91610824MA708T379E','91610824713561318K','91610824590274190L','9161082405479693X8','916108246949093223','91610800677905610M','91610000698419427R','91610824338748948N','91610824MA70A3CU40','91610824067940013A','91610824MA7031PR97','91610824093912366E','91610824052125550G','91610824555662817L','91610824586989142N','91610824MA70DAAF42','91610824667959658W','91610824338721649N','91610824305379402Y','916108245593799981','91610824583543516F','91610824MA703AAQXB','91610824675115447J','91610824MA70EFL6XT','91610824MABQ6NL698','91610824580788446J','91610824590270069T','91610824MA70D7CY47','91610824577821241J','916108003057434031','91610824059660193A','91610824559383436L','916108243054516389','91610824MA703BB57G','91610824MA708NC530','91610824305645491Q','91610824074522522Q','91610824093913721Q','91610824059693913C','91610824570657119L','91610825710058346A','916108253055799146','916108253057807499','91610825MA703BP13J','916108253055693618','91610825064823620K','91610825MA70GL610E','91610825MA703GRQ4W','91610825094008096H','91610825088196684W','916108250939461010','91610825305710716N','91610825MA7K37P61M','91610825MA70GHF09N','91610825MA70GGYFX2','916108253055136327','91610825MA708W73XU','9161082530550930XA','91610825056929273K','91610825MA7097R42F','91610825MA703D773C','91610825059666253M','916108250648189605','916108257100585227','916108255807949859','91610825770016498A','916108252241200281','916108257941124337','91610825305646531X','91610825305721597M','916108257869947471','916108255902738171','91610825MA704RJJ1E','91610825MA7031MW0H','916108250817158759','9161082530538018XY','91610825305557133R','916108253057060719','91610800MA70B34385','916108253054304235','916108253057555513','91610825MA70GGKC69','91610825338670220D','91610825MA7033R222','916108250939154365','916108257869852184','916108250881238057','91610825687982230J','91610825MA703C1J05','9161082569842068XN','916108250859923274','91610825MA70A9HP1Q','916108255521915890','9161082509390645XX','91610825MA70318W1T','91610826MA7033UK93','91610826MA70CG498N','91610800223930176L','91610826583507806A','916108266641120996','916108265593534054','91610800X236250463','9161082630547280X6','916108266949220255','91610826MA70333X3J','91610826MA703MQK5P','91610826064845432U','91610826MA70FQTC5E','91610826590266393W','91610827MA705CXF2K','91610827MA703CCPXF','91610827L04797814E','91610827MA7032FT4X','91610827MA70FK575C','916108277836898514','91610827061945163F','916100007552218217','91610827MA703BQ601','91610827552179360X','91610827MA708YWL19','9161082730562026XQ','91610827MA7094AT02','916108270596677901','91610827338623053A','91610827MA7040FP1T','91610828MA7030X571','91610828MA7MYJ0G84','9161082830551626X8','91610828MA703PB208','91610828MA709TR85H','91610828MA7035J19Q','91610828MA703GAP8E','93610828570669865Y','916108280679278399','93610828MA703MKE86','91610828562200557R','91610828MA7BC7C940','916108280504343257','93610828052138271M','91610828MA708R8J36','93610828559385247F','916108283057501879','9161082859666136XJ','91610828677919502C','91610828661169321T','91610828352252148H','936108285735062871','916108285966825230','91610828MA709KLN51','91610828088015337K','91610800735340321T','91610828MA708URR8U','91610828MA70E0UY5E','91610800MA70AG0322','916108283057533375','91610000305329931J','91610828575612673A','91610828MA70AN8Q0W','9161082806191181XW','91610828MA7032YP28','91610828MA703JJY7E','91610828773832562F','916108295933288522','91610829MA709G5C21','91610829067914085M','91610829MA704APE27','91610829MA70DMLR50','91610829790753292M','91610800790767520W','91610829786975896A','91610829586955030X','91610829MA709Q3L0H','91610830677947677D','916108307769831550','91610830MA7093KD0T','916108305756016913','91610830305391161C','91610830056947332A','916108303522676814','91610830786966180U','91610830MA7038868K','91610830MA70APFT59','91610830719750582L','916108307303912262','916108303056153482','91610830MA7033XL6Y','91610830719747359N','91610830224270253G','91610830MA7039PL1Q','91610830MA7095T23Y','91610830MA708W9N6T','91610830MA7036GJX0','916108303385722175','91610830MA70AC6U08','91610830MA704Q1P39','91610830338743565A','91610830MA7037YE6B','91610830710064009G','91610830555673671H','93610830MA703AM46D','91610830MA7036G47D','91610830MA7050672C','91610830305481837E','91610831MA70323L26','916108000880800750','91610831MA70B73M8U','91610831MA708WCR8K','91610831305767755N','91610831710065220P','91610831727348008E','91610831593341926R','916108310619449025','91610831MA703CJF2E','916108315706692914','91610831MA708R007W','91610831MA70B3MD4G','91610831MA70BD1H0Y','93610831562243709Y','916108313056661962','916108310939512426','91610831MA709N6L13','9161083157781616X6','916100000880671537','91610831MABN470E3F','93610831305343143G','9161082167793717XC','916108213055993173','91610821675105396U','91610821MA7035W466','91610821MA70EU1M4C','916108216847887231','91610800755204183B','91610821MA70978GX5','91610821MA70DTWN1E','91610821MA70DCFF18','91610821MA70FJR21E','91610821305767771W','916100007799307022','91610821677922285W','916100003053012185','91610000748616526W','91610821563791942T','91610821766308324Q','91610821786996363L','91610821MA70CNJ41E','91610000563779951C','916108217135706793','91610821MA70CRQ8X1','91610000732641997R','916100007799482405','91610000059666958C','91610821555656230Q','91610821583545538T','9161000075884416XH','91610821MA703FP576','916108215556937286','91610821MA70362218','91610821MA703BE64X','91610821664106588B','9161082156710197X5','91610821MA708QC6XL','91610821MA70C2Q06B','91610821MA70DKG314','916108217869920396','91610821MA70C2PMXJ','91610821719766197G','91610000710924961H','91610821067914579R','91610821071269955P','91610821MA70BJ9W0C','91610821MA7098JB19','916108215966606585','91610821MA70EC2YXC','91610821MA70CGPH7F','91610821583541000M','91610821559390206W','91610821667969549M','9161082175884213XN','916108217100521205','91610821MA703PT760','91610821338639979E','91610821MA703BHW6U','91610821783666617B','91610821MA70FXBP6K','91610821681596021L','916108216879709774','91610800598796380J','91610800691145662A','91610821691135173P','916108212240045457','91610821224000157T','91610821MA703GB77B','91610821681569381A','916100006847780075','916108216779159829','916108213385789858','91610821583543612D','9161082130538851XE','916108210648495992','91610000664134933F','91610821596655947E','91610821MA709DR57H','916108217552325636','91610821088123194U','91610000745041552L','91610000687992877M','91610821555689542X','91610821677901740Y','91610821677923560K','91610821MA70BFPF2Y','91610821MA709PAA2A','91610000748618126L','91610821MA70EGPX9H','916100002240051930','91610000924001302H','916108215835267406','916108215521580763','916108216751414422','916100005556584045','916108216779105329','91610821562228197B','91610821MA703XYH1W','91610821570659229E','916108217625768311','91610821305496043G','916108216815570738','91610821MA708GH3X4','9161082167510000XL','91610821MA703BEC31','91610821MA703LQGXF','916100000596527830','91610821MA703HPX1T','91610821305323134R','91610821MA703N1B5U','91610821573521946X','91610821MA703A1W1A','91610821305384658A','9161080066117645XR','91610821MA70955UX1','91610821MA70AY2U52','91610000577813989U','91610821MA70CG4B44','91610821694922041J','91610000064835373N','91610821562219223W','916100007100533691','91610000732652506M','916108217353454137','91610821MA7094W97M','91610821MA70381K88','91610000773808634G','91610821MA70BTPE2D','91610000305465386R','91610821MA70FPJ19J','91610821671501594M','91610821MA703316XP','91610000X23601087L','9161000071978482XF','916100006911194994','9161082158697432XC','91610821MA705B6C33','91610821MA6TGWKM6C','91610000710052067J','91610821770025546B','91610000924013143K','916108216911367669','91610821694911481E','91610000671527778F','91610000698436454D','91610821675114583D','91610821664110536C','916108215593819240','91610821681579985G','91610821563789930M','91610000766333263G','916108217869620404','91610821786962796K','91610821677948768B','91610821681588961J','91610000681597913D','91610821684790305C','91610821MA703BX57B','91610821MA703TKP7H','91610821MA70BKG73Y','91610821MA70FUAP10','91610821790760484L','91610821078617967F','91610821MA7098LA63','91610821MA7033LY65','91610821338609446E','91610821338609534W','91610821MA70B7PF0Y','91610821567141533Q','91610821681590201J','916108216815903409','91610000305340444Y','91610000059683299M','916100005622201527','916108215869581355','91610000924003893Q','916100005556826316','91610821684773951D','916100007135061593','91610821691126138J','9161000055219304XB','91610821061925744C','91610821L047817240','916108215807683576','916108216679832524','9161082155216971XT','91610821552195029G','91610821586985766Y','9161082155568221XT','9161082167792431X4','91610821081708229J','91610000773830604T','91610821675119317B','91610821061948145J','9161000092400161X7','91610821074501043F','916108215637906599','916108216847868401','91610821MA70AFP8XK','91610821790779716D','91610821MA709UHQ1A','91610821MA703J6063','9161082108806100X8','91610821305529570Q','91610821MA70950B2G','91610821MA703FB4X5','91610821MA7098KX87','91610821064821756N','91610821305712199T','91610821MA70GQ7Q5E','91610806691107682J','91610821580782810W','9161082155566904XM','91610821586998516U','916100007135040226','91610806741251560K','91610821MA7050AM0A','91610821766336843W','916108216879865066','91610821691147676K','916108216715437436','91610821755200326A','91610821687963603D','91610000224003921H','91610821050402075G','91610821675141573X','9161000071005330XQ','916108212240019344','91610821305412030B','9161082168796362X0','91610821586969627M','91610821MA703QH21F','91610821338655688X','91610821352278970Y','91610821562249481F','91610821MA704UUN2D','91610821MA704G7F25','91610821694924442T','91610000730404314B','91610821570668862F','9161000005968331X1','916108216879635742','91610000MA6TG40X28','91610821555699740T','916100005521646440','91610000710052833G','916100009240134699','91610000567129825J','9161000075521877X8','91610821770031292X','91610821776988992B','91610800MA708L7213','91610821MA703QFH1K','916108216715343455','91610821675144336D','91610893MA70C8BN5R','91610821MA70A5UY7L','91610821MA70ERP17F','91610000741268506A','91610821MA70DKQ35X','91610000577835002Q','916108216715106698','91610821563795767X','91610821770033124M','9161000071350425X8','91610821770043971N','91610821675117004H','916108212240035186','916108215556718796','91610821786998238R','916108216751367583','91610821684770443G','91610821677921872D','91610821338575709B','91610821698418344C','916108216847628063','916108217799382026','91610821698418854R','91610821713506992M','91610821681551456T','91610821MA709MTL5A','916108215593687385','91610821691122380D','91610000677916061H','91610821555684127G','91610000671528076Q','91610821694922463H','916108216911087145','916108216911169173','91610821675145697Y','916108216911304797','91610821583535073B','916108210817431714','91610821MA7034TW33','91610821671537108L','91610821580772153T','916108217552057845','916108215735218745','9161082173534658XU','9161082155219611XA','91610821071278958H','916108210569000161','9161000071005277XF','91610000691125485H','91610821577821532A','91610821671537060C','91610821MA704NDFX6','91610000710052665E','91610821071285285M','91610000577827678B','916108213056545585','916108213057842808','91610000694922009G','916108216715160756','91610821MA7094M77U','9161082155566058X5','9161082179790070XW','91610821338596382X','916108210712746156','91610821567114877Q','91610000577813946E','9161082169841866XW','916100000712732418','91610821586963639H','9161000071357096XA','91610821064803785M','91610821552174447C','91610821MA704JTT54','916108066879758908','91610000684797582J','91610806MA703A6H0H','91610806586970396Q','91610806MA703PKD8X','916108060547617531','91610800694936013W','91610806MA708E9E3C','916108007588273274','91610821MA708UWL0R','91610806MA70ADRH09','91610806305365529M','916108060786458026','91610806078612699W','91610821MA703DEQ3U','916100006879524029','91610800MA70F0X441','91610806586963508Y','91610806MA7096L418','91610800MA708M1D63','91610806MA70391C58','916100009240011505','916108065869736343','91610821758816740M','91610821794117621M','91610821MA70CG834J','91610821MA70D3P75Q','91610821MA70D3PF0H','91610821770017642B','91610821684784298E','91610821MA70AFDX8F','91610821664127514E','91610821MA70CXKU84','91610821MA70A9ND11','9161082176632501XX','91610821675145005D','91610821054788032W','916108217700423972','91610821770013983N','91610821MA70DT2L42','91610821681582199M','91610806MA709B0F16','91610821MA7037WA08','916108213057510246','91610821555695272T','916108215556868517','9161082158075123X8','91610821762578722U','91610821755222453P','91610821MA70A5L79B','916108213053142541','91610821074539041A','916108215637902455','91610806067948461P','91610821MA703AYG0F','91610800752146344X','9161082130559046X1','91610806755247976C','916108217588481798','91610821305664764W','91610821766312040R','91610821786998975C','91610821677912765F','91610821559365318N','91610821MA703BRT54','91610806735348921D','91610821748622598U','91610000748646493C','91610821MA709HTD69','916108215671436353','91610821338638431X','91610821305453131A','91610821305769822Y','91610821305337210M','916108213055523324','916108215807826858','91610821MA708P1D4Q','916100006984015773','916100007769889684','91610821MA708FCP5T','91610821305529183C','91610821677948098N') and xf_hydm is not null and gf_hydm is not null group by xf_shxydm, xf_nsrmc, xf_hydm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, spbm) test

# 核心企业做购方的购买金额数据汇总 3377
select * from (select gf_shxydm, gf_nsrmc, gf_hydm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj_2023_5 where gf_shxydm in ('9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70GG2Y1H','91610800MA703QPKXC','91610802061916178Q','916108007273435491','91610000567118552C','91610893713586830T','916108007869525471','916108007979389782','9161080271004864XG','91610800661152976L','91610802MA704Y7M9M','91610802MA70G53D6M','91610000797940752N','91610802MA7031R94N','91610802MA703WM89H','916108005778342619','91610800752124911M','916108007588338747','91610802223962506N','916108006751043185','91610800677914736G','916108007450080310','91610800773819114A','91610800794100846T','916108006641075481','916108003385989430','91610800552165727D','91610802577836419K','9161080266116183XF','91610800684799641Q','916108006984024304','916108006641132516','916108007625527097','91610802MA70BMX34M','91610800MA70BE0G9E','91610800MA703EW32C','91610800MA70AXHK6D','91610800MA7036C41M','91610800MA704GNH0Q','91610802MA703BWC8F','91610800059688380U','91610800305302050M','91610800MA709M13XP','91610893MA703DEY9X','91610800694918499D','91610800MA7037RA9M','91610822MA7097U17U','91610824MA704D9D0X','91610800067912098G','91610802MA7034XX71','916108933057186623','91610800730410562R','91610800593317221R','91610800MA70988L3J','916108006984411145','91610800677943334K','91610893MA703AHQ4K','91610800675112705T','91610800MA703DR38N','91610893MA703K4C4E','916108006611675026','91610893MA70C3XL5R','91610893MA70AKUT7Y','91610893MA7030276T','91610800067912397E','91610802081747367D','91610800078619321P','91610800223959534H','916108000679086973','91610104MA6U6WPP9U','91610822MA708UM64E','91610802MA70FP667W','91610800698417755J','91610802MA70F2QM2B','91610802MA70CCRJ58','916108003054061837','91610893MA70BUNR99','91610893567149850M','91610800694921348L','91610893MA7092E2XX','91610800MA70319P01','91610800776976764F','91610800555695483W','91610893MA7056GH49','91610800748618329J','91610800064810491X','91610802338656146X','9161080230559847XB','91610800691100720J','91610000661160001T','91610800MA7035TJX3','91610800305315580J','91610800074521407G','916108000679234595','91610802573522607J','91610800661157478J','91610800088193408Q','91610800MA704B3A2P','91610800088079154R','916108020569016092','91610800MA7036T07K','91610000X2358669XB','91610800061931191K','91610802338661033H','91610800694930906W','91610800575649337Q','916108023055573957','916108000939802706','916100002239672519','916108026879737809','916100007100518818','91610802MA70A6FN4M','91610000X23586219E','91610800671536834E','91610800691127712R','916100006815997672','91610802667987720K','91610802MA7038FT0M','91610802552178923N','91610000223967235K','916108006679710248','91610000745037668H','91610000223965838G','91610802570668811N','916108006751155433','91610000710051814A','91610000766346072G','91610800598779185W','91610802MA708F2G8K','91610802MA709Y9P37','916108007588160388','916108003054001360','91610800691131228D','91610000786958834R','91610800MA703RJE5P','91610802MA7055P507','916108020547928919','916100005593700612','91610802681595256T','91610800MA704B3W2A','916100006847510073','916108020880671880','91610000735388763B','91610000773844942E','91610800752129712H','916108007979285936','91610000X23586251R','91610000752131054Q','91610802MA7038KK91','91610802773841215D','916108005778349158','9161080009394451XQ','91610800061909963N','91610802MA7038GD6G','93610802MA703FR92F','91610000338700928R','91610800552158025D','91610802691100835M','91610802563788284P','91610802MA703J1L6T','91610800MA7036E79W','91610802061927221J','91610800590253023H','9161080276255849XM','91610802MA704D189B','91610821MA703YBK2D','91610800MA703QW35X','91610802305755922F','91610802078614125H','91610802MA70D7448F','91610802MA70ATEH12','91610000059676961F','91610802MA7038MAX1','91610000593327534K','91610802MA703HWE04','916108023386836119','9161080006481082X4','91610800590260549L','91610800MA70371J7F','91610000797938927G','91610800596685070J','9161000056714796XP','91610800684794322L','91610823555664310J','91610823755214998B','91610000755225347C','91610000758832169D','916108006815878706','916108006911278008','916100006984246809','916100006984339153','91610000710055196X','916100007273509845','91610000755214488A','91610000779910293J','91610000758810349Q','916108232240609348','91610000224061857H','916108935521883122','916100005556550942','916100006815955200','91610000687954301Y','91610000790751174B','91610823MA7035CD25','916108230880722358','91610823305604112X','916108230881953581','916108237907734373','91610823MA703ELY6H','91610823MA70CG8N8F','91610823671529749H','91610823MA7032KGXX','91610823MA70D1G21X','91610823MA70G73N3H','91610823MA703DDX4B','91610823MA7097DG1P','916108237941275554','91610823MA7034CJ9Q','91610823MA70GABW64','91610823MA7032U61A','91610823MA7030E21X','91610823MA709HYNXD','916108233054100434','91610823MA70EP4G0U','916100006984084291','916108230521323633','916108230882445003','91610823MA704R7J5F','91610000773800042F','91610000586982709M','91610823056903460W','91610823MA708N92XX','91610823305726101A','91610823MA708U9U8T','91610823MA70CM4M03','91610823MA70341H2K','91610893MA70F7MT80','91610823MA7030FM17','91610893MA70AULF26','91610800684795675B','91610893088247840A','916108930882378452','91610800567137999E','91610823MA70D01L9W','91610823MA7030WJ5A','91610822552193103H','91610822583542433E','916100006751366436','91610000687952218H','91610822694938211C','916100007769886696','9161082222403383X0','916108225593913130','91610822567139548R','91610800671529239R','916108222240319575','91610822MA70CYYF7Q','91610800223930635R','916108225593639100','91610822MA703A0A5W','91610822583526038L','91610822MA709EAN9R','91610822710053561Q','916108226715094065','91610822MA708J3C27','916108223387099570','9161082266411342X1','9161082267152645XB','916108226984006033','91610822MA703WCN8Q','91610822MA70DF1676','916108222240308741','916108222240302172','91610822224030882U','91610000577805794Q','91610822224032482K','916108227100544680','91610822766320980Q','91610822586971540Q','91610822071284629R','91610822MA70BE3W9P','91610822MA70F1FP48','91610822MA7BK9K43D','91610822MA709Y0M1T','91610822MA709Y7E08','91610822MA704D4D9G','91610822924032280H','91610822MA709N423A','91610822MA709HGW9P','91610822MA7040Q74E','91610822MA70G7EX5Y','91610000661170904Y','91610000224030946L','91610822797925923U','916108225933488027','91610822224032386A','91610000MA6TG1L91C','916108223056392271','9161082230539678XQ','91610822555655537N','91610822762574940L','9161082206868156X4','916100002240321328','91610822684783682Q','91610822735351493Q','916108227135586612','91610822752112793E','91610822224041784B','91610822573535061G','91610822698443996B','91610822552179029C','91610000580766765J','91610822081718611M','91610822677931237W','91610822MA703BPQ8E','91610000559374302W','91610822694908266R','91610822694929032M','91610000797915039F','91610822093914935U','91610822555666885M','916108223386303333','91610822583546661K','91610822MA70317P8A','91610822671525115L','91610822577800053T','91610822087963650K','91610822797937238T','91610822667973185H','916100005556762074','91610822687981123J','91610822567149623M','916108225869772994','9161082269494526XA','916108227663422157','916108225593843242','916108226911466229','91610822575627197H','91610822677911949C','91610822056917184G','91610822MA7059MH04','91610000770001587F','91610800719785144W','91610000567108776E','91610000758819036E','91610822786997948R','91610822563772274J','91610000224057049W','916108225521605984','91610822MA709DFT7W','916108225835113037','91610822MA703HB02A','916100000504386259','916108225869795931','91610822MA708MR72L','91610822735347005J','91610800770011101Y','91610800675123180G','9161000077382866XG','916108220569493036','91610822MA70DY9C9C','916100000817239128','91610000713558602R','91610822570658605X','91610000758843706Y','91610822MA7036PA3C','91610000061914033T','91610822224033581D','916108220619348353','91610822694914308A','9161082273039809X5','916100000521118534','91610822552168012N','91610000580775995H','916108227486349192','91610822766312091G','9161082276633061X2','91610822MA708Y049A','91610822710054302H','91610822687992703H','91610822766330636W','91610822MA70FGBR66','91610822078635727F','91610822078636033K','91610822MA704G881Y','91610822667965572L','916108227799344049','91610822MA70B4TQ8T','916108226911259879','916108222240334186','91610822MA70A6HJ4U','91610822MA709R8R4H','9161082258078527XY','91610822687983735A','91610822710054791E','91610822MA708R672M','91610822577843598F','916108225869837874','9161082257069098XN','91610822694927918T','91610822797911572J','916108225637837070','916108226949183843','916108226949327095','91610822MA704CN193','9161082271355912XF','91610822567124282E','916108225671304988','91610000710054484G','916108225735414537','916100002240314638','916108225778382625','91610822773826787T','91610000567136590B','91610822078640366D','91610822094023050T','9161000056710680XM','91610822681555895X','91610000924044062B','91610822552163879U','91610822305318853B','91610000567138334L','916108226879560162','91610822567139193Q','91610000681575589J','916108226679923350','91610821MA70343P02','916108930881217117','91610822305621270Y','916100006879777738','91610822050401881R','91610822MA704K1D9T','91610822748606993Y','91610822MA70B3171E','916108225807966659','91610822MA70AB7K22','91610822MA7030P565','91610822755225080Y','91610000X23606048P','916100005556688484','91610000598765891B','916100009240407903','91610822766305764C','91610000924030840U','9161000055935482XF','91610000567118544H','91610000719708587F','91610822562241391F','916108225521576557','91610822MA7030C7X0','9161082255566867X2','916108225735291204','916108226847965391','9161082271355816XC','91610000567129809W','91610000687954512E','91610822667970734M','91610822748616366T','91610000681575597D','91610000691106065G','916108227869769281','9161082269490584XB','916108223057890497','91610000562225156P','91610822737993836U','91610822MA7093A6XP','91610822563764290Y','916108225556745861','9161082266796860XC','91610822MA708N4B1M','91610000681575570M','9161082256712766XC','91610822727369589U','91610822580776779W','916108225902933593','916108220648211664','9161082207127640X1','91610822559385992C','9161082256224308XJ','9161082258078631XM','91610822590295784U','91610822593312017P','91610822596695017M','916108225987548772','916108226779018201','91610822681564126J','91610822MA709AGT5N','91610822MA70A9UE46','91610822MA708KQDXA','91610822552191175G','91610822555680564K','91610822552154948T','91610822671520461A','916100006847787282','91610822MA708YJX74','91610000567103318K','91610822563762148E','91610822562224372N','91610822555688793Q','9161082257351819X6','91610822583526636G','916100007099182651','916100005671063088','916108225807676532','91610822580784090L','9161000059027254XF','91610822MA7032CD47','916108220745134073','91610822766331508H','9161000056377267XH','916100005556789748','91610000567138887F','916108226715375708','916108227450323733','9161082230534244X0','9161082208819745XP','91610822573530391G','91610000681599492L','91610822MA7039625F','916108220786281646','91610000078649651R','91610000305788468G','91610822586987462E','91610822MA70384L5F','91610822MA703BUW9C','91610824MA708G4C05','916108245756426370','91610824352260279W','91610824338607352A','91610824MA709B7K7N','91610824MA7045E8X5','916108243055203063','91610824MA7043WQ6F','91610824MA7045KE7K','91610824684776511K','91610824MA7052RM4M','91610824305602280L','91610824MA70404L9N','91610824MA708T379E','91610824713561318K','91610824590274190L','9161082405479693X8','916108246949093223','91610800677905610M','91610000698419427R','91610824338748948N','91610824MA70A3CU40','91610824067940013A','91610824MA7031PR97','91610824093912366E','91610824052125550G','91610824555662817L','91610824586989142N','91610824MA70DAAF42','91610824667959658W','91610824338721649N','91610824305379402Y','916108245593799981','91610824583543516F','91610824MA703AAQXB','91610824675115447J','91610824MA70EFL6XT','91610824MABQ6NL698','91610824580788446J','91610824590270069T','91610824MA70D7CY47','91610824577821241J','916108003057434031','91610824059660193A','91610824559383436L','916108243054516389','91610824MA703BB57G','91610824MA708NC530','91610824305645491Q','91610824074522522Q','91610824093913721Q','91610824059693913C','91610824570657119L','91610825710058346A','916108253055799146','916108253057807499','91610825MA703BP13J','916108253055693618','91610825064823620K','91610825MA70GL610E','91610825MA703GRQ4W','91610825094008096H','91610825088196684W','916108250939461010','91610825305710716N','91610825MA7K37P61M','91610825MA70GHF09N','91610825MA70GGYFX2','916108253055136327','91610825MA708W73XU','9161082530550930XA','91610825056929273K','91610825MA7097R42F','91610825MA703D773C','91610825059666253M','916108250648189605','916108257100585227','916108255807949859','91610825770016498A','916108252241200281','916108257941124337','91610825305646531X','91610825305721597M','916108257869947471','916108255902738171','91610825MA704RJJ1E','91610825MA7031MW0H','916108250817158759','9161082530538018XY','91610825305557133R','916108253057060719','91610800MA70B34385','916108253054304235','916108253057555513','91610825MA70GGKC69','91610825338670220D','91610825MA7033R222','916108250939154365','916108257869852184','916108250881238057','91610825687982230J','91610825MA703C1J05','9161082569842068XN','916108250859923274','91610825MA70A9HP1Q','916108255521915890','9161082509390645XX','91610825MA70318W1T','91610826MA7033UK93','91610826MA70CG498N','91610800223930176L','91610826583507806A','916108266641120996','916108265593534054','91610800X236250463','9161082630547280X6','916108266949220255','91610826MA70333X3J','91610826MA703MQK5P','91610826064845432U','91610826MA70FQTC5E','91610826590266393W','91610827MA705CXF2K','91610827MA703CCPXF','91610827L04797814E','91610827MA7032FT4X','91610827MA70FK575C','916108277836898514','91610827061945163F','916100007552218217','91610827MA703BQ601','91610827552179360X','91610827MA708YWL19','9161082730562026XQ','91610827MA7094AT02','916108270596677901','91610827338623053A','91610827MA7040FP1T','91610828MA7030X571','91610828MA7MYJ0G84','9161082830551626X8','91610828MA703PB208','91610828MA709TR85H','91610828MA7035J19Q','91610828MA703GAP8E','93610828570669865Y','916108280679278399','93610828MA703MKE86','91610828562200557R','91610828MA7BC7C940','916108280504343257','93610828052138271M','91610828MA708R8J36','93610828559385247F','916108283057501879','9161082859666136XJ','91610828677919502C','91610828661169321T','91610828352252148H','936108285735062871','916108285966825230','91610828MA709KLN51','91610828088015337K','91610800735340321T','91610828MA708URR8U','91610828MA70E0UY5E','91610800MA70AG0322','916108283057533375','91610000305329931J','91610828575612673A','91610828MA70AN8Q0W','9161082806191181XW','91610828MA7032YP28','91610828MA703JJY7E','91610828773832562F','916108295933288522','91610829MA709G5C21','91610829067914085M','91610829MA704APE27','91610829MA70DMLR50','91610829790753292M','91610800790767520W','91610829786975896A','91610829586955030X','91610829MA709Q3L0H','91610830677947677D','916108307769831550','91610830MA7093KD0T','916108305756016913','91610830305391161C','91610830056947332A','916108303522676814','91610830786966180U','91610830MA7038868K','91610830MA70APFT59','91610830719750582L','916108307303912262','916108303056153482','91610830MA7033XL6Y','91610830719747359N','91610830224270253G','91610830MA7039PL1Q','91610830MA7095T23Y','91610830MA708W9N6T','91610830MA7036GJX0','916108303385722175','91610830MA70AC6U08','91610830MA704Q1P39','91610830338743565A','91610830MA7037YE6B','91610830710064009G','91610830555673671H','93610830MA703AM46D','91610830MA7036G47D','91610830MA7050672C','91610830305481837E','91610831MA70323L26','916108000880800750','91610831MA70B73M8U','91610831MA708WCR8K','91610831305767755N','91610831710065220P','91610831727348008E','91610831593341926R','916108310619449025','91610831MA703CJF2E','916108315706692914','91610831MA708R007W','91610831MA70B3MD4G','91610831MA70BD1H0Y','93610831562243709Y','916108313056661962','916108310939512426','91610831MA709N6L13','9161083157781616X6','916100000880671537','91610831MABN470E3F','93610831305343143G','9161082167793717XC','916108213055993173','91610821675105396U','91610821MA7035W466','91610821MA70EU1M4C','916108216847887231','91610800755204183B','91610821MA70978GX5','91610821MA70DTWN1E','91610821MA70DCFF18','91610821MA70FJR21E','91610821305767771W','916100007799307022','91610821677922285W','916100003053012185','91610000748616526W','91610821563791942T','91610821766308324Q','91610821786996363L','91610821MA70CNJ41E','91610000563779951C','916108217135706793','91610821MA70CRQ8X1','91610000732641997R','916100007799482405','91610000059666958C','91610821555656230Q','91610821583545538T','9161000075884416XH','91610821MA703FP576','916108215556937286','91610821MA70362218','91610821MA703BE64X','91610821664106588B','9161082156710197X5','91610821MA708QC6XL','91610821MA70C2Q06B','91610821MA70DKG314','916108217869920396','91610821MA70C2PMXJ','91610821719766197G','91610000710924961H','91610821067914579R','91610821071269955P','91610821MA70BJ9W0C','91610821MA7098JB19','916108215966606585','91610821MA70EC2YXC','91610821MA70CGPH7F','91610821583541000M','91610821559390206W','91610821667969549M','9161082175884213XN','916108217100521205','91610821MA703PT760','91610821338639979E','91610821MA703BHW6U','91610821783666617B','91610821MA70FXBP6K','91610821681596021L','916108216879709774','91610800598796380J','91610800691145662A','91610821691135173P','916108212240045457','91610821224000157T','91610821MA703GB77B','91610821681569381A','916100006847780075','916108216779159829','916108213385789858','91610821583543612D','9161082130538851XE','916108210648495992','91610000664134933F','91610821596655947E','91610821MA709DR57H','916108217552325636','91610821088123194U','91610000745041552L','91610000687992877M','91610821555689542X','91610821677901740Y','91610821677923560K','91610821MA70BFPF2Y','91610821MA709PAA2A','91610000748618126L','91610821MA70EGPX9H','916100002240051930','91610000924001302H','916108215835267406','916108215521580763','916108216751414422','916100005556584045','916108216779105329','91610821562228197B','91610821MA703XYH1W','91610821570659229E','916108217625768311','91610821305496043G','916108216815570738','91610821MA708GH3X4','9161082167510000XL','91610821MA703BEC31','91610821MA703LQGXF','916100000596527830','91610821MA703HPX1T','91610821305323134R','91610821MA703N1B5U','91610821573521946X','91610821MA703A1W1A','91610821305384658A','9161080066117645XR','91610821MA70955UX1','91610821MA70AY2U52','91610000577813989U','91610821MA70CG4B44','91610821694922041J','91610000064835373N','91610821562219223W','916100007100533691','91610000732652506M','916108217353454137','91610821MA7094W97M','91610821MA70381K88','91610000773808634G','91610821MA70BTPE2D','91610000305465386R','91610821MA70FPJ19J','91610821671501594M','91610821MA703316XP','91610000X23601087L','9161000071978482XF','916100006911194994','9161082158697432XC','91610821MA705B6C33','91610821MA6TGWKM6C','91610000710052067J','91610821770025546B','91610000924013143K','916108216911367669','91610821694911481E','91610000671527778F','91610000698436454D','91610821675114583D','91610821664110536C','916108215593819240','91610821681579985G','91610821563789930M','91610000766333263G','916108217869620404','91610821786962796K','91610821677948768B','91610821681588961J','91610000681597913D','91610821684790305C','91610821MA703BX57B','91610821MA703TKP7H','91610821MA70BKG73Y','91610821MA70FUAP10','91610821790760484L','91610821078617967F','91610821MA7098LA63','91610821MA7033LY65','91610821338609446E','91610821338609534W','91610821MA70B7PF0Y','91610821567141533Q','91610821681590201J','916108216815903409','91610000305340444Y','91610000059683299M','916100005622201527','916108215869581355','91610000924003893Q','916100005556826316','91610821684773951D','916100007135061593','91610821691126138J','9161000055219304XB','91610821061925744C','91610821L047817240','916108215807683576','916108216679832524','9161082155216971XT','91610821552195029G','91610821586985766Y','9161082155568221XT','9161082167792431X4','91610821081708229J','91610000773830604T','91610821675119317B','91610821061948145J','9161000092400161X7','91610821074501043F','916108215637906599','916108216847868401','91610821MA70AFP8XK','91610821790779716D','91610821MA709UHQ1A','91610821MA703J6063','9161082108806100X8','91610821305529570Q','91610821MA70950B2G','91610821MA703FB4X5','91610821MA7098KX87','91610821064821756N','91610821305712199T','91610821MA70GQ7Q5E','91610806691107682J','91610821580782810W','9161082155566904XM','91610821586998516U','916100007135040226','91610806741251560K','91610821MA7050AM0A','91610821766336843W','916108216879865066','91610821691147676K','916108216715437436','91610821755200326A','91610821687963603D','91610000224003921H','91610821050402075G','91610821675141573X','9161000071005330XQ','916108212240019344','91610821305412030B','9161082168796362X0','91610821586969627M','91610821MA703QH21F','91610821338655688X','91610821352278970Y','91610821562249481F','91610821MA704UUN2D','91610821MA704G7F25','91610821694924442T','91610000730404314B','91610821570668862F','9161000005968331X1','916108216879635742','91610000MA6TG40X28','91610821555699740T','916100005521646440','91610000710052833G','916100009240134699','91610000567129825J','9161000075521877X8','91610821770031292X','91610821776988992B','91610800MA708L7213','91610821MA703QFH1K','916108216715343455','91610821675144336D','91610893MA70C8BN5R','91610821MA70A5UY7L','91610821MA70ERP17F','91610000741268506A','91610821MA70DKQ35X','91610000577835002Q','916108216715106698','91610821563795767X','91610821770033124M','9161000071350425X8','91610821770043971N','91610821675117004H','916108212240035186','916108215556718796','91610821786998238R','916108216751367583','91610821684770443G','91610821677921872D','91610821338575709B','91610821698418344C','916108216847628063','916108217799382026','91610821698418854R','91610821713506992M','91610821681551456T','91610821MA709MTL5A','916108215593687385','91610821691122380D','91610000677916061H','91610821555684127G','91610000671528076Q','91610821694922463H','916108216911087145','916108216911169173','91610821675145697Y','916108216911304797','91610821583535073B','916108210817431714','91610821MA7034TW33','91610821671537108L','91610821580772153T','916108217552057845','916108215735218745','9161082173534658XU','9161082155219611XA','91610821071278958H','916108210569000161','9161000071005277XF','91610000691125485H','91610821577821532A','91610821671537060C','91610821MA704NDFX6','91610000710052665E','91610821071285285M','91610000577827678B','916108213056545585','916108213057842808','91610000694922009G','916108216715160756','91610821MA7094M77U','9161082155566058X5','9161082179790070XW','91610821338596382X','916108210712746156','91610821567114877Q','91610000577813946E','9161082169841866XW','916100000712732418','91610821586963639H','9161000071357096XA','91610821064803785M','91610821552174447C','91610821MA704JTT54','916108066879758908','91610000684797582J','91610806MA703A6H0H','91610806586970396Q','91610806MA703PKD8X','916108060547617531','91610800694936013W','91610806MA708E9E3C','916108007588273274','91610821MA708UWL0R','91610806MA70ADRH09','91610806305365529M','916108060786458026','91610806078612699W','91610821MA703DEQ3U','916100006879524029','91610800MA70F0X441','91610806586963508Y','91610806MA7096L418','91610800MA708M1D63','91610806MA70391C58','916100009240011505','916108065869736343','91610821758816740M','91610821794117621M','91610821MA70CG834J','91610821MA70D3P75Q','91610821MA70D3PF0H','91610821770017642B','91610821684784298E','91610821MA70AFDX8F','91610821664127514E','91610821MA70CXKU84','91610821MA70A9ND11','9161082176632501XX','91610821675145005D','91610821054788032W','916108217700423972','91610821770013983N','91610821MA70DT2L42','91610821681582199M','91610806MA709B0F16','91610821MA7037WA08','916108213057510246','91610821555695272T','916108215556868517','9161082158075123X8','91610821762578722U','91610821755222453P','91610821MA70A5L79B','916108213053142541','91610821074539041A','916108215637902455','91610806067948461P','91610821MA703AYG0F','91610800752146344X','9161082130559046X1','91610806755247976C','916108217588481798','91610821305664764W','91610821766312040R','91610821786998975C','91610821677912765F','91610821559365318N','91610821MA703BRT54','91610806735348921D','91610821748622598U','91610000748646493C','91610821MA709HTD69','916108215671436353','91610821338638431X','91610821305453131A','91610821305769822Y','91610821305337210M','916108213055523324','916108215807826858','91610821MA708P1D4Q','916100006984015773','916100007769889684','91610821MA708FCP5T','91610821305529183C','91610821677948098N') and xf_hydm is not null and gf_hydm is not null group by gf_shxydm, gf_nsrmc, gf_hydm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test
# 86028
select * from (select gf_shxydm, gf_nsrmc, gf_hydm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, spbm, sum(je) from datapool.t_data_sh_new_qyjfkpsj_2023_5 where gf_shxydm in ('9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70GG2Y1H','91610800MA703QPKXC','91610802061916178Q','916108007273435491','91610000567118552C','91610893713586830T','916108007869525471','916108007979389782','9161080271004864XG','91610800661152976L','91610802MA704Y7M9M','91610802MA70G53D6M','91610000797940752N','91610802MA7031R94N','91610802MA703WM89H','916108005778342619','91610800752124911M','916108007588338747','91610802223962506N','916108006751043185','91610800677914736G','916108007450080310','91610800773819114A','91610800794100846T','916108006641075481','916108003385989430','91610800552165727D','91610802577836419K','9161080266116183XF','91610800684799641Q','916108006984024304','916108006641132516','916108007625527097','91610802MA70BMX34M','91610800MA70BE0G9E','91610800MA703EW32C','91610800MA70AXHK6D','91610800MA7036C41M','91610800MA704GNH0Q','91610802MA703BWC8F','91610800059688380U','91610800305302050M','91610800MA709M13XP','91610893MA703DEY9X','91610800694918499D','91610800MA7037RA9M','91610822MA7097U17U','91610824MA704D9D0X','91610800067912098G','91610802MA7034XX71','916108933057186623','91610800730410562R','91610800593317221R','91610800MA70988L3J','916108006984411145','91610800677943334K','91610893MA703AHQ4K','91610800675112705T','91610800MA703DR38N','91610893MA703K4C4E','916108006611675026','91610893MA70C3XL5R','91610893MA70AKUT7Y','91610893MA7030276T','91610800067912397E','91610802081747367D','91610800078619321P','91610800223959534H','916108000679086973','91610104MA6U6WPP9U','91610822MA708UM64E','91610802MA70FP667W','91610800698417755J','91610802MA70F2QM2B','91610802MA70CCRJ58','916108003054061837','91610893MA70BUNR99','91610893567149850M','91610800694921348L','91610893MA7092E2XX','91610800MA70319P01','91610800776976764F','91610800555695483W','91610893MA7056GH49','91610800748618329J','91610800064810491X','91610802338656146X','9161080230559847XB','91610800691100720J','91610000661160001T','91610800MA7035TJX3','91610800305315580J','91610800074521407G','916108000679234595','91610802573522607J','91610800661157478J','91610800088193408Q','91610800MA704B3A2P','91610800088079154R','916108020569016092','91610800MA7036T07K','91610000X2358669XB','91610800061931191K','91610802338661033H','91610800694930906W','91610800575649337Q','916108023055573957','916108000939802706','916100002239672519','916108026879737809','916100007100518818','91610802MA70A6FN4M','91610000X23586219E','91610800671536834E','91610800691127712R','916100006815997672','91610802667987720K','91610802MA7038FT0M','91610802552178923N','91610000223967235K','916108006679710248','91610000745037668H','91610000223965838G','91610802570668811N','916108006751155433','91610000710051814A','91610000766346072G','91610800598779185W','91610802MA708F2G8K','91610802MA709Y9P37','916108007588160388','916108003054001360','91610800691131228D','91610000786958834R','91610800MA703RJE5P','91610802MA7055P507','916108020547928919','916100005593700612','91610802681595256T','91610800MA704B3W2A','916100006847510073','916108020880671880','91610000735388763B','91610000773844942E','91610800752129712H','916108007979285936','91610000X23586251R','91610000752131054Q','91610802MA7038KK91','91610802773841215D','916108005778349158','9161080009394451XQ','91610800061909963N','91610802MA7038GD6G','93610802MA703FR92F','91610000338700928R','91610800552158025D','91610802691100835M','91610802563788284P','91610802MA703J1L6T','91610800MA7036E79W','91610802061927221J','91610800590253023H','9161080276255849XM','91610802MA704D189B','91610821MA703YBK2D','91610800MA703QW35X','91610802305755922F','91610802078614125H','91610802MA70D7448F','91610802MA70ATEH12','91610000059676961F','91610802MA7038MAX1','91610000593327534K','91610802MA703HWE04','916108023386836119','9161080006481082X4','91610800590260549L','91610800MA70371J7F','91610000797938927G','91610800596685070J','9161000056714796XP','91610800684794322L','91610823555664310J','91610823755214998B','91610000755225347C','91610000758832169D','916108006815878706','916108006911278008','916100006984246809','916100006984339153','91610000710055196X','916100007273509845','91610000755214488A','91610000779910293J','91610000758810349Q','916108232240609348','91610000224061857H','916108935521883122','916100005556550942','916100006815955200','91610000687954301Y','91610000790751174B','91610823MA7035CD25','916108230880722358','91610823305604112X','916108230881953581','916108237907734373','91610823MA703ELY6H','91610823MA70CG8N8F','91610823671529749H','91610823MA7032KGXX','91610823MA70D1G21X','91610823MA70G73N3H','91610823MA703DDX4B','91610823MA7097DG1P','916108237941275554','91610823MA7034CJ9Q','91610823MA70GABW64','91610823MA7032U61A','91610823MA7030E21X','91610823MA709HYNXD','916108233054100434','91610823MA70EP4G0U','916100006984084291','916108230521323633','916108230882445003','91610823MA704R7J5F','91610000773800042F','91610000586982709M','91610823056903460W','91610823MA708N92XX','91610823305726101A','91610823MA708U9U8T','91610823MA70CM4M03','91610823MA70341H2K','91610893MA70F7MT80','91610823MA7030FM17','91610893MA70AULF26','91610800684795675B','91610893088247840A','916108930882378452','91610800567137999E','91610823MA70D01L9W','91610823MA7030WJ5A','91610822552193103H','91610822583542433E','916100006751366436','91610000687952218H','91610822694938211C','916100007769886696','9161082222403383X0','916108225593913130','91610822567139548R','91610800671529239R','916108222240319575','91610822MA70CYYF7Q','91610800223930635R','916108225593639100','91610822MA703A0A5W','91610822583526038L','91610822MA709EAN9R','91610822710053561Q','916108226715094065','91610822MA708J3C27','916108223387099570','9161082266411342X1','9161082267152645XB','916108226984006033','91610822MA703WCN8Q','91610822MA70DF1676','916108222240308741','916108222240302172','91610822224030882U','91610000577805794Q','91610822224032482K','916108227100544680','91610822766320980Q','91610822586971540Q','91610822071284629R','91610822MA70BE3W9P','91610822MA70F1FP48','91610822MA7BK9K43D','91610822MA709Y0M1T','91610822MA709Y7E08','91610822MA704D4D9G','91610822924032280H','91610822MA709N423A','91610822MA709HGW9P','91610822MA7040Q74E','91610822MA70G7EX5Y','91610000661170904Y','91610000224030946L','91610822797925923U','916108225933488027','91610822224032386A','91610000MA6TG1L91C','916108223056392271','9161082230539678XQ','91610822555655537N','91610822762574940L','9161082206868156X4','916100002240321328','91610822684783682Q','91610822735351493Q','916108227135586612','91610822752112793E','91610822224041784B','91610822573535061G','91610822698443996B','91610822552179029C','91610000580766765J','91610822081718611M','91610822677931237W','91610822MA703BPQ8E','91610000559374302W','91610822694908266R','91610822694929032M','91610000797915039F','91610822093914935U','91610822555666885M','916108223386303333','91610822583546661K','91610822MA70317P8A','91610822671525115L','91610822577800053T','91610822087963650K','91610822797937238T','91610822667973185H','916100005556762074','91610822687981123J','91610822567149623M','916108225869772994','9161082269494526XA','916108227663422157','916108225593843242','916108226911466229','91610822575627197H','91610822677911949C','91610822056917184G','91610822MA7059MH04','91610000770001587F','91610800719785144W','91610000567108776E','91610000758819036E','91610822786997948R','91610822563772274J','91610000224057049W','916108225521605984','91610822MA709DFT7W','916108225835113037','91610822MA703HB02A','916100000504386259','916108225869795931','91610822MA708MR72L','91610822735347005J','91610800770011101Y','91610800675123180G','9161000077382866XG','916108220569493036','91610822MA70DY9C9C','916100000817239128','91610000713558602R','91610822570658605X','91610000758843706Y','91610822MA7036PA3C','91610000061914033T','91610822224033581D','916108220619348353','91610822694914308A','9161082273039809X5','916100000521118534','91610822552168012N','91610000580775995H','916108227486349192','91610822766312091G','9161082276633061X2','91610822MA708Y049A','91610822710054302H','91610822687992703H','91610822766330636W','91610822MA70FGBR66','91610822078635727F','91610822078636033K','91610822MA704G881Y','91610822667965572L','916108227799344049','91610822MA70B4TQ8T','916108226911259879','916108222240334186','91610822MA70A6HJ4U','91610822MA709R8R4H','9161082258078527XY','91610822687983735A','91610822710054791E','91610822MA708R672M','91610822577843598F','916108225869837874','9161082257069098XN','91610822694927918T','91610822797911572J','916108225637837070','916108226949183843','916108226949327095','91610822MA704CN193','9161082271355912XF','91610822567124282E','916108225671304988','91610000710054484G','916108225735414537','916100002240314638','916108225778382625','91610822773826787T','91610000567136590B','91610822078640366D','91610822094023050T','9161000056710680XM','91610822681555895X','91610000924044062B','91610822552163879U','91610822305318853B','91610000567138334L','916108226879560162','91610822567139193Q','91610000681575589J','916108226679923350','91610821MA70343P02','916108930881217117','91610822305621270Y','916100006879777738','91610822050401881R','91610822MA704K1D9T','91610822748606993Y','91610822MA70B3171E','916108225807966659','91610822MA70AB7K22','91610822MA7030P565','91610822755225080Y','91610000X23606048P','916100005556688484','91610000598765891B','916100009240407903','91610822766305764C','91610000924030840U','9161000055935482XF','91610000567118544H','91610000719708587F','91610822562241391F','916108225521576557','91610822MA7030C7X0','9161082255566867X2','916108225735291204','916108226847965391','9161082271355816XC','91610000567129809W','91610000687954512E','91610822667970734M','91610822748616366T','91610000681575597D','91610000691106065G','916108227869769281','9161082269490584XB','916108223057890497','91610000562225156P','91610822737993836U','91610822MA7093A6XP','91610822563764290Y','916108225556745861','9161082266796860XC','91610822MA708N4B1M','91610000681575570M','9161082256712766XC','91610822727369589U','91610822580776779W','916108225902933593','916108220648211664','9161082207127640X1','91610822559385992C','9161082256224308XJ','9161082258078631XM','91610822590295784U','91610822593312017P','91610822596695017M','916108225987548772','916108226779018201','91610822681564126J','91610822MA709AGT5N','91610822MA70A9UE46','91610822MA708KQDXA','91610822552191175G','91610822555680564K','91610822552154948T','91610822671520461A','916100006847787282','91610822MA708YJX74','91610000567103318K','91610822563762148E','91610822562224372N','91610822555688793Q','9161082257351819X6','91610822583526636G','916100007099182651','916100005671063088','916108225807676532','91610822580784090L','9161000059027254XF','91610822MA7032CD47','916108220745134073','91610822766331508H','9161000056377267XH','916100005556789748','91610000567138887F','916108226715375708','916108227450323733','9161082230534244X0','9161082208819745XP','91610822573530391G','91610000681599492L','91610822MA7039625F','916108220786281646','91610000078649651R','91610000305788468G','91610822586987462E','91610822MA70384L5F','91610822MA703BUW9C','91610824MA708G4C05','916108245756426370','91610824352260279W','91610824338607352A','91610824MA709B7K7N','91610824MA7045E8X5','916108243055203063','91610824MA7043WQ6F','91610824MA7045KE7K','91610824684776511K','91610824MA7052RM4M','91610824305602280L','91610824MA70404L9N','91610824MA708T379E','91610824713561318K','91610824590274190L','9161082405479693X8','916108246949093223','91610800677905610M','91610000698419427R','91610824338748948N','91610824MA70A3CU40','91610824067940013A','91610824MA7031PR97','91610824093912366E','91610824052125550G','91610824555662817L','91610824586989142N','91610824MA70DAAF42','91610824667959658W','91610824338721649N','91610824305379402Y','916108245593799981','91610824583543516F','91610824MA703AAQXB','91610824675115447J','91610824MA70EFL6XT','91610824MABQ6NL698','91610824580788446J','91610824590270069T','91610824MA70D7CY47','91610824577821241J','916108003057434031','91610824059660193A','91610824559383436L','916108243054516389','91610824MA703BB57G','91610824MA708NC530','91610824305645491Q','91610824074522522Q','91610824093913721Q','91610824059693913C','91610824570657119L','91610825710058346A','916108253055799146','916108253057807499','91610825MA703BP13J','916108253055693618','91610825064823620K','91610825MA70GL610E','91610825MA703GRQ4W','91610825094008096H','91610825088196684W','916108250939461010','91610825305710716N','91610825MA7K37P61M','91610825MA70GHF09N','91610825MA70GGYFX2','916108253055136327','91610825MA708W73XU','9161082530550930XA','91610825056929273K','91610825MA7097R42F','91610825MA703D773C','91610825059666253M','916108250648189605','916108257100585227','916108255807949859','91610825770016498A','916108252241200281','916108257941124337','91610825305646531X','91610825305721597M','916108257869947471','916108255902738171','91610825MA704RJJ1E','91610825MA7031MW0H','916108250817158759','9161082530538018XY','91610825305557133R','916108253057060719','91610800MA70B34385','916108253054304235','916108253057555513','91610825MA70GGKC69','91610825338670220D','91610825MA7033R222','916108250939154365','916108257869852184','916108250881238057','91610825687982230J','91610825MA703C1J05','9161082569842068XN','916108250859923274','91610825MA70A9HP1Q','916108255521915890','9161082509390645XX','91610825MA70318W1T','91610826MA7033UK93','91610826MA70CG498N','91610800223930176L','91610826583507806A','916108266641120996','916108265593534054','91610800X236250463','9161082630547280X6','916108266949220255','91610826MA70333X3J','91610826MA703MQK5P','91610826064845432U','91610826MA70FQTC5E','91610826590266393W','91610827MA705CXF2K','91610827MA703CCPXF','91610827L04797814E','91610827MA7032FT4X','91610827MA70FK575C','916108277836898514','91610827061945163F','916100007552218217','91610827MA703BQ601','91610827552179360X','91610827MA708YWL19','9161082730562026XQ','91610827MA7094AT02','916108270596677901','91610827338623053A','91610827MA7040FP1T','91610828MA7030X571','91610828MA7MYJ0G84','9161082830551626X8','91610828MA703PB208','91610828MA709TR85H','91610828MA7035J19Q','91610828MA703GAP8E','93610828570669865Y','916108280679278399','93610828MA703MKE86','91610828562200557R','91610828MA7BC7C940','916108280504343257','93610828052138271M','91610828MA708R8J36','93610828559385247F','916108283057501879','9161082859666136XJ','91610828677919502C','91610828661169321T','91610828352252148H','936108285735062871','916108285966825230','91610828MA709KLN51','91610828088015337K','91610800735340321T','91610828MA708URR8U','91610828MA70E0UY5E','91610800MA70AG0322','916108283057533375','91610000305329931J','91610828575612673A','91610828MA70AN8Q0W','9161082806191181XW','91610828MA7032YP28','91610828MA703JJY7E','91610828773832562F','916108295933288522','91610829MA709G5C21','91610829067914085M','91610829MA704APE27','91610829MA70DMLR50','91610829790753292M','91610800790767520W','91610829786975896A','91610829586955030X','91610829MA709Q3L0H','91610830677947677D','916108307769831550','91610830MA7093KD0T','916108305756016913','91610830305391161C','91610830056947332A','916108303522676814','91610830786966180U','91610830MA7038868K','91610830MA70APFT59','91610830719750582L','916108307303912262','916108303056153482','91610830MA7033XL6Y','91610830719747359N','91610830224270253G','91610830MA7039PL1Q','91610830MA7095T23Y','91610830MA708W9N6T','91610830MA7036GJX0','916108303385722175','91610830MA70AC6U08','91610830MA704Q1P39','91610830338743565A','91610830MA7037YE6B','91610830710064009G','91610830555673671H','93610830MA703AM46D','91610830MA7036G47D','91610830MA7050672C','91610830305481837E','91610831MA70323L26','916108000880800750','91610831MA70B73M8U','91610831MA708WCR8K','91610831305767755N','91610831710065220P','91610831727348008E','91610831593341926R','916108310619449025','91610831MA703CJF2E','916108315706692914','91610831MA708R007W','91610831MA70B3MD4G','91610831MA70BD1H0Y','93610831562243709Y','916108313056661962','916108310939512426','91610831MA709N6L13','9161083157781616X6','916100000880671537','91610831MABN470E3F','93610831305343143G','9161082167793717XC','916108213055993173','91610821675105396U','91610821MA7035W466','91610821MA70EU1M4C','916108216847887231','91610800755204183B','91610821MA70978GX5','91610821MA70DTWN1E','91610821MA70DCFF18','91610821MA70FJR21E','91610821305767771W','916100007799307022','91610821677922285W','916100003053012185','91610000748616526W','91610821563791942T','91610821766308324Q','91610821786996363L','91610821MA70CNJ41E','91610000563779951C','916108217135706793','91610821MA70CRQ8X1','91610000732641997R','916100007799482405','91610000059666958C','91610821555656230Q','91610821583545538T','9161000075884416XH','91610821MA703FP576','916108215556937286','91610821MA70362218','91610821MA703BE64X','91610821664106588B','9161082156710197X5','91610821MA708QC6XL','91610821MA70C2Q06B','91610821MA70DKG314','916108217869920396','91610821MA70C2PMXJ','91610821719766197G','91610000710924961H','91610821067914579R','91610821071269955P','91610821MA70BJ9W0C','91610821MA7098JB19','916108215966606585','91610821MA70EC2YXC','91610821MA70CGPH7F','91610821583541000M','91610821559390206W','91610821667969549M','9161082175884213XN','916108217100521205','91610821MA703PT760','91610821338639979E','91610821MA703BHW6U','91610821783666617B','91610821MA70FXBP6K','91610821681596021L','916108216879709774','91610800598796380J','91610800691145662A','91610821691135173P','916108212240045457','91610821224000157T','91610821MA703GB77B','91610821681569381A','916100006847780075','916108216779159829','916108213385789858','91610821583543612D','9161082130538851XE','916108210648495992','91610000664134933F','91610821596655947E','91610821MA709DR57H','916108217552325636','91610821088123194U','91610000745041552L','91610000687992877M','91610821555689542X','91610821677901740Y','91610821677923560K','91610821MA70BFPF2Y','91610821MA709PAA2A','91610000748618126L','91610821MA70EGPX9H','916100002240051930','91610000924001302H','916108215835267406','916108215521580763','916108216751414422','916100005556584045','916108216779105329','91610821562228197B','91610821MA703XYH1W','91610821570659229E','916108217625768311','91610821305496043G','916108216815570738','91610821MA708GH3X4','9161082167510000XL','91610821MA703BEC31','91610821MA703LQGXF','916100000596527830','91610821MA703HPX1T','91610821305323134R','91610821MA703N1B5U','91610821573521946X','91610821MA703A1W1A','91610821305384658A','9161080066117645XR','91610821MA70955UX1','91610821MA70AY2U52','91610000577813989U','91610821MA70CG4B44','91610821694922041J','91610000064835373N','91610821562219223W','916100007100533691','91610000732652506M','916108217353454137','91610821MA7094W97M','91610821MA70381K88','91610000773808634G','91610821MA70BTPE2D','91610000305465386R','91610821MA70FPJ19J','91610821671501594M','91610821MA703316XP','91610000X23601087L','9161000071978482XF','916100006911194994','9161082158697432XC','91610821MA705B6C33','91610821MA6TGWKM6C','91610000710052067J','91610821770025546B','91610000924013143K','916108216911367669','91610821694911481E','91610000671527778F','91610000698436454D','91610821675114583D','91610821664110536C','916108215593819240','91610821681579985G','91610821563789930M','91610000766333263G','916108217869620404','91610821786962796K','91610821677948768B','91610821681588961J','91610000681597913D','91610821684790305C','91610821MA703BX57B','91610821MA703TKP7H','91610821MA70BKG73Y','91610821MA70FUAP10','91610821790760484L','91610821078617967F','91610821MA7098LA63','91610821MA7033LY65','91610821338609446E','91610821338609534W','91610821MA70B7PF0Y','91610821567141533Q','91610821681590201J','916108216815903409','91610000305340444Y','91610000059683299M','916100005622201527','916108215869581355','91610000924003893Q','916100005556826316','91610821684773951D','916100007135061593','91610821691126138J','9161000055219304XB','91610821061925744C','91610821L047817240','916108215807683576','916108216679832524','9161082155216971XT','91610821552195029G','91610821586985766Y','9161082155568221XT','9161082167792431X4','91610821081708229J','91610000773830604T','91610821675119317B','91610821061948145J','9161000092400161X7','91610821074501043F','916108215637906599','916108216847868401','91610821MA70AFP8XK','91610821790779716D','91610821MA709UHQ1A','91610821MA703J6063','9161082108806100X8','91610821305529570Q','91610821MA70950B2G','91610821MA703FB4X5','91610821MA7098KX87','91610821064821756N','91610821305712199T','91610821MA70GQ7Q5E','91610806691107682J','91610821580782810W','9161082155566904XM','91610821586998516U','916100007135040226','91610806741251560K','91610821MA7050AM0A','91610821766336843W','916108216879865066','91610821691147676K','916108216715437436','91610821755200326A','91610821687963603D','91610000224003921H','91610821050402075G','91610821675141573X','9161000071005330XQ','916108212240019344','91610821305412030B','9161082168796362X0','91610821586969627M','91610821MA703QH21F','91610821338655688X','91610821352278970Y','91610821562249481F','91610821MA704UUN2D','91610821MA704G7F25','91610821694924442T','91610000730404314B','91610821570668862F','9161000005968331X1','916108216879635742','91610000MA6TG40X28','91610821555699740T','916100005521646440','91610000710052833G','916100009240134699','91610000567129825J','9161000075521877X8','91610821770031292X','91610821776988992B','91610800MA708L7213','91610821MA703QFH1K','916108216715343455','91610821675144336D','91610893MA70C8BN5R','91610821MA70A5UY7L','91610821MA70ERP17F','91610000741268506A','91610821MA70DKQ35X','91610000577835002Q','916108216715106698','91610821563795767X','91610821770033124M','9161000071350425X8','91610821770043971N','91610821675117004H','916108212240035186','916108215556718796','91610821786998238R','916108216751367583','91610821684770443G','91610821677921872D','91610821338575709B','91610821698418344C','916108216847628063','916108217799382026','91610821698418854R','91610821713506992M','91610821681551456T','91610821MA709MTL5A','916108215593687385','91610821691122380D','91610000677916061H','91610821555684127G','91610000671528076Q','91610821694922463H','916108216911087145','916108216911169173','91610821675145697Y','916108216911304797','91610821583535073B','916108210817431714','91610821MA7034TW33','91610821671537108L','91610821580772153T','916108217552057845','916108215735218745','9161082173534658XU','9161082155219611XA','91610821071278958H','916108210569000161','9161000071005277XF','91610000691125485H','91610821577821532A','91610821671537060C','91610821MA704NDFX6','91610000710052665E','91610821071285285M','91610000577827678B','916108213056545585','916108213057842808','91610000694922009G','916108216715160756','91610821MA7094M77U','9161082155566058X5','9161082179790070XW','91610821338596382X','916108210712746156','91610821567114877Q','91610000577813946E','9161082169841866XW','916100000712732418','91610821586963639H','9161000071357096XA','91610821064803785M','91610821552174447C','91610821MA704JTT54','916108066879758908','91610000684797582J','91610806MA703A6H0H','91610806586970396Q','91610806MA703PKD8X','916108060547617531','91610800694936013W','91610806MA708E9E3C','916108007588273274','91610821MA708UWL0R','91610806MA70ADRH09','91610806305365529M','916108060786458026','91610806078612699W','91610821MA703DEQ3U','916100006879524029','91610800MA70F0X441','91610806586963508Y','91610806MA7096L418','91610800MA708M1D63','91610806MA70391C58','916100009240011505','916108065869736343','91610821758816740M','91610821794117621M','91610821MA70CG834J','91610821MA70D3P75Q','91610821MA70D3PF0H','91610821770017642B','91610821684784298E','91610821MA70AFDX8F','91610821664127514E','91610821MA70CXKU84','91610821MA70A9ND11','9161082176632501XX','91610821675145005D','91610821054788032W','916108217700423972','91610821770013983N','91610821MA70DT2L42','91610821681582199M','91610806MA709B0F16','91610821MA7037WA08','916108213057510246','91610821555695272T','916108215556868517','9161082158075123X8','91610821762578722U','91610821755222453P','91610821MA70A5L79B','916108213053142541','91610821074539041A','916108215637902455','91610806067948461P','91610821MA703AYG0F','91610800752146344X','9161082130559046X1','91610806755247976C','916108217588481798','91610821305664764W','91610821766312040R','91610821786998975C','91610821677912765F','91610821559365318N','91610821MA703BRT54','91610806735348921D','91610821748622598U','91610000748646493C','91610821MA709HTD69','916108215671436353','91610821338638431X','91610821305453131A','91610821305769822Y','91610821305337210M','916108213055523324','916108215807826858','91610821MA708P1D4Q','916100006984015773','916100007769889684','91610821MA708FCP5T','91610821305529183C','91610821677948098N') and xf_hydm is not null and gf_hydm is not null group by gf_shxydm, gf_nsrmc, gf_hydm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, spbm) test

# 提取核心企业的销售发票数据，分季度
select * from datapool.t_data_sh_new_qyjfkpsj_2023_5 where xf_shxydm in ('9161089357352467X6','91610800694925584L','91610802727360605W','91610800H13100629H','916108226679507926','916100006911035934','9161080072737250XT','91610000755202794R','916108005521962839','91610893735373473F','91610800436676327U','91610800766331292B','91610802562237448A','91610800691107981R','91610800710051400F','91610802684771219B','91610800MA703AJL6L','916108937869808637','91610800MA704K2B9J','91610800MA708G9G5C','916108007273474788','91610800MA7031YQ82','91610802050407888U','91610893687987058M','91610800MA7096937L','91610800677943473A','91610800748646530U','91610800MA7032RU93','91610800056930530N','91610800223931531Y','91610800776976043J','91610802MA703BFH02','91610800223930221M','916108006984228963','91610800MA70DD196U','91610802MA703F8M84','91610802567115634N','916108007663167036','91610800MA70AE5B4E','91610800061935707U','91610800563753049F','91610802050400635F','91610802MA705D6C9K','91610802MA70AWAY3D','91610802687968340T','91610800681582586L','91610800677948717M','91610802MA70GG2Y1H','91610800MA703QPKXC','91610802061916178Q','916108007273435491','91610000567118552C','91610893713586830T','916108007869525471','916108007979389782','9161080271004864XG','91610800661152976L','91610802MA704Y7M9M','91610802MA70G53D6M','91610000797940752N','91610802MA7031R94N','91610802MA703WM89H','916108005778342619','91610800752124911M','916108007588338747','91610802223962506N','916108006751043185','91610800677914736G','916108007450080310','91610800773819114A','91610800794100846T','916108006641075481','916108003385989430','91610800552165727D','91610802577836419K','9161080266116183XF','91610800684799641Q','916108006984024304','916108006641132516','916108007625527097','91610802MA70BMX34M','91610800MA70BE0G9E','91610800MA703EW32C','91610800MA70AXHK6D','91610800MA7036C41M','91610800MA704GNH0Q','91610802MA703BWC8F','91610800059688380U','91610800305302050M','91610800MA709M13XP','91610893MA703DEY9X','91610800694918499D','91610800MA7037RA9M','91610822MA7097U17U','91610824MA704D9D0X','91610800067912098G','91610802MA7034XX71','916108933057186623','91610800730410562R','91610800593317221R','91610800MA70988L3J','916108006984411145','91610800677943334K','91610893MA703AHQ4K','91610800675112705T','91610800MA703DR38N','91610893MA703K4C4E','916108006611675026','91610893MA70C3XL5R','91610893MA70AKUT7Y','91610893MA7030276T','91610800067912397E','91610802081747367D','91610800078619321P','91610800223959534H','916108000679086973','91610104MA6U6WPP9U','91610822MA708UM64E','91610802MA70FP667W','91610800698417755J','91610802MA70F2QM2B','91610802MA70CCRJ58','916108003054061837','91610893MA70BUNR99','91610893567149850M','91610800694921348L','91610893MA7092E2XX','91610800MA70319P01','91610800776976764F','91610800555695483W','91610893MA7056GH49','91610800748618329J','91610800064810491X','91610802338656146X','9161080230559847XB','91610800691100720J','91610000661160001T','91610800MA7035TJX3','91610800305315580J','91610800074521407G','916108000679234595','91610802573522607J','91610800661157478J','91610800088193408Q','91610800MA704B3A2P','91610800088079154R','916108020569016092','91610800MA7036T07K','91610000X2358669XB','91610800061931191K','91610802338661033H','91610800694930906W','91610800575649337Q','916108023055573957','916108000939802706','916100002239672519','916108026879737809','916100007100518818','91610802MA70A6FN4M','91610000X23586219E','91610800671536834E','91610800691127712R','916100006815997672','91610802667987720K','91610802MA7038FT0M','91610802552178923N','91610000223967235K','916108006679710248','91610000745037668H','91610000223965838G','91610802570668811N','916108006751155433','91610000710051814A','91610000766346072G','91610800598779185W','91610802MA708F2G8K','91610802MA709Y9P37','916108007588160388','916108003054001360','91610800691131228D','91610000786958834R','91610800MA703RJE5P','91610802MA7055P507','916108020547928919','916100005593700612','91610802681595256T','91610800MA704B3W2A','916100006847510073','916108020880671880','91610000735388763B','91610000773844942E','91610800752129712H','916108007979285936','91610000X23586251R','91610000752131054Q','91610802MA7038KK91','91610802773841215D','916108005778349158','9161080009394451XQ','91610800061909963N','91610802MA7038GD6G','93610802MA703FR92F','91610000338700928R','91610800552158025D','91610802691100835M','91610802563788284P','91610802MA703J1L6T','91610800MA7036E79W','91610802061927221J','91610800590253023H','9161080276255849XM','91610802MA704D189B','91610821MA703YBK2D','91610800MA703QW35X','91610802305755922F','91610802078614125H','91610802MA70D7448F','91610802MA70ATEH12','91610000059676961F','91610802MA7038MAX1','91610000593327534K','91610802MA703HWE04','916108023386836119','9161080006481082X4','91610800590260549L','91610800MA70371J7F','91610000797938927G','91610800596685070J','9161000056714796XP','91610800684794322L','91610823555664310J','91610823755214998B','91610000755225347C','91610000758832169D','916108006815878706','916108006911278008','916100006984246809','916100006984339153','91610000710055196X','916100007273509845','91610000755214488A','91610000779910293J','91610000758810349Q','916108232240609348','91610000224061857H','916108935521883122','916100005556550942','916100006815955200','91610000687954301Y','91610000790751174B','91610823MA7035CD25','916108230880722358','91610823305604112X','916108230881953581','916108237907734373','91610823MA703ELY6H','91610823MA70CG8N8F','91610823671529749H','91610823MA7032KGXX','91610823MA70D1G21X','91610823MA70G73N3H','91610823MA703DDX4B','91610823MA7097DG1P','916108237941275554','91610823MA7034CJ9Q','91610823MA70GABW64','91610823MA7032U61A','91610823MA7030E21X','91610823MA709HYNXD','916108233054100434','91610823MA70EP4G0U','916100006984084291','916108230521323633','916108230882445003','91610823MA704R7J5F','91610000773800042F','91610000586982709M','91610823056903460W','91610823MA708N92XX','91610823305726101A','91610823MA708U9U8T','91610823MA70CM4M03','91610823MA70341H2K','91610893MA70F7MT80','91610823MA7030FM17','91610893MA70AULF26','91610800684795675B','91610893088247840A','916108930882378452','91610800567137999E','91610823MA70D01L9W','91610823MA7030WJ5A','91610822552193103H','91610822583542433E','916100006751366436','91610000687952218H','91610822694938211C','916100007769886696','9161082222403383X0','916108225593913130','91610822567139548R','91610800671529239R','916108222240319575','91610822MA70CYYF7Q','91610800223930635R','916108225593639100','91610822MA703A0A5W','91610822583526038L','91610822MA709EAN9R','91610822710053561Q','916108226715094065','91610822MA708J3C27','916108223387099570','9161082266411342X1','9161082267152645XB','916108226984006033','91610822MA703WCN8Q','91610822MA70DF1676','916108222240308741','916108222240302172','91610822224030882U','91610000577805794Q','91610822224032482K','916108227100544680','91610822766320980Q','91610822586971540Q','91610822071284629R','91610822MA70BE3W9P','91610822MA70F1FP48','91610822MA7BK9K43D','91610822MA709Y0M1T','91610822MA709Y7E08','91610822MA704D4D9G','91610822924032280H','91610822MA709N423A','91610822MA709HGW9P','91610822MA7040Q74E','91610822MA70G7EX5Y','91610000661170904Y','91610000224030946L','91610822797925923U','916108225933488027','91610822224032386A','91610000MA6TG1L91C','916108223056392271','9161082230539678XQ','91610822555655537N','91610822762574940L','9161082206868156X4','916100002240321328','91610822684783682Q','91610822735351493Q','916108227135586612','91610822752112793E','91610822224041784B','91610822573535061G','91610822698443996B','91610822552179029C','91610000580766765J','91610822081718611M','91610822677931237W','91610822MA703BPQ8E','91610000559374302W','91610822694908266R','91610822694929032M','91610000797915039F','91610822093914935U','91610822555666885M','916108223386303333','91610822583546661K','91610822MA70317P8A','91610822671525115L','91610822577800053T','91610822087963650K','91610822797937238T','91610822667973185H','916100005556762074','91610822687981123J','91610822567149623M','916108225869772994','9161082269494526XA','916108227663422157','916108225593843242','916108226911466229','91610822575627197H','91610822677911949C','91610822056917184G','91610822MA7059MH04','91610000770001587F','91610800719785144W','91610000567108776E','91610000758819036E','91610822786997948R','91610822563772274J','91610000224057049W','916108225521605984','91610822MA709DFT7W','916108225835113037','91610822MA703HB02A','916100000504386259','916108225869795931','91610822MA708MR72L','91610822735347005J','91610800770011101Y','91610800675123180G','9161000077382866XG','916108220569493036','91610822MA70DY9C9C','916100000817239128','91610000713558602R','91610822570658605X','91610000758843706Y','91610822MA7036PA3C','91610000061914033T','91610822224033581D','916108220619348353','91610822694914308A','9161082273039809X5','916100000521118534','91610822552168012N','91610000580775995H','916108227486349192','91610822766312091G','9161082276633061X2','91610822MA708Y049A','91610822710054302H','91610822687992703H','91610822766330636W','91610822MA70FGBR66','91610822078635727F','91610822078636033K','91610822MA704G881Y','91610822667965572L','916108227799344049','91610822MA70B4TQ8T','916108226911259879','916108222240334186','91610822MA70A6HJ4U','91610822MA709R8R4H','9161082258078527XY','91610822687983735A','91610822710054791E','91610822MA708R672M','91610822577843598F','916108225869837874','9161082257069098XN','91610822694927918T','91610822797911572J','916108225637837070','916108226949183843','916108226949327095','91610822MA704CN193','9161082271355912XF','91610822567124282E','916108225671304988','91610000710054484G','916108225735414537','916100002240314638','916108225778382625','91610822773826787T','91610000567136590B','91610822078640366D','91610822094023050T','9161000056710680XM','91610822681555895X','91610000924044062B','91610822552163879U','91610822305318853B','91610000567138334L','916108226879560162','91610822567139193Q','91610000681575589J','916108226679923350','91610821MA70343P02','916108930881217117','91610822305621270Y','916100006879777738','91610822050401881R','91610822MA704K1D9T','91610822748606993Y','91610822MA70B3171E','916108225807966659','91610822MA70AB7K22','91610822MA7030P565','91610822755225080Y','91610000X23606048P','916100005556688484','91610000598765891B','916100009240407903','91610822766305764C','91610000924030840U','9161000055935482XF','91610000567118544H','91610000719708587F','91610822562241391F','916108225521576557','91610822MA7030C7X0','9161082255566867X2','916108225735291204','916108226847965391','9161082271355816XC','91610000567129809W','91610000687954512E','91610822667970734M','91610822748616366T','91610000681575597D','91610000691106065G','916108227869769281','9161082269490584XB','916108223057890497','91610000562225156P','91610822737993836U','91610822MA7093A6XP','91610822563764290Y','916108225556745861','9161082266796860XC','91610822MA708N4B1M','91610000681575570M','9161082256712766XC','91610822727369589U','91610822580776779W','916108225902933593','916108220648211664','9161082207127640X1','91610822559385992C','9161082256224308XJ','9161082258078631XM','91610822590295784U','91610822593312017P','91610822596695017M','916108225987548772','916108226779018201','91610822681564126J','91610822MA709AGT5N','91610822MA70A9UE46','91610822MA708KQDXA','91610822552191175G','91610822555680564K','91610822552154948T','91610822671520461A','916100006847787282','91610822MA708YJX74','91610000567103318K','91610822563762148E','91610822562224372N','91610822555688793Q','9161082257351819X6','91610822583526636G','916100007099182651','916100005671063088','916108225807676532','91610822580784090L','9161000059027254XF','91610822MA7032CD47','916108220745134073','91610822766331508H','9161000056377267XH','916100005556789748','91610000567138887F','916108226715375708','916108227450323733','9161082230534244X0','9161082208819745XP','91610822573530391G','91610000681599492L','91610822MA7039625F','916108220786281646','91610000078649651R','91610000305788468G','91610822586987462E','91610822MA70384L5F','91610822MA703BUW9C','91610824MA708G4C05','916108245756426370','91610824352260279W','91610824338607352A','91610824MA709B7K7N','91610824MA7045E8X5','916108243055203063','91610824MA7043WQ6F','91610824MA7045KE7K','91610824684776511K','91610824MA7052RM4M','91610824305602280L','91610824MA70404L9N','91610824MA708T379E','91610824713561318K','91610824590274190L','9161082405479693X8','916108246949093223','91610800677905610M','91610000698419427R','91610824338748948N','91610824MA70A3CU40','91610824067940013A','91610824MA7031PR97','91610824093912366E','91610824052125550G','91610824555662817L','91610824586989142N','91610824MA70DAAF42','91610824667959658W','91610824338721649N','91610824305379402Y','916108245593799981','91610824583543516F','91610824MA703AAQXB','91610824675115447J','91610824MA70EFL6XT','91610824MABQ6NL698','91610824580788446J','91610824590270069T','91610824MA70D7CY47','91610824577821241J','916108003057434031','91610824059660193A','91610824559383436L','916108243054516389','91610824MA703BB57G','91610824MA708NC530','91610824305645491Q','91610824074522522Q','91610824093913721Q','91610824059693913C','91610824570657119L','91610825710058346A','916108253055799146','916108253057807499','91610825MA703BP13J','916108253055693618','91610825064823620K','91610825MA70GL610E','91610825MA703GRQ4W','91610825094008096H','91610825088196684W','916108250939461010','91610825305710716N','91610825MA7K37P61M','91610825MA70GHF09N','91610825MA70GGYFX2','916108253055136327','91610825MA708W73XU','9161082530550930XA','91610825056929273K','91610825MA7097R42F','91610825MA703D773C','91610825059666253M','916108250648189605','916108257100585227','916108255807949859','91610825770016498A','916108252241200281','916108257941124337','91610825305646531X','91610825305721597M','916108257869947471','916108255902738171','91610825MA704RJJ1E','91610825MA7031MW0H','916108250817158759','9161082530538018XY','91610825305557133R','916108253057060719','91610800MA70B34385','916108253054304235','916108253057555513','91610825MA70GGKC69','91610825338670220D','91610825MA7033R222','916108250939154365','916108257869852184','916108250881238057','91610825687982230J','91610825MA703C1J05','9161082569842068XN','916108250859923274','91610825MA70A9HP1Q','916108255521915890','9161082509390645XX','91610825MA70318W1T','91610826MA7033UK93','91610826MA70CG498N','91610800223930176L','91610826583507806A','916108266641120996','916108265593534054','91610800X236250463','9161082630547280X6','916108266949220255','91610826MA70333X3J','91610826MA703MQK5P','91610826064845432U','91610826MA70FQTC5E','91610826590266393W','91610827MA705CXF2K','91610827MA703CCPXF','91610827L04797814E','91610827MA7032FT4X','91610827MA70FK575C','916108277836898514','91610827061945163F','916100007552218217','91610827MA703BQ601','91610827552179360X','91610827MA708YWL19','9161082730562026XQ','91610827MA7094AT02','916108270596677901','91610827338623053A','91610827MA7040FP1T','91610828MA7030X571','91610828MA7MYJ0G84','9161082830551626X8','91610828MA703PB208','91610828MA709TR85H','91610828MA7035J19Q','91610828MA703GAP8E','93610828570669865Y','916108280679278399','93610828MA703MKE86','91610828562200557R','91610828MA7BC7C940','916108280504343257','93610828052138271M','91610828MA708R8J36','93610828559385247F','916108283057501879','9161082859666136XJ','91610828677919502C','91610828661169321T','91610828352252148H','936108285735062871','916108285966825230','91610828MA709KLN51','91610828088015337K','91610800735340321T','91610828MA708URR8U','91610828MA70E0UY5E','91610800MA70AG0322','916108283057533375','91610000305329931J','91610828575612673A','91610828MA70AN8Q0W','9161082806191181XW','91610828MA7032YP28','91610828MA703JJY7E','91610828773832562F','916108295933288522','91610829MA709G5C21','91610829067914085M','91610829MA704APE27','91610829MA70DMLR50','91610829790753292M','91610800790767520W','91610829786975896A','91610829586955030X','91610829MA709Q3L0H','91610830677947677D','916108307769831550','91610830MA7093KD0T','916108305756016913','91610830305391161C','91610830056947332A','916108303522676814','91610830786966180U','91610830MA7038868K','91610830MA70APFT59','91610830719750582L','916108307303912262','916108303056153482','91610830MA7033XL6Y','91610830719747359N','91610830224270253G','91610830MA7039PL1Q','91610830MA7095T23Y','91610830MA708W9N6T','91610830MA7036GJX0','916108303385722175','91610830MA70AC6U08','91610830MA704Q1P39','91610830338743565A','91610830MA7037YE6B','91610830710064009G','91610830555673671H','93610830MA703AM46D','91610830MA7036G47D','91610830MA7050672C','91610830305481837E','91610831MA70323L26','916108000880800750','91610831MA70B73M8U','91610831MA708WCR8K','91610831305767755N','91610831710065220P','91610831727348008E','91610831593341926R','916108310619449025','91610831MA703CJF2E','916108315706692914','91610831MA708R007W','91610831MA70B3MD4G','91610831MA70BD1H0Y','93610831562243709Y','916108313056661962','916108310939512426','91610831MA709N6L13','9161083157781616X6','916100000880671537','91610831MABN470E3F','93610831305343143G','9161082167793717XC','916108213055993173','91610821675105396U','91610821MA7035W466','91610821MA70EU1M4C','916108216847887231','91610800755204183B','91610821MA70978GX5','91610821MA70DTWN1E','91610821MA70DCFF18','91610821MA70FJR21E','91610821305767771W','916100007799307022','91610821677922285W','916100003053012185','91610000748616526W','91610821563791942T','91610821766308324Q','91610821786996363L','91610821MA70CNJ41E','91610000563779951C','916108217135706793','91610821MA70CRQ8X1','91610000732641997R','916100007799482405','91610000059666958C','91610821555656230Q','91610821583545538T','9161000075884416XH','91610821MA703FP576','916108215556937286','91610821MA70362218','91610821MA703BE64X','91610821664106588B','9161082156710197X5','91610821MA708QC6XL','91610821MA70C2Q06B','91610821MA70DKG314','916108217869920396','91610821MA70C2PMXJ','91610821719766197G','91610000710924961H','91610821067914579R','91610821071269955P','91610821MA70BJ9W0C','91610821MA7098JB19','916108215966606585','91610821MA70EC2YXC','91610821MA70CGPH7F','91610821583541000M','91610821559390206W','91610821667969549M','9161082175884213XN','916108217100521205','91610821MA703PT760','91610821338639979E','91610821MA703BHW6U','91610821783666617B','91610821MA70FXBP6K','91610821681596021L','916108216879709774','91610800598796380J','91610800691145662A','91610821691135173P','916108212240045457','91610821224000157T','91610821MA703GB77B','91610821681569381A','916100006847780075','916108216779159829','916108213385789858','91610821583543612D','9161082130538851XE','916108210648495992','91610000664134933F','91610821596655947E','91610821MA709DR57H','916108217552325636','91610821088123194U','91610000745041552L','91610000687992877M','91610821555689542X','91610821677901740Y','91610821677923560K','91610821MA70BFPF2Y','91610821MA709PAA2A','91610000748618126L','91610821MA70EGPX9H','916100002240051930','91610000924001302H','916108215835267406','916108215521580763','916108216751414422','916100005556584045','916108216779105329','91610821562228197B','91610821MA703XYH1W','91610821570659229E','916108217625768311','91610821305496043G','916108216815570738','91610821MA708GH3X4','9161082167510000XL','91610821MA703BEC31','91610821MA703LQGXF','916100000596527830','91610821MA703HPX1T','91610821305323134R','91610821MA703N1B5U','91610821573521946X','91610821MA703A1W1A','91610821305384658A','9161080066117645XR','91610821MA70955UX1','91610821MA70AY2U52','91610000577813989U','91610821MA70CG4B44','91610821694922041J','91610000064835373N','91610821562219223W','916100007100533691','91610000732652506M','916108217353454137','91610821MA7094W97M','91610821MA70381K88','91610000773808634G','91610821MA70BTPE2D','91610000305465386R','91610821MA70FPJ19J','91610821671501594M','91610821MA703316XP','91610000X23601087L','9161000071978482XF','916100006911194994','9161082158697432XC','91610821MA705B6C33','91610821MA6TGWKM6C','91610000710052067J','91610821770025546B','91610000924013143K','916108216911367669','91610821694911481E','91610000671527778F','91610000698436454D','91610821675114583D','91610821664110536C','916108215593819240','91610821681579985G','91610821563789930M','91610000766333263G','916108217869620404','91610821786962796K','91610821677948768B','91610821681588961J','91610000681597913D','91610821684790305C','91610821MA703BX57B','91610821MA703TKP7H','91610821MA70BKG73Y','91610821MA70FUAP10','91610821790760484L','91610821078617967F','91610821MA7098LA63','91610821MA7033LY65','91610821338609446E','91610821338609534W','91610821MA70B7PF0Y','91610821567141533Q','91610821681590201J','916108216815903409','91610000305340444Y','91610000059683299M','916100005622201527','916108215869581355','91610000924003893Q','916100005556826316','91610821684773951D','916100007135061593','91610821691126138J','9161000055219304XB','91610821061925744C','91610821L047817240','916108215807683576','916108216679832524','9161082155216971XT','91610821552195029G','91610821586985766Y','9161082155568221XT','9161082167792431X4','91610821081708229J','91610000773830604T','91610821675119317B','91610821061948145J','9161000092400161X7','91610821074501043F','916108215637906599','916108216847868401','91610821MA70AFP8XK','91610821790779716D','91610821MA709UHQ1A','91610821MA703J6063','9161082108806100X8','91610821305529570Q','91610821MA70950B2G','91610821MA703FB4X5','91610821MA7098KX87','91610821064821756N','91610821305712199T','91610821MA70GQ7Q5E','91610806691107682J','91610821580782810W','9161082155566904XM','91610821586998516U','916100007135040226','91610806741251560K','91610821MA7050AM0A','91610821766336843W','916108216879865066','91610821691147676K','916108216715437436','91610821755200326A','91610821687963603D','91610000224003921H','91610821050402075G','91610821675141573X','9161000071005330XQ','916108212240019344','91610821305412030B','9161082168796362X0','91610821586969627M','91610821MA703QH21F','91610821338655688X','91610821352278970Y','91610821562249481F','91610821MA704UUN2D','91610821MA704G7F25','91610821694924442T','91610000730404314B','91610821570668862F','9161000005968331X1','916108216879635742','91610000MA6TG40X28','91610821555699740T','916100005521646440','91610000710052833G','916100009240134699','91610000567129825J','9161000075521877X8','91610821770031292X','91610821776988992B','91610800MA708L7213','91610821MA703QFH1K','916108216715343455','91610821675144336D','91610893MA70C8BN5R','91610821MA70A5UY7L','91610821MA70ERP17F','91610000741268506A','91610821MA70DKQ35X','91610000577835002Q','916108216715106698','91610821563795767X','91610821770033124M','9161000071350425X8','91610821770043971N','91610821675117004H','916108212240035186','916108215556718796','91610821786998238R','916108216751367583','91610821684770443G','91610821677921872D','91610821338575709B','91610821698418344C','916108216847628063','916108217799382026','91610821698418854R','91610821713506992M','91610821681551456T','91610821MA709MTL5A','916108215593687385','91610821691122380D','91610000677916061H','91610821555684127G','91610000671528076Q','91610821694922463H','916108216911087145','916108216911169173','91610821675145697Y','916108216911304797','91610821583535073B','916108210817431714','91610821MA7034TW33','91610821671537108L','91610821580772153T','916108217552057845','916108215735218745','9161082173534658XU','9161082155219611XA','91610821071278958H','916108210569000161','9161000071005277XF','91610000691125485H','91610821577821532A','91610821671537060C','91610821MA704NDFX6','91610000710052665E','91610821071285285M','91610000577827678B','916108213056545585','916108213057842808','91610000694922009G','916108216715160756','91610821MA7094M77U','9161082155566058X5','9161082179790070XW','91610821338596382X','916108210712746156','91610821567114877Q','91610000577813946E','9161082169841866XW','916100000712732418','91610821586963639H','9161000071357096XA','91610821064803785M','91610821552174447C','91610821MA704JTT54','916108066879758908','91610000684797582J','91610806MA703A6H0H','91610806586970396Q','91610806MA703PKD8X','916108060547617531','91610800694936013W','91610806MA708E9E3C','916108007588273274','91610821MA708UWL0R','91610806MA70ADRH09','91610806305365529M','916108060786458026','91610806078612699W','91610821MA703DEQ3U','916100006879524029','91610800MA70F0X441','91610806586963508Y','91610806MA7096L418','91610800MA708M1D63','91610806MA70391C58','916100009240011505','916108065869736343','91610821758816740M','91610821794117621M','91610821MA70CG834J','91610821MA70D3P75Q','91610821MA70D3PF0H','91610821770017642B','91610821684784298E','91610821MA70AFDX8F','91610821664127514E','91610821MA70CXKU84','91610821MA70A9ND11','9161082176632501XX','91610821675145005D','91610821054788032W','916108217700423972','91610821770013983N','91610821MA70DT2L42','91610821681582199M','91610806MA709B0F16','91610821MA7037WA08','916108213057510246','91610821555695272T','916108215556868517','9161082158075123X8','91610821762578722U','91610821755222453P','91610821MA70A5L79B','916108213053142541','91610821074539041A','916108215637902455','91610806067948461P','91610821MA703AYG0F','91610800752146344X','9161082130559046X1','91610806755247976C','916108217588481798','91610821305664764W','91610821766312040R','91610821786998975C','91610821677912765F','91610821559365318N','91610821MA703BRT54','91610806735348921D','91610821748622598U','91610000748646493C','91610821MA709HTD69','916108215671436353','91610821338638431X','91610821305453131A','91610821305769822Y','91610821305337210M','916108213055523324','916108215807826858','91610821MA708P1D4Q','916100006984015773','916100007769889684','91610821MA708FCP5T','91610821305529183C','91610821677948098N') and xf_hydm is not null and gf_hydm is not null and kpyf='202301'

In [ ]:
# 2023
# 碳中和 xf 44736
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000', '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000', '1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 碳中和 gf 81713
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000', '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000', '1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 铝镁及兰炭深加工产业链 xf 43370
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020503050000000000', '1070202110000000000', '1070214130000000000', '1070203010100000000', '1080314030200000000', '1080306020000000000', '1080307000000000000', '1080314010500000000', '1080316010000000000', '1090137010000000000', '1080417000000000000', '1080316020000000000', '1090136040000000000', '1090136990000000000', '1080306030000000000', '3040301990000000000', '3040105000000000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070213080000000000', '1080127030000000000', '1070201070000000000', '1070201220000000000', '1080130010000000000', '1080103090000000000', '1080314010700000000', '1080207040000000000', '1090234000000000000', '1090206010000000000', '1090519010000000000', '1080325000000000000', '1080209000000000000', '1100202030000000000', '1020203000000000000', '3040103000000000000', '1070101990000000000', '1070101030100000000', '1070101050100000000', '1070213019900000000', '1080419030000000000', '1090310019900000000', '2010500000000000000', '1020102000000000000', '3060000000000000000', '1070103020000000000', '1070202020199000064', '1080103080000000000', '1080208000000000000', '1080321010000000000', '1090206050000000000', '1090314040000000000', '1090509010000000000', '1090517000000000000', '1090500000000000000', '1090413029900000000', '3050100000000000000', '1100101010100000000', '1100201040000000000', '1100201990000000000', '1060509029900000000', '1020101000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 铝镁及兰炭深加工产业链 gf 71732
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020503050000000000', '1070202110000000000', '1070214130000000000', '1070203010100000000', '1080314030200000000', '1080306020000000000', '1080307000000000000', '1080314010500000000', '1080316010000000000', '1090137010000000000', '1080417000000000000', '1080316020000000000', '1090136040000000000', '1090136990000000000', '1080306030000000000', '3040301990000000000', '3040105000000000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070213080000000000', '1080127030000000000', '1070201070000000000', '1070201220000000000', '1080130010000000000', '1080103090000000000', '1080314010700000000', '1080207040000000000', '1090234000000000000', '1090206010000000000', '1090519010000000000', '1080325000000000000', '1080209000000000000', '1100202030000000000', '1020203000000000000', '3040103000000000000', '1070101990000000000', '1070101030100000000', '1070101050100000000', '1070213019900000000', '1080419030000000000', '1090310019900000000', '2010500000000000000', '1020102000000000000', '3060000000000000000', '1070103020000000000', '1070202020199000064', '1080103080000000000', '1080208000000000000', '1080321010000000000', '1090206050000000000', '1090314040000000000', '1090509010000000000', '1090517000000000000', '1090500000000000000', '1090413029900000000', '3050100000000000000', '1100101010100000000', '1100201040000000000', '1100201990000000000', '1060509029900000000', '1020101000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 能源化工装备制造产业链 xf 35455
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1090299000000000000', '1090253010000000000', '1090513040000000000', '3059900000000000000', '4010100000000000000', '1080207130000000000', '1090112140000000000', '1090202020000000000', '1090202090000000000', '1090202010000000000', '1090202080000000000', '1090136990000000000', '1090253100000000000', '1090207000000000000', '1090114010000000000', '1090201030000000000', '1090201070000000000', '1090202000000000000', '3050200000000000000', '1090306990000000000', '1080103030000000000', '3040205000000000000', '3040105000000000000', '1090117010000000000', '1090119010000000000', '1090127010000000000', '1090201020000000000', '1090203040000000000', '1090201990000000000', '1090120040000000000', '1090520000000000000', '3040200000000000000', '3040301010000000000', '3040502019900000256', '3049900000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 能源化工装备制造产业链 gf 49357
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1090299000000000000', '1090253010000000000', '1090513040000000000', '3059900000000000000', '4010100000000000000', '1080207130000000000', '1090112140000000000', '1090202020000000000', '1090202090000000000', '1090202010000000000', '1090202080000000000', '1090136990000000000', '1090253100000000000', '1090207000000000000', '1090114010000000000', '1090201030000000000', '1090201070000000000', '1090202000000000000', '3050200000000000000', '1090306990000000000', '1080103030000000000', '3040205000000000000', '3040105000000000000', '1090117010000000000', '1090119010000000000', '1090127010000000000', '1090201020000000000', '1090203040000000000', '1090201990000000000', '1090120040000000000', '1090520000000000000', '3040200000000000000', '3040301010000000000', '3040502019900000256', '3049900000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 轻纺产业链 xf 10297
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1040102010000000000', '1040102020000000000', '1040102040000000000', '1040101020000000000', '1040111020000000000', '1040112040000000000', '1090228180000000000', '1010399000000000000', '1040201990000000000', '3040301020000000000', '3040303010600000000', '1040201220000000000', '3040101000000000000', '1040201000000000000', '1040201190000000000', '1040114010000000000', '1040106990000000000', '1040102050000000000', '1010303050000000000', '1040200000000000000', '1040101060000000000', '1040199990000000000', '1040199060000000000', '1040101030000000000', '3040601000000000000', '3040302000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 轻纺产业链 gf 21157
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1040102010000000000', '1040102020000000000', '1040102040000000000', '1040101020000000000', '1040111020000000000', '1040112040000000000', '1090228180000000000', '1010399000000000000', '1040201990000000000', '3040301020000000000', '3040303010600000000', '1040201220000000000', '3040101000000000000', '1040201000000000000', '1040201190000000000', '1040114010000000000', '1040106990000000000', '1040102050000000000', '1010303050000000000', '1040200000000000000', '1040101060000000000', '1040199990000000000', '1040199060000000000', '1040101030000000000', '3040601000000000000', '3040302000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 氢能产业链 xf 21289
select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070214020000000000', '1080314030200000000', '1080410000000000000', '3040407990000000000', '1070212020000000000', '1070102020500000000', '1070201160000000000', '1070213050000000000', '1090121020000000000', '1090403030000000000', '3050100000000000000', '3010403000000000000', '3010402010000000000', '3010402000000000000', '1090128030000000000', '1090128060000000000', '1090306990000000000', '1090413040000000000', '1090413060000000000', '3010102020099990016', '1080103030000000000', '1090102990000000000', '1090118000000000000', '1090119010000000000', '1090119020000000000', '1090601040000000000', '1090113040000000000', '1100201010000000000', '1100201990000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf
# 氢能产业链 gf 18227
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070214020000000000', '1080314030200000000', '1080410000000000000', '3040407990000000000', '1070212020000000000', '1070102020500000000', '1070201160000000000', '1070213050000000000', '1090121020000000000', '1090403030000000000', '3050100000000000000', '3010403000000000000', '3010402010000000000', '3010402000000000000', '1090128030000000000', '1090128060000000000', '1090306990000000000', '1090413040000000000', '1090413060000000000', '3010102020099990016', '1080103030000000000', '1090102990000000000', '1090118000000000000', '1090119010000000000', '1090119020000000000', '1090601040000000000', '1090113040000000000', '1100201010000000000', '1100201990000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 文化旅游产业链 xf 21446
select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('3040301990000000000', '3040105000000000000', '3070301000000000000', '3070101000000000000', '3040304020000000000', '3040701040000000000', '3070102000000000000', '1060301030300000000', '3040301010000000000', '3040204000000000000', '3070302000000000000', '1090511010000000000', '1060500000000000000', '3070402000000000000', '3070401000000000000', '3040801990000000000', '3040701020000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf
# 文化旅游产业链 gf 96432
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('3040301990000000000', '3040105000000000000', '3070301000000000000', '3070101000000000000', '3040304020000000000', '3040701040000000000', '3070102000000000000', '1060301030300000000', '3040301010000000000', '3040204000000000000', '3070302000000000000', '1090511010000000000', '1060500000000000000', '3070402000000000000', '3070401000000000000', '3040801990000000000', '3040701020000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 现代煤化工产业链 xf 21917
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070202280000000000', '1070202170000000000', '1070202020103000064', '1070202200000000000', '1070202100000000000', '1070201010200000000', '1070202130000000000', '1070215990000000000', '1070213019900000000', '1070202010200000000', '1070213040000000000', '1070213080000000000', '1070213020000000000', '1070202310000000000', '1070202020102000000', '1070214010000000000', '1070599000000000000', '1070601130000000000', '1080307000000000000', '1070213030000000000', '1070101110100000000', '1070101120000000000', '1070101130100000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070201100000000000', '1070101070200000000', '1070213060000000000', '1070208010000000000', '1070213010100000000', '1070603990000000000', '1080314010700000000', '1060105040000000000', '1070214090000000000', '1070101040100000000', '1070101050100000000', '1070202250000000000', '1070202140000000000', '1070601140000000000', '1040199990000000000', '1040106990000000000', '1040199060000000000', '1070103020000000000', '1070101020100000000', '1070102020600000000', '1070202019900000000', '1070202010400000000', '1070202029900000000', '1070202020199000064', '1070202010300000000', '1070301040000000000', '1100101010100000000', '1100201040000000000', '1020101000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 现代煤化工产业链 gf 34858
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070202280000000000', '1070202170000000000', '1070202020103000064', '1070202200000000000', '1070202100000000000', '1070201010200000000', '1070202130000000000', '1070215990000000000', '1070213019900000000', '1070202010200000000', '1070213040000000000', '1070213080000000000', '1070213020000000000', '1070202310000000000', '1070202020102000000', '1070214010000000000', '1070599000000000000', '1070601130000000000', '1080307000000000000', '1070213030000000000', '1070101110100000000', '1070101120000000000', '1070101130100000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070201100000000000', '1070101070200000000', '1070213060000000000', '1070208010000000000', '1070213010100000000', '1070603990000000000', '1080314010700000000', '1060105040000000000', '1070214090000000000', '1070101040100000000', '1070101050100000000', '1070202250000000000', '1070202140000000000', '1070601140000000000', '1040199990000000000', '1040106990000000000', '1040199060000000000', '1070103020000000000', '1070101020100000000', '1070102020600000000', '1070202019900000000', '1070202010400000000', '1070202029900000000', '1070202020199000064', '1070202010300000000', '1070301040000000000', '1100101010100000000', '1100201040000000000', '1020101000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 现代特色农业产业链 xf 26379
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1010109010000000000', '1010102010000000000', '1030208000000000000', '1010115010100000000', '1010115012700000000', '1010115019900000000', '1060102040000000000', '1070223000000000000', '1070223040000000000', '1070301130000000000', '1010111990000000000', '3040101000000000000', '3040304020000000000', '3040303010600000000', '1010112000000000000', '1010104020000000000', '1010101030000000000', '1030107050000000000', '1010201000000000000', '1070205010000000000', '1070303010000000000', '1010303990000000000', '3000000000000000000', '1030199000000000000', '1030102010200000000', '1070304090000000000', '1010303050000000000', '2010500000000000000', '1030107030000000000', '1010115030300000000', '1010118000000000000', '1040200000000000000', '1040201000000000000', '1040199990000000000', '1040106990000000000', '1040199060000000000', '1070205020000000000', '1070204070000000000', '1070215040000000000', '1070307100000000000', '1070308010000000000', '1010301010000000000', '1010301020000000000', '1010301060000000000', '1010302990000000000', '3040105000000000000', '3060000000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 现代特色农业产业链 gf 44876
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1010109010000000000', '1010102010000000000', '1030208000000000000', '1010115010100000000', '1010115012700000000', '1010115019900000000', '1060102040000000000', '1070223000000000000', '1070223040000000000', '1070301130000000000', '1010111990000000000', '3040101000000000000', '3040304020000000000', '3040303010600000000', '1010112000000000000', '1010104020000000000', '1010101030000000000', '1030107050000000000', '1010201000000000000', '1070205010000000000', '1070303010000000000', '1010303990000000000', '3000000000000000000', '1030199000000000000', '1030102010200000000', '1070304090000000000', '1010303050000000000', '2010500000000000000', '1030107030000000000', '1010115030300000000', '1010118000000000000', '1040200000000000000', '1040201000000000000', '1040199990000000000', '1040106990000000000', '1040199060000000000', '1070205020000000000', '1070204070000000000', '1070215040000000000', '1070307100000000000', '1070308010000000000', '1010301010000000000', '1010301020000000000', '1010301060000000000', '1010302990000000000', '3040105000000000000', '3060000000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 新能源装备制造产业链 xf 35776
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020599010000000000', '1070218090000000000', '1070201010200000000', '1070214010000000000', '1070202010200000000', '1070212030000000000', '1070213019900000000', '1070213020000000000', '1070215060000000000', '1070403990000000000', '1070603060000000000', '1080119000000000000', '1080314010500000000', '1080414020000000000', '1080401010000000000', '1090124080000000000', '1090407010000000000', '1090401010000000000', '1090401020000000000', '1090402040000000000', '1090603070000000000', '1090122990000000000', '1090409990000000000', '1090619050000000000', '1090509020000000000', '1090520000000000000', '1090601020000000000', '1090601040000000000', '3040301010000000000', '3040301990000000000', '3040105000000000000', '1070101060000000000', '1070103010000000000', '1070202020199000064', '1070201100000000000', '1070213060000000000', '1070208010000000000', '1070601010200000000', '1070404000000000000', '1070401030000000000', '1080130010000000000', '1080108040000000000', '1080110029900000000', '1080310990000000000', '1080207110000000000', '1090103010000000000', '1090129050000000000', '1090129990000000000', '1090123000000000000', '1090124010000000000', '1090207010000000000', '1090408060000000000', '1090402010000000000', '1090404010000000000', '1090413030000000000', '1090310010400000000', '1090407040000000000', '1090519990000000000', '1080111030000000000', '1090615010000000000', '1070210050000000000', '1080116000000000000', '1090124030000000000', '1090404030000000000', '1090310010100000000', '1090516040000000000', '1090510030000000000', '1090413020200000000', '1090413060000000000', '1070201110000000000', '1070203070000000000', '1070213080000000000', '1090124020000000000', '1090310010300000000', '1080418990000000000', '1090102990000000000', '1090127040000000000', '1090120010000000000', '1090403990000000000', '1100101010600000000', '1100101010700000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 新能源装备制造产业链 gf 53698
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020599010000000000', '1070218090000000000', '1070201010200000000', '1070214010000000000', '1070202010200000000', '1070212030000000000', '1070213019900000000', '1070213020000000000', '1070215060000000000', '1070403990000000000', '1070603060000000000', '1080119000000000000', '1080314010500000000', '1080414020000000000', '1080401010000000000', '1090124080000000000', '1090407010000000000', '1090401010000000000', '1090401020000000000', '1090402040000000000', '1090603070000000000', '1090122990000000000', '1090409990000000000', '1090619050000000000', '1090509020000000000', '1090520000000000000', '1090601020000000000', '1090601040000000000', '3040301010000000000', '3040301990000000000', '3040105000000000000', '1070101060000000000', '1070103010000000000', '1070202020199000064', '1070201100000000000', '1070213060000000000', '1070208010000000000', '1070601010200000000', '1070404000000000000', '1070401030000000000', '1080130010000000000', '1080108040000000000', '1080110029900000000', '1080310990000000000', '1080207110000000000', '1090103010000000000', '1090129050000000000', '1090129990000000000', '1090123000000000000', '1090124010000000000', '1090207010000000000', '1090408060000000000', '1090402010000000000', '1090404010000000000', '1090413030000000000', '1090310010400000000', '1090407040000000000', '1090519990000000000', '1080111030000000000', '1090615010000000000', '1070210050000000000', '1080116000000000000', '1090124030000000000', '1090404030000000000', '1090310010100000000', '1090516040000000000', '1090510030000000000', '1090413020200000000', '1090413060000000000', '1070201110000000000', '1070203070000000000', '1070213080000000000', '1090124020000000000', '1090310010300000000', '1080418990000000000', '1090102990000000000', '1090127040000000000', '1090120010000000000', '1090403990000000000', '1100101010600000000', '1100101010700000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test

# 智能无人系统和航空产业链 xf 41644
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where xf_dsswjg like '%榆林市%' and xf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070213080000000000', '1080314019900000000', '1080325000000000000', '1080499000000000000', '1080400000000000000', '1090256010000000000', '3040401020200000000', '1090324040000000000', '1090253100000000000', '1090507010000000000', '1090507030000000000', '1090609040000000000', '3010300000000000000', '3040401029900000256', '1070603060000000000', '1090121020000000000', '1090404060000000000', '1090403030000000000', '1090407040000000000', '1090502010000000000', '3050100000000000000', '1060301030500000000', '3040401010200000000', '3040401019900000256', '3040101000000000000', '1090606080000000000', '3040203000000000000', '2020000000000000000', '1060301010100000000', '1090102020000000000', '1090256030000000000', '1090520000000000000', '1090413029900000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, kpyf) test
# 智能无人系统和航空产业链 gf 49296
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf, sum(je) from datapool.t_data_sh_new_qyjfkpsj where gf_dsswjg like '%榆林市%' and gf_shxydm not like '11%' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070213080000000000', '1080314019900000000', '1080325000000000000', '1080499000000000000', '1080400000000000000', '1090256010000000000', '3040401020200000000', '1090324040000000000', '1090253100000000000', '1090507010000000000', '1090507030000000000', '1090609040000000000', '3010300000000000000', '3040401029900000256', '1070603060000000000', '1090121020000000000', '1090404060000000000', '1090403030000000000', '1090407040000000000', '1090502010000000000', '3050100000000000000', '1060301030500000000', '3040401010200000000', '3040401019900000256', '3040101000000000000', '1090606080000000000', '3040203000000000000', '2020000000000000000', '1060301010100000000', '1090102020000000000', '1090256030000000000', '1090520000000000000', '1090413029900000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, kpyf) test


In [ ]:
xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'

In [29]:
# 各产业链各产品的销售企业和购买企业汇总
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
ls_file=os.listdir('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent')
df_chain_xf=pd.DataFrame()
df_chain_gf=pd.DataFrame()

for i in ls_chain:
    print(i)
    for j in ls_file:
        if j.startswith('2023'+i) and j.endswith('.csv'):
            if j.endswith('xf.csv'):
                df_fp=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent\\'+j,dtype='str')
                df_fp=df_fp.astype(dtype={'sum(je)':'float'})
                df_fp.rename(columns={'xf_shxydm':'shxydm','xf_nsrmc':'nsrmc','xf_hydm':'hydm','xf_sjswjg':'sjswjg','xf_dsswjg':'dsswjg','xf_qxswjg':'qxswjg','sum(je)':'xf_je'},inplace=True)
                df_fp.loc[:,'chain']=i
                df_fp.loc[:,'year']=df_fp['kpyf'].str[:4]
                df_chain_xf=pd.concat([df_chain_xf,df_fp],axis=0)
                
            elif j.endswith('gf.csv'):
                df_fp=pd.read_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent\\'+j,dtype='str')
                df_fp=df_fp.astype(dtype={'sum(je)':'float'})
                df_fp.rename(columns={'gf_shxydm':'shxydm','gf_nsrmc':'nsrmc','gf_hydm':'hydm','gf_sjswjg':'sjswjg','gf_dsswjg':'dsswjg','gf_qxswjg':'qxswjg','sum(je)':'gf_je'},inplace=True)
                df_fp.loc[:,'chain']=i
                df_fp.loc[:,'year']=df_fp['kpyf'].str[:4]
                df_chain_gf=pd.concat([df_chain_gf,df_fp],axis=0)

智能无人系统和航空产业链
轻纺产业链
“碳中和”产业链
氢能产业链
铝镁及兰炭深加工产业链
现代特色农业产业链
现代煤化工产业链
文化旅游产业链
能源化工装备制造产业链
新能源装备制造产业链


In [44]:
df_chain_xf_year=df_chain_xf.groupby(['shxydm','nsrmc','hydm','spbm','sjswjg','dsswjg','qxswjg','year','chain'])['xf_je'].sum().reset_index()
df_chain_gf_year=df_chain_gf.groupby(['shxydm','nsrmc','hydm','spbm','sjswjg','dsswjg','qxswjg','year','chain'])['gf_je'].sum().reset_index()
# 筛选金额je 大于 10000以上的数据
df_chain_xf_year_=df_chain_xf_year.loc[df_chain_xf_year['xf_je']>=10000]
df_chain_gf_year_=df_chain_gf_year.loc[df_chain_gf_year['gf_je']>=10000]
print(len(df_chain_xf_year),len(df_chain_xf_year_))
print(len(df_chain_gf_year),len(df_chain_gf_year_))

df_chain_ent=pd.merge(left=df_chain_xf_year_,right=df_chain_gf_year_,on=['shxydm','nsrmc','hydm','spbm','sjswjg','dsswjg','qxswjg','year','chain'],how='outer')
df_chain_ent_1=pd.merge(left=df_chain_ent,right=df_fp_name_,on=['spbm'],how='left')
df_chain_ent_1.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\base_data\\chain_ent\\chain_ent_detail.csv',index=False,mode='w')

220896 117789
392125 152638


In [ ]:
# 202302
# 碳中和 xf 27017
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000', '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000', '1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg) test
# 碳中和 gf 20161
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test
# 碳中和 gf 28001
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000','1080104040000000000', '1080420000000000000', '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000','1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test

# 铝镁及兰炭深加工产业链 xf 26103
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020503050000000000', '1070202110000000000', '1070214130000000000', '1070203010100000000', '1080314030200000000', '1080306020000000000', '1080307000000000000', '1080314010500000000', '1080316010000000000', '1090137010000000000', '1080417000000000000', '1080316020000000000', '1090136040000000000', '1090136990000000000', '1080306030000000000', '3040301990000000000', '3040105000000000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070213080000000000', '1080127030000000000', '1070201070000000000', '1070201220000000000', '1080130010000000000', '1080103090000000000', '1080314010700000000', '1080207040000000000', '1090234000000000000', '1090206010000000000', '1090519010000000000', '1080325000000000000', '1080209000000000000', '1100202030000000000', '1020203000000000000', '3040103000000000000', '1070101990000000000', '1070101030100000000', '1070101050100000000', '1070213019900000000', '1080419030000000000', '1090310019900000000', '2010500000000000000', '1020102000000000000', '3060000000000000000', '1070103020000000000', '1070202020199000064', '1080103080000000000', '1080208000000000000', '1080321010000000000', '1090206050000000000', '1090314040000000000', '1090509010000000000', '1090517000000000000', '1090500000000000000', '1090413029900000000', '3050100000000000000', '1100101010100000000', '1100201040000000000', '1100201990000000000', '1060509029900000000', '1020101000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg) test
# 铝镁及兰炭深加工产业链 gf 25053
select count(*) from (select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020503050000000000', '1070202110000000000', '1070214130000000000', '1070203010100000000', '1080314030200000000', '1080306020000000000', '1080307000000000000', '1080314010500000000', '1080316010000000000', '1090137010000000000', '1080417000000000000', '1080316020000000000', '1090136040000000000', '1090136990000000000', '1080306030000000000', '3040301990000000000', '3040105000000000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070213080000000000', '1080127030000000000', '1070201070000000000', '1070201220000000000', '1080130010000000000', '1080103090000000000', '1080314010700000000', '1080207040000000000', '1090234000000000000', '1090206010000000000', '1090519010000000000', '1080325000000000000', '1080209000000000000', '1100202030000000000', '1020203000000000000', '3040103000000000000', '1070101990000000000', '1070101030100000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test
# 铝镁及兰炭深加工产业链 gf 17807
select count(*) from (select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070101050100000000', '1070213019900000000', '1080419030000000000', '1090310019900000000', '2010500000000000000', '1020102000000000000', '3060000000000000000', '1070103020000000000', '1070202020199000064', '1080103080000000000', '1080208000000000000', '1080321010000000000', '1090206050000000000', '1090314040000000000', '1090509010000000000', '1090517000000000000', '1090500000000000000', '1090413029900000000', '3050100000000000000', '1100101010100000000', '1100201040000000000', '1100201990000000000', '1060509029900000000', '1020101000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test

# 能源化工装备制造产业链 xf 22655
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020503050000000000', '1070202110000000000', '1070214130000000000', '1070203010100000000', '1080314030200000000', '1080306020000000000', '1080307000000000000', '1080314010500000000', '1080316010000000000', '1090137010000000000', '1080417000000000000', '1080316020000000000', '1090136040000000000', '1090136990000000000', '1080306030000000000', '3040301990000000000', '3040105000000000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070213080000000000', '1080127030000000000', '1070201070000000000', '1070201220000000000', '1080130010000000000', '1080103090000000000', '1080314010700000000', '1080207040000000000', '1090234000000000000', '1090206010000000000', '1090519010000000000', '1080325000000000000', '1080209000000000000', '1100202030000000000', '1020203000000000000', '3040103000000000000', '1070101990000000000', '1070101030100000000', '1070101050100000000', '1070213019900000000', '1080419030000000000', '1090310019900000000', '2010500000000000000', '1020102000000000000', '3060000000000000000', '1070103020000000000', '1070202020199000064', '1080103080000000000', '1080208000000000000', '1080321010000000000', '1090206050000000000', '1090314040000000000', '1090509010000000000', '1090517000000000000', '1090500000000000000', '1090413029900000000', '3050100000000000000', '1100101010100000000', '1100201040000000000', '1100201990000000000', '1060509029900000000', '1020101000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg) test
# 能源化工装备制造产业链 gf 17635
select count(*) from (select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1090299000000000000', '1090253010000000000', '1090513040000000000', '3059900000000000000', '4010100000000000000', '1080207130000000000', '1090112140000000000', '1090202020000000000', '1090202090000000000', '1090202010000000000', '1090202080000000000', '1090136990000000000', '1090253100000000000', '1090207000000000000', '1090114010000000000', '1090201030000000000', '1090201070000000000', '1090202000000000000', '3050200000000000000', '1090306990000000000', '1080103030000000000', '3040205000000000000', '3040105000000000000', '1090117010000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test
# 能源化工装备制造产业链 gf 12462
select count(*) from (select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1090119010000000000', '1090127010000000000', '1090201020000000000', '1090203040000000000', '1090201990000000000', '1090120040000000000', '1090520000000000000', '3040200000000000000', '3040301010000000000', '3040502019900000256', '3049900000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test

# 轻纺产业链 xf 6643
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1020503050000000000', '1070202110000000000', '1070214130000000000', '1070203010100000000', '1080314030200000000', '1080306020000000000', '1080307000000000000', '1080314010500000000', '1080316010000000000', '1090137010000000000', '1080417000000000000', '1080316020000000000', '1090136040000000000', '1090136990000000000', '1080306030000000000', '3040301990000000000', '3040105000000000000', '1020199000000000000', '1070103010000000000', '1070102020500000000', '1070213080000000000', '1080127030000000000', '1070201070000000000', '1070201220000000000', '1080130010000000000', '1080103090000000000', '1080314010700000000', '1080207040000000000', '1090234000000000000', '1090206010000000000', '1090519010000000000', '1080325000000000000', '1080209000000000000', '1100202030000000000', '1020203000000000000', '3040103000000000000', '1070101990000000000', '1070101030100000000', '1070101050100000000', '1070213019900000000', '1080419030000000000', '1090310019900000000', '2010500000000000000', '1020102000000000000', '3060000000000000000', '1070103020000000000', '1070202020199000064', '1080103080000000000', '1080208000000000000', '1080321010000000000', '1090206050000000000', '1090314040000000000', '1090509010000000000', '1090517000000000000', '1090500000000000000', '1090413029900000000', '3050100000000000000', '1100101010100000000', '1100201040000000000', '1100201990000000000', '1060509029900000000', '1020101000000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg) test
# 轻纺产业链 gf 12918
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1040102010000000000', '1040102020000000000', '1040102040000000000', '1040101020000000000', '1040111020000000000', '1040112040000000000', '1090228180000000000', '1010399000000000000', '1040201990000000000', '3040301020000000000', '3040303010600000000', '1040201220000000000', '3040101000000000000', '1040201000000000000', '1040201190000000000', '1040114010000000000', '1040106990000000000', '1040102050000000000', '1010303050000000000', '1040200000000000000', '1040101060000000000', '1040199990000000000', '1040199060000000000', '1040101030000000000', '3040601000000000000', '3040302000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test

# 氢能产业链 xf 13008
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070214020000000000', '1080314030200000000', '1080410000000000000', '3040407990000000000', '1070212020000000000', '1070102020500000000', '1070201160000000000', '1070213050000000000', '1090121020000000000', '1090403030000000000', '3050100000000000000', '3010403000000000000', '3010402010000000000', '3010402000000000000', '1090128030000000000', '1090128060000000000', '1090306990000000000', '1090413040000000000', '1090413060000000000', '3010102020099990016', '1080103030000000000', '1090102990000000000', '1090118000000000000', '1090119010000000000', '1090119020000000000', '1090601040000000000', '1090113040000000000', '1100201010000000000', '1100201990000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg) test
# 氢能产业链 gf 11210
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1070214020000000000', '1080314030200000000', '1080410000000000000', '3040407990000000000', '1070212020000000000', '1070102020500000000', '1070201160000000000', '1070213050000000000', '1090121020000000000', '1090403030000000000', '3050100000000000000', '3010403000000000000', '3010402010000000000', '3010402000000000000', '1090128030000000000', '1090128060000000000', '1090306990000000000', '1090413040000000000', '1090413060000000000', '3010102020099990016', '1080103030000000000', '1090102990000000000', '1090118000000000000', '1090119010000000000', '1090119020000000000', '1090601040000000000', '1090113040000000000', '1100201010000000000', '1100201990000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test

# 文化旅游产业链 xf 24290
select count(*) from ( select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('3040301990000000000', '3040105000000000000', '3070301000000000000', '3070101000000000000', '3040304020000000000', '3040701040000000000', '3070102000000000000', '1060301030300000000', '3040301010000000000', '3040204000000000000', '3070302000000000000', '1090511010000000000', '1060500000000000000', '3070402000000000000', '3070401000000000000', '3040801990000000000', '3040701020000000000') group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg) test
# 文化旅游产业链 gf 11331
select count(*) from ( select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('3040301990000000000', '3040105000000000000', '3070301000000000000', '3070101000000000000', '3040304020000000000', '3040701040000000000', '3070102000000000000', '1060301030300000000', '3040301010000000000', '3040204000000000000', '3070302000000000000', '1090511010000000000', '1060500000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test
# 文化旅游产业链 gf 28998
select count(*) from (select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('3070402000000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test
# 文化旅游产业链 gf 29341
select count(*) from (select gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('3070401000000000000', '3040801990000000000', '3040701020000000000') group by gf_shxydm, gf_nsrmc, gf_hydm, spbm, gf_sjswjg, gf_dsswjg, gf_qxswjg) test

In [ ]:
select count(*) from ( select xf_shxydm from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000', '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000', '1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000') group by xf_shxydm) test

In [ ]:
# 碳中和'1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000'
# 碳中和, '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000', '1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000'

In [ ]:
select xf_shxydm from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and gf_shxydm != '-' and xf_shxydm != '-' and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000') group by xf_shxydm

In [ ]:
select xf_shxydm, xf_nsrmc, spbm, sum(je) from datapool.t_data_sh_new_qyjfkpsj group by xf_shxydm, xf_nsrmc, spbm limit 10

In [ ]:
select * from datapool.t_data_sh_new_qyjfkpsj limit 10

In [ ]:
select xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg, sum(je) from datapool.t_data_sh_new_qyjfkpsj group by xf_shxydm, xf_nsrmc, xf_hydm, spbm, xf_sjswjg, xf_dsswjg, xf_qxswjg limit 10

In [ ]:
# 不要行业代码为空的数据 19828
# select count(*) from ( select xf_shxydm from datapool.t_data_sh_new_qyjfkpsj where kpyf='202302' and xf_hydm is not null and gf_hydm is not null and gf_shxydm is not null and xf_shxydm is not null and gf_shxydm != '000000000000000' and xf_shxydm != '000000000000000'  and spbm in ('1030301990000000000', '1070219040000000000', '1070219010000000000', '1070214130000000000', '1070202100000000000', '1070214010000000000', '1070214020000000000', '1070201050000000000', '1070213040000000000', '1080127020000000000', '1020501020000000000', '1070601130000000000', '1080314030200000000', '1080306010000000000', '1080314019900000000', '1080325000000000000', '1080306030000000000', '1080499000000000000', '1080400000000000000', '1090299000000000000', '1090201010000000000', '3040301990000000000', '3040105000000000000', '1060101030000000000', '1090422020000000000', '1070102020500000000', '1020199000000000000', '1070103010000000000', '1070203060000000000', '1070201150000000000', '1060102060000000000', '1070219050000000000', '1070214150000000000', '1070204030000000000', '1070201070000000000', '1070201220000000000', '1080124020000000000', '1080126000000000000', '1080199010000000000', '1090204070000000000', '1090404010000000000', '1090413030000000000', '1070201020100000000', '3010403000000000000', '3010402010000000000', '3040101000000000000', '1090606080000000000', '3040103000000000000', '1030104010000000000', '1070201080000000000', '1070101030100000000', '1070214090000000000', '1070101040100000000', '1080104040000000000', '1080420000000000000', '1080307000000000000', '1090128030000000000', '1090128070000000000', '1090207000000000000', '1100101020100000000', '1090413050000000000', '1090603010000000000', '3059900000000000000', '1090409040000000000', '1060301010100000000', '3040204000000000000', '3040201990000000000', '3040202030000000000', '1070101150000000000', '2010400000000000000', '1050104010000000000', '1070102020600000000', '1070201010200000000', '1070205030000000000', '1010111010000000000', '1070202029900000000', '1080110020100000000', '1080101020000000000', '1090413029900000000', '1100101011100000000') group by xf_shxydm) test

In [2]:
# 发票数据
df_fp=pd.read_csv(r'E:\project_basedata\chain_supply\chain_result_0413\total_fp_2023_1.csv',dtype='str')
df_fp=df_fp.astype(dtype={'je':'float','sl':'float'})

In [ ]:

# 当年当季度数据
date=df_fp_6_2_['kpyf'].unique().tolist()
year=sorted([x[:4] for x in date])
quarter=sorted(list(set([x[-2:] for x in date])))
ls_date=[]
for i in quarter:
    date_i=year[-1]+i
    df_i=df_fp_6_2_.loc[df_fp_6_2_['kpyf']==date_i]
    ls_date.append(len(df_i))
latest=len([x for x in ls_date if x!=0])-1
latest_date=year[-1]+quarter[latest]
df_fp_4_2=df_fp_6_2_.loc[df_fp_6_2_['kpyf']==latest_date]
df_fp_4_allyear=df_fp_6_2_.loc[df_fp_6_2_['kpyf'].str.contains(year[-1])]

In [94]:
sorted(['202001','202103','202104','202101','202102','202202','202402'])

['202001', '202101', '202102', '202103', '202104', '202202', '202402']

In [97]:
# home_8产业链企业数量
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
df_reault_chain_market=pd.DataFrame()
df_reault_chain_market_allyear=pd.DataFrame()
ls_chain=[]
for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_ent=df_fp_4_allyear.loc[df_fp_4_allyear['spbm'].isin(ls_spbm)]
    ls1=df_chain_ent.loc[(df_chain_ent['xf_city']=='榆林市')&(df_chain_ent['xf_hydm'].str.contains('F'))&(df_chain_ent['xf_hydm'].str.contains('G'))]['xf_shxydm'].unique().tolist()
    ls2=df_chain_ent.loc[(df_chain_ent['gf_city']=='榆林市')&(df_chain_ent['gf_hydm'].str.contains('F'))&(df_chain_ent['gf_hydm'].str.contains('G'))]['gf_shxydm'].unique().tolist()
    ls=list(set(ls1+ls2))
    ls_chain.append([i,len(ls)])
pd.DataFrame(ls_chain,columns=['chain','ent_num']).to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\home_08_企业数量_全年.csv',index=False,mode='a+')

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_admin_pro,gf_admin_pro,xf_admin_city,gf_admin_city
289930,4855640,11,91110000MA019CJN99,北京国际度假区有限公司,NaN,NaN,NaN,NaN,916108213387027045,陕西博之源机电有限公司,C4190,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局大柳塔税务分局,*餐饮服务*餐饮服务,3070401000000000000,128.30,7.7,NaN,NaN,2023-01,2023-04-03 23:04:27,北京,北京市,NaN,陕西省,榆林市,神木市,11,NaN,1100,NaN,110000,NaN,北京,NaN,北京市,NaN
289931,4855641,11,91610802MA70D69637,榆林睿博洋文化传媒有限公司,L7251,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局驼峰路税务分局,11610802016082872W,榆阳区红石桥乡人民政府,S9100,NaN,NaN,NaN,*印刷品*亚克力卡卡,1060201990000000000,134.66,1.34,16.0,个,2023-01,2023-04-03 23:04:27,陕西省,榆林市,榆林市榆阳区,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610802,NaN,陕西省,NaN,榆林市
289932,4855642,11,91610825067908267G,定边县山丹丹苦荞产业发展有限责任公司,C1331,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局贺圈税务分局,1132102338858549A,宝应县市场监管管理局,NaN,NaN,NaN,NaN,*植物油*胡麻油,1030105010300000000,16065.00,0,357.0,瓶,2023-01,2023-04-03 23:04:27,陕西省,榆林市,定边县,江苏省,扬州市,NaN,NaN,32,NaN,3210,NaN,321023,NaN,江苏省,NaN,扬州市
289935,4855645,11,91110114076602948E,国家能源集团置业有限公司北京昌平中心,NaN,NaN,NaN,NaN,91610821770043971N,中国神华能源股份有限公司神东煤炭分公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理一股,*坚果*巴旦木,1010115031300000000,308.26,27.74,8.0,罐,2023-01,2023-04-03 23:04:27,北京,北京市,NaN,陕西省,榆林市,榆林市,11,NaN,1101,NaN,110114,NaN,北京,NaN,北京市,NaN
289936,4855646,11,91110106691666273C,北京贝林科技有限公司,NaN,NaN,NaN,NaN,91610806MA70391C58,国能榆林化工有限公司,C2614,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆神工业区税务局清水税务分局,*分析仪器*消解管光源,1090605990000000000,384.97,50.05,1.0,件,2023-01,2023-04-03 23:04:27,北京,北京市,NaN,陕西省,榆林市,榆林市榆神工业区,11,NaN,1101,NaN,110106,NaN,北京,NaN,北京市,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773281,6338991,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*家财险,3060302990000000000,167.93,10.07,1.0,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市
1773283,6338993,82,91610827MA7034FL46,陕西青创联盟电子商务股份有限公司,I6432,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局米脂县税务局银州税务分局,82445122MC6937324Q,饶平县樟溪镇工会工作委员会,NaN,NaN,NaN,NaN,*谷物加工品*黄小米,1030102010200000000,-352.29,-31.71,-8.0,袋,2023-01,2023-04-04 08:23:03,陕西省,榆林市,米脂县,广东省,潮州市,NaN,NaN,44,NaN,4451,NaN,445122,NaN,广东省,NaN,潮州市
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3060302990000000000,44.34,2.66,1.0,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3060302020000000000,492.92,29.58,1.0,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市


In [ ]:
# detail_1历年产业链销售总额及增速
# 
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
ls_result_chain_je=[]
for i in range(len(year)):
    for j in ls_chain:
        df_j=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==j]
        ls_spbm=df_i['spbm'].unique().tolist()
        df_ij=df_fp_6_2_1.loc[(df_fp_6_2_1['kpyf'].str.contains(year[i]))&(df_fp_6_2_1['spbm'].isin(ls_spbm))]
        chain_ij_je=df_ij['je'].sum()
        df_ij_last=df_fp_6_2_1.loc[(df_fp_6_2_1['kpyf'].str.contains(year[i-1]))&(df_fp_6_2_1['spbm'].isin(ls_spbm))]
        chain_ij_je_last=df_ij_last['je'].sum()
        if chain_ij_je_last=0:
            rate=0
        else:
            rate=(chain_ij_je-chain_ij_je_last)/chain_ij_je_last
        ls_result_chain_je.append(i,j,chain_ij_je,rate)
df_result_chain_je=pd.DataFrame(ls_result_chain_je,columns=['year','chain','je','rate'])
df_result_chain_je.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\detail_01_产业链历年销售金额_全年.csv',index=False,mode='a+')

# 增量运行
for j in ls_chain:
    df_ij=df_fp_6_2_.loc[(df_fp_6_2_['kpyf'].str.contains(year[i]))&(df_fp_6_2_['spbm'].isin(ls_spbm))]

In [9]:
pd.DataFrame(columns=['id'])['id'].sum()

0

In [ ]:
select  count(*) from (select xf_shxydm 
datapool.t_data_sh_new_qyjfkpsj where kpyf='202303' group by xf_shxydm 

In [79]:
# 当年当季度数据
date=df_result_1['kpyf'].unique().tolist()
year=sorted([x[:4] for x in date])
quarter=sorted(list(set([x[-2:] for x in date])))
ls_date=[]
for i in quarter:
    date_i=year[-1]+i
    df_i=df_result_1.loc[df_result_1['kpyf']==date_i]
    ls_date.append(len(df_i))
latest=len([x for x in ls_date if x!=0])
latest_date=year[-1]+quarter[latest]
df_fp_4_2=df_fp_4_2_.loc[df_fp_4_2_['kpyf']==latest_date]

'202301'

In [3]:
# 根据20230920在榆林绘制的产业链，更新产业链的构成数据
df_chain_name=pd.read_csv(r'E:\project_data\04yulin\connection\prediction\data\T_CHAIN_GRAPH_202309221528.csv',dtype='str')
df_chain_name_10=df_chain_name.loc[df_chain_name['DESCRIPTION']=='政策十条重点产业链']
df_chain_nodes=pd.read_csv(r'E:\project_data\04yulin\connection\prediction\data\T_CHAIN_GRAPH_NODE_202309211036.csv',dtype='str')
df_total_nodes=pd.read_csv(r'E:\project_data\04yulin\connection\prediction\data\T_CODE_PRODUCT_202309211038.csv',dtype='str')
df_chain_edge=pd.read_csv(r'E:\project_data\04yulin\connection\prediction\data\T_CHAIN_GRAPH_EDGE_202309211035.csv',dtype='str')

In [21]:
# 产业链产品对应数据
df_chain_nodes=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台验收\十条产业链的核心产品.xlsx',dtype='str')
# 发票数据
df_fp=pd.read_csv(r'E:\project_basedata\chain_supply\chain_result_0413\total_fp_2023_1.csv',dtype={'gf_sjswjg':'str','gf_dsswjg':'str','gf_qxswjg':'str','gf_shxydm':'str','spbm':'str'})
df_fp_4_=pd.read_csv(r'E:\project_项目\04榆林产业大脑\平台验收\frontpage_data\fp\fp_new.csv',dtype='str')
df_fp_4_=df_fp_4_.astype(dtype={'je':'float','sl':'float'})
df_fp_name=pd.read_excel(r'E:\project_data\04yulin\fp\t_data_sh_new_cpdm_202309261607.xlsx',dtype='str')
df_fp_name_=df_fp_name.loc[:,['spbm','spmc']]

In [4]:
df_fp_=df_fp.copy()
df_fp_.loc[df_fp_['xf_sjswjg'].notnull(),'xf_province']=df_fp_.loc[df_fp_['xf_sjswjg'].notnull()]['xf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['xf_dsswjg'].notnull(),'xf_city']=df_fp_.loc[df_fp_['xf_dsswjg'].notnull()]['xf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['xf_qxswjg'].notnull(),'xf_district']=df_fp_.loc[df_fp_['xf_qxswjg'].notnull()]['xf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()
df_fp_.loc[df_fp_['gf_sjswjg'].notnull(),'gf_province']=df_fp_.loc[df_fp_['gf_sjswjg'].notnull()]['gf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['gf_dsswjg'].notnull(),'gf_city']=df_fp_.loc[df_fp_['gf_dsswjg'].notnull()]['gf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_fp_.loc[df_fp_['gf_qxswjg'].notnull(),'gf_district']=df_fp_.loc[df_fp_['gf_qxswjg'].notnull()]['gf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()

In [5]:
df_fp_.loc[(df_fp_['xf_province'].isnull()),'xf_province_code']=df_fp_['xf_shxydm'].str[2:4]
df_fp_.loc[(df_fp_['gf_province'].isnull()),'gf_province_code']=df_fp_['gf_shxydm'].str[2:4]

df_fp_.loc[(df_fp_['xf_province'].isnull()),'xf_city_code']=df_fp_['xf_shxydm'].str[2:6]
df_fp_.loc[(df_fp_['gf_province'].isnull()),'gf_city_code']=df_fp_['gf_shxydm'].str[2:6]

df_fp_.loc[(df_fp_['xf_province'].isnull()),'xf_district_code']=df_fp_['xf_shxydm'].str[2:8]
df_fp_.loc[(df_fp_['gf_province'].isnull()),'gf_district_code']=df_fp_['gf_shxydm'].str[2:8]

In [7]:
df_province=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_province',dtype='str')
df_city=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_city',dtype='str')
df_dis=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',sheet_name='t_admin_division_code_city',dtype='str')
df_city=df_city.drop_duplicates(['type_4'],keep='first').reset_index(drop=True)

In [17]:
df_fp_1=pd.merge(left=df_fp_,right=df_province,left_on=['xf_province_code'],right_on=['type_4'],how='left')
df_fp_1.rename(columns={'admin_name':'xf_admin_pro'},inplace=True)
df_fp_1.drop(columns=['type_4'],inplace=True)
df_fp_2=pd.merge(left=df_fp_1,right=df_province,left_on=['gf_province_code'],right_on=['type_4'],how='left')
df_fp_2.rename(columns={'admin_name':'gf_admin_pro'},inplace=True)
df_fp_2.drop(columns=['type_4'],inplace=True)

df_fp_3=pd.merge(left=df_fp_2,right=df_city,left_on=['xf_city_code'],right_on=['type_4'],how='left')
df_fp_3.rename(columns={'admin_name':'xf_admin_city'},inplace=True)
df_fp_3.drop(columns=['type_4'],inplace=True)
df_fp_4=pd.merge(left=df_fp_3,right=df_city,left_on=['gf_city_code'],right_on=['type_4'],how='left')
df_fp_4.rename(columns={'admin_name':'gf_admin_city'},inplace=True)
df_fp_4.drop(columns=['type_4'],inplace=True)

In [19]:
df_fp_4_=df_fp_4.copy()
df_fp_4_.loc[df_fp_4_['xf_province'].isnull(),'xf_province']=df_fp_4_['xf_admin_pro']
df_fp_4_.loc[df_fp_4_['gf_province'].isnull(),'gf_province']=df_fp_4_['gf_admin_pro']
df_fp_4_.loc[df_fp_4_['xf_city'].isnull(),'xf_city']=df_fp_4_['xf_admin_city']
df_fp_4_.loc[df_fp_4_['gf_city'].isnull(),'gf_city']=df_fp_4_['gf_admin_city']

In [28]:
df_fp_4_1=df_fp_4_.loc[(df_fp_4_['xf_city'].notnull())&(df_fp_4_['gf_city'].notnull())]
df_chain_nodes_=df_chain_nodes.loc[df_chain_nodes['fp_code'].notnull()]
ls_chain=df_chain_nodes_['chain_x'].unique().tolist()
df_result=pd.DataFrame()
for i in ls_chain:
    df_i=df_chain_nodes_.loc[df_chain_nodes_['chain_x']==i]
    ls_spbm=df_i['spbm'].unique().tolist()
    df_chain_fp=df_fp_4_1.loc[df_fp_4_1['spbm'].isin(ls_spbm)]
    print(i,len(df_chain_fp))
    # 产业链榆林本地企业
    df_xf=df_chain_fp.groupby(['spbm','xf_shxydm','xf_nsrmc','xf_province','xf_city'])['je'].sum().reset_index()
    df_xf.rename(columns={'xf_shxydm':'shxydm','xf_nsrmc':'nsrmc','xf_province':'province','xf_city':'city','je':'xf_je'},inplace=True)
    df_gf=df_chain_fp.groupby(['spbm','gf_shxydm','gf_nsrmc','gf_province','gf_city'])['je'].sum().reset_index()
    df_gf.rename(columns={'gf_shxydm':'shxydm','gf_nsrmc':'nsrmc','gf_province':'province','gf_city':'city','je':'gf_je'},inplace=True)
    df_ent=pd.merge(left=df_xf,right=df_gf,on=['spbm','shxydm','nsrmc','province','city'],how='outer')
    
    # df_ent_chain_i=df_chain_fp.groupby(['spbm','xf_shxydm','xf_nsrmc','xf_province','xf_city','gf_shxydm','gf_nsrmc','gf_province','gf_city'])['je'].sum().reset_index()
    df_ent_1=pd.merge(left=df_ent,right=df_fp_name_,on=['spbm'],how='left')
    df_ent_1.loc[:,'chain']=i
    # df_ent_chain_1.loc[:,'chain']=i
    # df_market_pro=df_chain_fp.groupby(['spbm'])['je'].sum().reset_index()
    # df_market=df_chain_fp.groupby(['gf_province','gf_city'])['je'].sum().reset_index()
    # df_market_pro_=pd.merge(left=df_market_pro,right=df_fp_name_,on=['spbm'],how='left')
    # df_market_pro_1=df_market_pro_.loc[:,['spbm','spmc','je']]
    # df_market_pro_1.loc[:,'chain']=i
    # df_market_pro_2=df_market_pro_1.drop_duplicates()
    df_result=pd.concat([df_result,df_ent_1])
    # df_market_pro_.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\fp\\'+i+'_product_fp.csv',index=False,mode='w')
    # df_market.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\fp\\'+i+'_area_fp.csv',index=False,mode='w')

智能无人系统和航空产业链 42200
轻纺产业链 10505
“碳中和”产业链 51033
氢能产业链 17270
铝镁及兰炭深加工产业链 77607
现代特色农业产业链 71143
现代煤化工产业链 25056
文化旅游产业链 76911
能源化工装备制造产业链  37349
新能源装备制造产业链 33407


In [27]:
ls1=df_result['xf_nsrmc'].unique().tolist()
len(ls1)
ls2=df_result['gf_nsrmc'].unique().tolist()
len(ls2)
len(list(set(ls1+ls2)))

89987

In [30]:
df_chain_nodes_1=df_chain_nodes_.loc[:,['spbm','tupu_code_y','tupu_name_y']]
df_result_=df_result.loc[df_result['city']=='榆林市']
df_result_1=pd.merge(left=df_result_,right=df_chain_nodes_1,on=['spbm'],how='left')
df_result_2=df_result_1.drop_duplicates()
df_result_2.to_csv('E:\\project_项目\\04榆林产业大脑\\平台验收\\frontpage_data\\fp\\all_product_fp_ent1.csv',index=False,mode='w')

In [29]:
df_result.loc[df_result['city']=='榆林市']

,spbm,shxydm,nsrmc,province,city,xf_je,gf_je,spmc,chain
0,1060301010100000000,911101020973175805,北京天元启力科技有限公司,北京,北京市,5132.74,NaN,系统软件产品,智能无人系统和航空产业链
1,1060301010100000000,91110105MA005CH48R,北京火绒网络科技有限公司,北京,北京市,32743.36,NaN,系统软件产品,智能无人系统和航空产业链
2,1060301010100000000,911101080696430479,北京法宣在线科技有限公司,北京,北京市,283.19,NaN,系统软件产品,智能无人系统和航空产业链
3,1060301010100000000,911101085658469350,北京智达方通科技有限公司,北京,北京市,58407.08,NaN,系统软件产品,智能无人系统和航空产业链
4,1060301010100000000,911101086723891430,国能日新科技股份有限公司,北京,北京市,106194.69,NaN,系统软件产品,智能无人系统和航空产业链
...,...,...,...,...,...,...,...,...,...
26749,3040301990000000000,92610893MA707M6Q90,榆林市高新区普仁口腔门诊部,陕西省,榆林市,NaN,7029.70,其他设计服务,新能源装备制造产业链
26750,3040301990000000000,N2610823MF39111893,榆林市横山区怀仁路街道苏庄则村经济合作社,陕西省,榆林市,NaN,1743.57,其他设计服务,新能源装备制造产业链
26751,3040301990000000000,N2610824MF0438971P,靖边县宁条梁镇西园则村经济联合社,陕西省,榆林市,NaN,1524.27,其他设计服务,新能源装备制造产业链
26752,3040301990000000000,N2610828JTJJ01339D,佳县方塌镇谢家沟村经济合作社,陕西省,榆林市,NaN,15391.09,其他设计服务,新能源装备制造产业链


In [23]:
df_fp_4_.to_csv(r'E:\project_项目\04榆林产业大脑\平台验收\fp_new.csv',index=False)

In [22]:
df_fp_4_

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_admin_pro,gf_admin_pro,xf_admin_city,gf_admin_city
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3060302020000000000,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1100102010100000000,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33,陕西省,榆林市,神木市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1100202020000000000,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3060302990000000000,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3060302990000000000,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3060302020000000000,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1030112990000000000,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03,陕西省,榆林市,清涧县,陕西省,NaN,NaN,NaN,61,NaN,6100,NaN,61000M,NaN,陕西省,NaN,NaN
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,3060301990000000000,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市


In [20]:
df_fp_4_1=df_fp_4_.loc[(df_fp_4_['xf_city'].notnull())&(df_fp_4_['gf_city'].notnull())]


,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_admin_pro,gf_admin_pro,xf_admin_city,gf_admin_city
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3060302020000000000,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1100102010100000000,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33,陕西省,榆林市,神木市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1100202020000000000,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3060302990000000000,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3060302990000000000,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3060302020000000000,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1030112990000000000,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03,陕西省,榆林市,清涧县,陕西省,NaN,NaN,NaN,61,NaN,6100,NaN,61000M,NaN,陕西省,NaN,NaN
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,3060301990000000000,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03,陕西省,榆林市,榆林市,陕西省,榆林市,NaN,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市


In [21]:
df_city_code=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',dtype='str',sheet_name='t_admin_division_code_city')

In [19]:
df_chain_fp_.loc[(df_chain_fp_['xf_province'].isnull()),'xf_province_code']=df_chain_fp_['xf_shxydm'].str[2:4]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull()),'gf_province_code']=df_chain_fp_['gf_shxydm'].str[2:4]

df_chain_fp_.loc[(df_chain_fp_['xf_province'].isnull()),'xf_city_code']=df_chain_fp_['xf_shxydm'].str[2:6]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull()),'gf_city_code']=df_chain_fp_['gf_shxydm'].str[2:6]

df_chain_fp_.loc[(df_chain_fp_['xf_province'].isnull()),'xf_district_code']=df_chain_fp_['xf_shxydm'].str[2:8]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull()),'gf_district_code']=df_chain_fp_['gf_shxydm'].str[2:8]

In [20]:
df_chain_fp_

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1100102010100000000,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33,陕西省,榆林市,神木市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000
34,4565744,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,宋艳,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,3023.00,0.00,183.21,元/平方米,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000
46,4565756,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,陈艳芬,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,842.00,0.00,51.03,元/平方米,2023-01,2023-04-03 21:16:33,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000
55,4565765,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,李孝文,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1100102010100000000,4374.00,0.00,284.00,㎡,2023-01,2023-04-03 21:16:33,陕西省,榆林市,神木市,NaN,NaN,NaN,NaN,00,NaN,0000,NaN,000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772397,6338107,81,91610825MA70D6T01M,定边县百海实业有限公司,L7259,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局人民路税务分局,81610825MC678716X6,定边县油房庄乡工会联合会,NaN,NaN,NaN,NaN,*设计服务*广告设计与制作,3040301990000000000,44673.27,446.73,1.00,套,2023-01,2023-04-04 08:22:40,陕西省,榆林市,定边县,NaN,NaN,NaN,NaN,61,NaN,6108,NaN,610825
1772821,6338531,81,91610802MA70EPCM01,榆林市榆阳区恒生超市有限公司,F5212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局青山路税务分局,81610100MC4642223R,中化现代农业(陕西)有限公司工会委员会,NaN,NaN,NaN,NaN,*塑料制品*垃圾袋,1070601010200000000,77.88,10.12,4.00,包,2023-01,2023-04-04 08:22:40,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,61,NaN,6101,NaN,610100
1772923,6338633,81,91610802MA70ELX11A,陕西昱辰腾达广告传媒有限公司,L7259,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局航宇路税务分局,81610800MC636006XW,陕西陕煤榆北煤业有限公司榆林选煤分公司工会委员会,NaN,NaN,NaN,NaN,*设计服务*合影背景墙画面,3040301990000000000,712.87,7.13,36.00,平方米,2023-01,2023-04-04 08:22:40,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,NaN,61,NaN,6108,NaN,610800
1773094,6338804,81,91610825MA70D6T01M,定边县百海实业有限公司,L7259,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局人民路税务分局,81610825MC7315551C,定边县安边镇工会联合会,NaN,NaN,NaN,NaN,*设计服务*广告设计与制作,3040301990000000000,59405.94,594.06,NaN,NaN,2023-01,2023-04-04 08:23:03,陕西省,榆林市,定边县,NaN,NaN,NaN,NaN,61,NaN,6108,NaN,610825


In [ ]:
# province
df_chain_fp_.loc[(df_chain_fp_['xf_province'].isnull())&(df_chain_fp_['length_xf_shxydm']==18),'xf_province_code']=df_chain_fp_['xf_shxydm'].str[2:4]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull())&(df_chain_fp_['length_gf_shxydm']==18),'gf_province_code']=df_chain_fp_['gf_shxydm'].str[2:4]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull())&(df_chain_fp_['length_gf_nsrmc']==18),'gf_province_code']=df_chain_fp_['gf_shxydm'].str[2:4]
# city
df_chain_fp_.loc[(df_chain_fp_['xf_province'].isnull())&(df_chain_fp_['length_xf_shxydm']==18),'xf_city_code']=df_merge_fp['xf_shxydm'].str[2:6]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull())&(df_chain_fp_['length_gf_shxydm']==18),'gf_city_code']=df_merge_fp['gf_shxydm'].str[2:6]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull())&(df_chain_fp_['length_gf_nsrmc']==18),'gf_city_code']=df_merge_fp['gf_shxydm'].str[2:6]
# district
df_chain_fp_.loc[(df_chain_fp_['xf_province'].isnull())&(df_chain_fp_['length_xf_shxydm']==18),'xf_district_code']=df_merge_fp['xf_shxydm'].str[2:8]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull())&(df_chain_fp_['length_gf_shxydm']==18),'gf_district_code']=df_merge_fp['gf_shxydm'].str[2:8]
df_chain_fp_.loc[(df_chain_fp_['gf_province'].isnull())&(df_chain_fp_['length_gf_nsrmc']==18),'gf_district_code']=df_merge_fp['gf_shxydm'].str[2:8]

In [18]:
df_chain_fp_.loc[df_chain_fp_['xf_district'].isnull()]

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district
290033,4855743,11,91110115064882014Y,北京选晟乐机电科技有限公司,NaN,NaN,NaN,NaN,91610800MA703DR38N,华电榆林煤炭工程技术有限公司,E4790,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理一股,*齿轮*齿轮键,1090124080000000000,177.00,23.01,10.00,个,2023-01,2023-04-03 23:04:49,NaN,NaN,NaN,陕西省,榆林市,榆林高新技术产业开发区
290038,4855748,11,12100000400013601F,中国食品药品检定研究院,NaN,NaN,NaN,NaN,916108006984228963,榆林利君制药有限公司,C2720,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局麻黄梁税务分局,*化学试剂助剂*110857 五味子醇甲,1070214010000000000,472.57,61.43,2.00,支,2023-01,2023-04-03 23:04:49,NaN,NaN,NaN,陕西省,榆林市,榆林市榆阳区
290150,4855860,11,91110108558521630L,小米通讯技术有限公司,NaN,NaN,NaN,NaN,91610800MA70D46G0U,陕西毓锦昌盛建设工程有限公司,E4999,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理二股,*计算机网络设备*小米路由器4A千兆版（白色）,1090510030000000000,246.02,31.98,2.00,NaN,2023-01,2023-04-03 23:04:49,NaN,NaN,NaN,陕西省,榆林市,榆林高新技术产业开发区
290184,4855894,11,91110105801722513K,北京沃德科斯科技发展有限公司,NaN,NaN,NaN,NaN,91610806MA70391C58,国能榆林化工有限公司,C2614,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆神工业区税务局清水税务分局,*通用设备*夹紧螺栓,1090129050000000000,14236.28,1850.72,1.00,件,2023-01,2023-04-03 23:04:49,NaN,NaN,NaN,陕西省,榆林市,榆林市榆神工业区
290227,4855937,11,91110108558521630L,小米通讯技术有限公司,NaN,NaN,NaN,NaN,916108223056901102,府谷县七喜科技有限公司,F5279,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,*计算机网络设备*小米路由器AX6000,1090510030000000000,-213.72,-27.78,NaN,NaN,2023-01,2023-04-03 23:04:49,NaN,NaN,NaN,陕西省,榆林市,府谷县
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770190,6335900,65,91652200682706691G,哈密市伊州区恒信热力有限公司,NaN,NaN,NaN,NaN,91610821MA7050F37R,神木亨益伟机械制造安装有限公司,E4920,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局店塔税务分局,*热力*二级管网维护费,1100102010100000000,251.84,0.00,251.84,平方,2023-01,2023-04-04 08:21:54,NaN,NaN,NaN,陕西省,榆林市,神木市
1770205,6335915,65,91652201MA775JBPXB,哈密市驰骋伟业商贸有限责任公司,NaN,NaN,NaN,NaN,916108000881980060,陕西云普信息科技有限公司,I6599,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理二股,*研发和技术服务*技术服务费,3040105000000000000,130566.04,7833.96,2.00,项,2023-01,2023-04-04 08:21:54,NaN,NaN,NaN,陕西省,榆林市,榆林高新技术产业开发区
1770357,6336067,65,91650103MA78E3PN9B,乌鲁木齐中热融达节能科技有限公司,NaN,NaN,NaN,NaN,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,*电线电缆*线缆,1090409990000000000,11825.25,354.75,180.00,米,2023-01,2023-04-04 08:21:54,NaN,NaN,NaN,陕西省,榆林市,榆林市榆阳区
1770448,6336158,65,9165010905316464XJ,乌鲁木齐鸿润诚达钢构彩板有限公司,NaN,NaN,NaN,NaN,91610893MA70DN0H2R,榆林万鼎铭胜商贸有限公司,F5165,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理二股,*金属制品*钢结构,1080401010000000000,133450.42,17348.56,13.96,吨,2023-01,2023-04-04 08:21:54,NaN,NaN,NaN,陕西省,榆林市,榆林高新技术产业开发区


In [15]:
df_chain_fp_

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1100102010100000000,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33
34,4565744,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,宋艳,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,3023.00,0.00,183.21,元/平方米,2023-01,2023-04-03 21:16:33
46,4565756,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,陈艳芬,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1100102010100000000,842.00,0.00,51.03,元/平方米,2023-01,2023-04-03 21:16:33
55,4565765,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,李孝文,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1100102010100000000,4374.00,0.00,284.00,㎡,2023-01,2023-04-03 21:16:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772397,6338107,81,91610825MA70D6T01M,定边县百海实业有限公司,L7259,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局人民路税务分局,81610825MC678716X6,定边县油房庄乡工会联合会,NaN,NaN,NaN,NaN,*设计服务*广告设计与制作,3040301990000000000,44673.27,446.73,1.00,套,2023-01,2023-04-04 08:22:40
1772821,6338531,81,91610802MA70EPCM01,榆林市榆阳区恒生超市有限公司,F5212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局青山路税务分局,81610100MC4642223R,中化现代农业(陕西)有限公司工会委员会,NaN,NaN,NaN,NaN,*塑料制品*垃圾袋,1070601010200000000,77.88,10.12,4.00,包,2023-01,2023-04-04 08:22:40
1772923,6338633,81,91610802MA70ELX11A,陕西昱辰腾达广告传媒有限公司,L7259,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局航宇路税务分局,81610800MC636006XW,陕西陕煤榆北煤业有限公司榆林选煤分公司工会委员会,NaN,NaN,NaN,NaN,*设计服务*合影背景墙画面,3040301990000000000,712.87,7.13,36.00,平方米,2023-01,2023-04-04 08:22:40
1773094,6338804,81,91610825MA70D6T01M,定边县百海实业有限公司,L7259,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局人民路税务分局,81610825MC7315551C,定边县安边镇工会联合会,NaN,NaN,NaN,NaN,*设计服务*广告设计与制作,3040301990000000000,59405.94,594.06,NaN,NaN,2023-01,2023-04-04 08:23:03


In [9]:
df_fp_1=df_fp.loc[df_fp['spbm'].notnull()]
df_fp_1.astype(dtype={'spbm':'int'})

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,-2147483648,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,-2147483648,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,-2147483648,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,-2147483648,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,-2147483648,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,-2147483648,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,-2147483648,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,-2147483648,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,-2147483648,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03


In [10]:
df_fp_1

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1.100102e+18,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1.100102e+18,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1.100202e+18,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3.060303e+18,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3.060303e+18,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1.030113e+18,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,3.060302e+18,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03


In [6]:
df_fp.loc[33,'spbm']

1.10020202e+18

In [2]:
# 发票数据
df_fp=pd.read_csv(r'E:\project_basedata\chain_supply\chain_result_0413\total_fp_2023_1.csv',dtype={'gf_sjswjg':'str','gf_dsswjg':'str','gf_qxswjg':'str','gf_shxydm':'str'})
# 发票与国标产品对应关系
df_correspond=pd.read_csv(r'E:\project_basedata\chain_supply\全量税务产品与国标产品对应关系_0515_total.csv',dtype={'spbm':'float'},encoding='gbk')
df_correspond_f=df_correspond.loc[:,['spbm','spmc','tupu_code','tupu_name']]
df_correspond_f_drop=df_correspond_f.drop_duplicates(subset=['spbm'])
df_merge_fp=pd.merge(left=df_fp,right=df_correspond_f_drop,on=['spbm'],how='left')

In [4]:
ls_chain_name=df_chain_name_10['NAME'].unique().tolist()
ls_chain_code=df_chain_name_10['ID'].unique().tolist()

In [ ]:
# 经过验证，chain_nodes是total_nodes中的一部分

In [65]:
# 地区编码
df_city_code=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',dtype='str',sheet_name='t_admin_division_code')
df_area_code = pd.read_excel(r'E:\project_basedata\chain_supply\chain_result_0413\districtcode.xlsx',sheet_name = 'Sheet1',dtype ='str')

In [66]:
df_city_code.loc[:,'type_2']=df_city_code['type_4'].str[:2]
# province
df_province=df_city_code.loc[:,['type_2','admin_name']].drop_duplicates(['type_2'],keep='first').reset_index(drop=True)
df_province_1=df_area_code.loc[:,['code1','name1']].drop_duplicates(['code1'],keep='first').reset_index(drop=True)
# city
df_city=df_city_code.loc[:,['type_4','admin_name']].drop_duplicates(['type_4'],keep='first').reset_index(drop=True)
df_city_1=df_area_code.loc[:,['code2','name2']].drop_duplicates(['code2'],keep='first').reset_index(drop=True)
# district
df_district=df_city_code.loc[:,['type_code','admin_name']].drop_duplicates(['type_code'],keep='first').reset_index(drop=True)
df_district_1=df_area_code.loc[:,['code3','name3']].drop_duplicates(['code3'],keep='first').reset_index(drop=True)

In [77]:
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())|(df_merge_fp['xf_province'].isnull())]

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm,length_gf_shxydm,length_gf_nsrmc,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_province_name,gf_province_name,xf_city_name,gf_city_name,xf_district_name,gf_district_name
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1.100102e+18,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33,三北热力,4.402010e+09,热力,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33,机动车交通事故责任强制保险服务,7.002010e+09,交强险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,15,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1.100102e+18,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33,三北热力,4.402010e+09,热力,陕西省,榆林市,神木市,NaN,NaN,NaN,18,15,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1.100202e+18,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33,民用天然气,7.020000e+02,天然气,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3.060303e+18,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33,其他财产保险服务,7.002990e+09,其他财产保险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3.060303e+18,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03,其他财产保险服务,7.002990e+09,其他财产保险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,18,16,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市,NaN,榆林市
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03,机动车交通事故责任强制保险服务,7.002010e+09,交强险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,18,16,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市,NaN,榆林市
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1.030113e+18,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03,其他淀粉及淀粉制品,1.312990e+09,其他淀粉及淀粉制品,陕西省,榆林市,清涧县,NaN,NaN,NaN,18,17,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,3.060302e+18,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03,其他人身保险服务,7.001000e+03,人身保险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,18,16,NaN,61,NaN,6108,NaN,610800,NaN,陕西省,NaN,榆林市,NaN,榆林市


In [59]:
# 发票数据处理：
# 增加地址信息
# xf_province,xf_city,xf_district
# gf_province,gf_city,gf_district
df_merge_fp_1=df_merge_fp.copy()
df_merge_fp.loc[df_merge_fp['xf_sjswjg'].notnull(),'xf_province']=df_merge_fp.loc[df_merge_fp['xf_sjswjg'].notnull()]['xf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['xf_dsswjg'].notnull(),'xf_city']=df_merge_fp.loc[df_merge_fp['xf_dsswjg'].notnull()]['xf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['xf_qxswjg'].notnull(),'xf_district']=df_merge_fp.loc[df_merge_fp['xf_qxswjg'].notnull()]['xf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()
df_merge_fp.loc[df_merge_fp['gf_sjswjg'].notnull(),'gf_province']=df_merge_fp.loc[df_merge_fp['gf_sjswjg'].notnull()]['gf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['gf_dsswjg'].notnull(),'gf_city']=df_merge_fp.loc[df_merge_fp['gf_dsswjg'].notnull()]['gf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['gf_qxswjg'].notnull(),'gf_district']=df_merge_fp.loc[df_merge_fp['gf_qxswjg'].notnull()]['gf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()

In [60]:
df_merge_fp['gf_shxydm'].fillna('0',inplace=True)
df_merge_fp['gf_nsrmc'].fillna('0',inplace=True)
ls_len_xf = [len(x) for x in df_merge_fp['xf_shxydm'].tolist()]
ls_len_gf = [len(x) for x in df_merge_fp['gf_shxydm'].tolist()]
ls_len_gf_name=[len(x) for x in df_merge_fp['gf_nsrmc'].tolist()]

In [61]:
df_merge_fp.loc[:,'length_xf_shxydm'] = ls_len_xf
df_merge_fp.loc[:,'length_gf_shxydm'] = ls_len_gf
df_merge_fp.loc[:,'length_gf_nsrmc'] = ls_len_gf_name

In [62]:
# province
df_merge_fp.loc[(df_merge_fp['xf_province'].isnull())&(df_merge_fp['length_xf_shxydm']==18),'xf_province_code']=df_merge_fp['xf_shxydm'].str[2:4]
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())&(df_merge_fp['length_gf_shxydm']==18),'gf_province_code']=df_merge_fp['gf_shxydm'].str[2:4]
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())&(df_merge_fp['length_gf_nsrmc']==18),'gf_province_code']=df_merge_fp['gf_shxydm'].str[2:4]
# city
df_merge_fp.loc[(df_merge_fp['xf_province'].isnull())&(df_merge_fp['length_xf_shxydm']==18),'xf_city_code']=df_merge_fp['xf_shxydm'].str[2:6]
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())&(df_merge_fp['length_gf_shxydm']==18),'gf_city_code']=df_merge_fp['gf_shxydm'].str[2:6]
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())&(df_merge_fp['length_gf_nsrmc']==18),'gf_city_code']=df_merge_fp['gf_shxydm'].str[2:6]
# district
df_merge_fp.loc[(df_merge_fp['xf_province'].isnull())&(df_merge_fp['length_xf_shxydm']==18),'xf_district_code']=df_merge_fp['xf_shxydm'].str[2:8]
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())&(df_merge_fp['length_gf_shxydm']==18),'gf_district_code']=df_merge_fp['gf_shxydm'].str[2:8]
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())&(df_merge_fp['length_gf_nsrmc']==18),'gf_district_code']=df_merge_fp['gf_shxydm'].str[2:8]

In [71]:
df_district

,type_code,admin_name
0,100000,中国
1,110000,北京市
2,110100,北京市
3,110101,东城区
4,110102,西城区
...,...,...
4216,340221,芜湖县
4217,340222,繁昌县
4218,320602,崇川区
4219,320684,海门市


In [68]:
df_merge_fp=pd.merge(left=df_merge_fp,right=df_province,left_on=['xf_province_code'],right_on=['type_2'],how='left')
df_merge_fp.rename(columns={'admin_name':'xf_province_name'},inplace=True)
df_merge_fp.drop(columns=['type_2'],inplace=True)
df_merge_fp=pd.merge(left=df_merge_fp,right=df_province,left_on=['gf_province_code'],right_on=['type_2'],how='left')
df_merge_fp.rename(columns={'admin_name':'gf_province_name'},inplace=True)
df_merge_fp.drop(columns=['type_2'],inplace=True)

In [70]:
df_merge_fp=pd.merge(left=df_merge_fp,right=df_city,left_on=['xf_city_code'],right_on=['type_4'],how='left')
df_merge_fp.rename(columns={'admin_name':'xf_city_name'},inplace=True)
df_merge_fp.drop(columns=['type_4'],inplace=True)
df_merge_fp=pd.merge(left=df_merge_fp,right=df_city,left_on=['gf_city_code'],right_on=['type_4'],how='left')
df_merge_fp.rename(columns={'admin_name':'gf_city_name'},inplace=True)
df_merge_fp.drop(columns=['type_4'],inplace=True)

In [72]:
df_merge_fp=pd.merge(left=df_merge_fp,right=df_district,left_on=['xf_district_code'],right_on=['type_code'],how='left')
df_merge_fp.rename(columns={'admin_name':'xf_district_name'},inplace=True)
df_merge_fp.drop(columns=['type_code'],inplace=True)
df_merge_fp=pd.merge(left=df_merge_fp,right=df_district,left_on=['gf_district_code'],right_on=['type_code'],how='left')
df_merge_fp.rename(columns={'admin_name':'gf_district_name'},inplace=True)
df_merge_fp.drop(columns=['type_code'],inplace=True)

In [78]:
df_merge_fp.loc[df_merge_fp['xf_province'].isnull(),'xf_province']=df_merge_fp['xf_province_name']
df_merge_fp.loc[df_merge_fp['gf_province'].isnull(),'gf_province']=df_merge_fp['gf_province_name']

df_merge_fp.loc[df_merge_fp['xf_city'].isnull(),'xf_city']=df_merge_fp['xf_city_name']
df_merge_fp.loc[df_merge_fp['gf_city'].isnull(),'gf_city']=df_merge_fp['gf_city_name']

df_merge_fp.loc[df_merge_fp['xf_district'].isnull(),'xf_district']=df_merge_fp['xf_district_name']
df_merge_fp.loc[df_merge_fp['gf_district'].isnull(),'gf_district']=df_merge_fp['gf_district_name']

In [83]:
# 发票购销双方的数据通过信用代码补充后，购方销方的空间数据缺失从80w减少到40w
df_merge_fp.loc[(df_merge_fp['gf_province'].isnull())|(df_merge_fp['xf_province'].isnull())]

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm,length_gf_shxydm,length_gf_nsrmc,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_province_name,gf_province_name,xf_city_name,gf_city_name,xf_district_name,gf_district_name,xf_ind_type,gf_ind_type
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1.100102e+18,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33,三北热力,4.402010e+09,热力,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33,机动车交通事故责任强制保险服务,7.002010e+09,交强险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,15,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J,NaN
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1.100102e+18,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33,三北热力,4.402010e+09,热力,陕西省,榆林市,神木市,NaN,NaN,NaN,18,15,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1.100202e+18,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33,民用天然气,7.020000e+02,天然气,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3.060303e+18,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33,其他财产保险服务,7.002990e+09,其他财产保险服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773193,6338903,81,9161080068798448X4,榆林市新家源商贸有限责任公司,F5211,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局上郡路税务分局,81610800MC8332839,榆林华源电力有限责任公司工会委员会,NaN,NaN,NaN,NaN,*水果*橘子,1.010115e+18,762.39,68.61,59.44,公斤,2023-01,2023-04-04 08:23:03,苹果,1.160101e+06,苹果,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18,17,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN
1773205,6338915,81,91610828338640777Q,佳县兴茂农产品开发有限公司,A0313,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局佳州税务分局,81610828MC597544D,佳县招商服务中心工会委员会,NaN,NaN,NaN,NaN,*干豆*绿豆,1.010104e+18,700.00,0.00,10.00,箱,2023-01,2023-04-04 08:23:03,绿豆,1.040200e+04,绿豆,陕西省,榆林市,佳县,NaN,NaN,NaN,18,17,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN
1773282,6338992,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*谷物加工品*小米,1.030102e+18,6598.17,593.83,116.00,箱,2023-01,2023-04-04 08:23:03,小米,1.302012e+09,小米,陕西省,榆林市,清涧县,NaN,NaN,NaN,18,17,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1.030113e+18,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03,其他淀粉及淀粉制品,1.312990e+09,其他淀粉及淀粉制品,陕西省,榆林市,清涧县,NaN,NaN,NaN,18,17,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN


In [82]:
# 行业门类
df_merge_fp.loc[:,'xf_ind_type']=df_merge_fp['xf_hydm'].str[0]
df_merge_fp.loc[:,'gf_ind_type']=df_merge_fp['gf_hydm'].str[0]

In [140]:
# 计算每条产业链的数据：销售份额，
# 行业筛选['A','B','C','D','E','H','I','J','L','M','N','O','P','Q','R','S','T']
ls_ind=['A','B','C','D','E','H','I','J','L','M','N','O','P','Q','R','S','T']
deselect_pro=['原煤','液化石油气','柴油','燃料油']
count=1
ls_chain_je=[]
ls_chain_ent_num=[]
for i in ls_chain_name:
    print('number',count,'chain',i)
    ls_i_nodes_ori=re.findall('"text":"(.*?)"',df_chain_name_10.loc[df_chain_name_10['NAME']==i,'GRAPH'].values[0])
    ls_i_nodes_set=list(set(ls_i_nodes_ori))
    print('nodes length',len(ls_i_nodes_set))
    df_chain_fp=df_merge_fp.loc[(df_merge_fp['spmc'].isin(ls_i_nodes_set))|(df_merge_fp['tupu_name'].isin(ls_i_nodes_set))]
    print('fp length',len(df_chain_fp))
    count+=1
    
    # 筛选出本地非购销企业的销售发票数据
    df_chain_fp_filt=df_chain_fp.loc[(df_chain_fp['xf_city']=='榆林市')&(df_chain_fp['xf_ind_type'].isin(ls_ind))]
    
    # 产业链销售总额(亿元),税额（万元）
    sum_chain_sell=df_chain_fp_filt['je'].sum()
    sum_chain_sell_se=df_chain_fp_filt['se'].sum()
    ls_i_chain_je_se=[i,sum_chain_sell/100000000,sum_chain_sell_se/10000]
    ls_chain_je.append(ls_i_chain_je_se)
    
    # 本地产业链市场份额省市分布
    df_province=df_chain_fp_filt.groupby(['gf_province'])['je'].sum().reset_index()
    df_province.rename(columns={'je':'province_je'},inplace=True)
    df_city=df_chain_fp_filt.groupby(['gf_province','gf_city'])['je'].sum().reset_index()
    df_city.rename(columns={'je':'city_je'},inplace=True)
    df_area_market=pd.merge(left=df_city,right=df_province,on=['gf_province'],how='left')
    df_area_market.loc[:,'chain_name']=i
    df_area_market.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_市场份额.csv',index=False,mode='w')
    
    # 产业链企业数量
    chain_ent_num=len(df_chain_fp_filt['xf_nsrmc'].unique().tolist())
    ls_chain_ent_num.append([i,chain_ent_num])
    
    # 产业链企业区县分布
    df_district_ent_num=df_chain_fp_filt.groupby(['xf_district'])['xf_nsrmc'].count().reset_index()
    df_district_ent_num.loc[:,'chain_name']=i
    df_district_ent_num.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_企业区县分布.csv',index=False,mode='w')
    
    # 前三主要产品销售额
    df_top_pro=df_chain_fp_filt.groupby(['tupu_name'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    df_top_3=df_top_pro.loc[~df_top_pro['tupu_name'].isin(deselect_pro)].head(3)
    df_top_3.loc[:,'chain_name']=i
    df_top_3.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_前三产品销售.csv',index=False,mode='w')
    
    # 前十非煤油气生产企业，非B类企业
    df_chain_fp_filt_new=df_chain_fp.loc[(df_chain_fp['xf_city']=='榆林市')&(~df_chain_fp['xf_ind_type'].isin(['B','F','G']))&(~df_chain_fp_filt['tupu_name'].isin(deselect_pro))]
    # df_chain_fp_filt_new=df_chain_fp_filt.loc[~df_chain_fp_filt['tupu_name'].isin(deselect_pro)]
    df_produce_ent=df_chain_fp_filt_new.groupby(['xf_nsrmc'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    df_produce_ent_top_10=df_produce_ent.head(10)
    df_produce_ent_top_10.loc[:,'chain_name']=i
    df_produce_ent_top_10.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_前十非煤油气生产企业.csv',index=False,mode='w')
    
    # 生产企业分布
    df_chain_pro_ent_num=df_chain_fp_filt_new.groupby(['xf_district'])['xf_nsrmc'].count().reset_index()
    df_chain_pro_ent_num.loc[:,'chain_name']=i
    df_chain_pro_ent_num.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_生产企业区县分布.csv',index=False,mode='w')
    
    # 前十经销企业
    df_chain_fp_filt_new=df_chain_fp.loc[(df_chain_fp['xf_city']=='榆林市')&(df_chain_fp['xf_ind_type'].isin(['F','G']))]
    df_sell_ent=df_chain_fp_filt_new.groupby(['xf_nsrmc'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    df_sell_ent_top_10=df_sell_ent.head(10)
    df_sell_ent_top_10.loc[:,'chain_name']=i
    df_sell_ent_top_10.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_前十经销企业.csv',index=False,mode='w')
    
    # 产品晴雨表
    # 销售
    # 产业链交易情况
    df_top_pro.head(15).to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_销售_产业链交易情况.csv',index=False,mode='w')
    
    # 产品交易情况
    # 排名前五销售目的地，刨除经销环节
    ls_sell_3=df_top_3['tupu_name'].unique().tolist()
    df_sell_3=pd.DataFrame()
    for a in ls_sell_3:
        df_a=df_chain_fp_filt.loc[df_chain_fp_filt['tupu_name']==a]
        df_a_je=df_a.groupby(['gf_city'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
        df_a_sl=df_a.groupby(['gf_city'])['sl'].sum().reset_index()
        df_a_result=pd.merge(left=df_a_je,right=df_a_sl,on=['gf_city'],how='left')
        # 交易金额变化
        df_a_result.loc[:,'price']=df_a_result['je']/df_a_result['sl']
        df_a_result.loc[:,'product']=a
        df_sell_3=pd.concat([df_sell_3,df_a_result.head(5)],axis=0)
        
    df_sell_3.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_销售_产品交易情况.csv',index=False,mode='w')
        
    # 购买
    df_buy_fp=df_chain_fp.loc[(df_chain_fp['gf_city']=='榆林市')]
    # 产业链交易情况
    df_top_buy_pro=df_buy_fp.groupby(['tupu_name'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
    df_top_buy_pro.head(15).to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_购买_产业链交易情况.csv',index=False,mode='w')
    
    # 产品交易情况
    # 排名前五销售目的地，刨除经销环节
    df_buy_pro_3=df_top_buy_pro.head(3)
    ls_buy_3=df_buy_pro_3['tupu_name'].unique().tolist()
    df_buy_3=pd.DataFrame()
    for a in ls_buy_3:
        df_a=df_buy_fp.loc[df_buy_fp['tupu_name']==a]
        df_a_je=df_a.groupby(['xf_city'])['je'].sum().reset_index().sort_values(by='je',ascending=False)
        df_a_sl=df_a.groupby(['xf_city'])['sl'].sum().reset_index()
        df_a_result=pd.merge(left=df_a_je,right=df_a_sl,on=['xf_city'],how='left')
        # 交易金额变化
        df_a_result.loc[:,'price']=df_a_result['je']/df_a_result['sl']
        df_a_result.loc[:,'product']=a
        df_buy_3=pd.concat([df_buy_3,df_a_result.head(5)],axis=0)
        
    df_buy_3.to_csv('E:\\project_data\\04yulin\\frontpage\\'+i+'_购买_产品交易情况.csv',index=False,mode='w')
    
df_chain_je=pd.DataFrame(ls_chain_je,columns=['chain_name','je亿元','se万元'])
df_chain_je.to_csv('E:\\project_data\\04yulin\\frontpage\\销售金额及税额.csv',index=False,mode='w')

df_chain_ent_num=pd.DataFrame(ls_chain_ent_num,columns=['chain_name','企业数量'])
df_chain_ent_num.to_csv('E:\\project_data\\04yulin\\frontpage\\企业数量.csv',index=False,mode='w')

number 1 chain “碳中和”产业链
nodes length 106
fp length 23513
number 2 chain 现代特色农业产业链
nodes length 50


C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_produce_ent_top_10.loc[:,'chain_name']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sell_ent_top_10.loc[:,'chain_name']=i


fp length 57288
number 3 chain 氢能产业链
nodes length 39


C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_produce_ent_top_10.loc[:,'chain_name']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sell_ent_top_10.loc[:,'chain_name']=i


fp length 10206
number 4 chain 能源化工装备制造产业链 
nodes length 37
fp length 15520


...

C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_produce_ent_top_10.loc[:,'chain_name']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sell_ent_top_10.loc[:,'chain_name']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

number 9 chain 铝镁及兰炭深加工产业链
nodes length 66
fp length 28695


C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_produce_ent_top_10.loc[:,'chain_name']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sell_ent_top_10.loc[:,'chain_name']=i


number 10 chain 现代煤化工产业链
nodes length 78
fp length 9577


C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_produce_ent_top_10.loc[:,'chain_name']=i
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20304\3075012505.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sell_ent_top_10.loc[:,'chain_name']=i


In [17]:
# 税务数据对应关系整理
df_tax=pd.read_excel(r'E:\project_项目\04榆林产业大脑\平台修改_3\税务产品需求2023.xlsx',sheet_name='Sheet1',dtype='str')
df_chain=pd.read_excel(r'E:\project_data\04yulin\fp\cpdm_connect_1007.xlsx',sheet_name='result',dtype='str')

In [18]:
df_tax

,fp_code,fp_name,tupu_code,tupu_name
0,1070101070200000000,润滑脂,261306,油脂类高分子聚合物
1,1070204070000000000,复合肥、复混合肥,2605010200,有机-无机复混肥料
2,1090202010000000000,石油钻井设备,36020902,井下工具
3,1100202030000000000,液化天然气（LNG）,703000000,液化天然气
4,1090409040000000000,专用电缆,4702060102,高压电力输送线设施
...,...,...,...,...
779,1070102020700000000,生物炭,2620019900,其他动物炭黑
780,1070202080000000000,烃磺化、硝化或亚硝化衍生物,260208,烃磺化、硝化或亚硝化衍生物
781,1070102020800000000,变性燃料乙醇,150102,改性乙醇
782,1070202390000000000,正戊醇,2602090108,戊醇


In [19]:
ls_fp=df_tax['fp_code'].tolist()
ls_result_chain_nodes=[]
ls_result_chain=[]
for i in ls_fp:
    df_i=df_chain.loc[df_chain['fp_code']==i]
    print(len(df_i))
    ls_chain_name=df_i['chain_name'].unique().tolist()
    ls_chain=df_i['chain'].unique().tolist()
    ls_result_chain_nodes.append(';'.join(ls_chain_name))
    ls_result_chain.append(';'.join(ls_chain))

1
1
1
1
1
3
4
5
3
2
4
1
1
1
3
11
1
1
2
1
1
4
1
2
2
2
1
4
1
3
1
1
1
1
1
3
2
1
1
4
1
14
1
1
1
1
1
1
1
4
2
3
2
6
1
1
5
1
2
1
1
3
2
1
2
2
2
1
5
2
9
2
2
4
4
2
2
1
7
1
2
3
4
1
2
4
1
1
1
1
1
1
1
1
1
1
2
1
1
0
3
2
1
2
2
1
1
2
1
5
2
1
1
2
1
3
3
4
1
1
1
4
2
1
1
1
1
3
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
4
1
1
1
6
1
2
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
3
2
1
1
1
4
1
1
2
2
1
1
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
2
1
1
1
1
2
1
1
2
1
3
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
4
2
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
2
4
1
1
1
1
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [20]:
len(ls_result_chain_nodes)

784

In [21]:
df_tax.loc[:,'chain_node']=ls_result_chain_nodes
df_tax.loc[:,'chain']=ls_result_chain
df_tax.to_csv(r'E:\project_项目\04榆林产业大脑\平台修改_3\税务产品需求2023connect.csv',index=False)

In [13]:
ls_result_chain_nodes

['高端润滑油',
 '肥料',
 '井下工具',
 '液化天然气',
 '高压电力输送线设施',
 '1，4-丁二醇;乙二醇;精甲醇',
 'ABS树脂;丁腈橡胶;丁苯橡胶;聚氯乙烯树脂',
 'Liner Fresnel;吸热器;吸热部件;热交换装置;蒸发器',
 'N-甲基吡咯烷酮;环己酮;苯酚丙酮',
 'PET;聚对苯二甲酸乙二酯塑料板、片',
 'TDI;γ-丁内酯;二苯基甲烷二异氰酸酯（纯MDI）;己二酸己二胺盐（尼龙66盐）',
 'UPS电源',
 'α-烯烃',
 '丁二烯',
 '专业工程服务;工程环保设施服务;输气管道工程服务',
 '专业技术服务;二氧化碳驱油;塔式太阳能热发电系统技术服务;槽式、塔式、蝶式太阳能热发电系统技术服务;槽式太阳能热发电系统技术服务;煤矿有害气体监测服务;独杆拉索式;碟式太阳能热发电系统技术服务;碳中和;碳利用;食品检验服务',
 '专用材料',
 '专用设备',
 '专项化学用品;精细化学品',
 '丙烯',
 '丙烯腈',
 '丙烯酸;丙烯酸异辛酯;丙烯酸酯;己二酸',
 '中成药',
 '中草药材;黄芪',
 '主题公园管理服务;旅游景区管理服务',
 '乙烯;醋酸乙烯',
 '乙烯-乙酸乙烯（EVA）防水卷材',
 '乙烯聚合物;聚乙烯醇;聚乳酸;聚酰胺',
 '二次加工材，相关板材',
 '二次铝灰氟氮耦合脱除生产环保净水剂;水处理剂;软水、净水剂',
 '二氧化碳',
 '互联网信息服务',
 '井下填充',
 '产业用纺织品',
 '产品设备安装服务',
 '人工鱼礁;厂房、设备拆除服务;护岸护滩工程设施',
 '人造毛皮;己内酰胺',
 '企业投资服务',
 '传动装置',
 '传感器;编码器;风向标;风能监测设备',
 '传感器和易耗损部件生产',
 '低空旅游线路;旅游产品;旅游景区景点;旅行社服务;榆林古城;榆林特色旅游;沿长城\\\\\\\\黄河\\\\\\\\无定河;波浪谷;清平堡;石峁;转战陕北;镇北台;长城国家文化公园;黄河国家文化公园',
 '体育运动',
 '修理与维护服务',
 '偏航制动器',
 '偏航液压回路',
 '偏航计数器',
 '偏航轴承',
 '偏航驱动装置',
 '储氢材料储氢;储能;单一稀土金属;稀土金属合金',
 '催化剂;废催化剂',
 '光热工程技术服务

In [4]:
df_chain

,chain_pro,fp_name,fp_rate,tp_name,tp_rate,base_name,base_rate,tupu_name,chain,Unnamed: 9,fp_code,fp_name.1,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,化学（合成下游产品）,林产品;矿产品;化学合成材料;农业产品;园艺产品,67;67;67;50;50,林产品;化学矿;农业产品;园艺产品;林业产品,67;67;50;50;50,化学矿;合成炉;林产品;农业产品;园艺产品,67;67;67;50;50,化学（合成下游产品）,“碳中和”产业链,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,生物（用于促进植物生长）,长石;生物炭;货物;谷物;花生,67;67;50;50;50,长石;谷物;花生;生麻;生奶,67;50;50;50;50,植物生长仪;长石;长吻?;长裙;生物植物生长调节剂制剂,89;67;67;67;55,生物（用于促进植物生长）,“碳中和”产业链,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1，4-丁二醇,丁二烯;环醇;醚醇;醛醇;乘用车（排气量在1.0升以下（含））,67;50;50;50;43,丁二烯;环醇;醚醇;醛醇;戊醇,67;50;50;50;50,丁二醇;丁炔二醇（2-丁炔-1，4-二醇）;二元醇;1，2-丙二醇;新戊二醇（2，2-二甲基...,100;92;80;71;71,1，4-丁二醇,现代煤化工产业链,NaN,1070202090000000000,无环醇及其衍生物,260209,无环醇及其衍生物,NaN,NaN,NaN,NaN,NaN,NaN
3,ABS树脂,天然树脂、树胶;松脂岩;热固性树脂装饰层压板;聚对苯二甲酸乙二醇酯、PET、树脂;离子交换树脂,40;40;40;40;40,天然树脂、树胶;离子交换树脂;润滑脂;Spa水疗机;电力线宽带接入设备（BPL）,40;40;33;22;22,ABS树脂;树脂酸;AS树脂;防抱死制动系统（ABS）;树脂胶泥,100;80;75;75;67,ABS树脂,现代煤化工产业链,NaN,1070213019900000000,其他初级形态塑料,2651060,ABS树脂,NaN,NaN,NaN,NaN,NaN,NaN
4,Liner Fresnel,延安（金）;中华（硬）;红河（道）;中华（软）;牡丹（软）,25;25;25;25;25,衡器（秤）;液化天然气（LNG）供应量;粘土、砂石;钻石（矿类）;鲜、冷藏肉,25;23;20;20;20,还原蓝RSN;远端设备（RE）;LCR电桥;核糖核酸（RNA）;IC卡,50;43;40;38;33,Liner Fresnel,新能源装备制造产业链,NaN,1090129050000000000,热交换装置,353005,热交换装置,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,黄芪,红梅（软黄）;红梅（硬黄）;黄玉;黄酒;黄红麻织物,67;67;50;50;50,黄芪类饮片;黄酒;黄红麻织物;黄金;农业产品,100;50;50;50;0,黄芪类饮片;黄芪;吖啶黄;盐酸吖啶黄;荧光黄,100;100;67;67;67,黄芪,现代特色农业产业链,NaN,1010118000000000000,中草药材,119160000,黄芪,119160000,黄芪,NaN,NaN,NaN,NaN
579,黄芪类饮片,片烟;片基;片剂;植物类饮片;动物类饮片,67;67;67;60;60,黄芪类饮片;片烟;片基;片剂;动物类饮片,100;67;67;67;60,黄芪类饮片;黄芪;蒲黄类饮片;净天竺黄类饮片;大黄类饮片,100;100;89;89;89,黄芪类饮片,现代特色农业产业链,NaN,1070303010000000000,植物类饮片,2703010118,黄芪类饮片,2703010118,黄芪类饮片,1070303010000000000,植物类饮片,1070303010000000000,植物类饮片
580,齿轮,齿轮加工机床;齿轮、传动轴和驱动部件;齿轮传动轴;齿轮;齿轮传动装置（齿轮箱）,100;100;100;100;100,齿轮加工机床;齿轮、传动轴和驱动部件;齿轮传动轴;齿轮;齿轮、传动和驱动部件零件,100;100;100;100;100,其他齿轮传动轴;圆柱齿轮;粉末冶金圆柱齿轮;塑料圆柱齿轮;其他圆柱齿轮,100;100;100;100;100,齿轮,新能源装备制造产业链,NaN,1090124020000000000,齿轮,3453720,齿轮,3453720,齿轮,1090124020000000000,齿轮,1090124020000000000,齿轮
581,齿轮、传动和驱动部件零件,齿轮;齿轮、传动和驱动部件零件;齿轮、传动轴和驱动部件;齿轮传动轴;电动机零件,100;100;91;80;60,齿轮;齿轮、传动和驱动部件零件;齿轮、传动轴和驱动部件;齿轮传动轴;电动机零件,100;100;91;80;60,齿轮、传动和驱动部件零件;齿轮;齿轮、传动轴和驱动部件;齿轮传动轴;锥齿轮,100;100;91;80;67,齿轮、传动和驱动部件零件,新能源装备制造产业链,NaN,1090124080000000000,齿轮、传动和驱动部件零件,3525080000,齿轮、传动和驱动部件零件,3525080000,齿轮、传动和驱动部件零件,1090124080000000000,齿轮、传动和驱动部件零件,1090124080000000000,齿轮、传动和驱动部件零件


In [9]:
# 税务数据需求整理
# 先和产业链数据对应
ls_chain_name=df_chain_name_10['NAME'].unique().tolist()
count=1
ls_chain_nodes=[]
df_chain_nodes=pd.DataFrame()
for i in ls_chain_name:
    # print('number',count,'chain',i)
    ls_i_nodes_ori=re.findall('"text":"(.*?)"',df_chain_name_10.loc[df_chain_name_10['NAME']==i,'GRAPH'].values[0])
    ls_i_nodes_set=list(set(ls_i_nodes_ori))
    # print(len(ls_i_nodes_set))
    count+=1
    df_i=pd.DataFrame(ls_i_nodes_set,columns=['tupu_name'])
    df_i.loc[:,'chain']=i
    df_chain_nodes=pd.concat([df_chain_nodes,df_i],axis=0)
    ls_chain_nodes.extend(ls_i_nodes_set)
ls_chain_nodes=list(set(ls_chain_nodes))
# pd.DataFrame(ls_chain_nodes,columns=['tupu_name'])

In [73]:
# 读取全量发票产品名录
df_con=pd.read_excel(r'E:\project_data\04yulin\fp\全量税务产品与国标产品对应关系_0515_total.xlsx',dtype='str')
# df_con_new=df_con.drop_duplicates(['spbm','spmc']).loc[:,['spbm','spmc']]
df_fp_pro=pd.read_excel(r'E:\project_data\04yulin\fp\t_data_sh_new_cpdm_202309261607.xlsx',dtype='str')
df_fp_pro_new=df_fp_pro.drop_duplicates(['spbm','spmc']).loc[:,['spbm','spmc']]
ls_fp_pro_new=df_fp_pro_new['spmc'].unique().tolist()
ls_con=df_con.loc[df_con['tupu_code']!='0','tupu_name'].unique().tolist()

# 读取全量产品基底数据
df_pro=pd.read_csv(r'E:\project_data\04yulin\connection\prediction\data\T_CODE_PRODUCT_202309271100.csv',dtype='str')
ls_pro=df_pro['NAME'].unique().tolist()

In [74]:
# 35084 35051
# len(df_pro['NAME'].unique().tolist())
len(df_pro.loc[df_pro['NAME'].duplicated()])
len(ls_pro)

35051

In [8]:
df=pd.read_excel(r'E:\project_data\04yulin\fp\cpdm_connect_1007.xlsx',sheet_name='result')
df_fp=pd.read_excel(r'E:\project_basedata\2023090801产业生长图谱\202301fpspbm.xlsx',sheet_name='202301fpspbm')
df_fp

,spbm,spmc,Unnamed: 2
0,1.100102e+18,三北热力,NaN
1,3.060302e+18,机动车交通事故责任强制保险服务,NaN
2,1.100202e+18,民用天然气,NaN
3,3.060303e+18,其他财产保险服务,NaN
4,3.030000e+18,电信服务,电信服务
...,...,...,...
2759,1.020302e+18,人造富锰矿,NaN
2760,1.040307e+18,加工填充用羽绒,NaN
2761,1.070216e+18,木质素,木质素
2762,1.020511e+18,其他玉石,NaN


In [13]:
# df=df_chain_nodes.groupby(['tupu_name'])['chain'].apply(lambda x:';'.join(x)).reset_index()
ls_tupu_name=df['chain_name'].unique().tolist()
ls_pro=df_fp['spmc'].unique().tolist()
ls_pro=[x for x in ls_pro if type(x)!=float]

ls_result=[]
for i in ls_tupu_name:
    # result_i_1 = process.extractBests(i,ls_fp_pro_new,scorer=fuzz.partial_ratio)
    # result_i_2 = process.extractBests(i,ls_con,scorer=fuzz.partial_ratio)
    result_i_3 = process.extractBests(i,ls_pro,scorer=fuzz.partial_ratio)
    
    # ls_i_1_1 = [x[0] for x in result_i_1]
    # ls_i_1_2 = [str(x[1]) for x in result_i_1]
    # ls_i_2_1 = [x[0] for x in result_i_2]
    # ls_i_2_2 = [str(x[1]) for x in result_i_2]
    ls_i_3_1 = [x[0] for x in result_i_3]
    ls_i_3_2 = [str(x[1]) for x in result_i_3]
    
    # str_1 = ';'.join(ls_i_1_1)
    # str_2 = ';'.join(ls_i_1_2)
    # str_3 = ';'.join(ls_i_2_1)
    # str_4 = ';'.join(ls_i_2_2)
    str_5 = ';'.join(ls_i_3_1)
    str_6 = ';'.join(ls_i_3_2)
    # ,str_1,str_2,str_3,str_4
    ls_i = [i,str_5,str_6]
    ls_result.append(ls_i)

In [15]:
df_connect=pd.DataFrame(ls_result,columns=['chain_pro','fp_name','fp_rate'])
df_connect.to_csv(r'E:\project_data\04yulin\fp\cpdm_connect_fp.csv',index=False,mode='w')

In [80]:
df_connect=pd.DataFrame(ls_result,columns=['chain_pro','fp_name','fp_rate'])
df_1=pd.merge(left=df_connect,right=df,left_on=['chain_pro'],right_on=['tupu_name'],how='inner')
df_1.to_csv(r'E:\project_data\04yulin\fp\cpdm_connect.csv',index=False,mode='w')
# pd.merge(left=df_1,right=df_fp_pro_new,left_on=[''])

In [79]:
df

,tupu_name,chain
0,化学（合成下游产品）,“碳中和”产业链
1,生物（用于促进植物生长）,“碳中和”产业链
2,1，4-丁二醇,现代煤化工产业链
3,ABS树脂,现代煤化工产业链
4,Liner Fresnel,新能源装备制造产业链
...,...,...
578,黄芪,现代特色农业产业链
579,黄芪类饮片,现代特色农业产业链
580,齿轮,新能源装备制造产业链
581,齿轮、传动和驱动部件零件,新能源装备制造产业链


In [54]:
df1=pd.merge(left=df_fp_pro_new,right=df_con_new,on=['spmc'],how='outer')
df1.loc[df1['spbm_y'].duplicated()]

,spbm_x,spmc,spbm_y
285,1020401070000000000,铝土矿,1020401070000000000
355,1020499020000000000,铍矿石,1020499020000000000
357,1020499030000000000,铯矿,1020499030000000000
359,1020499040000000000,钽矿,1020499040000000000
361,1020499050000000000,铌矿石,1020499050000000000
...,...,...,...
1002,1030402050101010048,红塔山（硬经典）,1030402050101010048
1003,1030402050101010048,乙类卷烟预留3,1030402050101010048
1004,1030402050101010048,乙类卷烟预留4,1030402050101010048
4259,3010101020101009920,国内长途汽车旅客运输服务,3010101020101000192


In [38]:
df_con.drop_duplicates(['spbm','spmc'])

,id,spbm,spmc,spbmjc,hzx,update_time,tupu_code,tupu_name,特殊,Unnamed: 9,Unnamed: 10
0,23551,1000000000000000000,货物,货物,Y,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
1,23322,1010000000000000000,农、林、牧、渔业类产品,农产品,Y,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
2,23267,1010100000000000000,农业产品,农业产品,Y,2023-03-13 20:34:00,1,农业产品,NaN,NaN,True
3,23024,1010101000000000000,谷物,谷物,Y,2023-03-13 20:34:00,101,谷物,NaN,NaN,True
4,21554,1010101010000000000,稻谷,谷物,N,2023-03-13 20:34:00,1310030,大米,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4730,19657,6120000000000000000,建筑服务预收款,建筑服务预收款,N,2023-03-13 20:33:00,0,0,NaN,NaN,NaN
4731,21865,6130000000000000000,代收民航发展基金,代收民航发展基金,N,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
4732,19620,6140000000000000000,拍卖行受托拍卖文物艺术品代收货款,受托拍卖文物艺术品代收货款,N,2023-03-13 20:33:00,6330020300,文物拍卖服务,NaN,NaN,NaN
4733,22155,6150000000000000000,与销售行为不挂钩的财政补贴收入,与销售行为不挂钩的财政补贴收入,N,2023-03-13 20:34:00,0,0,NaN,NaN,NaN


In [36]:
pd.merge(left=df_fp_pro,right=df_con,on=['spbm'],how='outer')

,id_x,spbm,spmc_x,spbmjc_x,hzx_x,update_time_x,select,id_y,spmc_y,spbmjc_y,hzx_y,update_time_y,tupu_code,tupu_name,特殊,Unnamed: 9,Unnamed: 10
0,23551,1000000000000000000,货物,货物,Y,2023-03-13 20:34:31,NaN,23551,货物,货物,Y,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
1,23322,1010000000000000000,农、林、牧、渔业类产品,农产品,Y,2023-03-13 20:34:31,NaN,23322,农、林、牧、渔业类产品,农产品,Y,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
2,23267,1010100000000000000,农业产品,农业产品,Y,2023-03-13 20:34:31,NaN,23267,农业产品,农业产品,Y,2023-03-13 20:34:00,1,农业产品,NaN,NaN,True
3,23024,1010101000000000000,谷物,谷物,Y,2023-03-13 20:34:31,NaN,23024,谷物,谷物,Y,2023-03-13 20:34:00,101,谷物,NaN,NaN,True
4,21554,1010101010000000000,稻谷,谷物,N,2023-03-13 20:34:10,NaN,21554,稻谷,谷物,N,2023-03-13 20:34:00,1310030,大米,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32258,NaN,3040502010099990016,NaN,NaN,NaN,NaN,NaN,20398,光租业务,经营租赁,N,2023-03-13 20:34:00,7399,其他租赁服务,NaN,NaN,NaN
32259,NaN,3040502020099990016,NaN,NaN,NaN,NaN,NaN,22983,住宅经营租赁服务,经营租赁,Y,2023-03-13 20:34:00,73,租赁服务,NaN,NaN,NaN
32260,NaN,3040703030099990016,NaN,NaN,NaN,NaN,NaN,20018,现场电影放映服务,广播影视服务,N,2023-03-13 20:34:00,8904,电影放映服务,NaN,NaN,NaN
32261,NaN,3040802010099990016,NaN,NaN,NaN,NaN,NaN,22354,国内货物运输代理服务,经纪代理服务,N,2023-03-13 20:34:00,5702,运输代理服务,NaN,NaN,NaN


In [34]:
df_fp_pro

,id,spbm,spmc,spbmjc,hzx,update_time,select
0,23551,1000000000000000000,货物,货物,Y,2023-03-13 20:34:31,NaN
1,23322,1010000000000000000,农、林、牧、渔业类产品,农产品,Y,2023-03-13 20:34:31,NaN
2,23267,1010100000000000000,农业产品,农业产品,Y,2023-03-13 20:34:31,NaN
3,23024,1010101000000000000,谷物,谷物,Y,2023-03-13 20:34:31,NaN
4,21554,1010101010000000000,稻谷,谷物,N,2023-03-13 20:34:10,NaN
...,...,...,...,...,...,...,...
4730,19657,6120000000000000000,建筑服务预收款,建筑服务预收款,N,2023-03-13 20:33:50,NaN
4731,21865,6130000000000000000,代收民航发展基金,代收民航发展基金,N,2023-03-13 20:34:10,NaN
4732,19620,6140000000000000000,拍卖行受托拍卖文物艺术品代收货款,受托拍卖文物艺术品代收货款,N,2023-03-13 20:33:50,NaN
4733,22155,6150000000000000000,与销售行为不挂钩的财政补贴收入,与销售行为不挂钩的财政补贴收入,N,2023-03-13 20:34:21,NaN


In [33]:
df_con

,id,spbm,spmc,spbmjc,hzx,update_time,tupu_code,tupu_name,特殊,Unnamed: 9,Unnamed: 10
0,23551,1000000000000000000,货物,货物,Y,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
1,23322,1010000000000000000,农、林、牧、渔业类产品,农产品,Y,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
2,23267,1010100000000000000,农业产品,农业产品,Y,2023-03-13 20:34:00,1,农业产品,NaN,NaN,True
3,23024,1010101000000000000,谷物,谷物,Y,2023-03-13 20:34:00,101,谷物,NaN,NaN,True
4,21554,1010101010000000000,稻谷,谷物,N,2023-03-13 20:34:00,1310030,大米,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4730,19657,6120000000000000000,建筑服务预收款,建筑服务预收款,N,2023-03-13 20:33:00,0,0,NaN,NaN,NaN
4731,21865,6130000000000000000,代收民航发展基金,代收民航发展基金,N,2023-03-13 20:34:00,0,0,NaN,NaN,NaN
4732,19620,6140000000000000000,拍卖行受托拍卖文物艺术品代收货款,受托拍卖文物艺术品代收货款,N,2023-03-13 20:33:00,6330020300,文物拍卖服务,NaN,NaN,NaN
4733,22155,6150000000000000000,与销售行为不挂钩的财政补贴收入,与销售行为不挂钩的财政补贴收入,N,2023-03-13 20:34:00,0,0,NaN,NaN,NaN


In [26]:
len(df_con['spbm'].unique().tolist())

218

In [27]:
df_con['spbm'].unique().tolist()

['1E+18',
 '1.01E+18',
 '1.0101E+18',
 '1.01011E+18',
 '1.01012E+18',
 '1.0102E+18',
 '1.0103E+18',
 '1.0104E+18',
 '1.01041E+18',
 '1.0105E+18',
 '1.01051E+18',
 '1.0106E+18',
 '1.02E+18',
 '1.0201E+18',
 '1.0202E+18',
 '1.02021E+18',
 '1.0203E+18',
 '1.0204E+18',
 '1.0205E+18',
 '1.02051E+18',
 '1.0206E+18',
 '1.03E+18',
 '1.0301E+18',
 '1.03011E+18',
 '1.03012E+18',
 '1.0302E+18',
 '1.03021E+18',
 '1.0303E+18',
 '1.03031E+18',
 '1.0304E+18',
 '1.03041E+18',
 '1.04E+18',
 '1.0401E+18',
 '1.04011E+18',
 '1.0402E+18',
 '1.04021E+18',
 '1.0403E+18',
 '1.04031E+18',
 '1.05E+18',
 '1.0501E+18',
 '1.05011E+18',
 '1.0502E+18',
 '1.06E+18',
 '1.0601E+18',
 '1.06011E+18',
 '1.0602E+18',
 '1.06021E+18',
 '1.0603E+18',
 '1.0604E+18',
 '1.06041E+18',
 '1.0605E+18',
 '1.06051E+18',
 '1.07E+18',
 '1.0701E+18',
 '1.0702E+18',
 '1.07021E+18',
 '1.07022E+18',
 '1.07023E+18',
 '1.0703E+18',
 '1.07031E+18',
 '1.0704E+18',
 '1.0705E+18',
 '1.07051E+18',
 '1.0706E+18',
 '1.08E+18',
 '1.0801E+18',
 '1.080

In [11]:
df_chain_nodes.to_csv(r'E:\project_data\04yulin\fp\chain_nodes.csv',index=False,mode='w')

In [18]:
df=df_chain_nodes.groupby(['tupu_name'])['chain'].apply(lambda x:';'.join(x)).reset_index()
df.loc[df['chain'].str.contains(';')]

,tupu_name,chain
16,专业技术服务,“碳中和”产业链;铝镁及兰炭深加工产业链
18,专用设备,“碳中和”产业链;能源化工装备制造产业链
21,丙烯腈,新能源装备制造产业链;现代煤化工产业链
29,乙烯,新能源装备制造产业链;现代煤化工产业链
46,传感器,能源化工装备制造产业链 ;智能无人系统和航空产业链
61,兰炭,“碳中和”产业链;现代煤化工产业链
77,冰乙酸（冰醋酸）,新能源装备制造产业链;现代煤化工产业链
84,制氢设备,“碳中和”产业链;氢能产业链
95,化学试剂和助剂,“碳中和”产业链;新能源装备制造产业链
105,原煤,铝镁及兰炭深加工产业链;现代煤化工产业链


In [130]:
df_chain_fp_filt['tupu_name']==

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm,length_gf_shxydm,length_gf_nsrmc,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_province_name,gf_province_name,xf_city_name,gf_city_name,xf_district_name,gf_district_name,xf_ind_type,gf_ind_type
1171,4566881,0,91610828790767838A,佳县宏达燃气有限公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局通镇税务分局,000000000000000,佳县坑镇敬老院,NaN,NaN,NaN,NaN,*液化气*液化石油气,1.070101e+18,7000.00,0.00,700.00,公斤,2023-01,2023-04-03 21:16:59,液化石油气,2.502110e+09,液化石油气,陕西省,榆林市,佳县,NaN,NaN,NaN,18,15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
54983,4620693,0,916100007099182651,陕西南梁矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局第二税务分局,96160000580766765J,陕西省清水川能源股份有限公司,NaN,NaN,NaN,NaN,*煤炭*沫煤,1.020101e+18,-3357189.03,-436434.57,-4863.62,吨,2023-01,2023-04-03 21:37:03,原煤,6.010000e+08,原煤,陕西省,榆林市,府谷县,NaN,NaN,NaN,18,18,14,NaN,16,NaN,1600,NaN,160000,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN
80140,4645850,0,91610824MA705CU194,靖边县弘丰新能源有限公司,C2511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局靖边县税务局杨桥畔税务分局,000000000000000,零星销售,NaN,NaN,NaN,NaN,*液化气*饱和液化气,1.070101e+18,1193631.10,107426.80,241.94,吨,2023-01,2023-04-03 21:46:47,液化石油气,2.502110e+09,液化石油气,陕西省,榆林市,靖边县,NaN,NaN,NaN,18,15,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN
213533,4779243,0,916100007099182651,陕西南梁矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局第二税务分局,96160000580766765J,陕西清水川能源股份有限公司,NaN,NaN,NaN,NaN,*原煤*沫煤,1.020101e+18,3357189.03,436434.57,4863.62,吨,2023-01,2023-04-03 22:36:24,原煤,6.010000e+08,原煤,陕西省,榆林市,府谷县,NaN,NaN,NaN,18,18,13,NaN,16,NaN,1600,NaN,160000,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN
226304,4792014,0,91610828790767838A,佳县宏达燃气有限公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局通镇税务分局,000000000000000,佳县乌镇敬老院,NaN,NaN,NaN,NaN,*液化气*液化石油气,1.070101e+18,7000.00,70.00,700.00,公斤,2023-01,2023-04-03 22:41:08,液化石油气,2.502110e+09,液化石油气,陕西省,榆林市,佳县,NaN,NaN,NaN,18,15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768236,6333946,65,91610821MA709GJH2C,神木市准神兴泰煤业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局大柳塔税务分局,91650103333108343N,新疆光海晋毅商贸有限公司,NaN,NaN,NaN,NaN,*原煤*块煤,1.020101e+18,585368.85,76097.95,612.56,吨,2023-01,2023-04-04 08:21:08,原煤,6.010000e+08,原煤,陕西省,榆林市,神木市,新疆维吾尔自治区,乌鲁木齐市,沙依巴克区,18,18,12,NaN,65,NaN,6501,NaN,650103,NaN,新疆维吾尔自治区,NaN,乌鲁木齐市,NaN,沙依巴克区,B,NaN
1768311,6334021,65,91610800MA70DQ140K,榆林华领新能供应链管理有限公司,L7224,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局牛家梁税务分局,91650502MABJJW6W51,新疆陕域能源供应链有限公司,NaN,NaN,NaN,NaN,*原煤*沫煤,1.020101e+18,171213.10,22257.70,117.97,吨,2023-01,2023-04-04 08:21:08,原煤,6.010000e+08,原煤,陕西省,榆林市,榆林市榆阳区,新疆维吾尔自治区,哈密市,伊州区,18,18,13,NaN,65,NaN,6505,NaN,650502,NaN,新疆维吾尔自治区,NaN,哈密市,NaN,伊州区,L,NaN
1769854,6335564,65,916108230880722358,横山县星顺煤业有限责任公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市横山区税务局波罗税务分局,91652300572528273E,中疆物流有限责任公司,NaN,NaN,NaN,NaN,*原煤*原煤,1.020101e+18,133625.52,17371.32,177.44,吨,2023-01,2023-04-04 08:21:31,原煤,6.010000e+08,原煤,陕西省,榆林市,榆林市横山区,新疆维吾尔自治区,昌吉回族自治州,昌吉回族自治州,18,18,10,NaN,65,NaN,6523,NaN,652300,NaN,新疆维吾尔自治区,NaN,昌吉回族自治州,NaN,昌吉回族自治州,B,NaN
1769879,6335589,65,9161082169114722XY,神木市瑞鑫洗煤有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局高家堡税务分局,916540216763420199,新疆睿石硅业有限公司,NaN,NaN,NaN,NaN,*原煤*精煤,1.020101e+18,407996.55,53039.55,250.37,吨,2023-01,2023-04-04 08:21:31,原煤,6.010000e+08,原煤,陕西省,榆林市,神木市,新疆维吾尔自治区,伊犁哈萨克自治州,伊宁县,18,18,10,NaN,65,NaN,6540,NaN,654021,NaN,新疆维吾尔自治区,NaN,伊犁哈萨克自治州,NaN,伊宁县,B,NaN


In [129]:
df_chain_fp_filt_new=df_chain_fp.loc[(df_chain_fp['xf_city']=='榆林市')&(df_chain_fp['xf_ind_type'].isin(['F','G']))]
df_chain_fp_filt_new.groupby(['xf_nsrmc'])['je'].sum().reset_index().sort_values(by='je',ascending=False)

,xf_nsrmc,je
726,陕煤运销集团榆中销售有限公司,2.355643e+09
13,国能销售集团榆林结算有限公司,1.881341e+09
821,陕西榆林能源集团榆神煤电有限公司,5.608023e+08
727,陕煤运销集团榆林销售有限公司,3.496046e+08
822,陕西榆林能源集团煤炭运销有限公司,2.712982e+08
...,...,...
180,榆林兴旺源清洁能源有限公司,-8.842211e+04
165,榆林亿泽顺商贸有限公司,-1.334196e+05
915,靖边县达旺物流有限公司,-1.608633e+05
608,神木市宏瑞泰能源有限公司,-1.609890e+05


In [120]:
df_chain_fp.loc[(df_chain_fp['xf_city']=='榆林市')&(~df_chain_fp['xf_ind_type'].isin(['B','F','G']))&(~df_chain_fp_filt['tupu_name'].isin(deselect_pro))]

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm,length_gf_shxydm,length_gf_nsrmc,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_province_name,gf_province_name,xf_city_name,gf_city_name,xf_district_name,gf_district_name,xf_ind_type,gf_ind_type
226506,4792216,0,916108007412961609,陕西易初明通工程机械维修服务有限公司榆林分公司,O8111,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局大柳塔税务分局,000000000000000,王文浩,NaN,NaN,NaN,NaN,*润滑油*机油,1.070101e+18,4293.00,558.09,180.00,升,2023-01,2023-04-03 22:41:08,润滑油,2.502040e+09,润滑油,陕西省,榆林市,神木市,NaN,NaN,NaN,18,15,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN
293398,4859108,11,91610821748622598U,陕西北元化工集团股份有限公司,C2651,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局第二税务分局,91110105797588795E,中恒华信国际贸易（北京）有限公司,NaN,NaN,NaN,NaN,*化学合成材料*聚氯乙烯,1.070213e+18,5356991.16,696408.84,956.00,吨,2023-01,2023-04-03 23:05:58,聚对苯二甲酸乙二醇酯、PET、树脂,3.001021e+09,聚对苯二甲酸乙二酯塑料板、片,陕西省,榆林市,神木市,北京市,北京市,朝阳区,18,18,16,NaN,11,NaN,1101,NaN,110105,NaN,北京市,NaN,北京市,NaN,朝阳区,C,NaN
309586,4875296,11,9161082271355816XC,陕西三忻（集团）实业有限责任公司,C3217,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局三道沟税务分局,91110116MA01QQ0Y9H,北京富星恒通科贸有限公司,NaN,NaN,NaN,NaN,*有色金属冶炼压延品*镁锭,1.080307e+18,193805.31,25194.69,10.00,吨,2023-01,2023-04-03 23:12:01,镁,3.217010e+06,镁,陕西省,榆林市,府谷县,北京市,北京市,怀柔区,18,18,12,NaN,11,NaN,1101,NaN,110116,NaN,北京市,NaN,北京市,NaN,怀柔区,C,NaN
314532,4880242,11,91610821748622598U,陕西北元化工集团股份有限公司,C2651,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局第二税务分局,911103025674229926,北京亿兆华盛股份有限公司,NaN,NaN,NaN,NaN,*化学合成材料*聚氯乙烯,1.070213e+18,13926902.66,1810497.34,2522.00,吨,2023-01,2023-04-03 23:13:52,聚对苯二甲酸乙二醇酯、PET、树脂,3.001021e+09,聚对苯二甲酸乙二酯塑料板、片,陕西省,榆林市,神木市,北京市,NaN,NaN,18,18,12,NaN,11,NaN,1103,NaN,110302,NaN,北京市,NaN,NaN,NaN,NaN,C,NaN
316216,4881926,11,91610821MA703AYG0F,陕西精益化工有限公司,C2619,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局锦界税务分局,91110111766766827K,北京汇能橡塑化工有限公司,NaN,NaN,NaN,NaN,*有机化学原料*苯,1.070202e+18,2562964.25,333185.35,456.84,吨,2023-01,2023-04-03 23:14:38,纯苯,2.614100e+06,纯苯,陕西省,榆林市,神木市,北京市,北京市,房山区,18,18,12,NaN,11,NaN,1101,NaN,110111,NaN,北京市,NaN,北京市,NaN,房山区,C,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763762,6329472,64,91610822575627197H,府谷县颐河镁业有限责任公司,C3217,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局新民税务分局,91640100MA7M3PMP9E,宁夏德信佰铖金属材料有限公司,NaN,NaN,NaN,NaN,*有色金属冶炼压延品*镁锭,1.080307e+18,648577.03,84315.01,33.31,吨,2023-01,2023-04-04 08:19:17,镁,3.217010e+06,镁,陕西省,榆林市,府谷县,宁夏回族自治区,银川市,银川市,18,18,14,NaN,64,NaN,6401,NaN,640100,NaN,宁夏回族自治区,NaN,银川市,NaN,银川市,C,NaN
1765125,6330835,64,9161082271355816XC,陕西三忻（集团）实业有限责任公司,C3217,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局三道沟税务分局,91640521317837968K,宁夏鹏波工贸有限公司,NaN,NaN,NaN,NaN,*有色金属冶炼压延品*镁锭,1.080307e+18,636637.17,82762.83,33.00,吨,2023-01,2023-04-04 08:20:02,镁,3.217010e+06,镁,陕西省,榆林市,府谷县,宁夏回族自治区,中卫市,中宁县,18,18,10,NaN,64,NaN,6405,NaN,640521,NaN,宁夏回族自治区,NaN,中卫市,NaN,中宁县,C,NaN
1765558,6331268,64,916108217625768311,神木市鑫庆金属镁业有限公司,C3217,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局店塔税务分局,91640100MABP164H62,宁夏诚镁实业有限公司,NaN,NaN,NaN,NaN,*有色金属冶炼压延品*镁锭,1.080307e+18,192920.35,25079.65,10.00,吨,2023-01,2023-04-04 08:20:02,镁,3.217010e+06,镁,陕西省,榆林市,神木市,宁夏回族自治区,银川市,银川市,18,18,10,NaN,64,NaN,6401,NaN,640100,NaN,宁夏回族自治区,NaN,银川市,NaN,银川市,C,NaN
1766547,6332257,64,91610802223963568A,榆林恒泰汽车维修技术服务站,O8111,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局牛家梁税务分局,91640300MA7LLQN35M,宁夏路捷物流有限公司,NaN,NaN,NaN,NaN,*润滑油*柴油机油,1.070101e+18,398.23,51.77,18.00,升,2023-01,2023-04-04 08:20:25,润滑油,2.502040e+09,润滑油,陕西省,榆林市,榆林市榆阳区,宁夏回族自治区,吴忠市,吴忠市,18,18,10,NaN,64,NaN,6403,NaN,640300,NaN,宁夏回族自治区,NaN,吴忠市,NaN,吴忠市,O,NaN


In [118]:
df_chain_fp_filt_new=df_chain_fp_filt.loc[~df_chain_fp_filt['tupu_name'].isin(deselect_pro)]
df_chain_fp_filt_new.groupby(['xf_nsrmc'])['je'].sum().reset_index().sort_values(by='je',ascending=False).head(10)

,xf_nsrmc,je
137,陕西北元化工集团股份有限公司,3.360620e+08
141,陕西延长中煤榆林能源化工股份有限公司,1.807150e+08
142,陕西延长石油（集团）有限责任公司榆林核算部,9.957622e+07
166,陕西金泰氯碱化工有限公司,9.147037e+07
145,陕西恒源投资集团焦化有限公司,4.143861e+07
...,...,...
140,陕西同多泰商贸有限公司,0.000000e+00
75,榆林百达顺汽车服务有限公司,0.000000e+00
174,靖边县武通实业有限公司,0.000000e+00
70,榆林市鑫福泰汽车维修有限公司,-2.831900e+02


In [110]:
df_chain_fp_filt.groupby(['tupu_name'])['je'].sum().reset_index().sort_values(by='je',ascending=False)

,tupu_name,je
2,原煤,5.960142e+09
7,煤焦油,5.711174e+08
12,聚对苯二甲酸乙二酯塑料板、片,4.276091e+08
13,镁,3.686918e+08
10,石脑油,2.562158e+08
6,液化石油气,1.780042e+08
3,对二甲苯（PX）,7.802832e+07
8,燃料油,7.172474e+07
11,纯苯,3.598321e+07
0,丙烯,7.255623e+06


In [106]:
df_chain_fp_filt

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm,length_gf_shxydm,length_gf_nsrmc,xf_province_code,gf_province_code,xf_city_code,gf_city_code,xf_district_code,gf_district_code,xf_province_name,gf_province_name,xf_city_name,gf_city_name,xf_district_name,gf_district_name,xf_ind_type,gf_ind_type
1171,4566881,0,91610828790767838A,佳县宏达燃气有限公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局通镇税务分局,000000000000000,佳县坑镇敬老院,NaN,NaN,NaN,NaN,*液化气*液化石油气,1.070101e+18,7000.00,0.00,700.00,公斤,2023-01,2023-04-03 21:16:59,液化石油气,2.502110e+09,液化石油气,陕西省,榆林市,佳县,NaN,NaN,NaN,18,15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
54983,4620693,0,916100007099182651,陕西南梁矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局第二税务分局,96160000580766765J,陕西省清水川能源股份有限公司,NaN,NaN,NaN,NaN,*煤炭*沫煤,1.020101e+18,-3357189.03,-436434.57,-4863.62,吨,2023-01,2023-04-03 21:37:03,原煤,6.010000e+08,原煤,陕西省,榆林市,府谷县,NaN,NaN,NaN,18,18,14,NaN,16,NaN,1600,NaN,160000,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN
80140,4645850,0,91610824MA705CU194,靖边县弘丰新能源有限公司,C2511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局靖边县税务局杨桥畔税务分局,000000000000000,零星销售,NaN,NaN,NaN,NaN,*液化气*饱和液化气,1.070101e+18,1193631.10,107426.80,241.94,吨,2023-01,2023-04-03 21:46:47,液化石油气,2.502110e+09,液化石油气,陕西省,榆林市,靖边县,NaN,NaN,NaN,18,15,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN
213533,4779243,0,916100007099182651,陕西南梁矿业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局府谷县税务局第二税务分局,96160000580766765J,陕西清水川能源股份有限公司,NaN,NaN,NaN,NaN,*原煤*沫煤,1.020101e+18,3357189.03,436434.57,4863.62,吨,2023-01,2023-04-03 22:36:24,原煤,6.010000e+08,原煤,陕西省,榆林市,府谷县,NaN,NaN,NaN,18,18,13,NaN,16,NaN,1600,NaN,160000,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN
226304,4792014,0,91610828790767838A,佳县宏达燃气有限公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局佳县税务局通镇税务分局,000000000000000,佳县乌镇敬老院,NaN,NaN,NaN,NaN,*液化气*液化石油气,1.070101e+18,7000.00,70.00,700.00,公斤,2023-01,2023-04-03 22:41:08,液化石油气,2.502110e+09,液化石油气,陕西省,榆林市,佳县,NaN,NaN,NaN,18,15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768236,6333946,65,91610821MA709GJH2C,神木市准神兴泰煤业有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局大柳塔税务分局,91650103333108343N,新疆光海晋毅商贸有限公司,NaN,NaN,NaN,NaN,*原煤*块煤,1.020101e+18,585368.85,76097.95,612.56,吨,2023-01,2023-04-04 08:21:08,原煤,6.010000e+08,原煤,陕西省,榆林市,神木市,新疆维吾尔自治区,乌鲁木齐市,沙依巴克区,18,18,12,NaN,65,NaN,6501,NaN,650103,NaN,新疆维吾尔自治区,NaN,乌鲁木齐市,NaN,沙依巴克区,B,NaN
1768311,6334021,65,91610800MA70DQ140K,榆林华领新能供应链管理有限公司,L7224,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局牛家梁税务分局,91650502MABJJW6W51,新疆陕域能源供应链有限公司,NaN,NaN,NaN,NaN,*原煤*沫煤,1.020101e+18,171213.10,22257.70,117.97,吨,2023-01,2023-04-04 08:21:08,原煤,6.010000e+08,原煤,陕西省,榆林市,榆林市榆阳区,新疆维吾尔自治区,哈密市,伊州区,18,18,13,NaN,65,NaN,6505,NaN,650502,NaN,新疆维吾尔自治区,NaN,哈密市,NaN,伊州区,L,NaN
1769854,6335564,65,916108230880722358,横山县星顺煤业有限责任公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市横山区税务局波罗税务分局,91652300572528273E,中疆物流有限责任公司,NaN,NaN,NaN,NaN,*原煤*原煤,1.020101e+18,133625.52,17371.32,177.44,吨,2023-01,2023-04-04 08:21:31,原煤,6.010000e+08,原煤,陕西省,榆林市,榆林市横山区,新疆维吾尔自治区,昌吉回族自治州,昌吉回族自治州,18,18,10,NaN,65,NaN,6523,NaN,652300,NaN,新疆维吾尔自治区,NaN,昌吉回族自治州,NaN,昌吉回族自治州,B,NaN
1769879,6335589,65,9161082169114722XY,神木市瑞鑫洗煤有限公司,B0610,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局高家堡税务分局,916540216763420199,新疆睿石硅业有限公司,NaN,NaN,NaN,NaN,*原煤*精煤,1.020101e+18,407996.55,53039.55,250.37,吨,2023-01,2023-04-04 08:21:31,原煤,6.010000e+08,原煤,陕西省,榆林市,神木市,新疆维吾尔自治区,伊犁哈萨克自治州,伊宁县,18,18,10,NaN,65,NaN,6540,NaN,654021,NaN,新疆维吾尔自治区,NaN,伊犁哈萨克自治州,NaN,伊宁县,B,NaN


In [97]:
df_province=df_chain_fp_filt.groupby(['gf_province'])['je'].sum().reset_index()
df_province.rename(columns={'je':'province_je'},inplace=True)
df_city=df_chain_fp_filt.groupby(['gf_province','gf_city'])['je'].sum().reset_index()
df_city.rename(columns={'je':'city_je'},inplace=True)
df_area_market=pd.merge(left=df_city,right=df_province,on=['gf_province'],how='left')

In [98]:
df_area_market

,gf_province,gf_city,city_je,province_je
0,上海市,上海市,52949700.38,5.294970e+07
1,云南省,大理白族自治州,107415.93,2.381878e+05
2,云南省,曲靖市,130771.86,2.381878e+05
3,内蒙古自治区,乌兰察布市,568794.51,4.935594e+08
4,内蒙古自治区,乌海市,4488534.32,4.935594e+08
...,...,...,...,...
185,陕西省,陕西省,23029507.63,5.436768e+09
186,青海省,西宁市,36892.04,3.689204e+04
187,黑龙江省,哈尔滨市,204186.91,4.744312e+05
188,黑龙江省,大庆市,185288.50,4.744312e+05


In [110]:
# 发票数据处理：
# 增加地址信息
# xf_province,xf_city,xf_district
# gf_province,gf_city,gf_district
df_merge_fp_1=df_merge_fp.copy()
df_merge_fp.loc[df_merge_fp['xf_sjswjg'].notnull(),'xf_province']=df_merge_fp.loc[df_merge_fp['xf_sjswjg'].notnull()]['xf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['xf_dsswjg'].notnull(),'xf_city']=df_merge_fp.loc[df_merge_fp['xf_dsswjg'].notnull()]['xf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['xf_qxswjg'].notnull(),'xf_district']=df_merge_fp.loc[df_merge_fp['xf_qxswjg'].notnull()]['xf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()
df_merge_fp.loc[df_merge_fp['gf_sjswjg'].notnull(),'gf_province']=df_merge_fp.loc[df_merge_fp['gf_sjswjg'].notnull()]['gf_sjswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['gf_dsswjg'].notnull(),'gf_city']=df_merge_fp.loc[df_merge_fp['gf_dsswjg'].notnull()]['gf_dsswjg'].str.extract(r'国家税务总局(.*?)税务局')[0].tolist()
df_merge_fp.loc[df_merge_fp['gf_qxswjg'].notnull(),'gf_district']=df_merge_fp.loc[df_merge_fp['gf_qxswjg'].notnull()]['gf_qxswjg'].str.extract(r'国家税务总局(.*?)税务局.*?')[0].tolist()

df_merge_fp['gf_shxydm'].fillna('0',inplace=True)
df_merge_fp['gf_nsrmc'].fillna('0',inplace=True)

ls_len_xf = [len(x) for x in df_merge_fp['xf_shxydm'].tolist()]
df_merge_fp.loc[:,'length_xf_shxydm'] = ls_len_xf
ls_len_gf = [len(x) for x in df_merge_fp['gf_shxydm'].tolist()]
df_merge_fp.loc[:,'length_gf_shxydm'] = ls_len_gf

df_merge_fp.loc[(df_merge_fp['ls_len_xf']==18)&(df_merge_fp['xf_sjswjg'].isnull()),'xf_province_code']=df_merge_fp['xf_shxydm'].str[2:4]
df_merge_fp.loc[(df_merge_fp['ls_len_xf']==18)&(df_merge_fp['xf_sjswjg'].isnull()),'xf_city_code']=df_merge_fp['xf_shxydm'].str[2:6]
df_merge_fp.loc[(df_merge_fp['ls_len_xf']==18)&(df_merge_fp['xf_sjswjg'].isnull()),'xf_district_code']=df_merge_fp['xf_shxydm'].str[2:8]
df_merge_fp.loc[(df_merge_fp['ls_len_gf']==18)&(df_merge_fp['gf_sjswjg'].isnull()),'gf_province_code']=df_merge_fp['gf_shxydm'].str[2:4]
df_merge_fp.loc[(df_merge_fp['ls_len_gf']==18)&(df_merge_fp['gf_sjswjg'].isnull()),'gf_city_code']=df_merge_fp['gf_shxydm'].str[2:6]
df_merge_fp.loc[(df_merge_fp['ls_len_gf']==18)&(df_merge_fp['gf_sjswjg'].isnull()),'gf_district_code']=df_merge_fp['gf_shxydm'].str[2:8]

TypeError: object of type 'float' has no len()

In [121]:
df_merge_fp.loc[df_merge_fp['gf_nsrmc'].str.startswith('91')]

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm
4885,4570595,0,916108253054710499,定边县百绿坊农牧业发展有限公司,E5090,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局郝滩税务分局,0,91610825067936014F,NaN,NaN,NaN,NaN,*建筑服务*工程服务,3.050100e+18,0.00,0.00,NaN,NaN,2023-01,2023-04-03 21:18:15,工程服务,4.701021e+09,输气管道工程服务,陕西省,榆林市,定边县,NaN,NaN,NaN,18
17270,4582980,0,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理一股,000000000000000,9113023MA09AKY19P,NaN,NaN,NaN,NaN,*柴油*0#车用柴油（国VI）,1.070101e+18,1769.91,230.09,266.31,升,2023-01,2023-04-03 21:23:06,柴油,2.502030e+09,柴油,陕西省,榆林市,榆林高新技术产业开发区,NaN,NaN,NaN,18
77668,4643378,0,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理一股,000000000000000,91130230MA09AKY19P,NaN,NaN,NaN,NaN,*柴油*0#车用柴油（国VI）,1.070101e+18,884.96,115.04,133.16,升,2023-01,2023-04-03 21:45:42,柴油,2.502030e+09,柴油,陕西省,榆林市,榆林高新技术产业开发区,NaN,NaN,NaN,18
115847,4681557,0,91610826MA70C22X3B,绥德县合晟商贸有限责任公司,G6020,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局绥德县税务局田庄税务分局,0,91610826064826557T,NaN,NaN,NaN,NaN,*物流辅助服务*派送服务,3.040409e+18,0.00,0.00,NaN,NaN,2023-01,2023-04-03 21:59:55,派送服务,5.702990e+09,物流代理服务,陕西省,榆林市,绥德县,NaN,NaN,NaN,18
116482,4682192,0,91610800223930221M,中国石油天然气股份有限公司陕西榆林销售分公司,F5265,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局上郡路税务分局,000000000000000,91z,NaN,NaN,NaN,NaN,*柴油*-20号 车用柴油(Ⅵ),1.070101e+18,460.18,59.82,64.12,升,2023-01,2023-04-03 22:00:17,柴油,2.502030e+09,柴油,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18
160357,4726067,0,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理一股,000000000000000,911400005998733769,NaN,NaN,NaN,NaN,*汽油*92号车用汽油(ⅥB),1.070101e+18,123.89,16.11,18.25,升,2023-01,2023-04-03 22:16:49,汽油,2.502010e+09,汽油,陕西省,榆林市,榆林高新技术产业开发区,NaN,NaN,NaN,18
225613,4791323,0,91610800552165727D,中国石化销售股份有限公司陕西榆林石油分公司,F5162,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林高新技术产业开发区税务局税源管理一股,000000000000000,916108007663110809,NaN,NaN,NaN,NaN,*汽油*95号车用汽油(ⅥB),1.070101e+18,442.48,57.52,63.05,升,2023-01,2023-04-03 22:40:46,汽油,2.502010e+09,汽油,陕西省,榆林市,榆林高新技术产业开发区,NaN,NaN,NaN,18
380055,4945765,13,91610802MA7HURTH6X,陕西隆盛瀚宝能源有限公司,F5281,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局牛家梁税务分局,91130481MA08K1LL11,91130481MA08K1LL11,NaN,NaN,NaN,NaN,*原煤*煤炭,1.020101e+18,0.00,0.00,0.00,吨,2023-01,2023-04-03 23:38:24,原煤,6.010000e+08,原煤,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18
586462,5152172,37,91610800552185445P,陕西中交榆佳高速公路有限公司,K7040,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理一股,91371302MA3W9TP5XE,91371302MA3W9TP5XE,NaN,NaN,NaN,NaN,*经营租赁*通行费,3.040502e+18,165.64,4.97,20221017.00,货车,2023-01,2023-04-04 00:56:10,高速公路通行服务,7.607080e+09,道路、桥梁、隧道工程综合服务,陕西省,榆林市,榆林市,NaN,NaN,NaN,18
591078,5156788,37,91610800661191780F,陕西榆林榆神高速公路有限公司,L7119,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,91371302MA3W9TP5XE,91371302MA3W9TP5XE,NaN,NaN,NaN,NaN,*经营租赁*通行费,3.040502e+18,7.93,0.24,20221017.00,货车,2023-01,2023-04-04 00:58:04,高速公路通行服务,7.607080e+09,道路、桥梁、隧道工程综合服务,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18


In [117]:
df_merge_fp.loc[df_merge_fp['gf_shxydm']=='0']

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province,xf_city,xf_district,gf_province,gf_city,gf_district,length_xf_shxydm
399,4566109,0,916108303387495619,清涧县康家圪塔农业光伏发电公司,D4416,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局店则沟税务分局,0,陕西黄原生物工程有限公司,NaN,NaN,NaN,NaN,*美容护肤品*洗发水,1.070223e+18,0.0,0.0,0.0,桶,2023-01,2023-04-03 21:16:33,清洁类化妆品,2.623010e+05,清洁类化妆品,陕西省,榆林市,清涧县,NaN,NaN,NaN,18
4440,4570150,0,916108243056031791,靖边县杰潮园林绿化工程有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局靖边县税务局杨米涧税务分局,0,孙三（个人）,NaN,NaN,NaN,NaN,*泵*绿化工程,1.090117e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 21:18:15,泵,3.441010e+06,泵,陕西省,榆林市,靖边县,NaN,NaN,NaN,18
4885,4570595,0,916108253054710499,定边县百绿坊农牧业发展有限公司,E5090,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局定边县税务局郝滩税务分局,0,91610825067936014F,NaN,NaN,NaN,NaN,*建筑服务*工程服务,3.050100e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 21:18:15,工程服务,4.701021e+09,输气管道工程服务,陕西省,榆林市,定边县,NaN,NaN,NaN,18
5091,4570801,0,936108295706588322,陕西省吴堡县小康红枣专业合作社,F5299,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局吴堡县税务局郭家沟税务所,0,吴堡县公安局交通警察大队,NaN,NaN,NaN,NaN,*水果*红枣,1.010115e+18,0.0,0.0,NaN,斤,2023-01,2023-04-03 21:18:38,枣,1.160199e+08,枣,陕西省,榆林市,吴堡县,NaN,NaN,NaN,18
5542,4571252,0,91610824MA703X4606,靖边县伟元实业有限公司,E4821,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局靖边县税务局东坑税务分局,0,张燕,NaN,NaN,NaN,NaN,*建筑服务*工程服务,3.050100e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 21:18:38,工程服务,4.701021e+09,输气管道工程服务,陕西省,榆林市,靖边县,NaN,NaN,NaN,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282474,4848184,0,91610824MA709D792B,靖边县田中田中药材有限公司,A0179,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局靖边县税务局杨米涧税务分局,0,王小（个人）,NaN,NaN,NaN,NaN,*中草药材*中草药材,1.010118e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 23:01:55,中草药材,1.190000e+02,中草药材,陕西省,榆林市,靖边县,NaN,NaN,NaN,18
286858,4852568,0,91610802MA708QHD9R,榆林市铭达电子科技有限公司,F5273,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局青山路税务分局,0,赵燕,NaN,NaN,NaN,NaN,*乐器*中乐器,1.060407e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 23:03:22,中乐器,2.407010e+05,中乐器,陕西省,榆林市,榆林市榆阳区,NaN,NaN,NaN,18
287039,4852749,0,91610826305691658C,绥德县长欣绿野有限责任公司,A0181,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局绥德县税务局田庄税务分局,0,绥德县林业局,NaN,NaN,NaN,NaN,*建筑服务*建筑服务,3.050100e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 23:03:43,工程服务,4.701021e+09,输气管道工程服务,陕西省,榆林市,绥德县,NaN,NaN,NaN,18
287634,4853344,0,91610823MA70A0MH3H,榆林新动向旅游开发有限公司,L7282,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市横山区税务局石湾税务所,0,榆林市横山区农嘉牧种养殖专业合作社,NaN,NaN,NaN,NaN,*塑料制品*日用塑料制品,1.070601e+18,0.0,0.0,NaN,NaN,2023-01,2023-04-03 23:03:43,日用塑料制品,2.927010e+06,日用塑料制品,陕西省,榆林市,榆林市横山区,NaN,NaN,NaN,18


In [120]:
df_merge_fp['gf_shxydm'].fillna('0',inplace=True)
df_merge_fp['gf_nsrmc'].fillna('0',inplace=True)

In [105]:
df_merge_fp_1=pd.merge(left=df_merge_fp,right=df_province_1,left_on=['xf_province_code'],right_on=['code1'],how='left')

str

In [79]:
df_merge_fp.loc[df_merge_fp['xf_sjswjg'].notnull()]

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time,spmc,tupu_code,tupu_name,xf_province
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1.100102e+18,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33,三北热力,4.402010e+09,热力,陕西省
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33,机动车交通事故责任强制保险服务,7.002010e+09,交强险服务,陕西省
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1.100102e+18,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33,三北热力,4.402010e+09,热力,陕西省
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1.100202e+18,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33,民用天然气,7.020000e+02,天然气,陕西省
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3.060303e+18,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33,其他财产保险服务,7.002990e+09,其他财产保险服务,陕西省
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3.060303e+18,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03,其他财产保险服务,7.002990e+09,其他财产保险服务,陕西省
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03,机动车交通事故责任强制保险服务,7.002010e+09,交强险服务,陕西省
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1.030113e+18,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03,其他淀粉及淀粉制品,1.312990e+09,其他淀粉及淀粉制品,陕西省
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,3.060302e+18,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03,其他人身保险服务,7.001000e+03,人身保险服务,陕西省


In [67]:
# 计算每条产业链的数据：销售份额，
count=1
for i in ls_chain_name:
    print('number',count,'chain',i)
    ls_i_nodes_ori=re.findall('"text":"(.*?)"',df_chain_name_10.loc[df_chain_name_10['NAME']==i,'GRAPH'].values[0])
    ls_i_nodes_set=list(set(ls_i_nodes_ori))
    print('nodes length',len(ls_i_nodes_set))
    df_chain_fp=df_merge_fp.loc[(df_merge_fp['spmc'].isin(ls_i_nodes_set))|(df_merge_fp['tupu_name'].isin(ls_i_nodes_set))]
    print('fp length',len(df_chain_fp))
    count+=1
    # 产业链销售份额
    
    

number 1 chain “碳中和”产业链
nodes length 106
fp length 23513
number 2 chain 现代特色农业产业链
nodes length 50
fp length 57288
number 3 chain 氢能产业链
nodes length 39
fp length 10206
number 4 chain 能源化工装备制造产业链 
nodes length 37
fp length 15520
number 5 chain 轻纺产业链
nodes length 36
fp length 8173
number 6 chain 文化旅游产业链
nodes length 39
fp length 2602
number 7 chain 智能无人系统和航空产业链
nodes length 45
fp length 5829
number 8 chain 新能源装备制造产业链
nodes length 128
fp length 9829
number 9 chain 铝镁及兰炭深加工产业链
nodes length 66
fp length 28695
number 10 chain 现代煤化工产业链
nodes length 78
fp length 9577


In [6]:
pd.DataFrame(df_fp['spbm'].unique().tolist(),columns=['spbm']).to_csv(r'E:\project_data\04yulin\fp\202301fpspbm.csv',index=False)

In [5]:
df_fp

,id,xzqh,xf_shxydm,xf_nsrmc,xf_hydm,xf_sjswjg,xf_dsswjg,xf_qxswjg,gf_shxydm,gf_nsrmc,gf_hydm,gf_sjswjg,gf_dsswjg,gf_qxswjg,hwmc,spbm,je,se,sl,jldw,kpyf,update_time
0,4565710,0,91610800MA70DD196U,榆林市供热有限公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,张甫东,NaN,NaN,NaN,NaN,*热力*2022-2023采暖费,1.100102e+18,1714.00,0.00,103.88,元/平方米,2023-01,2023-04-03 21:16:33
1,4565711,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,孙涛,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,582.55,34.95,1.00,份,2023-01,2023-04-03 21:16:33
2,4565712,0,916108212240045457,神木市供热有限责任公司,D4430,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局神木市税务局麟州税务分局,000000000000000,奥磊,NaN,NaN,NaN,NaN,*热力*采暖费—2022.11.01-2023.04.15,1.100102e+18,1901.00,0.00,123.43,㎡,2023-01,2023-04-03 21:16:33
3,4565713,0,91610893713586830T,榆林榆川天然气有限责任公司,D4511,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市榆阳区税务局第二税务分局,000000000000000,郭晓鹏,NaN,NaN,NaN,NaN,*燃气*民用天然气,1.100202e+18,183.49,16.51,1.00,无,2023-01,2023-04-03 21:16:33
4,4565714,0,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,000000000000000,谢春富,NaN,NaN,NaN,NaN,*保险服务*家财险,3.060303e+18,113.22,6.78,1.00,份,2023-01,2023-04-03 21:16:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773284,6338994,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*意外伤害保险,3.060303e+18,44.34,2.66,1.00,份,2023-01,2023-04-04 08:23:03
1773285,6338995,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*机动车交通事故责任强制保险,3.060302e+18,492.92,29.58,1.00,份,2023-01,2023-04-04 08:23:03
1773286,6338996,82,91610830MA7054CF73,陕西果业集团清涧有限公司,A0212,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局清涧县税务局宽州税务分局,8261000MC6070433K,陕西河湖生态维护治理有限公司工会,NaN,NaN,NaN,NaN,*淀粉制品*粉条,1.030113e+18,10642.20,957.80,116.00,箱,2023-01,2023-04-04 08:23:03
1773287,6338997,82,916108007552208292,中国平安财产保险股份有限公司榆林中心支公司,J6820,国家税务总局陕西省税务局,国家税务总局榆林市税务局,国家税务总局榆林市税务局第二税务分局征收管理二股,82610800MC7392178B,榆林市中级人民法院工会工作委员会,NaN,NaN,NaN,NaN,*保险服务*人身意外险,3.060302e+18,103.78,6.22,1.00,份,2023-01,2023-04-04 08:23:03


In [ ]:
df_fp.loc[:,['']]